## Technical Analysis

This part of the code is 100% inspired by Ben, a friend of mine in HKU, who performs manual trading for more than 3 years, and has achieved 20x gain using this strategy

#### Initial Version

Copied from VSCode testing env folder

In [ ]:
from AlgoAPI import AlgoAPIUtil, AlgoAPI_Backtest
import pandas as pd
import numpy as np
import mplfinance as mpf
    
    
class BearishFSM: # Finite State Machine

        # ==============================================================
        # ========================  __init__ ===========================
        # ==============================================================
    def __init__(self, df, instrument, evt, account_balance=0, pip_size = 0.0001, pip_value_per_lot = 10):
        
        self.df = df
        self.instrument = instrument
        self.evt = evt
        self.account_balance = account_balance
        
        self.state = "IDLE"
        self.low_price = None
        
        self.low_price_idx = None
        self.high_price = None
        self.high_price_idx = None

        self.fib_labels = []
        self.fib_dict = {} # not a must to initialize, just for good habit
        self.entry_signal_idx = None
        self.engulfing_candle_idx = None
        self.trend_changing_idx = 3 # do not reset this, always keep record of 
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None

        # do not reset the folloiwng parameters, test for optimal values
        self.risk_reward_ratio = 2.5
        self.atr_period = 14
        self.atr_multiplier = 3.0 # pip_size = atr_multiplier * atr_value, atr_multiplier = 2.0 means stop loss is 2 ATRs away
        self.risk_percent = 0.5
        
        # this is unique in every func call
        self.instrument = instrument
        self.pip_size = pip_size
        self.pip_value_per_lot = pip_value_per_lot
        
        # ==============================================================
        # ========================  calculate_lot_size  =============
        # ==============================================================
        
    def log_state_change(self, old_state, new_state, idx, label=None):
        date_str = (
            str(self.df['Datetime'].iloc[idx])
            if 'Datetime' in self.df.columns and idx < len(self.df)
            else str(idx)
        )
        msg = f"[{date_str}] State: {old_state} → {new_state}"
        if label:
            msg += f" ({label})"
        self.evt.consoleLog(msg)
        
        # ==============================================================
        # ========================  calculate_lot_size  =============
        # ==============================================================
        
    def calculate_lot_size(self, risk_percent, stop_loss_pips, pip_value_per_lot=10):
        risk_amount = self.account_balance * (risk_percent / 100)
        lot_size = risk_amount / (stop_loss_pips * pip_value_per_lot)
        return round(lot_size, 4)
        
        # ==============================================================
        # ========================  calc_fibonacci_levels  =============
        # ==============================================================
        
    def calc_fibonacci_levels(self, start_idx, end_idx, levels=None, trend="bearish",
                         entry_signal=None, engulfing_candle=None,
                         R=None, atr_period=None, atr_multiplier=None):
                             
        if levels is None:
            levels = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]
            
        if atr_period is None:
            atr_period = self.atr_period
            
        if atr_multiplier is None:
            atr_multiplier = self.atr_multiplier
        
        if R is None:
            R = self.risk_reward_ratio
            
        df = self.df
        df_slice = df.iloc[start_idx:end_idx + 2 + 1].copy()
        min_price = df_slice['Low'].min()
        max_price = df_slice['High'].max()
    
        fib_labels = []
        for level in levels:
            if trend == "bearish":
                y = min_price + (max_price - min_price) * level
            else:
                y = max_price - (max_price - min_price) * level
            fib_labels.append((level, y))
    
        # Calculate ATR for stop loss & take profit sizing
        # Calculate True Range (TR)
        df['H-L'] = df['High'] - df['Low']
        df['H-PC'] = abs(df['High'] - df['Close'].shift(1))
        df['L-PC'] = abs(df['Low'] - df['Close'].shift(1))
        df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
        # ATR: rolling mean of TR
        df['ATR'] = df['TR'].rolling(window=atr_period).mean()
        # Use ATR at the most recent engulfing_candle or latest in slice
        if engulfing_candle is not None:
            atr_value = df['ATR'].iloc[engulfing_candle]
        else:
            atr_value = df['ATR'].iloc[end_idx]
        pip_size = atr_multiplier * atr_value
    
        stop_loss_price = None
        take_profit_price = None
    
        if R is not None and engulfing_candle is not None:
            candle = df.iloc[engulfing_candle]
            close_price = candle['Close']
    
            if trend == "bearish":
                stop_loss_price = close_price + pip_size
                take_profit_price = close_price - R * pip_size
            else:  # bullish
                stop_loss_price = close_price - pip_size
                take_profit_price = close_price + R * pip_size
    
        return stop_loss_price, take_profit_price, fib_labels
        
    # ==============================================================
    # ========================  update()  =============
    # ==============================================================
    
    def update(self, idx):

        if idx < 3:
            return
        
        trend = self.df['trend'].values
        greens = self.df["Green"]
        opens = self.df["Open"].values
        closes = self.df["Close"].values
        highs = self.df["High"].values
        lows = self.df["Low"].values
        lowest_body_price = np.select([greens == True, greens == False], [opens, closes], default= np.inf) # array-based logic, avoid direct truthiness check
        highest_body_price = np.select([greens == True, greens == False], [closes, opens], default= -np.inf) # default=np.nan ensures that rows where "Green" is missing won't cause errors
        
        current_trend = trend[idx]
        prev_trend = trend[idx - 1]

        if current_trend != prev_trend :
            self.trend_changing_idx = idx
            if self.state != "ENTER_MARKET":
                self.reset()

        if (current_trend == "bearish" and self.enter_market_trade_side is not "bullish") or (current_trend == "bullish" and self.enter_market_trade_side is "bearish"):
            if self.state == "IDLE":
                if self.check_pullback(greens, opens, closes, idx, "bearish"): # pullback should eat and cover the previous red candle
                    self.low_price = lowest_body_price[self.trend_changing_idx:idx+1].min()
                    self.low_price_idx = (self.trend_changing_idx) + np.argmin(
                                            lowest_body_price[self.trend_changing_idx:idx+1])
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, idx, "bearish pullback detected")
                    

            elif self.state == "PULLBACK_DETECTED":
                if lowest_body_price[idx] < self.low_price:

                    self.high_price = highest_body_price[self.low_price_idx:idx].max()
                    self.high_price_idx = self.low_price_idx + np.argmax(highest_body_price[self.low_price_idx:idx])
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, idx, "bearish new low after pullback")

                    
            elif self.state == "BREAK_OF_STRUCTURE":
                # If price breaks the pev high (LH)
                if highest_body_price[idx] > self.high_price:

                    self.low_price = lowest_body_price[self.high_price_idx:idx].min()
                    self.low_price_idx = self.high_price_idx + np.argmin(
                        lowest_body_price[self.high_price_idx:idx])
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, idx, "bearish break of structure")

                elif self.check_pullback(greens, opens, closes, idx, "bearish"):
                    
                    if lowest_body_price[idx -2] < self.low_price: #### Change from idx to idx - 2

                        # Low price for confirming the next Lower high
                        self.low_price = lowest_body_price[idx-3:idx+1].min()
                        self.low_price_idx = (idx-3) + np.argmin(
                            lowest_body_price[idx-3:idx+1])

                        _,_,self.fib_labels = self.calc_fibonacci_levels( self.high_price_idx, self.low_price_idx, trend="bearish")
                        self.fib_dict = dict(self.fib_labels)
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, idx, "bearish waiting entry")

                    elif highest_body_price[idx] >= self.low_price:

                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, idx, "bearish back to pullback")

            elif self.state == "WAITING_ENTRY":
                fib_618_price = self.fib_dict.get(0.618)
                fib_000_price = self.fib_dict.get(0.000)
                if fib_618_price and highs[idx] >= fib_618_price: # price rebound back to 0.618
                
                    self.entry_signal_idx = idx
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, idx, "bearish price rebound to 0.618")

                elif fib_618_price and lowest_body_price[idx] <= fib_000_price: # price lower than 0.000
                    # # break of structure, set new high point
                    self.high_price = highest_body_price[self.low_price_idx:idx].max()
                    self.high_price_idx = self.low_price_idx + np.argmax(
                        highest_body_price[self.low_price_idx:idx])
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, idx, "bearish breaks 0.000 before entry")
            
            elif self.state == "ENTRY_SIGNAL": 
                fib_100_price = self.fib_dict.get(1.000)
                fib_000_price = self.fib_dict.get(0.000)

                if highest_body_price[idx] >= fib_100_price: # no matter what, if the candle pass 1.00, we do not trade
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, idx, "bearish cancels: price > 1.0")

                # Bearish Engulfing candle  
                elif (greens.iloc[idx - 1] is True) and (greens.iloc[idx] is not True) and opens[idx] > closes[idx - 1] and closes[idx] < opens[idx - 1]:  #### run enter market logic
                    
                    self.engulfing_candle_idx = idx
                    self.stop_loss_price, self.take_profit_price,_ = self.calc_fibonacci_levels(self.high_price_idx, 
                    self.low_price_idx, trend="bearish", entry_signal=self.entry_signal_idx, 
                    engulfing_candle=self.engulfing_candle_idx
                    )
                    
                    entry_price = closes[idx]
                    stop_loss_pips = abs(self.stop_loss_price - entry_price) / self.pip_size
                    
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips, self.pip_value_per_lot)
                    
                    # Log for debug
                    self.evt.consoleLog(f"Balance: {self.account_balance}, Risk%: {self.risk_percent}, Stop Loss (pips): {stop_loss_pips}, Pip Value: {self.pip_value_per_lot}, Lot Size: {lot_size}")
                    
                    # place selling order
                    self.place_order(idx, instrument = self.instrument,  trend = 'bearish', order_type = 0, volume = lot_size, 
                    stoploss = self.stop_loss_price, takeprofit = self.take_profit_price, holdtime = None)
                    
                    old_state = self.state
                    self.state = "ENTER_MARKET"
                    self.enter_market_trade_side = "bearish"
                    self.log_state_change(old_state, self.state, idx, "bearish entry: engulfing candle")
                    
                elif lowest_body_price[idx] <= fib_000_price: # If candle breaks 0.00 before there is a Bearish ENGULFING candle
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, idx, "bearish breaks 0.000 before engulfing")
            
            elif self.state == "ENTER_MARKET":
                current_high = self.df['High'].iloc[idx]
                current_low = self.df["Low"].iloc[idx]
                hit = None

                if current_high >= self.stop_loss_price: #### run exit market logic
                    hit = "STOP LOSS"
                elif current_low <= self.take_profit_price:
                    hit = "TAKE PROFIT"

                if hit is not None:
                    self.reset()
                    
            
        # ==============================================================
        # ========================  BULLISH  ============================
        # ==============================================================

        if (current_trend == "bullish" and self.enter_market_trade_side != "bearish") or (current_trend == "bearish" and self.enter_market_trade_side == "bullish"):
            if self.state == "IDLE":    
                if self.check_pullback(greens, opens, closes, idx, "bullish"):
                    self.high_price = highest_body_price[self.trend_changing_idx:idx+1].max()
                    self.high_price_idx = self.trend_changing_idx + np.argmax(highest_body_price[self.trend_changing_idx : idx+1])
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, idx, "bullish pullback detected")

            elif self.state == "PULLBACK_DETECTED":
                if highest_body_price[idx] > self.high_price:
                    self.low_price = lowest_body_price[self.high_price_idx: idx].min()
                    self.low_price_idx = self.high_price_idx + np.argmin(lowest_body_price[self.high_price_idx:idx])
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, idx, "bullish new high after pullback")

   
            elif self.state == "BREAK_OF_STRUCTURE":
                # structure broken downward -> restart pull-back search
                if lowest_body_price[idx] < self.low_price:
                    self.high_price = highest_body_price[self.low_price_idx:idx].max()
                    self.high_price_idx = self.low_price_idx + np.argmax(highest_body_price[self.low_price_idx:idx])
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, idx, "bullish break of structure")

                # new 3 red pull-back -> possible Fib leg
                elif self.check_pullback(greens, opens, closes, idx, "bullish"):
                    
                    if highest_body_price[idx - 2] > self.high_price: # fresh HH
                        self.high_price = highest_body_price[idx-3: idx+1].max()
                        self.high_price_idx = (idx-3) + np.argmax(highest_body_price[idx-3:idx+1])
                        _,_, self.fib_labels = self.calc_fibonacci_levels(self.low_price_idx, self.high_price_idx, trend="bullish")
                        self.fib_dict = dict(self.fib_labels)
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, idx, "bullish waiting entry")

                    # price failed to beat prev high: treat as new pull-back
                    elif lowest_body_price[idx] <= self.high_price:
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, idx, "bullish back to pullback")
                        

            elif self.state == "WAITING_ENTRY":
                fib_618_bullish = self.fib_dict.get(0.618)
                fib_000_bullish = self.fib_dict.get(0.000)

                # price retraced to 0.618 – ready for bullish entry
                if fib_618_bullish and lows[idx] <= fib_618_bullish: # count the tip of the candlestick instead of the body
                    self.entry_signal_idx = idx
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, idx, "bullish price retraced to 0.618")

                # price breaks above 0.000 (HH) before touching 0.618
                elif fib_618_bullish and highest_body_price[idx] > fib_000_bullish:
                    self.low_price = lowest_body_price[self.high_price_idx:idx].min()
                    self.low_price_idx = self.high_price_idx + np.argmin(lowest_body_price[self.high_price_idx:idx])
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, idx, "bullish breaks above 0.000 before entry")
                
            elif self.state == "ENTRY_SIGNAL":
                fib_100_bullish = self.fib_dict.get(1.000)
                fib_000_bullish = self.fib_dict.get(0.000)

                # cancel if price dumps below 1.000 extension
                if lowest_body_price[idx] <= fib_100_bullish:
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, idx, "bullish cancels: price < 1.0")

                # Bullish Engulfing candle
                elif ((greens.iloc[idx-1] is not True) and (greens.iloc[idx] is True) and (opens[idx] <= closes[idx - 1]) and (closes[idx] >= opens[idx - 1])): #### Run Enter Logic
                    self.engulfing_candle_idx = idx
                    
                    self.stop_loss_price, self.take_profit_price,_ = self.calc_fibonacci_levels(self.low_price_idx, 
                    self.high_price_idx,trend="bullish", entry_signal=self.entry_signal_idx,
                    engulfing_candle=self.engulfing_candle_idx)
                    
                    entry_price = closes[idx]
                    stop_loss_pips = abs(self.stop_loss_price - entry_price) / self.pip_size
                    
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips, self.pip_value_per_lot)
                    
                    self.evt.consoleLog(f"Balance: {self.account_balance}, Risk%: {self.risk_percent}, Stop Loss (pips): {stop_loss_pips}, Pip Value: {self.pip_value_per_lot}, Lot Size: {lot_size}")
                    # place buying order
                    self.place_order(idx, instrument = self.instrument,  trend = 'bullish', order_type = 0, volume = lot_size, stoploss = self.stop_loss_price, takeprofit = self.take_profit_price, holdtime = None)
                    
                    old_state = self.state
                    self.state = "ENTER_MARKET"
                    self.enter_market_trade_side = "bullish"
                    self.log_state_change(old_state, self.state, idx, "bullish entry: engulfing candle")
                    
                # invalidated if price breaks above HH first
                elif highest_body_price[idx] >= fib_000_bullish:
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, idx, "bullish breaks above HH first")


            
            elif self.state == "ENTER_MARKET":
                current_high = self.df['High'].iloc[idx]
                current_low = self.df["Low"].iloc[idx]
                hit = None
               
                if current_high >= self.take_profit_price or np.isclose(current_high, self.take_profit_price, atol=1e-5):
                    hit = "TAKE PROFIT"
                elif current_low <= self.stop_loss_price or np.isclose(current_low, self.stop_loss_price, atol=1e-5):
                    hit = "STOP LOSS"

                if hit is not None: #### run exit logic
                    self.reset()
                    
        # ==============================================================
        # ========================  place_order()  ========================
        # ==============================================================     
        
    def place_order(self, idx, instrument=None,  trend = 'bullish', order_type = 0, volume = 0, stoploss = None, takeprofit = None, holdtime = None):
        
        order = AlgoAPIUtil.OrderObject()
        order.instrument = instrument
        order.orderRef = idx
        order.openclose = 'open'
        order.buysell = 1 if trend == "bullish" else -1   #1=buy, -1=sell
        order.ordertype = order_type  #0=market, 1=limit, 2=stop
        order.volume = volume
        order.takeProfitLevel = takeprofit
        order.stopLossLevel = stoploss
        order.holdtime = holdtime
        self.evt.sendOrder(order)
        
        # ==============================================================
        # ========================  pullback()  ========================
        # ==============================================================     
        
    def check_pullback(self,greens, opens, closes, idx, trend):
        if trend == "bearish":
            # Check for 3, 4, 5, or 6 consecutive green candles.
            for window in [3, 4, 5, 6]:
                start_idx = idx - window + 1
                prev_idx = start_idx - 1
                # Boundary check
                if prev_idx < 0:
                    continue
                # Check if all are green
                if all(greens.iloc[idx - i] is True for i in range(window)):
                    # Previous candle must be red
                    if greens.iloc[prev_idx] is False:
                        # Any green candle closes above the open of previous red candle
                        if any(closes[i] >= opens[prev_idx] for i in range(start_idx, idx + 1)):
                            return True
            return False
        
        if trend == "bullish":
            for window in [3,4,5,6]:
                start_idx = idx - window + 1
                prev_idx = start_idx - 1
                if prev_idx < 0:
                    continue
                if all(greens.iloc[idx - i] is not True for i in range(window)): # all previous not green candles 
                    if greens.iloc[prev_idx] is True:
                        if any(closes[i] <= opens[prev_idx] for i in range(start_idx, idx + 1)):
                            return True
            return False

    def reset(self):
        self.state = "IDLE"
        self.low_price  = self.low_price_idx  =  None
        self.high_price = self.high_price_idx =  None
        self.fib_labels = []
        self.fib_dict = {} 
        self.entry_signal_idx = None
        self.engulfing_candle_idx = None
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None




class AlgoEvent:

    def __init__(self):
        self.data = pd.DataFrame(columns=[
            'Datetime', 'Open', 'High', 'Low', 'Close', 'Green', 'Instrument', 'Market', 'EMA_30', 'EMA_120', 'trend'
        ])
        self.fsm = None # Only create fsm after enough bars
        
    def start(self, mEvt):
        self.evt = AlgoAPI_Backtest.AlgoEvtHandler(self, mEvt)
        self.evt.start()  # ← Backtest runs here (blocking call)

        
    def on_marketdatafeed(self, md, ab):

        
        if md.lastPrice > md.openPrice:
            green_value = True
        elif md.lastPrice < md.openPrice:
            green_value = False
        else:
            green_value = pd.NA
        
        bar = {
            'Datetime': pd.to_datetime(md.timestamp),
            'Open': md.openPrice, 
            'High': md.highPrice,
            'Low': md.lowPrice,
            'Close': md.lastPrice,
            'Green': green_value, 
            'Market': md.market
        }
        # Append the new row
        self.data = pd.concat([self.data, pd.DataFrame([bar])], ignore_index=True)
        
        self.evt.consoleLog(self.data['Datetime'].iloc[-1].date())
        
        # Calculate EMAs and trend for the whole DataFrame
        self.data['EMA_30'] = self.data['Close'].ewm(span=30, adjust=False).mean()
        self.data['EMA_120'] = self.data['Close'].ewm(span=120, adjust=False).mean()
        self.data['trend'] = np.where(
            self.data['EMA_30'] > self.data['EMA_120'], 'bullish',
            np.where(self.data['EMA_30'] < self.data['EMA_120'], 'bearish', np.nan)
        )
        
        # Only start after you have enough bars
        MIN_BARS = 10  # or 100, as needed
        if len(self.data) < MIN_BARS:
            return
        
        available_balance = ab['availableBalance']
        self.evt.consoleLog(available_balance)
        
        if self.fsm is None:
            self.fsm = BearishFSM(self.data, instrument=md.instrument, evt=self.evt, account_balance=available_balance, 
            pip_size= 0.0001, pip_value_per_lot = 10) # change for JPY
        else:
            self.fsm.df = self.data
            self.fsm.account_balance = available_balance
            self.fsm.update(len(self.data) - 1)
            
    def on_newsdatafeed(self, nd):
        pass

    def on_weatherdatafeed(self, wd):
        pass
    
    def on_econsdatafeed(self, ed):
        pass
        
    def on_corpAnnouncement(self, ca):
        pass

    def on_orderfeed(self, of):
        pass

    def on_dailyPLfeed(self, pl):
        pass

    def on_openPositionfeed(self, op, oo, uo):
        pass






#### Version 2 Improvements: 
1. Since Algogene provides 5 mins data, but we want 15-mins data, so we aggregate three 5-minute bars into a 15-minute .

2. Define a separate function to calculate the atr ( average true return)

3. Trends are now represented by True (bullish) and False (bearish) instead of strings, making the code more memory efficient

4. Enhanced Risk Management: A simple but effective rule (min(10, lot_size)) was added to cap the maximum trade volume, preventing oversized positions.

In [ ]:
from AlgoAPI import AlgoAPIUtil, AlgoAPI_Backtest
import pandas as pd
import numpy as np
import mplfinance as mpf
from collections import deque
from datetime import datetime, timedelta

    
class BearishFSM: # Finite State Machine

        # ==============================================================
        # ========================  __init__ ===========================
        # ==============================================================
    def __init__(self, df, instrument, evt, account_balance=0, pip_size = 0.0001, pip_value_per_lot = 10):
        
        self.df = df
        self.instrument = instrument
        self.evt = evt
        self.account_balance = account_balance
        
        self.state = "IDLE"
        self.low_price = None
        
        self.low_price_idx = None
        self.high_price = None
        self.high_price_idx = None

        self.fib_labels = []
        self.fib_dict = {} # not a must to initialize, just for good habit
        self.entry_signal_idx = None
        self.engulfing_candle_idx = None
        self.trend_changing_idx = 3 # do not reset this, always keep record of 
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None

        # do not reset the folloiwng parameters, test for optimal values
        self.risk_reward_ratio = 2.5
        self.atr_period = 14
        self.atr_multiplier = 10.0 # pip_size = atr_multiplier * atr_value, atr_multiplier = 2.0 means stop loss is 2 ATRs away
        self.risk_percent = 1
        
        # this is unique in every func call
        self.instrument = instrument
        self.pip_size = pip_size
        self.pip_value_per_lot = pip_value_per_lot
        
        # ==============================================================
        # ========================  calculate_log_state_change =========
        # ==============================================================
        
    def log_state_change(self, old_state, new_state, idx, label=None):
        date_str = (
            str(self.df['Datetime'].iloc[idx])
            if 'Datetime' in self.df.columns and idx < len(self.df)
            else str(idx)
        )
        msg = f"[{date_str}] State: {old_state} → {new_state}"
        if label:
            msg += f" ({label})"
        self.evt.consoleLog(msg)
        
        # ==============================================================
        # ========================  calculate_lot_size  ================
        # ==============================================================
        
    def calculate_lot_size(self, risk_percent, stop_loss_pips, pip_value_per_lot=10):
        risk_amount = self.account_balance * (risk_percent / 100)
        lot_size = risk_amount / (stop_loss_pips * pip_value_per_lot)
        return round(lot_size, 4)
        
        # ==============================================================
        # ===== Wilder’s EMA: recursive smoothing with alpha = 1/n  ====
        # ==============================================================
        
    def wwma(self, values: pd.Series, n: int) -> pd.Series:
        # Wilder’s EMA: recursive smoothing with alpha = 1/n
        return values.ewm(alpha=1/n, adjust=False).mean()
        
        # ==============================================================
        # ========================  atr  ================
        # ==============================================================
        
    def atr(self, df: pd.DataFrame, n: int = 14) -> pd.Series:
        high = df["High"]
        low = df["Low"]
        close = df["Close"]
    
        tr0 = (high - low).abs()
        tr1 = (high - close.shift()).abs()
        tr2 = (low  - close.shift()).abs()
    
        tr = pd.concat([tr0, tr1, tr2], axis=1).max(axis=1)
    
        return self.wwma(tr, n)
        
        # ==============================================================
        # ========================  calc_fibonacci_levels  =============
        # ==============================================================
        
    def calc_fibonacci_levels(self, start_idx, end_idx, levels=None, trend=False,
                         entry_signal=None, engulfing_candle=None,
                         R=None, atr_period=None, atr_multiplier=None):
                             
        if levels is None:
            levels = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]
            
        if atr_period is None:
            atr_period = self.atr_period
            
        if atr_multiplier is None:
            atr_multiplier = self.atr_multiplier
        
        if R is None:
            R = self.risk_reward_ratio
        
        
        df = self.df
        df_slice = df.iloc[start_idx:end_idx + 2 + 1].copy() # slice
        min_price = df_slice['Low'].min()
        max_price = df_slice['High'].max()
    
        fib_labels = []
        for level in levels:
            if trend == False:
                y = min_price + (max_price - min_price) * level
            else:
                y = max_price - (max_price - min_price) * level
            fib_labels.append((level, y))
        
        if engulfing_candle is None:
            return fib_labels
        
        
        # Normal ATR
        df['H-L'] = df['High'] - df['Low']
        df['H-PC'] = abs(df['High'] - df['Close'].shift(1))
        df['L-PC'] = abs(df['Low'] - df['Close'].shift(1))
        df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
        
        # ATR: rolling mean of TR
        df['ATR'] = df['TR'].rolling(window=atr_period).mean()
        
        """
        
        # Wilder’s EMA
        self.df['ATR'] = self.atr(self.df, n=atr_period)
        atr_value = self.df['ATR'].iloc[engulfing_candle]
         """
         
        # Retrieve ATR value at the engulfing candle index
        atr_value = df['ATR'].iloc[engulfing_candle]
        pip_size = atr_multiplier * atr_value
        
        # Retrieve the close price of the engulfing candle
        candle = self.df.iloc[engulfing_candle]
        close_price = candle['Close']
        
        # Initialize stop loss and take profit prices
        stop_loss_price = None
        take_profit_price = None
    
    
        if trend == False:
            stop_loss_price = close_price + pip_size
            take_profit_price = close_price - R * pip_size
        else:  # bullish
            stop_loss_price = close_price - pip_size
            take_profit_price = close_price + R * pip_size
    
        return stop_loss_price, take_profit_price
        
    # ==============================================================
    # ========================  update()  =============
    # ==============================================================
    
    def update(self, idx):

        if idx < 3:
            return
        
        trend = self.df['trend'].values
        greens = self.df["Green"]
        opens = self.df["Open"].values
        closes = self.df["Close"].values
        highs = self.df["High"].values
        lows = self.df["Low"].values
        lowest_body_price = np.select([greens == True, greens == False], [opens, closes], default= np.inf) # array-based logic, avoid direct truthiness check
        highest_body_price = np.select([greens == True, greens == False], [closes, opens], default= -np.inf) # default=np.nan ensures that rows where "Green" is missing won't cause errors
        
        current_trend = trend[idx]
        prev_trend = trend[idx - 1]

        if current_trend != prev_trend :
            self.trend_changing_idx = idx
            if self.state != "ENTER_MARKET":
                self.reset()

        if (current_trend == False and self.enter_market_trade_side is not False) or (current_trend == True and self.enter_market_trade_side is False):
            if self.state == "IDLE":
                if self.check_pullback(greens, opens, closes, idx, trend=False): # pullback should eat and cover the previous red candle
                    self.low_price = lowest_body_price[self.trend_changing_idx:idx+1].min()
                    self.low_price_idx = (self.trend_changing_idx) + np.argmin(
                                            lowest_body_price[self.trend_changing_idx:idx+1])
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, idx, "bearish pullback detected")
                    

            elif self.state == "PULLBACK_DETECTED":
                if lowest_body_price[idx] < self.low_price:

                    self.high_price = highest_body_price[self.low_price_idx:idx].max()
                    self.high_price_idx = self.low_price_idx + np.argmax(highest_body_price[self.low_price_idx:idx])
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, idx, "bearish new low after pullback")

                    
            elif self.state == "BREAK_OF_STRUCTURE":
                # If price breaks the pev high (LH)
                if highest_body_price[idx] > self.high_price:

                    self.low_price = lowest_body_price[self.high_price_idx:idx].min()
                    self.low_price_idx = self.high_price_idx + np.argmin(
                        lowest_body_price[self.high_price_idx:idx])
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, idx, "bearish break of structure")

                elif self.check_pullback(greens, opens, closes, idx, trend=False):
                    
                    if lowest_body_price[idx -2] < self.low_price: #### Change from idx to idx - 2

                        # Low price for confirming the next Lower high
                        self.low_price = lowest_body_price[idx-3:idx+1].min()
                        self.low_price_idx = (idx-3) + np.argmin(
                            lowest_body_price[idx-3:idx+1])

                        self.fib_labels = self.calc_fibonacci_levels( self.high_price_idx, self.low_price_idx, trend=False)
                        self.fib_dict = dict(self.fib_labels)
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, idx, "bearish waiting entry")

                    elif highest_body_price[idx] >= self.low_price:

                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, idx, "bearish back to pullback")

            elif self.state == "WAITING_ENTRY":
                fib_618_price = self.fib_dict.get(0.618)
                fib_000_price = self.fib_dict.get(0.000)
                if fib_618_price and highs[idx] >= fib_618_price: # price rebound back to 0.618
                
                    self.entry_signal_idx = idx
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, idx, "bearish price rebound to 0.618")

                elif fib_618_price and lowest_body_price[idx] <= fib_000_price: # price lower than 0.000
                    # # break of structure, set new high point
                    self.high_price = highest_body_price[self.low_price_idx:idx].max()
                    self.high_price_idx = self.low_price_idx + np.argmax(
                        highest_body_price[self.low_price_idx:idx])
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, idx, "bearish breaks 0.000 before entry")
            
            elif self.state == "ENTRY_SIGNAL": 
                fib_100_price = self.fib_dict.get(1.000)
                fib_000_price = self.fib_dict.get(0.000)

                if highest_body_price[idx] >= fib_100_price: # no matter what, if the candle pass 1.00, we do not trade
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, idx, "bearish cancels: price > 1.0")

                # Bearish Engulfing candle  
                elif (greens.iloc[idx - 1] is True) and (greens.iloc[idx] is not True) and opens[idx] > closes[idx - 1] and closes[idx] < opens[idx - 1]:  #### run enter market logic
                    
                    self.engulfing_candle_idx = idx
                    self.stop_loss_price, self.take_profit_price = self.calc_fibonacci_levels(self.high_price_idx, 
                    self.low_price_idx, trend=False, entry_signal=self.entry_signal_idx, 
                    engulfing_candle=self.engulfing_candle_idx
                    )
                    
                    entry_price = closes[idx]
                    stop_loss_pips = abs(self.stop_loss_price - entry_price) / self.pip_size
                    
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips, self.pip_value_per_lot)
                    
                    # Log for debug
                    self.evt.consoleLog(f"Balance: {self.account_balance}, Risk%: {self.risk_percent}, Stop Loss (pips): {stop_loss_pips}, Pip Value: {self.pip_value_per_lot}, Lot Size: {lot_size}")
                    
                    # place selling order
                    self.place_order(idx, instrument = self.instrument,  trend = False, order_type = 0, volume = min(10,lot_size), 
                    stoploss = self.stop_loss_price, takeprofit = self.take_profit_price, holdtime = None)
                    
                    old_state = self.state
                    self.state = "ENTER_MARKET"
                    self.enter_market_trade_side = False
                    self.log_state_change(old_state, self.state, idx, "bearish entry: engulfing candle")
                    
                elif lowest_body_price[idx] <= fib_000_price: # If candle breaks 0.00 before there is a Bearish ENGULFING candle
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, idx, "bearish breaks 0.000 before engulfing")
            
            elif self.state == "ENTER_MARKET":
                current_high = self.df['High'].iloc[idx]
                current_low = self.df["Low"].iloc[idx]
                hit = None

                if current_high >= self.stop_loss_price: #### run exit market logic
                    hit = "STOP LOSS"
                elif current_low <= self.take_profit_price:
                    hit = "TAKE PROFIT"

                if hit is not None:
                    self.reset()
                    
            
        # ==============================================================
        # ========================  BULLISH  ============================
        # ==============================================================

        if (current_trend == True and self.enter_market_trade_side != False) or (current_trend == False and self.enter_market_trade_side == True):
            if self.state == "IDLE":    
                if self.check_pullback(greens, opens, closes, idx, trend=True):
                    self.high_price = highest_body_price[self.trend_changing_idx:idx+1].max()
                    self.high_price_idx = self.trend_changing_idx + np.argmax(highest_body_price[self.trend_changing_idx : idx+1])
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, idx, "bullish pullback detected")

            elif self.state == "PULLBACK_DETECTED":
                if highest_body_price[idx] > self.high_price:
                    self.low_price = lowest_body_price[self.high_price_idx: idx].min()
                    self.low_price_idx = self.high_price_idx + np.argmin(lowest_body_price[self.high_price_idx:idx])
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, idx, "bullish new high after pullback")

   
            elif self.state == "BREAK_OF_STRUCTURE":
                # structure broken downward -> restart pull-back search
                if lowest_body_price[idx] < self.low_price:
                    self.high_price = highest_body_price[self.low_price_idx:idx].max()
                    self.high_price_idx = self.low_price_idx + np.argmax(highest_body_price[self.low_price_idx:idx])
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, idx, "bullish break of structure")

                # new 3 red pull-back -> possible Fib leg
                elif self.check_pullback(greens, opens, closes, idx, trend=True):
                    
                    if highest_body_price[idx - 2] > self.high_price: # fresh HH
                        self.high_price = highest_body_price[idx-3: idx+1].max()
                        self.high_price_idx = (idx-3) + np.argmax(highest_body_price[idx-3:idx+1])
                        self.fib_labels = self.calc_fibonacci_levels(self.low_price_idx, self.high_price_idx, trend=True)
                        self.fib_dict = dict(self.fib_labels)
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, idx, "bullish waiting entry")

                    # price failed to beat prev high: treat as new pull-back
                    elif lowest_body_price[idx] <= self.high_price:
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, idx, "bullish back to pullback")
                        

            elif self.state == "WAITING_ENTRY":
                fib_618_bullish = self.fib_dict.get(0.618)
                fib_000_bullish = self.fib_dict.get(0.000)

                # price retraced to 0.618 – ready for bullish entry
                if fib_618_bullish and lows[idx] <= fib_618_bullish: # count the tip of the candlestick instead of the body
                    self.entry_signal_idx = idx
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, idx, "bullish price retraced to 0.618")

                # price breaks above 0.000 (HH) before touching 0.618
                elif fib_618_bullish and highest_body_price[idx] > fib_000_bullish:
                    self.low_price = lowest_body_price[self.high_price_idx:idx].min()
                    self.low_price_idx = self.high_price_idx + np.argmin(lowest_body_price[self.high_price_idx:idx])
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, idx, "bullish breaks above 0.000 before entry")
                
            elif self.state == "ENTRY_SIGNAL":
                fib_100_bullish = self.fib_dict.get(1.000)
                fib_000_bullish = self.fib_dict.get(0.000)

                # cancel if price dumps below 1.000 extension
                if lowest_body_price[idx] <= fib_100_bullish:
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, idx, "bullish cancels: price < 1.0")

                # Bullish Engulfing candle
                elif ((greens.iloc[idx-1] is not True) and (greens.iloc[idx] is True) and (opens[idx] <= closes[idx - 1]) and (closes[idx] >= opens[idx - 1])): #### Run Enter Logic
                    self.engulfing_candle_idx = idx
                    
                    self.stop_loss_price, self.take_profit_price = self.calc_fibonacci_levels(self.low_price_idx, 
                    self.high_price_idx,trend=True, entry_signal=self.entry_signal_idx,
                    engulfing_candle=self.engulfing_candle_idx)
                    
                    entry_price = closes[idx]
                    stop_loss_pips = abs(self.stop_loss_price - entry_price) / self.pip_size
                    
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips, self.pip_value_per_lot)
                    
                    self.evt.consoleLog(f"Balance: {self.account_balance}, Risk%: {self.risk_percent}, Stop Loss (pips): {stop_loss_pips}, Pip Value: {self.pip_value_per_lot}, Lot Size: {lot_size}")
                    # place buying order
                    self.place_order(idx, instrument = self.instrument,  trend = True, order_type = 0, volume = min(10,lot_size), stoploss = self.stop_loss_price, takeprofit = self.take_profit_price, holdtime = None)
                    
                    old_state = self.state
                    self.state = "ENTER_MARKET"
                    self.enter_market_trade_side = True
                    self.log_state_change(old_state, self.state, idx, "bullish entry: engulfing candle")
                    
                # invalidated if price breaks above HH first
                elif highest_body_price[idx] >= fib_000_bullish:
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, idx, "bullish breaks above HH first")


            
            elif self.state == "ENTER_MARKET":
                current_high = self.df['High'].iloc[idx]
                current_low = self.df["Low"].iloc[idx]
                hit = None
               
                if current_high >= self.take_profit_price or np.isclose(current_high, self.take_profit_price, atol=1e-5):
                    hit = "TAKE PROFIT"
                elif current_low <= self.stop_loss_price or np.isclose(current_low, self.stop_loss_price, atol=1e-5):
                    hit = "STOP LOSS"

                if hit is not None: #### run exit logic
                    self.reset()
                    
        # ==============================================================
        # ========================  place_order()  ========================
        # ==============================================================     
        
    def place_order(self, idx, instrument=None,  trend = True, order_type = 0, volume = 0, stoploss = None, takeprofit = None, holdtime = None):
        
        order = AlgoAPIUtil.OrderObject()
        order.instrument = instrument
        order.orderRef = idx
        order.openclose = 'open'
        order.buysell = 1 if trend == True else -1   #1=buy, -1=sell
        order.ordertype = order_type  #0=market, 1=limit, 2=stop
        order.volume =  volume
        order.takeProfitLevel = takeprofit
        order.stopLossLevel = stoploss
        order.holdtime = holdtime
        self.evt.sendOrder(order)
        
        # ==============================================================
        # ========================  pullback()  ========================
        # ==============================================================     
        
    def check_pullback(self,greens, opens, closes, idx, trend):
        if trend == False:
            # Check for 3, 4, 5, or 6 consecutive green candles.
            for window in [3, 4, 5, 6]:
                start_idx = idx - window + 1
                prev_idx = start_idx - 1
                # Boundary check
                if prev_idx < 0:
                    continue
                # Check if all are green
                if all(greens.iloc[idx - i] is True for i in range(window)):
                    # Previous candle must be red
                    if greens.iloc[prev_idx] is False:
                        # Any green candle closes above the open of previous red candle
                        if any(closes[i] >= opens[prev_idx] for i in range(start_idx, idx + 1)):
                            return True
            return False
        
        if trend == True:
            for window in [3,4,5,6]:
                start_idx = idx - window + 1
                prev_idx = start_idx - 1
                if prev_idx < 0:
                    continue
                if all(greens.iloc[idx - i] is not True for i in range(window)): # all previous not green candles 
                    if greens.iloc[prev_idx] is True:
                        if any(closes[i] <= opens[prev_idx] for i in range(start_idx, idx + 1)):
                            return True
            return False

    def reset(self):
        self.state = "IDLE"
        self.low_price  = self.low_price_idx  =  None
        self.high_price = self.high_price_idx =  None
        self.fib_labels = []
        self.fib_dict = {} 
        self.entry_signal_idx = None
        self.engulfing_candle_idx = None
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None




class AlgoEvent:
    
    

    def __init__(self):
        
        self.last_time = datetime(2010,1,1)
        self.fsm = None
        # Initialize DataFrame with columns
        self.raw_5min_data = pd.DataFrame(columns=[
            'Datetime', 'Open', 'High', 'Low', 'Close', 'Green', 'Instrument', 'Market'
        ])
        self.raw_5min_data = self.raw_5min_data.astype({
            'Datetime': 'datetime64[ns]',
            'Open': 'float32',
            'High': 'float32',
            'Low': 'float32',
            'Close': 'float32',
            'Green': 'bool',
            'Instrument': 'object',
            'Market': 'object'
        })
        self.data = pd.DataFrame(columns=[
        'Datetime', 'Open', 'High', 'Low', 'Close', 'Green', 'Instrument', 'Market', 'EMA_30', 'EMA_120', 'trend'
        ])
        # Set data types for each column
        self.data = self.data.astype({
            'Datetime': 'datetime64[ns]',
            'Open': 'float32',
            'High': 'float32',
            'Low': 'float32',
            'Close': 'float32',
            'Green': 'bool',
            'Instrument': 'object',
            'Market': 'object',
            'EMA_30': 'float32',
            'EMA_120': 'float32',
            'trend': 'bool' 
        })

    def start(self, mEvt):
        self.evt = AlgoAPI_Backtest.AlgoEvtHandler(self, mEvt)
        self.evt.start()

    def on_marketdatafeed(self, md, ab):

        # Build the bar
        green = True if md.lastPrice > md.openPrice else False if md.lastPrice < md.openPrice else pd.NA
        five_min_bar = {
            'Datetime': pd.to_datetime(md.timestamp),
            'Open': md.openPrice,
            'High': md.highPrice,
            'Low': md.lowPrice,
            'Close': md.lastPrice,
            'Green': green,
            'Instrument': md.instrument,
            'Market': md.market
        }
        self.raw_5min_data = pd.concat([self.raw_5min_data, pd.DataFrame([five_min_bar])], ignore_index=True)
        
        # Only aggregate when 15 min has passed
        if md.timestamp >= self.last_time + timedelta(minutes=15):
            self.last_time = md.timestamp
    
            # Get the last 3 rows (the last 15 minutes)
            last3 = self.raw_5min_data.tail(3)
            if len(last3) < 3:
                return  # not enough data yet
    
            # Aggregate into a real 15-min bar
            bar = {
                'Datetime': last3['Datetime'].iloc[-1],  # end time of the 15-min bar
                'Open': last3['Open'].iloc[0],
                'High': last3['High'].max(),
                'Low': last3['Low'].min(),
                'Close': last3['Close'].iloc[-1],
                'Green': True if last3['Close'].iloc[-1] > last3['Open'].iloc[0] else False if last3['Close'].iloc[-1] < last3['Open'].iloc[0] else pd.NA,
                'Instrument': last3['Instrument'].iloc[-1],
                'Market': last3['Market'].iloc[-1]
            }
    
            # Append the aggregated bar to your main 15-min data
            self.data = pd.concat([self.data, pd.DataFrame([bar])], ignore_index=True)
            
            # clear up memory space
            self.raw_5min_data = self.raw_5min_data.iloc[0:0].copy()
            
            # For EMA/trend as before
            MIN_BARS = 10
            if len(self.data) < MIN_BARS:
                return
    
            self.data['EMA_30'] = self.data['Close'].ewm(span=30, adjust=False).mean()
            self.data['EMA_120'] = self.data['Close'].ewm(span=120, adjust=False).mean()
            self.data['trend'] = np.where(
                self.data['EMA_30'] > self.data['EMA_120'], True,
                np.where(self.data['EMA_30'] < self.data['EMA_120'], False, np.nan)
            )
    
            available_balance = ab['availableBalance']
            self.evt.consoleLog(available_balance)
    
            # FSM logic as before, updating with new self.data
            if self.fsm is None:
                self.fsm = BearishFSM(
                    self.data,
                    instrument=md.instrument,
                    evt=self.evt,
                    account_balance=np.float32(available_balance),
                    pip_size=np.float32(0.0001),
                    pip_value_per_lot=int(10)
                )
            else:
                self.fsm.df = self.data
                self.fsm.account_balance = available_balance
                idx = len(self.data) - 1
                self.fsm.update(idx)
            
            
        """
        # Append the new row
        self.data = pd.concat([self.data, pd.DataFrame([bar])], ignore_index=True)
        
        MIN_BARS = 10  # adjust as needed
        if len(self.data) < MIN_BARS:
            return
        
        # Calculate EMAs and trend for the whole DataFrame
        self.data['EMA_30'] = self.data['Close'].ewm(span=30, adjust=False).mean()
        self.data['EMA_120'] = self.data['Close'].ewm(span=120, adjust=False).mean()
        self.data['trend'] = np.where(
            self.data['EMA_30'] > self.data['EMA_120'], True,
            np.where(self.data['EMA_30'] < self.data['EMA_120'], False, np.nan) # True = "bullish", False = "bearish"
        )
        # self.evt.consoleLog("trend dtype:", df_recent['trend'].dtype)
        
        available_balance = ab['availableBalance']
        self.evt.consoleLog(available_balance)
        
        
        # Initialize or update FSM
        if self.fsm is None:
            self.fsm = BearishFSM(
                self.data,
                instrument=md.instrument,
                evt=self.evt,
                account_balance=np.float32(available_balance),
                pip_size=np.float32(0.0001),
                pip_value_per_lot=int(10)
            )
        else:
            self.fsm.df = self.data
            self.fsm.account_balance = available_balance

            # Update FSM at the latest index
            idx = len(self.data) - 1
            self.fsm.update(idx)
            
        """
                
    def on_newsdatafeed(self, nd):
        pass

    def on_weatherdatafeed(self, wd):
        pass
    
    def on_econsdatafeed(self, ed):
        pass
        
    def on_corpAnnouncement(self, ca):
        pass

    def on_orderfeed(self, of):
        pass

    def on_dailyPLfeed(self, pl):
        pass

    def on_openPositionfeed(self, op, oo, uo):
        pass









#### Version 3 Improvements:

1. Using the pandas nullable 'boolean' dtype for the trend column, to hold True, False, and pd.NA values, which is a more accurate representation of the data, especially at the start of the series.

2. updating EMAs incrementally -- O(1), rather than recalculating them entirely for each new bar -- O(n)

In [ ]:
from AlgoAPI import AlgoAPIUtil, AlgoAPI_Backtest
import pandas as pd
import numpy as np
import mplfinance as mpf
from collections import deque
from datetime import datetime, timedelta
    
class BearishFSM: # Finite State Machine

        # ==============================================================
        # ========================  __init__ ===========================
        # ==============================================================
    def __init__(self, df, instrument, evt, account_balance=0, pip_size = 0.0001, pip_value_per_lot = 10):
        
        self.df = df
        self.instrument = instrument
        self.evt = evt
        self.account_balance = account_balance
        
        self.state = "IDLE"
        self.low_price = None
        
        self.low_price_idx = None
        self.high_price = None
        self.high_price_idx = None

        self.fib_labels = []
        self.fib_dict = {} # not a must to initialize, just for good habit
        self.entry_signal_idx = None
        self.engulfing_candle_idx = None
        self.trend_changing_idx = 3 # do not reset this, always keep record of 
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None

        # do not reset the folloiwng parameters, test for optimal values
        self.risk_reward_ratio = 2.5
        self.atr_period = 14
        self.atr_multiplier = 10.0 # pip_size = atr_multiplier * atr_value, atr_multiplier = 2.0 means stop loss is 2 ATRs away
        self.risk_percent = 1
        
        # this is unique in every func call
        self.instrument = instrument
        self.pip_size = pip_size
        self.pip_value_per_lot = pip_value_per_lot
        
        # ==============================================================
        # ========================  calculate_log_state_change =========
        # ==============================================================
        
    def log_state_change(self, old_state, new_state, idx, label=None):
        date_str = (
            str(self.df['Datetime'].iloc[idx])
            if 'Datetime' in self.df.columns and idx < len(self.df)
            else str(idx)
        )
        msg = f"[{date_str}] State: {old_state} → {new_state}"
        if label:
            msg += f" ({label})"
        self.evt.consoleLog(msg)
        
        # ==============================================================
        # ========================  calculate_lot_size  ================
        # ==============================================================
        
    def calculate_lot_size(self, risk_percent, stop_loss_pips, pip_value_per_lot=10):
        risk_amount = self.account_balance * (risk_percent / 100)
        lot_size = risk_amount / (stop_loss_pips * pip_value_per_lot)
        return round(lot_size, 4)
        
        # ==============================================================
        # ===== Wilder’s EMA: recursive smoothing with alpha = 1/n  ====
        # ==============================================================
        
    def wwma(self, values: pd.Series, n: int) -> pd.Series:
        # Wilder’s EMA: recursive smoothing with alpha = 1/n
        return values.ewm(alpha=1/n, adjust=False).mean()
        
        # ==============================================================
        # ========================  atr  ================
        # ==============================================================
        
    def atr(self, df: pd.DataFrame, n: int = 14) -> pd.Series:
        high = df["High"]
        low = df["Low"]
        close = df["Close"]
    
        tr0 = (high - low).abs()
        tr1 = (high - close.shift()).abs()
        tr2 = (low  - close.shift()).abs()
    
        tr = pd.concat([tr0, tr1, tr2], axis=1).max(axis=1)
    
        return self.wwma(tr, n)
        
        # ==============================================================
        # ========================  calc_fibonacci_levels  =============
        # ==============================================================
        
    def calc_fibonacci_levels(self, start_idx, end_idx, levels=None, trend=False,
                         entry_signal=None, engulfing_candle=None,
                         R=None, atr_period=None, atr_multiplier=None):
                             
        if levels is None:
            levels = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]
            
        if atr_period is None:
            atr_period = self.atr_period
            
        if atr_multiplier is None:
            atr_multiplier = self.atr_multiplier
        
        if R is None:
            R = self.risk_reward_ratio
        
        
        df = self.df
        df_slice = df.iloc[start_idx:end_idx + 2 + 1].copy() # slice
        min_price = df_slice['Low'].min()
        max_price = df_slice['High'].max()
    
        fib_labels = []
        for level in levels:
            if trend == False:
                y = min_price + (max_price - min_price) * level
            else:
                y = max_price - (max_price - min_price) * level
            fib_labels.append((level, y))
        
        if engulfing_candle is None:
            return fib_labels
        
        
        # Normal ATR
        df['H-L'] = df['High'] - df['Low']
        df['H-PC'] = abs(df['High'] - df['Close'].shift(1))
        df['L-PC'] = abs(df['Low'] - df['Close'].shift(1))
        df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
        
        # ATR: rolling mean of TR
        df['ATR'] = df['TR'].rolling(window=atr_period).mean()
        
        """
        
        # Wilder’s EMA
        self.df['ATR'] = self.atr(self.df, n=atr_period)
        atr_value = self.df['ATR'].iloc[engulfing_candle]
         """
         
        # Retrieve ATR value at the engulfing candle index
        atr_value = df['ATR'].iloc[engulfing_candle]
        pip_size = atr_multiplier * atr_value
        
        # Retrieve the close price of the engulfing candle
        candle = self.df.iloc[engulfing_candle]
        close_price = candle['Close']
        
        # Initialize stop loss and take profit prices
        stop_loss_price = None
        take_profit_price = None
    
    
        if trend == False:
            stop_loss_price = close_price + pip_size
            take_profit_price = close_price - R * pip_size
        else:  # bullish
            stop_loss_price = close_price - pip_size
            take_profit_price = close_price + R * pip_size
    
        return stop_loss_price, take_profit_price
        
    # ==============================================================
    # ========================  update()  =============
    # ==============================================================
    
    def update(self, idx):

        if idx < 3:
            return
        
        trend = self.df['trend'].values
        greens = self.df["Green"]
        opens = self.df["Open"].values
        closes = self.df["Close"].values
        highs = self.df["High"].values
        lows = self.df["Low"].values
        lowest_body_price = np.select([greens == True, greens == False], [opens, closes], default= np.inf) # array-based logic, avoid direct truthiness check
        highest_body_price = np.select([greens == True, greens == False], [closes, opens], default= -np.inf) # default=np.nan ensures that rows where "Green" is missing won't cause errors
        
        current_trend = trend[idx]
        prev_trend = trend[idx - 1]

        if current_trend != prev_trend :
            self.trend_changing_idx = idx
            if self.state != "ENTER_MARKET":
                self.reset()

        if (current_trend == False and self.enter_market_trade_side is not False) or (current_trend == True and self.enter_market_trade_side is False):
            if self.state == "IDLE":
                if self.check_pullback(greens, opens, closes, idx, trend=False): # pullback should eat and cover the previous red candle
                    self.low_price = lowest_body_price[self.trend_changing_idx:idx+1].min()
                    self.low_price_idx = (self.trend_changing_idx) + np.argmin(
                                            lowest_body_price[self.trend_changing_idx:idx+1])
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, idx, "bearish pullback detected")
                    

            elif self.state == "PULLBACK_DETECTED":
                if lowest_body_price[idx] < self.low_price:

                    self.high_price = highest_body_price[self.low_price_idx:idx].max()
                    self.high_price_idx = self.low_price_idx + np.argmax(highest_body_price[self.low_price_idx:idx])
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, idx, "bearish new low after pullback")

                    
            elif self.state == "BREAK_OF_STRUCTURE":
                # If price breaks the pev high (LH)
                if highest_body_price[idx] > self.high_price:

                    self.low_price = lowest_body_price[self.high_price_idx:idx].min()
                    self.low_price_idx = self.high_price_idx + np.argmin(
                        lowest_body_price[self.high_price_idx:idx])
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, idx, "bearish break of structure")

                elif self.check_pullback(greens, opens, closes, idx, trend=False):
                    
                    if lowest_body_price[idx -2] < self.low_price: #### Change from idx to idx - 2

                        # Low price for confirming the next Lower high
                        self.low_price = lowest_body_price[idx-3:idx+1].min()
                        self.low_price_idx = (idx-3) + np.argmin(
                            lowest_body_price[idx-3:idx+1])

                        self.fib_labels = self.calc_fibonacci_levels( self.high_price_idx, self.low_price_idx, trend=False)
                        self.fib_dict = dict(self.fib_labels)
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, idx, "bearish waiting entry")

                    elif highest_body_price[idx] >= self.low_price:

                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, idx, "bearish back to pullback")

            elif self.state == "WAITING_ENTRY":
                fib_618_price = self.fib_dict.get(0.618)
                fib_000_price = self.fib_dict.get(0.000)
                if fib_618_price and highs[idx] >= fib_618_price: # price rebound back to 0.618
                
                    self.entry_signal_idx = idx
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, idx, "bearish price rebound to 0.618")

                elif fib_618_price and lowest_body_price[idx] <= fib_000_price: # price lower than 0.000
                    # # break of structure, set new high point
                    self.high_price = highest_body_price[self.low_price_idx:idx].max()
                    self.high_price_idx = self.low_price_idx + np.argmax(
                        highest_body_price[self.low_price_idx:idx])
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, idx, "bearish breaks 0.000 before entry")
            
            elif self.state == "ENTRY_SIGNAL": 
                fib_100_price = self.fib_dict.get(1.000)
                fib_000_price = self.fib_dict.get(0.000)

                if highest_body_price[idx] >= fib_100_price: # no matter what, if the candle pass 1.00, we do not trade
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, idx, "bearish cancels: price > 1.0")

                # Bearish Engulfing candle  
                elif (greens.iloc[idx - 1] is True) and (greens.iloc[idx] is not True) and opens[idx] > closes[idx - 1] and closes[idx] < opens[idx - 1]:  #### run enter market logic
                    
                    self.engulfing_candle_idx = idx
                    self.stop_loss_price, self.take_profit_price = self.calc_fibonacci_levels(self.high_price_idx, 
                    self.low_price_idx, trend=False, entry_signal=self.entry_signal_idx, 
                    engulfing_candle=self.engulfing_candle_idx
                    )
                    
                    entry_price = closes[idx]
                    stop_loss_pips = abs(self.stop_loss_price - entry_price) / self.pip_size
                    
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips, self.pip_value_per_lot)
                    
                    # Log for debug
                    self.evt.consoleLog(f"Balance: {self.account_balance}, Risk%: {self.risk_percent}, Stop Loss (pips): {stop_loss_pips}, Pip Value: {self.pip_value_per_lot}, Lot Size: {lot_size}")
                    
                    # place selling order
                    self.place_order(idx, instrument = self.instrument,  trend = False, order_type = 0, volume = min(10,lot_size), 
                    stoploss = self.stop_loss_price, takeprofit = self.take_profit_price, holdtime = None)
                    
                    old_state = self.state
                    self.state = "ENTER_MARKET"
                    self.enter_market_trade_side = False
                    self.log_state_change(old_state, self.state, idx, "bearish entry: engulfing candle")
                    
                elif lowest_body_price[idx] <= fib_000_price: # If candle breaks 0.00 before there is a Bearish ENGULFING candle
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, idx, "bearish breaks 0.000 before engulfing")
            
            elif self.state == "ENTER_MARKET":
                current_high = self.df['High'].iloc[idx]
                current_low = self.df["Low"].iloc[idx]
                hit = None

                if current_high >= self.stop_loss_price: #### run exit market logic
                    hit = "STOP LOSS"
                elif current_low <= self.take_profit_price:
                    hit = "TAKE PROFIT"

                if hit is not None:
                    self.reset()
                    
            
        # ==============================================================
        # ========================  BULLISH  ============================
        # ==============================================================

        if (current_trend == True and self.enter_market_trade_side != False) or (current_trend == False and self.enter_market_trade_side == True):
            if self.state == "IDLE":    
                if self.check_pullback(greens, opens, closes, idx, trend=True):
                    self.high_price = highest_body_price[self.trend_changing_idx:idx+1].max()
                    self.high_price_idx = self.trend_changing_idx + np.argmax(highest_body_price[self.trend_changing_idx : idx+1])
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, idx, "bullish pullback detected")

            elif self.state == "PULLBACK_DETECTED":
                if highest_body_price[idx] > self.high_price:
                    self.low_price = lowest_body_price[self.high_price_idx: idx].min()
                    self.low_price_idx = self.high_price_idx + np.argmin(lowest_body_price[self.high_price_idx:idx])
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, idx, "bullish new high after pullback")

   
            elif self.state == "BREAK_OF_STRUCTURE":
                # structure broken downward -> restart pull-back search
                if lowest_body_price[idx] < self.low_price:
                    self.high_price = highest_body_price[self.low_price_idx:idx].max()
                    self.high_price_idx = self.low_price_idx + np.argmax(highest_body_price[self.low_price_idx:idx])
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, idx, "bullish break of structure")

                # new 3 red pull-back -> possible Fib leg
                elif self.check_pullback(greens, opens, closes, idx, trend=True):
                    
                    if highest_body_price[idx - 2] > self.high_price: # fresh HH
                        self.high_price = highest_body_price[idx-3: idx+1].max()
                        self.high_price_idx = (idx-3) + np.argmax(highest_body_price[idx-3:idx+1])
                        self.fib_labels = self.calc_fibonacci_levels(self.low_price_idx, self.high_price_idx, trend=True)
                        self.fib_dict = dict(self.fib_labels)
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, idx, "bullish waiting entry")

                    # price failed to beat prev high: treat as new pull-back
                    elif lowest_body_price[idx] <= self.high_price:
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, idx, "bullish back to pullback")
                        

            elif self.state == "WAITING_ENTRY":
                fib_618_bullish = self.fib_dict.get(0.618)
                fib_000_bullish = self.fib_dict.get(0.000)

                # price retraced to 0.618 – ready for bullish entry
                if fib_618_bullish and lows[idx] <= fib_618_bullish: # count the tip of the candlestick instead of the body
                    self.entry_signal_idx = idx
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, idx, "bullish price retraced to 0.618")

                # price breaks above 0.000 (HH) before touching 0.618
                elif fib_618_bullish and highest_body_price[idx] > fib_000_bullish:
                    self.low_price = lowest_body_price[self.high_price_idx:idx].min()
                    self.low_price_idx = self.high_price_idx + np.argmin(lowest_body_price[self.high_price_idx:idx])
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, idx, "bullish breaks above 0.000 before entry")
                
            elif self.state == "ENTRY_SIGNAL":
                fib_100_bullish = self.fib_dict.get(1.000)
                fib_000_bullish = self.fib_dict.get(0.000)

                # cancel if price dumps below 1.000 extension
                if lowest_body_price[idx] <= fib_100_bullish:
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, idx, "bullish cancels: price < 1.0")

                # Bullish Engulfing candle
                elif ((greens.iloc[idx-1] is not True) and (greens.iloc[idx] is True) and (opens[idx] <= closes[idx - 1]) and (closes[idx] >= opens[idx - 1])): #### Run Enter Logic
                    self.engulfing_candle_idx = idx
                    
                    self.stop_loss_price, self.take_profit_price = self.calc_fibonacci_levels(self.low_price_idx, 
                    self.high_price_idx,trend=True, entry_signal=self.entry_signal_idx,
                    engulfing_candle=self.engulfing_candle_idx)
                    
                    entry_price = closes[idx]
                    stop_loss_pips = abs(self.stop_loss_price - entry_price) / self.pip_size
                    
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips, self.pip_value_per_lot)
                    
                    self.evt.consoleLog(f"Balance: {self.account_balance}, Risk%: {self.risk_percent}, Stop Loss (pips): {stop_loss_pips}, Pip Value: {self.pip_value_per_lot}, Lot Size: {lot_size}")
                    # place buying order
                    self.place_order(idx, instrument = self.instrument,  trend = True, order_type = 0, volume = min(10,lot_size), stoploss = self.stop_loss_price, takeprofit = self.take_profit_price, holdtime = None)
                    
                    old_state = self.state
                    self.state = "ENTER_MARKET"
                    self.enter_market_trade_side = True
                    self.log_state_change(old_state, self.state, idx, "bullish entry: engulfing candle")
                    
                # invalidated if price breaks above HH first
                elif highest_body_price[idx] >= fib_000_bullish:
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, idx, "bullish breaks above HH first")


            
            elif self.state == "ENTER_MARKET":
                current_high = self.df['High'].iloc[idx]
                current_low = self.df["Low"].iloc[idx]
                hit = None
               
                if current_high >= self.take_profit_price or np.isclose(current_high, self.take_profit_price, atol=1e-5):
                    hit = "TAKE PROFIT"
                elif current_low <= self.stop_loss_price or np.isclose(current_low, self.stop_loss_price, atol=1e-5):
                    hit = "STOP LOSS"

                if hit is not None: #### run exit logic
                    self.reset()
                    
        # ==============================================================
        # ========================  place_order()  ========================
        # ==============================================================     
        
    def place_order(self, idx, instrument=None,  trend = True, order_type = 0, volume = 0, stoploss = None, takeprofit = None, holdtime = None):
        
        order = AlgoAPIUtil.OrderObject()
        order.instrument = instrument
        order.orderRef = idx
        order.openclose = 'open'
        order.buysell = 1 if trend == True else -1   #1=buy, -1=sell
        order.ordertype = order_type  #0=market, 1=limit, 2=stop
        order.volume =  volume
        order.takeProfitLevel = takeprofit
        order.stopLossLevel = stoploss
        order.holdtime = holdtime
        self.evt.sendOrder(order)
        
        # ==============================================================
        # ========================  pullback()  ========================
        # ==============================================================     
        
    def check_pullback(self,greens, opens, closes, idx, trend):
        if trend == False:
            # Check for 3, 4, 5, or 6 consecutive green candles.
            for window in [3, 4, 5, 6]:
                start_idx = idx - window + 1
                prev_idx = start_idx - 1
                # Boundary check
                if prev_idx < 0:
                    continue
                # Check if all are green
                if all(greens.iloc[idx - i] is True for i in range(window)):
                    # Previous candle must be red
                    if greens.iloc[prev_idx] is False:
                        # Any green candle closes above the open of previous red candle
                        if any(closes[i] >= opens[prev_idx] for i in range(start_idx, idx + 1)):
                            return True
            return False
        
        if trend == True:
            for window in [3,4,5,6]:
                start_idx = idx - window + 1
                prev_idx = start_idx - 1
                if prev_idx < 0:
                    continue
                if all(greens.iloc[idx - i] is not True for i in range(window)): # all previous not green candles 
                    if greens.iloc[prev_idx] is True:
                        if any(closes[i] <= opens[prev_idx] for i in range(start_idx, idx + 1)):
                            return True
            return False

    def reset(self):
        self.state = "IDLE"
        self.low_price  = self.low_price_idx  =  None
        self.high_price = self.high_price_idx =  None
        self.fib_labels = []
        self.fib_dict = {} 
        self.entry_signal_idx = None
        self.engulfing_candle_idx = None
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None




class AlgoEvent:
    
    

    def __init__(self):
        
        self.last_time = datetime(2010,1,1)
        self.fsm = None
        # Initialize DataFrame with columns
        self.raw_5min_data = pd.DataFrame(columns=[
            'Datetime', 'Open', 'High', 'Low', 'Close', 'Green', 'Instrument', 'Market'
        ])
        self.raw_5min_data = self.raw_5min_data.astype({
            'Datetime': 'datetime64[ns]',
            'Open': 'float32',
            'High': 'float32',
            'Low': 'float32',
            'Close': 'float32',
            'Green': 'bool',
            'Instrument': 'object',
            'Market': 'object'
        })
        self.data = pd.DataFrame(columns=[
        'Datetime', 'Open', 'High', 'Low', 'Close', 'Green', 'Instrument', 'Market', 'EMA_30', 'EMA_120', 'trend'
        ])
        # Set data types for each column
        self.data = self.data.astype({
            'Datetime': 'datetime64[ns]',
            'Open': 'float32',
            'High': 'float32',
            'Low': 'float32',
            'Close': 'float32',
            'Green': 'bool',
            'Instrument': 'object',
            'Market': 'object',
            'EMA_30': 'float32',
            'EMA_120': 'float32',
            'trend': 'boolean' 
        })

    def start(self, mEvt):
        self.evt = AlgoAPI_Backtest.AlgoEvtHandler(self, mEvt)
        self.evt.start()
        
    def add_new_rows(self, n, last3, alpha_30 = 2/31, alpha_120 = 2/121):
                
        df = self.data
        # Aggregate into a real 15-min bar
        bar = {
            'Datetime': last3['Datetime'].iloc[-1],  # end time of the 15-min bar
            'Open': last3['Open'].iloc[0],
            'High': last3['High'].max(),
            'Low': last3['Low'].min(),
            'Close': last3['Close'].iloc[-1],
            'Green': True if last3['Close'].iloc[-1] > last3['Open'].iloc[0] else False if last3['Close'].iloc[-1] < last3['Open'].iloc[0] else pd.NA,
            'Instrument': last3['Instrument'].iloc[-1],
            'Market': last3['Market'].iloc[-1]
        }
        
        if n == 0:
            EMA_30 = bar['Close']
            EMA_120 = bar['Close']
        else:
            EMA_30 = (1-alpha_30) * df.iloc[n-1]['EMA_30'] + alpha_30 * bar['Close']
            EMA_120 = (1-alpha_120) * df.iloc[n-1]['EMA_120'] + alpha_120 * bar['Close']
            
        
        bar['EMA_30'] = EMA_30
        bar['EMA_120'] = EMA_120
        
        if EMA_30 > EMA_120:
            bar['trend'] = True
        elif EMA_30 < EMA_120:
            bar['trend'] = False
        else:
            bar['trend'] = pd.NA
        
        df = pd.concat([df, pd.DataFrame([bar])], ignore_index=True)
        # self.evt.consoleLog(df.tail(1))
        return df

    def on_marketdatafeed(self, md, ab):
        
        # Build the bar
        green = True if md.lastPrice > md.openPrice else False if md.lastPrice < md.openPrice else pd.NA
        
        five_min_bar = {
            'Datetime': pd.to_datetime(md.timestamp),
            'Open': md.openPrice,
            'High': md.highPrice,
            'Low': md.lowPrice,
            'Close': md.lastPrice,
            'Green': green,
            'Instrument': md.instrument,
            'Market': md.market
        }
        
        self.raw_5min_data = pd.concat([self.raw_5min_data, pd.DataFrame([five_min_bar])], ignore_index=True)
        
        # Only aggregate when 15 min has passed
        if md.timestamp >= self.last_time + timedelta(minutes=15):
            self.last_time = md.timestamp
    
            # Get the last 3 rows (the last 15 minutes)
            last3 = self.raw_5min_data.tail(3)
            if len(last3) < 3:
                return  # not enough data yet
            
            # Append the aggregated bar to your main 15-min data
            self.data = self.add_new_rows(n = len(self.data),last3 = last3)
            # self.evt.consoleLog(self.data.tail(1))
            
            # clear up memory space
            self.raw_5min_data = self.raw_5min_data.iloc[0:0].copy()
            
            # For EMA/trend as before
            MIN_BARS = 3

            if len(self.data) < MIN_BARS:
                return
    
            available_balance = ab['availableBalance']
            self.evt.consoleLog(available_balance)
    
            # FSM logic as before, updating with new self.data
            if self.fsm is None:
                self.fsm = BearishFSM(
                    self.data,
                    instrument=md.instrument,
                    evt=self.evt,
                    account_balance=np.float32(available_balance),
                    pip_size=np.float32(0.0001),
                    pip_value_per_lot=int(10)
                )
            else:
                self.fsm.df = self.data
                self.fsm.account_balance = available_balance
                idx = len(self.data) - 1
                self.fsm.update(idx)
            
    def on_newsdatafeed(self, nd):
        pass

    def on_weatherdatafeed(self, wd):
        pass
    
    def on_econsdatafeed(self, ed):
        pass
        
    def on_corpAnnouncement(self, ca):
        pass

    def on_orderfeed(self, of):
        pass

    def on_dailyPLfeed(self, pl):
        pass

    def on_openPositionfeed(self, op, oo, uo):
        pass









#### Version 4 Improvements: 

1. The algorithm now prevents new trades while a position is active. 

2. Improved Trend Stability:
    - Before: Definition of Bullish = EMA30 > EMA120
    - Now: Definition of Bullish = EMA30 / EMA120 > 1.001, filter out noise and effectively prevent trades around Golden cross/ death cross.

In [ ]:
from AlgoAPI import AlgoAPIUtil, AlgoAPI_Backtest
import pandas as pd
import numpy as np
import mplfinance as mpf
from collections import deque
from datetime import datetime, timedelta
    
class BearishFSM: # Finite State Machine

        # ==============================================================
        # ========================  __init__ ===========================
        # ==============================================================
    def __init__(self, df, instrument, evt, account_balance=0, pip_size = 0.0001, pip_value_per_lot = 10, ts = None):
        
        self.df = df
        self.instrument = instrument
        self.evt = evt
        self.account_balance = account_balance
        
        self.state = "IDLE"
        self.low_price = None
        
        self.low_price_ts = None
        self.high_price = None
        self.high_price_ts = None

        self.fib_labels = []
        self.fib_dict = {} # not a must to initialize, just for good habit
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.trend_changing_ts = ts # do not reset this, always keep record of 
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        
        self.position_open = False

        # do not reset the folloiwng parameters, test for optimal values
        self.risk_reward_ratio = 2
        self.atr_period = 14
        self.atr_multiplier = 8.0 # pip_size = atr_multiplier * atr_value, atr_multiplier = 2.0 means stop loss is 2 ATRs away
        self.risk_percent = 1
        
        # this is unique in every func call
        self.instrument = instrument
        self.pip_size = pip_size
        self.pip_value_per_lot = pip_value_per_lot
        
        # ==============================================================
        # ========================  calculate_log_state_change =========
        # ==============================================================
        
    def log_state_change(self, old_state, new_state, ts, label=None):
        if 'Datetime' in self.df.columns and ts in self.df.index:
            date_str = str(self.df.loc[ts, 'Datetime'])
        else:
            date_str = str(ts)
        msg = f"[{date_str}] State: {old_state} → {new_state}"
        if label:
            msg += f" ({label})"
        self.evt.consoleLog(msg)
        
        # ==============================================================
        # ========================  calculate_lot_size  ================
        # ==============================================================
        
    def calculate_lot_size(self, risk_percent, stop_loss_pips, pip_value_per_lot=10):
        risk_amount = self.account_balance * (risk_percent / 100)
        lot_size = risk_amount / (stop_loss_pips * pip_value_per_lot)
        return round(lot_size, 4)
        
        # ==============================================================
        # ===== Wilder’s EMA: recursive smoothing with alpha = 1/n  ====
        # ==============================================================
        
    def wwma(self, values: pd.Series, n: int) -> pd.Series:
        # Wilder’s EMA: recursive smoothing with alpha = 1/n
        return values.ewm(alpha=1/n, adjust=False).mean()
        
        # ==============================================================
        # ========================  atr  ================
        # ==============================================================
        
    def atr(self, df: pd.DataFrame, n: int = 14) -> pd.Series:
        high = df["High"]
        low = df["Low"]
        close = df["Close"]
    
        tr0 = (high - low).abs()
        tr1 = (high - close.shift()).abs()
        tr2 = (low  - close.shift()).abs()
    
        tr = pd.concat([tr0, tr1, tr2], axis=1).max(axis=1)
    
        return self.wwma(tr, n)
        
        # ==============================================================
        # ========================  calc_fibonacci_levels  =============
        # ==============================================================
        
    def calc_fibonacci_levels(self, start_ts, end_ts, levels=None, trend="NO",
                         entry_signal=None, engulfing_candle=None,
                         R=None, atr_period=None, atr_multiplier=None):
                             
        if levels is None:
            levels = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]
            
        if atr_period is None:
            atr_period = self.atr_period
            
        if atr_multiplier is None:
            atr_multiplier = self.atr_multiplier
        
        if R is None:
            R = self.risk_reward_ratio
        
        
        df = self.df
        start_idx =  df.index.get_loc(start_ts)
        end_idx = df.index.get_loc(end_ts)
        
        start_ts = df.index[start_idx] # still a timestamp
        end_ts = df.index[end_idx]
        window = df.loc[start_ts : end_ts]
                            
        min_price = window['Low'].min()
        max_price = window['High'].max()
    
        fib_labels = []
        for level in levels:
            if trend == "BE":
                y = min_price + (max_price - min_price) * level
            else:
                y = max_price - (max_price - min_price) * level
            fib_labels.append((level, y))
        
        if engulfing_candle is None:
            return fib_labels
        
        
        # Normal ATR
        df['H-L'] = df['High'] - df['Low']
        df['H-PC'] = abs(df['High'] - df['Close'].shift(1))
        df['L-PC'] = abs(df['Low'] - df['Close'].shift(1))
        df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
        
        # ATR: rolling mean of TR
        df['ATR'] = df['TR'].rolling(window=atr_period).mean()
         
        # Retrieve ATR value at the engulfing candle index
        atr_value = df['ATR'].loc[engulfing_candle]
        pip_size = atr_multiplier * atr_value
        
        # Retrieve the close price of the engulfing candle
        candle = self.df.loc[engulfing_candle]
        close_price = candle['Close']
        
        # Initialize stop loss and take profit prices
        stop_loss_price = None
        take_profit_price = None
    
    
        if trend == "BE" :
            stop_loss_price = close_price + pip_size
            take_profit_price = close_price - R * pip_size
        else:  # bullish
            stop_loss_price = close_price - pip_size
            take_profit_price = close_price + R * pip_size
    
        return stop_loss_price, take_profit_price
        
    # ==============================================================
    # ========================  update()  =============
    # ==============================================================
    
    def update(self, ts: pd.Timestamp):
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        prev_ts = df.index[pos - 1] # still a timestamp
        
        if df.index.get_loc(ts) < 3: # Get current timestamp location
            return
        
        #--- 2. prepare frequently-used columns (all remain Series!) ------
        trend   = df["trend"]               # string
        greens  = df["Green"]               # string
        opens   = df["Open"]
        closes  = df["Close"]
        
        lowest_body_price = opens.where(greens == 'G', closes) # if greens == 'G', use values in open, else use values in closes 
        highest_body_price = closes.where(greens == 'G', opens) # the index of this series keeps the same Datetime index as the original df
        
        # selects the row in df where the index = ts
        curr   = df.loc[ts]
        prev   = df.shift(1).loc[ts]       # one-bar shift gives previous values
        
        current_trend = curr['trend']
        prev_trend    = prev['trend']

        if current_trend != prev_trend : 
            self.trend_changing_ts = ts
            if self.state != "ENTER_MARKET":
                self.reset()

        if (current_trend == "BE" and self.enter_market_trade_side != "BE") or (current_trend == "BU" and self.enter_market_trade_side == "BE"):
            if self.state == "IDLE":
                if self.check_pullback(greens, opens, closes, ts, trend="BE"): # pullback should eat and cover the previous red candle
                    # logic
                    slices = lowest_body_price.loc[self.trend_changing_ts:ts] # Slicing with .loc[start:end] in pandas includes both endpoints
                    self.low_price = slices.min()
                    self.low_price_ts  = slices.idxmin()   # Return index(Datetime) of first occurrence of minimum over requested axis.
                    # debug
                    old_state = self.state  
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish pullback detected")
                    

            elif self.state == "PULLBACK_DETECTED":
                if lowest_body_price.loc[ts] < self.low_price:
                    # logic
                    slices = highest_body_price.loc[self.low_price_ts:ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    # debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish new low after pullback")

                    
            elif self.state == "BREAK_OF_STRUCTURE":
                # If price breaks the pev high (LH)
                if highest_body_price.loc[ts] > self.high_price:
                    
                    # logic
                    slices = lowest_body_price.loc[self.high_price_ts: ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish break of structure")

                elif self.check_pullback(greens, opens, closes, ts, trend="BE"):
                    
                    if lowest_body_price.shift(2).loc[ts] < self.low_price: #### use 2 timeframe before current ts , can incorporate to 3,4,5 or 6
                    
                        # Logic: Low price for confirming the next Lower high 
                        if pos >= 3:   
                            window = lowest_body_price.iloc[pos-3 : pos+1]
                            self.low_price     = window.min()
                            self.low_price_ts  = window.idxmin()

                        self.fib_labels = self.calc_fibonacci_levels(self.high_price_ts, self.low_price_ts, trend="BE")
                        self.fib_dict = dict(self.fib_labels)
                        
                        # debug
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, ts, "bearish waiting entry")

                    elif highest_body_price.loc[ts] >= self.low_price:
                        
                        # debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bearish back to pullback")

            elif self.state == "WAITING_ENTRY":
                
                fib_618_price = self.fib_dict.get(0.618)
                fib_000_price = self.fib_dict.get(0.000)
                
                # price rebound back to 0.618
                if fib_618_price and df["High"].loc[ts] >= fib_618_price: # reference the "High" price at the exact timestamp
                
                    # debug
                    self.entry_signal_ts = ts
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, ts, "bearish price rebound to 0.618")

                elif fib_618_price and lowest_body_price.loc[ts] <= fib_000_price: # price lower than 0.000
                
                    # Logic: break of structure, set new high point
                    slices = highest_body_price.loc[self.low_price_ts: ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish breaks 0.000 before entry")
            
            elif self.state == "ENTRY_SIGNAL": 
                fib_100_price = self.fib_dict.get(1.000)
                fib_000_price = self.fib_dict.get(0.000)
                

                if highest_body_price.loc[ts] >= fib_100_price: # no matter what, if the candle pass 1.00, we do not trade
                
                    # debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish cancels: price > 1.0")

                # Logic: Bearish Engulfing candle  
                elif (greens.loc[prev_ts] == 'G') and (greens.loc[ts] != 'G') and opens.loc[ts] > closes.loc[prev_ts] and closes.loc[ts] < opens.loc[prev_ts]: # enter market
                    # Debug
                    self.engulfing_candle_ts = ts
                    self.stop_loss_price, self.take_profit_price = self.calc_fibonacci_levels(self.high_price_ts, 
                            self.low_price_ts, trend="BE", entry_signal=self.entry_signal_ts, engulfing_candle=self.engulfing_candle_ts
                            )
                    
                    # Logic to calculate lot_size
                    entry_price = closes.loc[ts]
                    stop_loss_pips = abs(self.stop_loss_price - entry_price) / self.pip_size
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips, self.pip_value_per_lot)
                    
                    # Log for debug
                    self.evt.consoleLog(f"Balance: {self.account_balance}, Risk%: {self.risk_percent}, Stop Loss (pips): {stop_loss_pips}, Pip Value: {self.pip_value_per_lot}, Lot Size: {lot_size}")
                    
                    # place selling order
                    self.place_order(ts, instrument = self.instrument,  trend = "BE", order_type = 0, volume = min(10,lot_size), 
                    stoploss = self.stop_loss_price, takeprofit = self.take_profit_price, holdtime = None)
                    
                    # Debug
                    old_state = self.state
                    self.state = "ENTER_MARKET"
                    self.enter_market_trade_side = "BE"
                    self.log_state_change(old_state, self.state, ts, "bearish entry: engulfing candle")
                    
                elif lowest_body_price.loc[ts] <= fib_000_price: # If candle breaks 0.00 before there is a Bearish ENGULFING candle
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish breaks 0.000 before engulfing")
            
            elif self.state == "ENTER_MARKET":
                
                current_high = df['High'].loc[ts]
                current_low =  df["Low"].loc[ts]

                if current_high >= self.stop_loss_price: #### run exit market logic
                    self.evt.consoleLog(f"Price touched stop_loss at {current_high:.2f} (threshold: {self.stop_loss_price:.2f}) – waiting for broker to close.")
                elif current_low <= self.take_profit_price:
                    self.evt.consoleLog(f"Price touched stop_loss at {current_low:.2f} (threshold: {self.take_profit_price:.2f}) – waiting for broker to close.")
            
        # ==============================================================
        # ========================  BULLISH  ============================
        # ==============================================================

        if (current_trend == "BU" and self.enter_market_trade_side != "BE") or (current_trend == "BE" and self.enter_market_trade_side == "BU"):
            if self.state == "IDLE":    
                if self.check_pullback(greens, opens, closes, ts, trend="BU"):
                    
                    # Logic
                    slices = highest_body_price[self.trend_changing_ts:ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish pullback detected")

            elif self.state == "PULLBACK_DETECTED":
                if highest_body_price.loc[ts] > self.high_price:
                    
                    # Logic
                    slices = lowest_body_price[self.high_price_ts: ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish new high after pullback")

   
            elif self.state == "BREAK_OF_STRUCTURE":
                # structure broken downward -> restart pull-back search
                if lowest_body_price.loc[ts] < self.low_price:
                    
                    # Logic
                    slices = highest_body_price[self.low_price_ts : ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish break of structure")

                # new 3 red pull-back -> possible Fib leg
                elif self.check_pullback(greens, opens, closes, ts, trend="BU"):
                    
                    if highest_body_price.shift(2).loc[ts] > self.high_price: # fresh HH
                        
                        # Logic
                        if pos >= 3:   
                            window = highest_body_price.iloc[pos-3 : pos+1]
                            self.high_price     = window.max()
                            self.high_price_ts  = window.idxmax()
                        
                        self.fib_labels = self.calc_fibonacci_levels(self.low_price_ts, self.high_price_ts, trend="BU")
                        self.fib_dict = dict(self.fib_labels)
                        
                        # Debug
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, ts, "bullish waiting entry")

                    # price failed to beat prev high: treat as new pull-back
                    elif lowest_body_price.loc[ts] <= self.high_price:
                        
                        # Debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bullish back to pullback")
                        

            elif self.state == "WAITING_ENTRY":
                fib_618_bullish = self.fib_dict.get(0.618)
                fib_000_bullish = self.fib_dict.get(0.000)

                # price retraced to 0.618 – ready for bullish entry
                if fib_618_bullish and df['Low'].loc[ts] <= fib_618_bullish: # count the tip of the candlestick instead of the body
                
                    # Debug
                    self.entry_signal_ts = ts
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, ts, "bullish price retraced to 0.618")

                # price breaks above 0.000 (HH) before touching 0.618
                elif fib_618_bullish and highest_body_price.loc[ts] > fib_000_bullish:
                    
                    # Logic
                    slices = lowest_body_price[self.high_price_ts:ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish breaks above 0.000 before entry")
                
            elif self.state == "ENTRY_SIGNAL":
                fib_100_bullish = self.fib_dict.get(1.000)
                fib_000_bullish = self.fib_dict.get(0.000)

                # cancel if price dumps below 1.000 extension
                if lowest_body_price.loc[ts] <= fib_100_bullish:
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish cancels: price < 1.0")

                # Bullish Engulfing candle
                elif ((greens.loc[prev_ts] != 'G') and (greens.loc[ts] == 'G') and (opens.loc[ts] <= closes.loc[prev_ts]) and (closes.loc[ts] >= opens.loc[prev_ts])):# Enter Market 
                    self.engulfing_candle_ts = ts
                    
                    self.stop_loss_price, self.take_profit_price = self.calc_fibonacci_levels(self.low_price_ts, 
                    self.high_price_ts,trend="BU", entry_signal=self.entry_signal_ts,
                    engulfing_candle=self.engulfing_candle_ts)
                    
                    # calculate volume
                    entry_price = closes.loc[ts]
                    stop_loss_pips = abs(self.stop_loss_price - entry_price) / self.pip_size
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips, self.pip_value_per_lot)
                    
                    self.evt.consoleLog(f"Balance: {self.account_balance}, Risk%: {self.risk_percent}, Stop Loss (pips): {stop_loss_pips}, Pip Value: {self.pip_value_per_lot}, Lot Size: {lot_size}")
                    # place buying order
                    self.place_order(ts, instrument = self.instrument,  trend = "BU", order_type = 0, volume = min(10,lot_size), stoploss = self.stop_loss_price, takeprofit = self.take_profit_price, holdtime = None)
                    
                    # Debug
                    old_state = self.state
                    self.state = "ENTER_MARKET"
                    self.enter_market_trade_side = "BU"
                    self.log_state_change(old_state, self.state, ts, "bullish entry: engulfing candle")
                    
                # invalidated if price breaks above HH first
                elif highest_body_price.loc[ts] >= fib_000_bullish:
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish breaks above HH first")


            elif self.state == "ENTER_MARKET":
                current_high = df['High'].loc[ts]
                current_low = df["Low"].loc[ts]
               
                if current_high >= self.take_profit_price :
                    self.evt.consoleLog(f"Price touched take_profit at {current_high:.2f} (threshold: {self.take_profit_price:.2f}) – waiting for broker to close")
                elif current_low <= self.stop_loss_price :
                    self.evt.consoleLog(f"Price touched stop_loss at {current_low:.2f} (threshold: {self.stop_loss_price:.2f}) – waiting for broker to close")

                    
        # ==============================================================
        # ========================  place_order()  ========================
        # ==============================================================     
        
    def place_order(self, ts, instrument=None,  trend = "NO", order_type = 0, volume = 0, stoploss = None, takeprofit = None, holdtime = None):
        
        self.position_open = True     # remember that we have an open trade
        
        order = AlgoAPIUtil.OrderObject()
        order.instrument = instrument
        order.orderRef = ts
        order.openclose = 'open'
        order.buysell = 1 if trend == "BU" else -1   #1=buy, -1=sell
        order.ordertype = order_type  #0=market, 1=limit, 2=stop
        order.volume =  volume
        order.takeProfitLevel = takeprofit
        order.stopLossLevel = stoploss
        order.holdtime = holdtime
        self.evt.sendOrder(order)
        
        # ==============================================================
        # ========================  pullback()  ========================
        # ==============================================================     
        
    def check_pullback(self, greens, opens, closes, ts, trend):
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        
        if trend == "BE":
            # Check for 3, 4, 5, or 6 consecutive green candles.
            for window in [3, 4, 5, 6]:

                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                # Boundary check
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                # Check if all are green
                if all(greens.loc[ts_] == 'G' for ts_ in window_ts):
                    # Previous candle must be red
                    if greens.loc[prev_ts] == 'R':
                        # Any green candle closes above the open of previous red candle
                        if any(closes.loc[ts_] >= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False
        
        if trend == "BU":
            for window in [3,4,5,6]:
                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                if all(greens.loc[ts_]!= 'G' for ts_ in window_ts): # all previous not green candles 
                    if greens.loc[prev_ts] == 'G':
                        if any(closes.loc[ts_] <= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False

    def reset(self):
        self.state = "IDLE"
        self.low_price  = self.low_price_ts  =  None
        self.high_price = self.high_price_ts =  None
        self.fib_labels = []
        self.fib_dict = {} 
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None




class AlgoEvent:

    def __init__(self):
        
        self.last_time = datetime(2010,1,1)
        self.fsm = None
        # Initialize DataFrame with columns
        self.raw_5min_data = pd.DataFrame(columns=[
            'Datetime', 'Open', 'High', 'Low', 'Close', 'Green', 'Instrument', 'Market'
        ])
        nullable_float = pd.Float32Dtype()
        self.raw_5min_data = self.raw_5min_data.astype({
            'Datetime': 'datetime64[ns]',
            'Open': nullable_float,
            'High': nullable_float,
            'Low': nullable_float,
            'Close': nullable_float,
            'Green': 'string', 
            'Instrument': 'object',
            'Market': 'object'
        })
        self.data = pd.DataFrame(columns=[
        'Datetime', 'Open', 'High', 'Low', 'Close', 'Green', 'Instrument', 'Market', 'EMA_30', 'EMA_120', 'trend'
        ])
        # Set data types for each column
        self.data = self.data.astype({
            'Datetime': 'datetime64[ns]',
            'Open': nullable_float,
            'High': nullable_float,
            'Low': nullable_float,
            'Close': nullable_float,
            'Green': 'string', 
            'Instrument': 'object',
            'Market': 'object',
            'EMA_30': nullable_float,
            'EMA_120': nullable_float,
            'trend': 'string' 
        })

    def start(self, mEvt):
        self.evt = AlgoAPI_Backtest.AlgoEvtHandler(self, mEvt)
        self.evt.start()
        
    def add_new_rows(self, n, last3, alpha_30 = 2/31, alpha_120 = 2/121):
                
        df = self.data
        # Aggregate into a real 15-min bar
        bar = {
            'Datetime': last3['Datetime'].iloc[-1],  # end time of the 15-min bar
            'Open': last3['Open'].iloc[0],
            'High': last3['High'].max(),
            'Low': last3['Low'].min(),
            'Close': last3['Close'].iloc[-1],
            'Green': 'G' if last3['Close'].iloc[-1] > last3['Open'].iloc[0] else 'R' if last3['Close'].iloc[-1] < last3['Open'].iloc[0] else 'NO',
            'Instrument': last3['Instrument'].iloc[-1],
            'Market': last3['Market'].iloc[-1]
        }
        
        if n == 0:
            EMA_30 = bar['Close']
            EMA_120 = bar['Close']
        else:
            EMA_30 = (1-alpha_30) * df.iloc[n-1]['EMA_30'] + alpha_30 * bar['Close']
            EMA_120 = (1-alpha_120) * df.iloc[n-1]['EMA_120'] + alpha_120 * bar['Close']
            
        
        bar['EMA_30'] = EMA_30
        bar['EMA_120'] = EMA_120
        
        if (EMA_30/EMA_120) > 1.001 :
            bar['trend'] = "BU"
        elif (EMA_30/EMA_120) < 0.999:
            bar['trend'] = "BE"
        else:
            bar['trend'] = "NO"
        
        df = pd.concat([df, pd.DataFrame([bar])], ignore_index=True)
        
        # self.evt.consoleLog(df.tail(1))
        return df

    def on_marketdatafeed(self, md, ab):
        
        # Build the bar
        green = 'G' if md.lastPrice > md.openPrice else 'R' if md.lastPrice < md.openPrice else 'NO'
        
        five_min_bar = {
            'Datetime': pd.to_datetime(md.timestamp),
            'Open': md.openPrice,
            'High': md.highPrice,
            'Low': md.lowPrice,
            'Close': md.lastPrice,
            'Green': green,
            'Instrument': md.instrument,
            'Market': md.market
        }
        
        self.raw_5min_data = pd.concat([self.raw_5min_data, pd.DataFrame([five_min_bar])], ignore_index=True)
        # Only aggregate when 15 min has passed
        if md.timestamp >= self.last_time + timedelta(minutes=15):
            self.last_time = md.timestamp
    
            # Get the last 3 rows (the last 15 minutes)
            last3 = self.raw_5min_data.tail(3)
            if len(last3) < 3:
                return  # not enough data yet
            
            # Append the aggregated bar to your main 15-min data
            self.data = self.add_new_rows(n = len(self.data),last3 = last3)
            # The Datetime column is now the index.
            self.data.set_index("Datetime", drop=False, inplace = True)
            
            # last_label = self.data.tail(1).index.item()
            # self.evt.consoleLog(f"Last row LABEL: {last_label}")
            
            # clear up memory space
            self.raw_5min_data = self.raw_5min_data.iloc[0:0].copy()
            
            # For EMA/trend as before
            MIN_BARS = 3

            if len(self.data) < MIN_BARS:
                return
            
            MAX_BARS = 250
            if len(self.data) > MAX_BARS:
                self.data = self.data.iloc[-MAX_BARS:]
    
            available_balance = ab['availableBalance']
            self.evt.consoleLog(available_balance)
    
            # FSM logic as before, updating with new self.data
            if self.fsm is None:
                self.fsm = BearishFSM(
                    self.data,
                    instrument=md.instrument,
                    evt=self.evt,
                    account_balance=np.float32(available_balance),
                    pip_size=np.float32(0.0001),
                    pip_value_per_lot=int(10),
                    ts = self.last_time
                )
            else:
                self.fsm.df = self.data
                self.fsm.account_balance = available_balance
                ts = self.last_time
                self.fsm.update(ts)
            
            
    def on_newsdatafeed(self, nd):
        pass

    def on_weatherdatafeed(self, wd):
        pass
    
    def on_econsdatafeed(self, ed):
        pass
        
    def on_corpAnnouncement(self, ca):
        pass

    def on_orderfeed(self, of):
        if of.openclose == 'close' and of.status == 'success': # return 'success' if an order is opened/ closed successfully, or when a limit/stop order is filled
            self.fsm.position_open = False
            self.fsm.reset()

    def on_dailyPLfeed(self, pl):
        pass

    def on_openPositionfeed(self, op, oo, uo):
        pass










#### Version 5 Improvements

1. The code is now structured to handle multiple trading instruments simultaneously

In [ ]:
from AlgoAPI import AlgoAPIUtil, AlgoAPI_Backtest
import pandas as pd
import numpy as np
import mplfinance as mpf
from collections import deque
from datetime import datetime, timedelta
    
class BearishFSM: # Finite State Machine

        # ==============================================================
        # ========================  __init__ ===========================
        # ==============================================================
    def __init__(self, df, instrument, evt, account_balance=0, pip_size = 0.0001, pip_value_per_lot = 10, ts = None):
        
        self.df = df
        self.instrument = instrument
        self.evt = evt
        self.account_balance = account_balance
        
        self.state = "IDLE"
        self.low_price = None
        
        self.low_price_ts = None
        self.high_price = None
        self.high_price_ts = None

        self.fib_labels = []
        self.fib_dict = {} # not a must to initialize, just for good habit
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.trend_changing_ts = ts # do not reset this, always keep record of 
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        
        self.position_open = False

        # do not reset the folloiwng parameters, test for optimal values
        self.risk_reward_ratio = 2
        self.atr_period = 14
        self.atr_multiplier = 8.0 # pip_size = atr_multiplier * atr_value, atr_multiplier = 2.0 means stop loss is 2 ATRs away
        self.risk_percent = 1
        
        # this is unique in every func call
        self.instrument = instrument
        self.pip_size = pip_size
        self.pip_value_per_lot = pip_value_per_lot
        
        # ==============================================================
        # ========================  calculate_log_state_change =========
        # ==============================================================
        
    def log_state_change(self, old_state, new_state, ts, label=None):
        if 'Datetime' in self.df.columns and ts in self.df.index:
            date_str = str(self.df.loc[ts, 'Datetime'])
        else:
            date_str = str(ts)
        msg = f"[{date_str}] State: {old_state} → {new_state}"
        if label:
            msg += f" ({label})"
        self.evt.consoleLog(msg)
        
        # ==============================================================
        # ========================  calculate_lot_size  ================
        # ==============================================================
        
    def calculate_lot_size(self, risk_percent, stop_loss_pips, pip_value_per_lot=10):
        risk_amount = self.account_balance * (risk_percent / 100)
        lot_size = risk_amount / (stop_loss_pips * pip_value_per_lot)
        return round(lot_size, 4)
        
        # ==============================================================
        # ===== Wilder’s EMA: recursive smoothing with alpha = 1/n  ====
        # ==============================================================
        
    def wwma(self, values: pd.Series, n: int) -> pd.Series:
        # Wilder’s EMA: recursive smoothing with alpha = 1/n
        return values.ewm(alpha=1/n, adjust=False).mean()
        
        # ==============================================================
        # ========================  atr  ================
        # ==============================================================
        
    def atr(self, df: pd.DataFrame, n: int = 14) -> pd.Series:
        high = df["High"]
        low = df["Low"]
        close = df["Close"]
    
        tr0 = (high - low).abs()
        tr1 = (high - close.shift()).abs()
        tr2 = (low  - close.shift()).abs()
    
        tr = pd.concat([tr0, tr1, tr2], axis=1).max(axis=1)
    
        return self.wwma(tr, n)
        
        # ==============================================================
        # ========================  calc_fibonacci_levels  =============
        # ==============================================================
        
    def calc_fibonacci_levels(self, start_ts, end_ts, levels=None, trend="NO",
                         entry_signal=None, engulfing_candle=None,
                         R=None, atr_period=None, atr_multiplier=None):
                             
        if levels is None:
            levels = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]
            
        if atr_period is None:
            atr_period = self.atr_period
            
        if atr_multiplier is None:
            atr_multiplier = self.atr_multiplier
        
        if R is None:
            R = self.risk_reward_ratio
        
        
        df = self.df
        start_idx =  df.index.get_loc(start_ts)
        end_idx = df.index.get_loc(end_ts)
        
        start_ts = df.index[start_idx] # still a timestamp
        end_ts = df.index[end_idx]
        window = df.loc[start_ts : end_ts]
                            
        min_price = window['Low'].min()
        max_price = window['High'].max()
    
        fib_labels = []
        for level in levels:
            if trend == "BE":
                y = min_price + (max_price - min_price) * level
            else:
                y = max_price - (max_price - min_price) * level
            fib_labels.append((level, y))
        
        if engulfing_candle is None:
            return fib_labels
        
        
        # Normal ATR
        df['H-L'] = df['High'] - df['Low']
        df['H-PC'] = abs(df['High'] - df['Close'].shift(1))
        df['L-PC'] = abs(df['Low'] - df['Close'].shift(1))
        df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
        
        # ATR: rolling mean of TR
        df['ATR'] = df['TR'].rolling(window=atr_period).mean()
         
        # Retrieve ATR value at the engulfing candle index
        atr_value = df['ATR'].loc[engulfing_candle]
        pip_size = atr_multiplier * atr_value
        
        # Retrieve the close price of the engulfing candle
        candle = self.df.loc[engulfing_candle]
        close_price = candle['Close']
        
        # Initialize stop loss and take profit prices
        stop_loss_price = None
        take_profit_price = None
    
    
        if trend == "BE" :
            stop_loss_price = close_price + pip_size
            take_profit_price = close_price - R * pip_size
        else:  # bullish
            stop_loss_price = close_price - pip_size
            take_profit_price = close_price + R * pip_size
    
        return stop_loss_price, take_profit_price
        
    # ==============================================================
    # ========================  update()  =============
    # ==============================================================
    
    def update(self, ts: pd.Timestamp):
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        prev_ts = df.index[pos - 1] # still a timestamp
        
        if df.index.get_loc(ts) < 3: # Get current timestamp location
            return
        
        #--- 2. prepare frequently-used columns (all remain Series!) ------
        trend   = df["trend"]               # string
        greens  = df["Green"]               # string
        opens   = df["Open"]
        closes  = df["Close"]
        
        lowest_body_price = opens.where(greens == 'G', closes) # if greens == 'G', use values in open, else use values in closes 
        highest_body_price = closes.where(greens == 'G', opens) # the index of this series keeps the same Datetime index as the original df
        
        # selects the row in df where the index = ts
        curr   = df.loc[ts]
        prev   = df.shift(1).loc[ts]       # one-bar shift gives previous values
        
        current_trend = curr['trend']
        prev_trend    = prev['trend']

        if current_trend != prev_trend : 
            self.trend_changing_ts = ts
            if self.state != "ENTER_MARKET":
                self.reset()

        if (current_trend == "BE" and self.enter_market_trade_side != "BE") or (current_trend == "BU" and self.enter_market_trade_side == "BE"):
            if self.state == "IDLE":
                if self.check_pullback(greens, opens, closes, ts, trend="BE"): # pullback should eat and cover the previous red candle
                    # logic
                    slices = lowest_body_price.loc[self.trend_changing_ts:ts] # Slicing with .loc[start:end] in pandas includes both endpoints
                    self.low_price = slices.min()
                    self.low_price_ts  = slices.idxmin()   # Return index(Datetime) of first occurrence of minimum over requested axis.
                    # debug
                    old_state = self.state  
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish pullback detected")
                    

            elif self.state == "PULLBACK_DETECTED":
                if lowest_body_price.loc[ts] < self.low_price:
                    # logic
                    slices = highest_body_price.loc[self.low_price_ts:ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    # debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish new low after pullback")

                    
            elif self.state == "BREAK_OF_STRUCTURE":
                # If price breaks the pev high (LH)
                if highest_body_price.loc[ts] > self.high_price:
                    
                    # logic
                    slices = lowest_body_price.loc[self.high_price_ts: ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish break of structure")

                elif self.check_pullback(greens, opens, closes, ts, trend="BE"):
                    
                    if lowest_body_price.shift(2).loc[ts] < self.low_price: #### use 2 timeframe before current ts , can incorporate to 3,4,5 or 6
                    
                        # Logic: Low price for confirming the next Lower high 
                        if pos >= 3:   
                            window = lowest_body_price.iloc[pos-3 : pos+1]
                            self.low_price     = window.min()
                            self.low_price_ts  = window.idxmin()

                        self.fib_labels = self.calc_fibonacci_levels(self.high_price_ts, self.low_price_ts, trend="BE")
                        self.fib_dict = dict(self.fib_labels)
                        
                        # debug
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, ts, "bearish waiting entry")

                    elif highest_body_price.loc[ts] >= self.low_price:
                        
                        # debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bearish back to pullback")

            elif self.state == "WAITING_ENTRY":
                
                fib_618_price = self.fib_dict.get(0.618)
                fib_000_price = self.fib_dict.get(0.000)
                
                # price rebound back to 0.618
                if fib_618_price and df["High"].loc[ts] >= fib_618_price: # reference the "High" price at the exact timestamp
                
                    # debug
                    self.entry_signal_ts = ts
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, ts, "bearish price rebound to 0.618")

                elif fib_618_price and lowest_body_price.loc[ts] <= fib_000_price: # price lower than 0.000
                
                    # Logic: break of structure, set new high point
                    slices = highest_body_price.loc[self.low_price_ts: ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish breaks 0.000 before entry")
            
            elif self.state == "ENTRY_SIGNAL": 
                fib_100_price = self.fib_dict.get(1.000)
                fib_000_price = self.fib_dict.get(0.000)
                

                if highest_body_price.loc[ts] >= fib_100_price: # no matter what, if the candle pass 1.00, we do not trade
                
                    # debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish cancels: price > 1.0")

                # Logic: Bearish Engulfing candle  
                elif (greens.loc[prev_ts] == 'G') and (greens.loc[ts] != 'G') and opens.loc[ts] > closes.loc[prev_ts] and closes.loc[ts] < opens.loc[prev_ts]: # enter market
                    # Debug
                    self.engulfing_candle_ts = ts
                    self.stop_loss_price, self.take_profit_price = self.calc_fibonacci_levels(self.high_price_ts, 
                            self.low_price_ts, trend="BE", entry_signal=self.entry_signal_ts, engulfing_candle=self.engulfing_candle_ts
                            )
                    
                    # Logic to calculate lot_size
                    entry_price = closes.loc[ts]
                    stop_loss_pips = abs(self.stop_loss_price - entry_price) / self.pip_size
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips, self.pip_value_per_lot)
                    
                    # Log for debug
                    self.evt.consoleLog(f"Balance: {self.account_balance}, Risk%: {self.risk_percent}, Stop Loss (pips): {stop_loss_pips}, Pip Value: {self.pip_value_per_lot}, Lot Size: {lot_size}")
                    
                    # place selling order
                    self.place_order(ts, instrument = self.instrument,  trend = "BE", order_type = 0, volume = min(10,lot_size), 
                    stoploss = self.stop_loss_price, takeprofit = self.take_profit_price, holdtime = None)
                    
                    # Debug
                    old_state = self.state
                    self.state = "ENTER_MARKET"
                    self.enter_market_trade_side = "BE"
                    self.log_state_change(old_state, self.state, ts, "bearish entry: engulfing candle")
                    
                elif lowest_body_price.loc[ts] <= fib_000_price: # If candle breaks 0.00 before there is a Bearish ENGULFING candle
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish breaks 0.000 before engulfing")
            
            elif self.state == "ENTER_MARKET":
                
                current_high = df['High'].loc[ts]
                current_low =  df["Low"].loc[ts]

                if current_high >= self.stop_loss_price: #### run exit market logic
                    self.evt.consoleLog(f"Price touched stop_loss at {current_high:.5f} (stop_loss: {self.stop_loss_price:.5f}) – waiting for broker to close.")
                elif current_low <= self.take_profit_price:
                    self.evt.consoleLog(f"Price touched take_profit at {current_low:.5f} (take_profit: {self.take_profit_price:.5f}) – waiting for broker to close.")
            
        # ==============================================================
        # ========================  BULLISH  ============================
        # ==============================================================

        if (current_trend == "BU" and self.enter_market_trade_side != "BE") or (current_trend == "BE" and self.enter_market_trade_side == "BU"):
            if self.state == "IDLE":    
                if self.check_pullback(greens, opens, closes, ts, trend="BU"):
                    
                    # Logic
                    slices = highest_body_price[self.trend_changing_ts:ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish pullback detected")

            elif self.state == "PULLBACK_DETECTED":
                if highest_body_price.loc[ts] > self.high_price:
                    
                    # Logic
                    slices = lowest_body_price[self.high_price_ts: ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish new high after pullback")

   
            elif self.state == "BREAK_OF_STRUCTURE":
                # structure broken downward -> restart pull-back search
                if lowest_body_price.loc[ts] < self.low_price:
                    
                    # Logic
                    slices = highest_body_price[self.low_price_ts : ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish break of structure")

                # new 3 red pull-back -> possible Fib leg
                elif self.check_pullback(greens, opens, closes, ts, trend="BU"):
                    
                    if highest_body_price.shift(2).loc[ts] > self.high_price: # fresh HH
                        
                        # Logic
                        if pos >= 3:   
                            window = highest_body_price.iloc[pos-3 : pos+1]
                            self.high_price     = window.max()
                            self.high_price_ts  = window.idxmax()
                        
                        self.fib_labels = self.calc_fibonacci_levels(self.low_price_ts, self.high_price_ts, trend="BU")
                        self.fib_dict = dict(self.fib_labels)
                        
                        # Debug
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, ts, "bullish waiting entry")

                    # price failed to beat prev high: treat as new pull-back
                    elif lowest_body_price.loc[ts] <= self.high_price:
                        
                        # Debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bullish back to pullback")
                        

            elif self.state == "WAITING_ENTRY":
                fib_618_bullish = self.fib_dict.get(0.618)
                fib_000_bullish = self.fib_dict.get(0.000)

                # price retraced to 0.618 – ready for bullish entry
                if fib_618_bullish and df['Low'].loc[ts] <= fib_618_bullish: # count the tip of the candlestick instead of the body
                
                    # Debug
                    self.entry_signal_ts = ts
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, ts, "bullish price retraced to 0.618")

                # price breaks above 0.000 (HH) before touching 0.618
                elif fib_618_bullish and highest_body_price.loc[ts] > fib_000_bullish:
                    
                    # Logic
                    slices = lowest_body_price[self.high_price_ts:ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish breaks above 0.000 before entry")
                
            elif self.state == "ENTRY_SIGNAL":
                fib_100_bullish = self.fib_dict.get(1.000)
                fib_000_bullish = self.fib_dict.get(0.000)

                # cancel if price dumps below 1.000 extension
                if lowest_body_price.loc[ts] <= fib_100_bullish:
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish cancels: price < 1.0")

                # Bullish Engulfing candle
                elif ((greens.loc[prev_ts] != 'G') and (greens.loc[ts] == 'G') and (opens.loc[ts] <= closes.loc[prev_ts]) and (closes.loc[ts] >= opens.loc[prev_ts])):# Enter Market 
                    self.engulfing_candle_ts = ts
                    
                    self.stop_loss_price, self.take_profit_price = self.calc_fibonacci_levels(self.low_price_ts, 
                    self.high_price_ts,trend="BU", entry_signal=self.entry_signal_ts,
                    engulfing_candle=self.engulfing_candle_ts)
                    
                    # calculate volume
                    entry_price = closes.loc[ts]
                    stop_loss_pips = abs(self.stop_loss_price - entry_price) / self.pip_size
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips, self.pip_value_per_lot)
                    
                    self.evt.consoleLog(f"Balance: {self.account_balance}, Risk%: {self.risk_percent}, Stop Loss (pips): {stop_loss_pips}, Pip Value: {self.pip_value_per_lot}, Lot Size: {lot_size}")
                    # place buying order
                    self.place_order(ts, instrument = self.instrument,  trend = "BU", order_type = 0, volume = min(10,lot_size), stoploss = self.stop_loss_price, takeprofit = self.take_profit_price, holdtime = None)
                    
                    # Debug
                    old_state = self.state
                    self.state = "ENTER_MARKET"
                    self.enter_market_trade_side = "BU"
                    self.log_state_change(old_state, self.state, ts, "bullish entry: engulfing candle")
                    
                # invalidated if price breaks above HH first
                elif highest_body_price.loc[ts] >= fib_000_bullish:
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish breaks above HH first")


            elif self.state == "ENTER_MARKET":
                current_high = df['High'].loc[ts]
                current_low = df["Low"].loc[ts]
               
                if current_high >= self.take_profit_price :
                    self.evt.consoleLog(f"Price touched take_profit at {current_high:.5f} (take_profit: {self.take_profit_price:.5f}) – waiting for broker to close")
                elif current_low <= self.stop_loss_price :
                    self.evt.consoleLog(f"Price touched stop_loss at {current_low:.5f} (stop_loss: {self.stop_loss_price:.5f}) – waiting for broker to close")

                    
        # ==============================================================
        # ========================  place_order()  ========================
        # ==============================================================     
        
    def place_order(self, ts, instrument=None,  trend = "NO", order_type = 0, volume = 0, stoploss = None, takeprofit = None, holdtime = None):
        
        self.position_open = True     # remember that we have an open trade
        
        order = AlgoAPIUtil.OrderObject()
        order.instrument = instrument
        order.orderRef = ts
        order.openclose = 'open'
        order.buysell = 1 if trend == "BU" else -1   #1=buy, -1=sell
        order.ordertype = order_type  #0=market, 1=limit, 2=stop
        order.volume =  volume
        order.takeProfitLevel = takeprofit
        order.stopLossLevel = stoploss
        order.holdtime = holdtime
        self.evt.sendOrder(order)
        
        # ==============================================================
        # ========================  pullback()  ========================
        # ==============================================================     
        
    def check_pullback(self, greens, opens, closes, ts, trend):
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        
        if trend == "BE":
            # Check for 3, 4, 5, or 6 consecutive green candles.
            for window in [3, 4, 5, 6]:

                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                # Boundary check
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                # Check if all are green
                if all(greens.loc[ts_] == 'G' for ts_ in window_ts):
                    # Previous candle must be red
                    if greens.loc[prev_ts] == 'R':
                        # Any green candle closes above the open of previous red candle
                        if any(closes.loc[ts_] >= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False
        
        if trend == "BU":
            for window in [3,4,5,6]:
                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                if all(greens.loc[ts_]!= 'G' for ts_ in window_ts): # all previous not green candles 
                    if greens.loc[prev_ts] == 'G':
                        if any(closes.loc[ts_] <= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False

    def reset(self):
        self.state = "IDLE"
        self.low_price  = self.low_price_ts  =  None
        self.high_price = self.high_price_ts =  None
        self.fib_labels = []
        self.fib_dict = {} 
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None


class InstrumentBar:
    
    """Bar builder + FSM for ONE instrument."""

    ALPHA_30  = 2 / 31
    ALPHA_120 = 2 / 121
    MIN_BARS  = 3
    MAX_BARS  = 250
    
    
    def __init__(self, symbol: str, evt):
        
        self.symbol    = symbol
        self.evt       = evt
        self.last_time = datetime(2010, 1, 1)
        self.fsm       = None
        
        NFLOAT = "float64"
        
        # ------------- 5-min buffer ---------------------------------
        self.raw_5m = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument"])
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "string",
                "Instrument": "object"}))
                
        # ------------- 15-min table ---------------------------------
        self.data = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument", "EMA_30", "EMA_120", "trend"])
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "string", "Instrument": "object",
                "EMA_30": NFLOAT, "EMA_120": NFLOAT,
                "trend": "string"})
            .set_index("Datetime", drop=False))
        
    def update(self, snap: dict, available_balance: float):
        
        """
        snap = bd[symbol]  – single-instrument snapshot from on_bulkdatafeed
        """
        # -------------------- append 5-min bar ------------------------
        green = ("G" if snap["lastPrice"] > snap["openPrice"]
                 else "R" if snap["lastPrice"] < snap["openPrice"]
                 else "NO")
        
        bar5 = {
            "Datetime": pd.to_datetime(snap["timestamp"]),
            "Open":  snap["openPrice"],
            "High":  snap["highPrice"],
            "Low":   snap["lowPrice"],
            "Close": snap["lastPrice"],
            "Green": green,
            "Instrument": snap["instrument"],
        }
        
        self.raw_5m = pd.concat([self.raw_5m,
                                 pd.DataFrame([bar5])], ignore_index=True)
        
        # ------------------- finished 15-min? -------------------------
        ts = pd.to_datetime(snap["timestamp"])
        if ts < self.last_time + timedelta(minutes=15):
            return
        self.last_time = ts
        
        last3 = self.raw_5m.tail(3)
        if len(last3) < 3:
            return                          # need full 15-minute slice
        self.raw_5m = self.raw_5m.iloc[0:0]  # clear buffer
        
        self._aggregate_15m(last3)
        
        # -------------------- FSM ------------------------------------
        if self.fsm is None:
            self.fsm = BearishFSM(
                self.data.copy(), instrument=self.symbol, evt=self.evt,
                account_balance=np.float32(available_balance),
                pip_size=np.float32(0.0001), pip_value_per_lot=10,
                ts=self.last_time)
        else:
            self.fsm.df = self.data
            self.fsm.account_balance = available_balance
            self.fsm.update(self.last_time)
            
    def _aggregate_15m(self, last3):
        
        n   = len(self.data)
        op  = last3["Open"].iloc[0]
        cl  = last3["Close"].iloc[-1]
        bar = {
            "Datetime": last3["Datetime"].iloc[-1],
            "Open": op,
            "High": last3["High"].max(),
            "Low":  last3["Low"].min(),
            "Close": cl,
            "Green": "G" if cl > op else "R" if cl < op else "NO",
            "Instrument": self.symbol,
        }

        if n == 0:
            ema30 = ema120 = cl
        else:
            prev  = self.data.iloc[n - 1]
            ema30  = (1 - self.ALPHA_30)  * prev["EMA_30"]  + self.ALPHA_30  * cl
            ema120 = (1 - self.ALPHA_120) * prev["EMA_120"] + self.ALPHA_120 * cl
        
        bar["EMA_30"]  = ema30
        bar["EMA_120"] = ema120
        r = ema30 / ema120
        bar["trend"] = "BU" if r > 1.001 else "BE" if r < 0.999 else "NO" ################### change to detect slope

        self.data = pd.concat([self.data,
                               pd.DataFrame([bar]).set_index("Datetime",
                                                             drop=False)])

        

class AlgoEvent:

    def __init__(self):
        self.evt   = None
        self.books = {}          # symbol → InstrumentBar

    def start(self, mEvt):
        self.evt = AlgoAPI_Backtest.AlgoEvtHandler(self, mEvt)
        self.evt.start()
            
    def on_bulkdatafeed(self, isSync, bd, ab):
        
        balance = ab["availableBalance"]
        
        for sym, snap in bd.items():        # sym: AUDNZD, EURUSD, ... snap: {'instrument': 'AUDNZD', 'timestamp': datetime.datetime(1970, 1, 1, 0, 0), 'askPrice': -1, ...}
            if sym not in self.books:       # first sight → create book
                self.books[sym] = InstrumentBar(sym, self.evt)
            else: 
                self.books[sym].update(snap, balance)

    def on_orderfeed(self, of):
        if of.openclose == 'close' and of.status == 'success': # return 'success' if an order is opened/ closed successfully, or when a limit/stop order is filled
            sym = of.instrument
            if sym in self.books and self.books[sym].fsm:
                self.books[sym].fsm.position_open = False
                self.books[sym].fsm.reset()
    
    
    def on_newsdatafeed(self, nd):
        pass

    def on_weatherdatafeed(self, wd):
        pass
    
    def on_econsdatafeed(self, ed):
        pass
        
    def on_corpAnnouncement(self, ca):
        pass
    
    def on_dailyPLfeed(self, pl):
        pass

    def on_openPositionfeed(self, op, oo, uo):
        pass










#### Version 6 Improvements

1. Asynchronous Order Management: Instead of assumeing orders are filled instantly when the "ENTER_MARKET" state is hit, we now waits for broker confirmation before considering a position to be live.


In [ ]:
from AlgoAPI import AlgoAPIUtil, AlgoAPI_Backtest
import pandas as pd
import numpy as np
import mplfinance as mpf
from collections import deque
from datetime import datetime, timedelta
    
class BearishFSM: # Finite State Machine

        # ==============================================================
        # ========================  __init__ ===========================
        # ==============================================================
    def __init__(self, df, instrument, evt, account_balance=0, pip_size = 0.0001, pip_value_per_lot = 10, ts = None):
        
        self.df = df
        self.instrument = instrument
        self.evt = evt
        self.account_balance = account_balance
        
        # Enter market flag
        self.position_open = False
        self.pending = None       # ← dict that holds the ticket waiting
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.open_volume = None
        
        self.state = "IDLE"
        self.low_price = None
        
        self.low_price_ts = None
        self.high_price = None
        self.high_price_ts = None

        self.fib_labels = []
        self.fib_dict = {} # not a must to initialize, just for good habit
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.trend_changing_ts = ts # do not reset this, always keep record of 

        # do not reset the folloiwng parameters, test for optimal values
        self.risk_reward_ratio = 2.2
        self.atr_period = 14
        self.atr_multiplier = 7.5 # pip_size = atr_multiplier * atr_value, atr_multiplier = 2.0 means stop loss is 2 ATRs away
        self.risk_percent = 1
        
        # this is unique in every func call
        self.instrument = instrument
        self.pip_size = pip_size
        self.pip_value_per_lot = pip_value_per_lot
        
        # ==============================================================
        # ========================  calculate_log_state_change =========
        # ==============================================================
        
    def log_state_change(self, old_state, new_state, ts, label=None):
        if 'Datetime' in self.df.columns and ts in self.df.index:
            date_str = str(self.df.loc[ts, 'Datetime'])
        else:
            date_str = str(ts)
        msg = f" State: {old_state} → {new_state}"
        if label:
            msg += f" ({label})"
        self.evt.consoleLog(msg)
        
        # ==============================================================
        # ========================  calculate_lot_size  ================
        # ==============================================================
        
    def calculate_lot_size(self, risk_percent, stop_loss_pips, pip_value_per_lot=10):
        risk_amount = self.account_balance * (risk_percent / 100)
        lot_size = risk_amount / (stop_loss_pips * pip_value_per_lot)
        return round(lot_size, 4)
        
        # ==============================================================
        # ===== Wilder’s EMA: recursive smoothing with alpha = 1/n  ====
        # ==============================================================
        
    def wwma(self, values: pd.Series, n: int) -> pd.Series:
        # Wilder’s EMA: recursive smoothing with alpha = 1/n
        return values.ewm(alpha=1/n, adjust=False).mean()
        
        # ==============================================================
        # ========================  atr  ================
        # ==============================================================
        
    def atr(self, df: pd.DataFrame, n: int = 14) -> pd.Series:
        high = df["High"]
        low = df["Low"]
        close = df["Close"]
    
        tr0 = (high - low).abs()
        tr1 = (high - close.shift()).abs()
        tr2 = (low  - close.shift()).abs()
    
        tr = pd.concat([tr0, tr1, tr2], axis=1).max(axis=1)
    
        return self.wwma(tr, n)
        
        # ==============================================================
        # ========================  calc_fibonacci_levels  =============
        # ==============================================================
        
    def calc_fibonacci_levels(self, start_ts, end_ts, levels=None, trend="NO",
                         entry_signal=None, engulfing_candle=None,
                         R=None, atr_period=None, atr_multiplier=None):
                             
        if levels is None:
            levels = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]
            
        if atr_period is None:
            atr_period = self.atr_period
            
        if atr_multiplier is None:
            atr_multiplier = self.atr_multiplier
        
        if R is None:
            R = self.risk_reward_ratio
        
        
        df = self.df
        start_idx =  df.index.get_loc(start_ts)
        end_idx = df.index.get_loc(end_ts)
        
        start_ts = df.index[start_idx] # still a timestamp
        end_ts = df.index[end_idx]
        window = df.loc[start_ts : end_ts]
                            
        min_price = window['Low'].min()
        max_price = window['High'].max()
    
        fib_labels = []
        for level in levels:
            if trend == "BE": # bearish, sell
                y = min_price + (max_price - min_price) * level
            else:
                y = max_price - (max_price - min_price) * level
            fib_labels.append((level, y))
        
        if engulfing_candle is None:
            return fib_labels
        
        
        # Normal ATR
        df['H-L'] = df['High'] - df['Low']
        df['H-PC'] = abs(df['High'] - df['Close'].shift(1))
        df['L-PC'] = abs(df['Low'] - df['Close'].shift(1))
        df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
        
        # ATR: rolling mean of TR
        df['ATR'] = df['TR'].rolling(window=atr_period).mean()
         
        # Retrieve ATR value at the engulfing candle index
        atr_value = df['ATR'].loc[engulfing_candle]
        pip_size = atr_multiplier * atr_value
        
        # Retrieve the close price of the engulfing candle
        candle = self.df.loc[engulfing_candle]
        close_price = candle['Close']
        
        # Initialize stop loss and take profit prices
        stop_loss_price = None
        take_profit_price = None
    
        if trend == "BE" :
            stop_loss_price = close_price + pip_size
            take_profit_price = close_price - R * pip_size
        else:  # bullish
            stop_loss_price = close_price - pip_size
            take_profit_price = close_price + R * pip_size
    
        return stop_loss_price, take_profit_price
        
    # ==============================================================
    # ======================== send_entry_order  =============
    # ==============================================================
    
    def send_entry_order(self, ts, trend, entry_price,
                         stop, take, volume):

        # remember what we just asked the broker to do
        self.pending = dict(
            orderRef = ts,         # we use the timestamp as reference
            trend    = trend,      # "BU" / "BE"
            stop     = stop,
            take     = take,
            volume   = volume
        )

        order = AlgoAPIUtil.OrderObject()
        order.instrument   = self.instrument
        order.orderRef     = ts
        order.openclose    = "open"
        order.buysell      = 1 if trend == "BU" else -1
        order.ordertype    = 0          # market
        order.volume       = volume
        order.stopLossLevel= stop
        order.takeProfitLevel = take
        self.evt.sendOrder(order)

        # NOTE: do NOT set state / flags here;
        #       wait for on_orderfeed()
    
    # -------------------------------------------------------------
    # monitor_exit()  – runs while we are in the trade
    # -------------------------------------------------------------
    def monitor_exit(self, ts):
        """
        Called every bar only while self.position_open is True.
        Produces the same log lines you had in your old ENTER_MARKET
        branch.  If you additionally want to *close* the trade,
        send a close-order here.
        """
        if self.take_profit_price is None or self.stop_loss_price is None:
            return      # safety – nothing to do

        high = self.df["High"].loc[ts]
        low  = self.df["Low"].loc[ts]

        if self.enter_market_trade_side == "BU":          # long trade
            if high >= self.take_profit_price:
                self.evt.consoleLog(
                    f"Price touched take_profit at {high:.5f} "
                    f"(take_profit: {self.take_profit_price:.5f}) – "
                    "waiting for broker to close"
                )
            elif low <= self.stop_loss_price:
                self.evt.consoleLog(
                    f"Price touched stop_loss at {low:.5f} "
                    f"(stop_loss: {self.stop_loss_price:.5f}) – "
                    "waiting for broker to close"
                )

        else:                                             # short trade
            if low <= self.take_profit_price:
                self.evt.consoleLog(
                    f"Price touched take_profit at {low:.5f} "
                    f"(take_profit: {self.take_profit_price:.5f}) – "
                    "waiting for broker to close"
                )
            elif high >= self.stop_loss_price:
                self.evt.consoleLog(
                    f"Price touched stop_loss at {high:.5f} "
                    f"(stop_loss: {self.stop_loss_price:.5f}) – "
                    "waiting for broker to close"
                )
                
    # ==============================================================
    # ========================  update()  =============
    # ==============================================================
    
    def update(self, ts: pd.Timestamp):
        
        # 1️⃣ When a position is open we do only exit-logic and leave
        #    the rest of the gigantic state-machine untouched.
        #    (So IDLE / PULLBACK_DETECTED / … cannot fire again.)
        if self.position_open:
            self.monitor_exit(ts)        # <-- will emit console logs
            return                       # <-- early exit
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        prev_ts = df.index[pos - 1] # still a timestamp
        
        if df.index.get_loc(ts) < 3: # Get current timestamp location
            return
        
        #--- 2. prepare frequently-used columns (all remain Series!) ------
        trend   = df["trend"]               # string
        greens  = df["Green"]               # string
        opens   = df["Open"]
        closes  = df["Close"]
        
        lowest_body_price = opens.where(greens == 'G', closes) # if greens == 'G', use values in open, else use values in closes 
        highest_body_price = closes.where(greens == 'G', opens) # the index of this series keeps the same Datetime index as the original df
        
        # selects the row in df where the index = ts
        curr   = df.loc[ts]
        prev   = df.shift(1).loc[ts]       # one-bar shift gives previous values
        
        current_trend = curr['trend']
        prev_trend    = prev['trend']

        if current_trend != prev_trend : 
            self.trend_changing_ts = ts
            if self.state != "ENTER_MARKET":
                self.reset()

        if (current_trend == "BE" and self.enter_market_trade_side == None):
            if self.state == "IDLE":
                if self.check_pullback(greens, opens, closes, ts, trend="BE"): # pullback should eat and cover the previous red candle
                    # logic
                    slices = lowest_body_price.loc[self.trend_changing_ts:ts] # Slicing with .loc[start:end] in pandas includes both endpoints
                    self.low_price = slices.min()
                    self.low_price_ts  = slices.idxmin()   # Return index(Datetime) of first occurrence of minimum over requested axis.
                    # debug
                    old_state = self.state  
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish pullback detected")
                    

            elif self.state == "PULLBACK_DETECTED":
                if lowest_body_price.loc[ts] < self.low_price:
                    # logic
                    slices = highest_body_price.loc[self.low_price_ts:ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    # debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish new low after pullback")

                    
            elif self.state == "BREAK_OF_STRUCTURE":
                # If price breaks the pev high (LH)
                if highest_body_price.loc[ts] > self.high_price:
                    
                    # logic
                    slices = lowest_body_price.loc[self.high_price_ts: ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish break of structure")

                elif self.check_pullback(greens, opens, closes, ts, trend="BE"):
                    
                    if lowest_body_price.shift(2).loc[ts] < self.low_price: #### use 2 timeframe before current ts , can incorporate to 3,4,5 or 6
                    
                        # Logic: Low price for confirming the next Lower high 
                        if pos >= 3:   
                            window = lowest_body_price.iloc[pos-3 : pos+1]
                            self.low_price     = window.min()
                            self.low_price_ts  = window.idxmin()

                        self.fib_labels = self.calc_fibonacci_levels(self.high_price_ts, self.low_price_ts, trend="BE")
                        self.fib_dict = dict(self.fib_labels)
                        
                        # debug
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, ts, "bearish waiting entry")

                    elif highest_body_price.loc[ts] >= self.low_price:
                        
                        # debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bearish back to pullback")

            elif self.state == "WAITING_ENTRY":
                
                fib_618_price = self.fib_dict.get(0.618)
                fib_000_price = self.fib_dict.get(0.000)
                
                # price rebound back to 0.618
                if fib_618_price and df["High"].loc[ts] >= fib_618_price: # reference the "High" price at the exact timestamp
                
                    # debug
                    self.entry_signal_ts = ts
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, ts, "bearish price rebound to 0.618")

                elif fib_618_price and lowest_body_price.loc[ts] <= fib_000_price: # price lower than 0.000
                
                    # Logic: break of structure, set new high point
                    slices = highest_body_price.loc[self.low_price_ts: ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish breaks 0.000 before entry")
            
            elif self.state == "ENTRY_SIGNAL": 
                fib_100_price = self.fib_dict.get(1.000)
                fib_000_price = self.fib_dict.get(0.000)
                

                if highest_body_price.loc[ts] >= fib_100_price: # no matter what, if the candle pass 1.00, we do not trade
                
                    # debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish cancels: price > 1.0")

                # Logic: Bearish Engulfing candle  
                elif (greens.loc[prev_ts] == 'G') and (greens.loc[ts] != 'G') and opens.loc[ts] > closes.loc[prev_ts] and closes.loc[ts] < opens.loc[prev_ts]: # enter market
                    # Debug
                    self.engulfing_candle_ts = ts
                    
                    stop, take = self.calc_fibonacci_levels(self.high_price_ts, 
                            self.low_price_ts, trend="BE", entry_signal=self.entry_signal_ts, engulfing_candle=self.engulfing_candle_ts
                            )
                    
                    # Logic to calculate lot_size
                    entry_price = closes.loc[ts]
                    stop_loss_pips = abs(stop - entry_price) / self.pip_size
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips, self.pip_value_per_lot)
                    
                    # Log for debug
                    self.evt.consoleLog(
                        f"DEBUG ticket – side:SELL, instrument:{self.instrument},ab:{self.account_balance}, entry≈{entry_price:.5f} "
                        f"SL={stop:.5f} TP={take:.5f}"
                    )
                    
                    # self.evt.consoleLog(f"Balance: {self.account_balance}, Risk%: {self.risk_percent}, Stop Loss (pips): {stop_loss_pips}, Pip Value: {self.pip_value_per_lot}, Lot Size: {lot_size}")
                    
                    # place selling order
                    self.send_entry_order(ts, trend="BE",
                                entry_price=entry_price,
                                stop=stop, take=take,
                                volume=min(10, lot_size))
                    
                    # Debug
                    old_state = self.state
                    self.state = "PRE_ENTER_MARKET"
                    self.log_state_change(old_state, self.state, ts, "bearish entry: engulfing candle")
                    
                elif lowest_body_price.loc[ts] <= fib_000_price: # If candle breaks 0.00 before there is a Bearish ENGULFING candle
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish breaks 0.000 before engulfing")
            
        # ==============================================================
        # ========================  BULLISH  ============================
        # ==============================================================

        if (current_trend == "BU" and self.enter_market_trade_side == None):
            if self.state == "IDLE":    
                if self.check_pullback(greens, opens, closes, ts, trend="BU"):
                    
                    # Logic
                    slices = highest_body_price[self.trend_changing_ts:ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish pullback detected")

            elif self.state == "PULLBACK_DETECTED":
                if highest_body_price.loc[ts] > self.high_price:
                    
                    # Logic
                    slices = lowest_body_price[self.high_price_ts: ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish new high after pullback")

   
            elif self.state == "BREAK_OF_STRUCTURE":
                # structure broken downward -> restart pull-back search
                if lowest_body_price.loc[ts] < self.low_price:
                    
                    # Logic
                    slices = highest_body_price[self.low_price_ts : ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish break of structure")

                # new 3 red pull-back -> possible Fib leg
                elif self.check_pullback(greens, opens, closes, ts, trend="BU"):
                    
                    if highest_body_price.shift(2).loc[ts] > self.high_price: # fresh HH
                        
                        # Logic
                        if pos >= 3:   
                            window = highest_body_price.iloc[pos-3 : pos+1]
                            self.high_price     = window.max()
                            self.high_price_ts  = window.idxmax()
                        
                        self.fib_labels = self.calc_fibonacci_levels(self.low_price_ts, self.high_price_ts, trend="BU")
                        self.fib_dict = dict(self.fib_labels)
                        
                        # Debug
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, ts, "bullish waiting entry")

                    # price failed to beat prev high: treat as new pull-back
                    elif lowest_body_price.loc[ts] <= self.high_price:
                        
                        # Debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bullish back to pullback")
                        

            elif self.state == "WAITING_ENTRY":
                fib_618_bullish = self.fib_dict.get(0.618)
                fib_000_bullish = self.fib_dict.get(0.000)

                # price retraced to 0.618 – ready for bullish entry
                if fib_618_bullish and df['Low'].loc[ts] <= fib_618_bullish: # count the tip of the candlestick instead of the body
                
                    # Debug
                    self.entry_signal_ts = ts
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, ts, "bullish price retraced to 0.618")

                # price breaks above 0.000 (HH) before touching 0.618
                elif fib_618_bullish and highest_body_price.loc[ts] > fib_000_bullish:
                    
                    # Logic
                    slices = lowest_body_price[self.high_price_ts:ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish breaks above 0.000 before entry")
                
            elif self.state == "ENTRY_SIGNAL":
                fib_100_bullish = self.fib_dict.get(1.000)
                fib_000_bullish = self.fib_dict.get(0.000)

                # cancel if price dumps below 1.000 extension
                if lowest_body_price.loc[ts] <= fib_100_bullish:
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish cancels: price < 1.0")

                # Bullish Engulfing candle
                elif ((greens.loc[prev_ts] != 'G') and (greens.loc[ts] == 'G') and (opens.loc[ts] <= closes.loc[prev_ts]) and (closes.loc[ts] >= opens.loc[prev_ts])):# Enter Market 
                    self.engulfing_candle_ts = ts
                    
                    stop, take = self.calc_fibonacci_levels(self.low_price_ts, 
                    self.high_price_ts,trend="BU", entry_signal=self.entry_signal_ts,
                    engulfing_candle=self.engulfing_candle_ts)
                    
                    # calculate volume
                    entry_price = closes.loc[ts]
                    stop_loss_pips = abs(stop - entry_price) / self.pip_size
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips, self.pip_value_per_lot)
                    
                    self.evt.consoleLog(
                        f"DEBUG ticket – side:BUY, instrument: {self.instrument}, ab:{self.account_balance}, entry≈{entry_price:.5f} "
                        f"SL={stop:.5f} TP={take:.5f}"
                    )
                    # place buying order
                    self.send_entry_order(ts, trend="BU",
                              entry_price=entry_price,
                              stop=stop, take=take,
                              volume=min(10, lot_size))
                    # self.evt.consoleLog(f"Balance: {self.account_balance}, Risk%: {self.risk_percent}, Stop Loss (pips): {stop_loss_pips}, Pip Value: {self.pip_value_per_lot}, Lot Size: {lot_size}")
                    
                    # Debug
                    old_state = self.state
                    self.state = "PRE_ENTER_MARKET"
                    self.log_state_change(old_state, self.state, ts, "bullish entry: engulfing candle")
                    
                # invalidated if price breaks above HH first
                elif highest_body_price.loc[ts] >= fib_000_bullish:
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish breaks above HH first")

            elif self.state == "ENTER_MARKET":
                current_high = df['High'].loc[ts]
                current_low = df["Low"].loc[ts]
               
                if current_high >= self.take_profit_price :
                    self.evt.consoleLog(f"Price touched take_profit at {current_high:.5f} (take_profit: {self.take_profit_price:.5f}) – waiting for broker to close")
                elif current_low <= self.stop_loss_price :
                    self.evt.consoleLog(f"Price touched stop_loss at {current_low:.5f} (stop_loss: {self.stop_loss_price:.5f}) – waiting for broker to close")
        
        # ==============================================================
        # ========================  pullback()  ========================
        # ==============================================================     
        
    def check_pullback(self, greens, opens, closes, ts, trend):
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        
        if trend == "BE":
            # Check for 3, 4, 5, or 6 consecutive green candles.
            for window in [3, 4, 5, 6]:

                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                # Boundary check
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                # Check if all are green
                if all(greens.loc[ts_] == 'G' for ts_ in window_ts):
                    # Previous candle must be red
                    if greens.loc[prev_ts] == 'R':
                        # Any green candle closes above the open of previous red candle
                        if any(closes.loc[ts_] >= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False
        
        if trend == "BU":
            for window in [3,4,5,6]:
                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                if all(greens.loc[ts_]!= 'G' for ts_ in window_ts): # all previous not green candles 
                    if greens.loc[prev_ts] == 'G':
                        if any(closes.loc[ts_] <= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False

    def reset(self):
        self.state = "IDLE"
        self.low_price  = self.low_price_ts  =  None
        self.high_price = self.high_price_ts =  None
        self.fib_labels = []
        self.fib_dict = {} 
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.position_open = False
        self.open_volume = None


class InstrumentBar:
    
    """Bar builder + FSM for ONE instrument."""

    ALPHA_30  = 2 / 31
    ALPHA_120 = 2 / 121
    MIN_BARS  = 3
    MAX_BARS  = 225
    
    
    def __init__(self, symbol: str, evt):
        
        self.symbol    = symbol
        self.evt       = evt
        self.last_time = datetime(2010, 1, 1)
        self.fsm       = None
        
        NFLOAT = "float64"
        
        # ------------- 5-min buffer ---------------------------------
        self.raw_5m = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument"])
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "string",
                "Instrument": "object"}))
                
        # ------------- 15-min table ---------------------------------
        self.data = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument", "EMA_30", "EMA_120", "trend"])
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "string", "Instrument": "object",
                "EMA_30": NFLOAT, "EMA_120": NFLOAT,
                "trend": "string"})
            .set_index("Datetime", drop=False))
        
    def update(self, snap: dict, available_balance: float):
        
        """
        snap = bd[symbol]  – single-instrument snapshot from on_bulkdatafeed
        """
        # -------------------- append 5-min bar ------------------------
        green = ("G" if snap["lastPrice"] > snap["openPrice"]
                 else "R" if snap["lastPrice"] < snap["openPrice"]
                 else "NO")
        
        bar5 = {
            "Datetime": pd.to_datetime(snap["timestamp"]),
            "Open":  snap["openPrice"],
            "High":  snap["highPrice"],
            "Low":   snap["lowPrice"],
            "Close": snap["lastPrice"],
            "Green": green,
            "Instrument": snap["instrument"],
        }
        
        self.raw_5m = pd.concat([self.raw_5m,
                                 pd.DataFrame([bar5])], ignore_index=True)
        
        # ------------------- finished 15-min? -------------------------
        ts = pd.to_datetime(snap["timestamp"])
        if ts < self.last_time + timedelta(minutes=15):
            return
        self.last_time = ts
        
        last3 = self.raw_5m.tail(3)
        if len(last3) < 3:
            return                          # need full 15-minute slice
        self.raw_5m = self.raw_5m.iloc[0:0]  # clear buffer
        
        self._aggregate_15m(last3)
        
        # -------------------- FSM ------------------------------------
        if self.fsm is None:
            self.fsm = BearishFSM(
                self.data.copy(), instrument=self.symbol, evt=self.evt,
                account_balance=np.float32(available_balance),
                pip_size=np.float32(0.0001), pip_value_per_lot=10,
                ts=self.last_time)
        else:
            self.fsm.df = self.data
            self.fsm.account_balance = available_balance
            self.fsm.update(self.last_time)
            
    def _aggregate_15m(self, last3):
        
        n   = len(self.data)
        op  = last3["Open"].iloc[0]
        cl  = last3["Close"].iloc[-1]
        bar = {
            "Datetime": last3["Datetime"].iloc[-1],
            "Open": op,
            "High": last3["High"].max(),
            "Low":  last3["Low"].min(),
            "Close": cl,
            "Green": "G" if cl > op else "R" if cl < op else "NO",
            "Instrument": self.symbol,
        }

        if n == 0:
            ema30 = ema120 = cl
        else:
            prev  = self.data.iloc[n - 1]
            ema30  = (1 - self.ALPHA_30)  * prev["EMA_30"]  + self.ALPHA_30  * cl
            ema120 = (1 - self.ALPHA_120) * prev["EMA_120"] + self.ALPHA_120 * cl
        
        bar["EMA_30"]  = ema30
        bar["EMA_120"] = ema120
        r = ema30 / ema120
        bar["trend"] = "BU" if r > 1.001 else "BE" if r < 0.999 else "NO" ################### change to detect slope

        self.data = pd.concat([self.data,
                               pd.DataFrame([bar]).set_index("Datetime",
                                                             drop=False)])

        

class AlgoEvent:

    def __init__(self):
        self.evt   = None
        self.books = {}          # symbol → InstrumentBar

    def start(self, mEvt):
        self.evt = AlgoAPI_Backtest.AlgoEvtHandler(self, mEvt)
        self.evt.start()
            
    def on_bulkdatafeed(self, isSync, bd, ab):
        
        balance = ab["availableBalance"]
        
        for sym, snap in bd.items():        # sym: AUDNZD, EURUSD, ... snap: {'instrument': 'AUDNZD', 'timestamp': datetime.datetime(1970, 1, 1, 0, 0), 'askPrice': -1, ...}
            if sym not in self.books:       # first sight → create book
                self.books[sym] = InstrumentBar(sym, self.evt)
            else: 
                self.books[sym].update(snap, balance)

    def on_orderfeed(self, of):
        
        sym = of.instrument
        if sym not in self.books:
            return                        # safety
        
        fsm = self.books[sym].fsm
        
        # -------------- order filled -----------------
        if of.openclose == "open" and of.status == "success":
            # Is this the order we are waiting for?
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.position_open      = True
                fsm.stop_loss_price    = fsm.pending["stop"]
                fsm.take_profit_price  = fsm.pending["take"]
                fsm.open_volume        = fsm.pending["volume"]
                fsm.enter_market_trade_side = fsm.pending["trend"]
    
                fsm.pending = None      # clear
                old_state   = fsm.state
                fsm.state   = "ENTER_MARKET"
                fsm.log_state_change(old_state, fsm.state, of.orderRef,
                                     "broker filled order")
        # -------------- order rejected ---------------
        elif of.openclose == "open" and of.status != "success":
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.pending = None
                old_state = fsm.state
                fsm.state = "ENTRY_SIGNAL"     # back to previous state
                fsm.log_state_change(old_state, fsm.state, of.orderRef,
                                     "order rejected")
                                     
        # -------------- position closed --------------
        if of.openclose == 'close' and of.status == 'success': # return 'success' if an order is opened/ closed successfully, or when a limit/stop order is filled
            fsm.reset()
    
    
    def on_newsdatafeed(self, nd):
        pass

    def on_weatherdatafeed(self, wd):
        pass
    
    def on_econsdatafeed(self, ed):
        pass
        
    def on_corpAnnouncement(self, ca):
        pass
    
    def on_dailyPLfeed(self, pl):
        pass

    def on_openPositionfeed(self, op, oo, uo):
        pass











#### Version 7 Improvements

1. Dynamically adjust the ATR lookback period to be the length of the previous price swing, which allows wider stops after volatile moves and tighter stops after quieter ones.

2. Using EMA30 and EMA120 slope to confirm momentum and filter out patterns during trend exhaustion or potential reversals. ( However it doesn't work quite well in the backtesting)

In [ ]:
from AlgoAPI import AlgoAPIUtil, AlgoAPI_Backtest
import pandas as pd
import numpy as np
import mplfinance as mpf
from collections import deque
from datetime import datetime, timedelta
    
class BearishFSM: # Finite State Machine

        # ==============================================================
        # ========================  __init__ ===========================
        # ==============================================================
    def __init__(self, df, instrument, evt, account_balance=0, pip_size = 0.0001, pip_value_per_lot = 10, ts = None):
        
        self.df = df
        self.instrument = instrument
        self.evt = evt
        self.account_balance = account_balance
        
        # Enter market flag
        self.position_open = False
        self.pending = None       # ← dict that holds the ticket waiting
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.open_volume = None
        
        self.state = "IDLE"
        self.low_price = None
        
        self.low_price_ts = None
        self.high_price = None
        self.high_price_ts = None

        self.fib_labels = []
        self.fib_dict = {} # not a must to initialize, just for good habit
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.trend_changing_ts = ts # do not reset this, always keep record of 

        # do not reset the folloiwng parameters, test for optimal values
        self.risk_reward_ratio = 2.2
        self.atr_period = 14
        self.atr_multiplier = 7.5 # pip_size = atr_multiplier * atr_value, atr_multiplier = 2.0 means stop loss is 2 ATRs away
        self.risk_percent = 1
        self.dynamic_atr_window = True
        # this is unique in every func call
        self.instrument = instrument
        self.pip_size = pip_size
        self.pip_value_per_lot = pip_value_per_lot
        
        # ==============================================================
        # ========================  calculate_log_state_change =========
        # ==============================================================
        
    def log_state_change(self, old_state, new_state, ts, label=None):
        if 'Datetime' in self.df.columns and ts in self.df.index:
            date_str = str(self.df.loc[ts, 'Datetime'])
        else:
            date_str = str(ts)
        msg = f" State: {old_state} → {new_state}"
        if label:
            msg += f" ({label})"
        self.evt.consoleLog(msg)
        
        # ==============================================================
        # ========================  calculate_lot_size  ================
        # ==============================================================
        
    def calculate_lot_size(self, risk_percent, stop_loss_pips, pip_value_per_lot=10):
        risk_amount = self.account_balance * (risk_percent / 100)
        lot_size = risk_amount / (stop_loss_pips * pip_value_per_lot)
        return round(lot_size, 4)
        
        # ==============================================================
        # ===== Wilder’s EMA: recursive smoothing with alpha = 1/n  ====
        # ==============================================================
        
    def wwma(self, values: pd.Series, n: int) -> pd.Series:
        # Wilder’s EMA: recursive smoothing with alpha = 1/n
        return values.ewm(alpha=1/n, adjust=False).mean()
        
        # ==============================================================
        # ========================  atr  ================
        # ==============================================================
        
    def atr(self, df: pd.DataFrame, n: int = 14) -> pd.Series:
        high = df["High"]
        low = df["Low"]
        close = df["Close"]
    
        tr0 = (high - low).abs()
        tr1 = (high - close.shift()).abs()
        tr2 = (low  - close.shift()).abs()
    
        tr = pd.concat([tr0, tr1, tr2], axis=1).max(axis=1)
    
        return self.wwma(tr, n)
        
        # ==============================================================
        # ========================  calc_fibonacci_levels  =============
        # ==============================================================
        
    def calc_fibonacci_levels(self, start_ts, end_ts, levels=None, trend="NO",
                         entry_signal=None, engulfing_candle=None,
                         R=None, atr_period=None, atr_multiplier=None,dynamic_atr_window = None):
                             
        if levels is None:
            levels = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]
            
        if dynamic_atr_window is None:
            dynamic_atr_window = self.dynamic_atr_window
            
        if atr_multiplier is None:
            atr_multiplier = self.atr_multiplier
        
        if R is None:
            R = self.risk_reward_ratio
        
        
        df = self.df
        start_idx =  df.index.get_loc(start_ts)
        end_idx = df.index.get_loc(end_ts)
        
        start_ts = df.index[start_idx] # still a timestamp
        end_ts = df.index[end_idx]
        window = df.loc[start_ts : end_ts]
                            
        min_price = window['Low'].min()
        max_price = window['High'].max()
    
        fib_labels = []
        for level in levels:
            if trend == "BE": # bearish, sell
                y = min_price + (max_price - min_price) * level
            else:
                y = max_price - (max_price - min_price) * level
            fib_labels.append((level, y))
        
        if engulfing_candle is None:
            return fib_labels
        
        if atr_period is None:

            if dynamic_atr_window == False:
                # Fixed atr period
                atr_period = self.atr_period
            else: 
                # dynamic atr period
                start_idx = df.index.get_loc(start_ts)
                enter_idx = df.index.get_loc(engulfing_candle)
            
                dynamic_period = enter_idx - start_idx
                atr_period = max(10, min(dynamic_period, 50))
        
        # Normal ATR
        df['H-L'] = df['High'] - df['Low']
        df['H-PC'] = abs(df['High'] - df['Close'].shift(1))
        df['L-PC'] = abs(df['Low'] - df['Close'].shift(1))
        df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
        
        # ATR: rolling mean of TR
        df['ATR'] = df['TR'].rolling(window=atr_period).mean()
         
        # Retrieve ATR value at the engulfing candle index
        atr_value = df['ATR'].loc[engulfing_candle]
        pip_size = atr_multiplier * atr_value
        
        # Retrieve the close price of the engulfing candle
        candle = self.df.loc[engulfing_candle]
        close_price = candle['Close']
        
        # Initialize stop loss and take profit prices
        stop_loss_price = None
        take_profit_price = None
    
        if trend == "BE" :
            stop_loss_price = close_price + pip_size
            take_profit_price = close_price - R * pip_size
        else:  # bullish
            stop_loss_price = close_price - pip_size
            take_profit_price = close_price + R * pip_size
    
        return stop_loss_price, take_profit_price
        
    # ==============================================================
    # ======================== send_entry_order  =============
    # ==============================================================
    
    def send_entry_order(self, ts, trend, entry_price,
                         stop, take, volume):

        # remember what we just asked the broker to do
        self.pending = dict(
            orderRef = ts,         # we use the timestamp as reference
            trend    = trend,      # "BU" / "BE"
            stop     = stop,
            take     = take,
            volume   = volume
        )

        order = AlgoAPIUtil.OrderObject()
        order.instrument   = self.instrument
        order.orderRef     = ts
        order.openclose    = "open"
        order.buysell      = 1 if trend == "BU" else -1
        order.ordertype    = 0          # market
        order.volume       = volume
        order.stopLossLevel= stop
        order.takeProfitLevel = take
        self.evt.sendOrder(order)

        # NOTE: do NOT set state / flags here;
        #       wait for on_orderfeed()
    
    # -------------------------------------------------------------
    # monitor_exit()  – runs while we are in the trade
    # -------------------------------------------------------------
    def monitor_exit(self, ts):
        """
        Called every bar only while self.position_open is True.
        Produces the same log lines you had in your old ENTER_MARKET
        branch.  If you additionally want to *close* the trade,
        send a close-order here.
        """
        if self.take_profit_price is None or self.stop_loss_price is None:
            return      # safety – nothing to do

        high = self.df["High"].loc[ts]
        low  = self.df["Low"].loc[ts]

        if self.enter_market_trade_side == "BU":          # long trade
            if high >= self.take_profit_price:
                self.evt.consoleLog(
                    f"Price touched take_profit at {high:.5f} "
                    f"(take_profit: {self.take_profit_price:.5f}) – "
                    "waiting for broker to close"
                )
            elif low <= self.stop_loss_price:
                self.evt.consoleLog(
                    f"Price touched stop_loss at {low:.5f} "
                    f"(stop_loss: {self.stop_loss_price:.5f}) – "
                    "waiting for broker to close"
                )

        else:                                             # short trade
            if low <= self.take_profit_price:
                self.evt.consoleLog(
                    f"Price touched take_profit at {low:.5f} "
                    f"(take_profit: {self.take_profit_price:.5f}) – "
                    "waiting for broker to close"
                )
            elif high >= self.stop_loss_price:
                self.evt.consoleLog(
                    f"Price touched stop_loss at {high:.5f} "
                    f"(stop_loss: {self.stop_loss_price:.5f}) – "
                    "waiting for broker to close"
                )
                
    # ==============================================================
    # ========================  update()  =============
    # ==============================================================
    
    def update(self, ts: pd.Timestamp):
        
        # 1️⃣ When a position is open we do only exit-logic and leave
        #    the rest of the gigantic state-machine untouched.
        #    (So IDLE / PULLBACK_DETECTED / … cannot fire again.)
        if self.position_open:
            self.monitor_exit(ts)        # <-- will emit console logs
            return                       # <-- early exit
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        prev_ts = df.index[pos - 1] # still a timestamp
        
        if df.index.get_loc(ts) < 3: # Get current timestamp location
            return
        
        #--- 2. prepare frequently-used columns (all remain Series!) ------
        trend   = df["trend"]               # string
        greens  = df["Green"]               # string
        opens   = df["Open"]
        closes  = df["Close"]
        
        lowest_body_price = opens.where(greens == 'G', closes) # if greens == 'G', use values in open, else use values in closes 
        highest_body_price = closes.where(greens == 'G', opens) # the index of this series keeps the same Datetime index as the original df
        
        # selects the row in df where the index = ts
        curr   = df.loc[ts]
        prev   = df.shift(1).loc[ts]       # one-bar shift gives previous values
        
        current_trend = curr['trend']
        prev_trend    = prev['trend']

        if current_trend != prev_trend : 
            self.trend_changing_ts = ts
            if self.state != "ENTER_MARKET":
                self.reset()

        if (current_trend == "BE" and self.enter_market_trade_side == None):
            if self.state == "IDLE":
                if self.check_pullback(greens, opens, closes, ts, trend="BE"): # pullback should eat and cover the previous red candle
                    # logic
                    slices = lowest_body_price.loc[self.trend_changing_ts:ts] # Slicing with .loc[start:end] in pandas includes both endpoints
                    self.low_price = slices.min()
                    self.low_price_ts  = slices.idxmin()   # Return index(Datetime) of first occurrence of minimum over requested axis.
                    # debug
                    old_state = self.state  
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish pullback detected")
                    

            elif self.state == "PULLBACK_DETECTED":
                if lowest_body_price.loc[ts] < self.low_price:
                    # logic
                    slices = highest_body_price.loc[self.low_price_ts:ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    # debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish new low after pullback")

                    
            elif self.state == "BREAK_OF_STRUCTURE":
                # If price breaks the pev high (LH)
                if highest_body_price.loc[ts] > self.high_price:
                    
                    # logic
                    slices = lowest_body_price.loc[self.high_price_ts: ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish break of structure")

                elif self.check_pullback(greens, opens, closes, ts, trend="BE"):
                    
                    if lowest_body_price.shift(2).loc[ts] < self.low_price: #### use 2 timeframe before current ts , can incorporate to 3,4,5 or 6
                    
                        # Logic: Low price for confirming the next Lower high 
                        if pos >= 3:   
                            window = lowest_body_price.iloc[pos-3 : pos+1]
                            self.low_price     = window.min()
                            self.low_price_ts  = window.idxmin()

                        self.fib_labels = self.calc_fibonacci_levels(self.high_price_ts, self.low_price_ts, trend="BE")
                        self.fib_dict = dict(self.fib_labels)
                        
                        # debug
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, ts, "bearish waiting entry")

                    elif highest_body_price.loc[ts] >= self.low_price:
                        
                        # debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bearish back to pullback")

            elif self.state == "WAITING_ENTRY":
                
                fib_618_price = self.fib_dict.get(0.618)
                fib_000_price = self.fib_dict.get(0.000)
                
                # price rebound back to 0.618
                if fib_618_price and df["High"].loc[ts] >= fib_618_price: # reference the "High" price at the exact timestamp
                
                    # debug
                    self.entry_signal_ts = ts
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, ts, "bearish price rebound to 0.618")

                elif fib_618_price and lowest_body_price.loc[ts] <= fib_000_price: # price lower than 0.000
                
                    # Logic: break of structure, set new high point
                    slices = highest_body_price.loc[self.low_price_ts: ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish breaks 0.000 before entry")
            
            elif self.state == "ENTRY_SIGNAL": 
                fib_100_price = self.fib_dict.get(1.000)
                fib_000_price = self.fib_dict.get(0.000)
                

                if highest_body_price.loc[ts] >= fib_100_price: # no matter what, if the candle pass 1.00, we do not trade
                
                    # debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish cancels: price > 1.0")

                # Logic: Bearish Engulfing candle  
                elif (greens.loc[prev_ts] == 'G') and (greens.loc[ts] != 'G') and opens.loc[ts] > closes.loc[prev_ts] and closes.loc[ts] < opens.loc[prev_ts]: # enter market
                    current_slope_30 = df.loc[ts]['slope30']
                    current_slope_120 = df.loc[ts]['slope120']
                    
                    # In bearish trend, a crossover early signal emerges when EMA30 turns upward, while EMA120 is still trending downward or flat.
                    
                    if current_slope_30 > 0 and current_slope_120 < 0: # EMA_30 rising, but EMA_120 decreasing
                        return # no trade, wait for next entry
        
                    # Debug
                    self.engulfing_candle_ts = ts
                    
                    stop, take = self.calc_fibonacci_levels(self.high_price_ts, 
                            self.low_price_ts, trend="BE", entry_signal=self.entry_signal_ts, engulfing_candle=self.engulfing_candle_ts
                            )
                    
                    # Logic to calculate lot_size
                    entry_price = closes.loc[ts]
                    stop_loss_pips = abs(stop - entry_price) / self.pip_size
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips, self.pip_value_per_lot)
                    
                    # Log for debug
                    self.evt.consoleLog(
                        f"DEBUG ticket – side:SELL, instrument:{self.instrument},ab:{self.account_balance}, entry≈{entry_price:.5f} "
                        f"SL={stop:.5f} TP={take:.5f}"
                    )
                    
                    # self.evt.consoleLog(f"Balance: {self.account_balance}, Risk%: {self.risk_percent}, Stop Loss (pips): {stop_loss_pips}, Pip Value: {self.pip_value_per_lot}, Lot Size: {lot_size}")
                    
                    # place selling order
                    self.send_entry_order(ts, trend="BE",
                                entry_price=entry_price,
                                stop=stop, take=take,
                                volume=min(10, lot_size))
                    
                    # Debug
                    old_state = self.state
                    self.state = "PRE_ENTER_MARKET"
                    self.log_state_change(old_state, self.state, ts, "bearish entry: engulfing candle")
                    
                elif lowest_body_price.loc[ts] <= fib_000_price: # If candle breaks 0.00 before there is a Bearish ENGULFING candle
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish breaks 0.000 before engulfing")
            
        # ==============================================================
        # ========================  BULLISH  ============================
        # ==============================================================

        if (current_trend == "BU" and self.enter_market_trade_side == None):
            if self.state == "IDLE":    
                if self.check_pullback(greens, opens, closes, ts, trend="BU"):
                    
                    # Logic
                    slices = highest_body_price[self.trend_changing_ts:ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish pullback detected")

            elif self.state == "PULLBACK_DETECTED":
                if highest_body_price.loc[ts] > self.high_price:
                    
                    # Logic
                    slices = lowest_body_price[self.high_price_ts: ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish new high after pullback")

   
            elif self.state == "BREAK_OF_STRUCTURE":
                # structure broken downward -> restart pull-back search
                if lowest_body_price.loc[ts] < self.low_price:
                    
                    # Logic
                    slices = highest_body_price[self.low_price_ts : ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish break of structure")

                # new 3 red pull-back -> possible Fib leg
                elif self.check_pullback(greens, opens, closes, ts, trend="BU"):
                    
                    if highest_body_price.shift(2).loc[ts] > self.high_price: # fresh HH
                        
                        # Logic
                        if pos >= 3:   
                            window = highest_body_price.iloc[pos-3 : pos+1]
                            self.high_price     = window.max()
                            self.high_price_ts  = window.idxmax()
                        
                        self.fib_labels = self.calc_fibonacci_levels(self.low_price_ts, self.high_price_ts, trend="BU")
                        self.fib_dict = dict(self.fib_labels)
                        
                        # Debug
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, ts, "bullish waiting entry")

                    # price failed to beat prev high: treat as new pull-back
                    elif lowest_body_price.loc[ts] <= self.high_price:
                        
                        # Debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bullish back to pullback")
                        

            elif self.state == "WAITING_ENTRY":
                fib_618_bullish = self.fib_dict.get(0.618)
                fib_000_bullish = self.fib_dict.get(0.000)

                # price retraced to 0.618 – ready for bullish entry
                if fib_618_bullish and df['Low'].loc[ts] <= fib_618_bullish: # count the tip of the candlestick instead of the body
                
                    # Debug
                    self.entry_signal_ts = ts
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, ts, "bullish price retraced to 0.618")

                # price breaks above 0.000 (HH) before touching 0.618
                elif fib_618_bullish and highest_body_price.loc[ts] > fib_000_bullish:
                    
                    # Logic
                    slices = lowest_body_price[self.high_price_ts:ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish breaks above 0.000 before entry")
                
            elif self.state == "ENTRY_SIGNAL":
                fib_100_bullish = self.fib_dict.get(1.000)
                fib_000_bullish = self.fib_dict.get(0.000)

                # cancel if price dumps below 1.000 extension
                if lowest_body_price.loc[ts] <= fib_100_bullish:
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish cancels: price < 1.0")

                # Bullish Engulfing candle
                elif ((greens.loc[prev_ts] != 'G') and (greens.loc[ts] == 'G') and (opens.loc[ts] <= closes.loc[prev_ts]) and (closes.loc[ts] >= opens.loc[prev_ts])):# Enter Market 
                
                    current_slope_30 = df.loc[ts]['slope30']
                    current_slope_120 = df.loc[ts]['slope120']
                    
                    # In bullish trend, a crossover early signal emerges when EMA30 turns downward, while EMA120 is still trending upward or flat.
                    
                    if current_slope_30 < 0 and current_slope_120 > 0: # EMA_30 rising, but EMA_120 decreasing
                        return # no trade, wait for next entry
                    
                    self.engulfing_candle_ts = ts
                    
                    stop, take = self.calc_fibonacci_levels(self.low_price_ts, 
                    self.high_price_ts,trend="BU", entry_signal=self.entry_signal_ts,
                    engulfing_candle=self.engulfing_candle_ts)
                    
                    # calculate volume
                    entry_price = closes.loc[ts]
                    stop_loss_pips = abs(stop - entry_price) / self.pip_size
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips, self.pip_value_per_lot)
                    
                    self.evt.consoleLog(
                        f"DEBUG ticket – side:BUY, instrument: {self.instrument}, ab:{self.account_balance}, entry≈{entry_price:.5f} "
                        f"SL={stop:.5f} TP={take:.5f}"
                    )
                    # place buying order
                    self.send_entry_order(ts, trend="BU",
                              entry_price=entry_price,
                              stop=stop, take=take,
                              volume=min(10, lot_size))
                    # self.evt.consoleLog(f"Balance: {self.account_balance}, Risk%: {self.risk_percent}, Stop Loss (pips): {stop_loss_pips}, Pip Value: {self.pip_value_per_lot}, Lot Size: {lot_size}")
                    
                    # Debug
                    old_state = self.state
                    self.state = "PRE_ENTER_MARKET"
                    self.log_state_change(old_state, self.state, ts, "bullish entry: engulfing candle")
                    
                # invalidated if price breaks above HH first
                elif highest_body_price.loc[ts] >= fib_000_bullish:
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish breaks above HH first")

            elif self.state == "ENTER_MARKET":
                current_high = df['High'].loc[ts]
                current_low = df["Low"].loc[ts]
               
                if current_high >= self.take_profit_price :
                    self.evt.consoleLog(f"Price touched take_profit at {current_high:.5f} (take_profit: {self.take_profit_price:.5f}) – waiting for broker to close")
                elif current_low <= self.stop_loss_price :
                    self.evt.consoleLog(f"Price touched stop_loss at {current_low:.5f} (stop_loss: {self.stop_loss_price:.5f}) – waiting for broker to close")
        
        # ==============================================================
        # ========================  pullback()  ========================
        # ==============================================================     
        
    def check_pullback(self, greens, opens, closes, ts, trend):
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        
        if trend == "BE":
            # Check for 3, 4, 5, or 6 consecutive green candles.
            for window in [3, 4, 5, 6]:

                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                # Boundary check
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                # Check if all are green
                if all(greens.loc[ts_] == 'G' for ts_ in window_ts):
                    # Previous candle must be red
                    if greens.loc[prev_ts] == 'R':
                        # Any green candle closes above the open of previous red candle
                        if any(closes.loc[ts_] >= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False
        
        if trend == "BU":
            for window in [3,4,5,6]:
                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                if all(greens.loc[ts_]!= 'G' for ts_ in window_ts): # all previous not green candles 
                    if greens.loc[prev_ts] == 'G':
                        if any(closes.loc[ts_] <= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False

    def reset(self):
        self.state = "IDLE"
        self.low_price  = self.low_price_ts  =  None
        self.high_price = self.high_price_ts =  None
        self.fib_labels = []
        self.fib_dict = {} 
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.position_open = False
        self.open_volume = None


class InstrumentBar:
    
    """Bar builder + FSM for ONE instrument."""

    ALPHA_30  = 2 / 31
    ALPHA_120 = 2 / 121
    MIN_BARS  = 3
    MAX_BARS  = 225
    
    
    def __init__(self, symbol: str, evt):
        
        self.symbol    = symbol
        self.evt       = evt
        self.last_time = datetime(2010, 1, 1)
        self.fsm       = None
        
        NFLOAT = "float32"
        
        # ------------- 5-min buffer ---------------------------------
        self.raw_5m = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument"])
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "string",
                "Instrument": "object"}))
                
        # ------------- 15-min table ---------------------------------
        self.data = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument", "EMA_30", "EMA_120", "trend"])
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "string", "Instrument": "object",
                "EMA_30": NFLOAT, "EMA_120": NFLOAT,
                "trend": "string",
                "slope30": NFLOAT,
                "slope120": NFLOAT
            })
            .set_index("Datetime", drop=False))
        
    def update(self, snap: dict, available_balance: float):
        
        """
        snap = bd[symbol]  – single-instrument snapshot from on_bulkdatafeed
        """
        # -------------------- append 5-min bar ------------------------
        green = ("G" if snap["lastPrice"] > snap["openPrice"]
                 else "R" if snap["lastPrice"] < snap["openPrice"]
                 else "NO")
        
        bar5 = {
            "Datetime": pd.to_datetime(snap["timestamp"]),
            "Open":  snap["openPrice"],
            "High":  snap["highPrice"],
            "Low":   snap["lowPrice"],
            "Close": snap["lastPrice"],
            "Green": green,
            "Instrument": snap["instrument"],
        }
        
        self.raw_5m = pd.concat([self.raw_5m,
                                 pd.DataFrame([bar5])], ignore_index=True)
        
        # ------------------- finished 15-min? -------------------------
        ts = pd.to_datetime(snap["timestamp"])
        if ts < self.last_time + timedelta(minutes=15):
            return
        self.last_time = ts
        
        last3 = self.raw_5m.tail(3)
        if len(last3) < 3:
            return                          # need full 15-minute slice
        self.raw_5m = self.raw_5m.iloc[0:0]  # clear buffer
        
        self._aggregate_15m(last3)
        
        # -------------------- FSM ------------------------------------
        if self.fsm is None:
            self.fsm = BearishFSM(
                self.data.copy(), instrument=self.symbol, evt=self.evt,
                account_balance=np.float32(available_balance),
                pip_size=np.float32(0.0001), pip_value_per_lot=10,
                ts=self.last_time)
        else:
            self.fsm.df = self.data
            self.fsm.account_balance = available_balance
            self.fsm.update(self.last_time)
            
    def _aggregate_15m(self, last3):
        
        n   = len(self.data)
        op  = last3["Open"].iloc[0]
        cl  = last3["Close"].iloc[-1]
        bar = {
            "Datetime": last3["Datetime"].iloc[-1],
            "Open": op,
            "High": last3["High"].max(),
            "Low":  last3["Low"].min(),
            "Close": cl,
            "Green": "G" if cl > op else "R" if cl < op else "NO",
            "Instrument": self.symbol,
        }

        if n == 0:
            ema30 = ema120 = cl
            slope30 = slope120 = 0
        elif n < 30: 
            prev  = self.data.iloc[n - 1]
            ema30  = (1 - self.ALPHA_30)  * prev["EMA_30"]  + self.ALPHA_30  * cl
            ema120 = (1 - self.ALPHA_120) * prev["EMA_120"] + self.ALPHA_120 * cl
            slope30 = slope120 = 0
        else:
            prev  = self.data.iloc[n - 3] ###########################################  optimize the number of bars for calculating the slope: n-bar slope (e.g. 5) bars)
            ema30  = (1 - self.ALPHA_30)  * prev["EMA_30"]  + self.ALPHA_30  * cl
            ema120 = (1 - self.ALPHA_120) * prev["EMA_120"] + self.ALPHA_120 * cl
            slope30 = ema30 - prev["EMA_30"]
            slope120 = ema120 - prev["EMA_120"]
        
        bar["EMA_30"]  = ema30
        bar["EMA_120"] = ema120
        r = ema30 / ema120
        bar["trend"] = "BU" if r > 1.001 else "BE" if r < 0.999 else "NO" ################### change to detect slope
        bar['slope30'] = slope30
        bar['slope120'] = slope120

        self.data = pd.concat([self.data,
                               pd.DataFrame([bar]).set_index("Datetime",
                                                             drop=False)])

        

class AlgoEvent:

    def __init__(self):
        self.evt   = None
        self.books = {}          # symbol → InstrumentBar

    def start(self, mEvt):
        self.evt = AlgoAPI_Backtest.AlgoEvtHandler(self, mEvt)
        self.evt.start()
            
    def on_bulkdatafeed(self, isSync, bd, ab):
        
        balance = ab["availableBalance"]
        
        for sym, snap in bd.items():        # sym: AUDNZD, EURUSD, ... snap: {'instrument': 'AUDNZD', 'timestamp': datetime.datetime(1970, 1, 1, 0, 0), 'askPrice': -1, ...}
            if sym not in self.books:       # first sight → create book
                self.books[sym] = InstrumentBar(sym, self.evt)
            else: 
                self.books[sym].update(snap, balance)

    def on_orderfeed(self, of):
        
        sym = of.instrument
        if sym not in self.books:
            return                        # safety
        
        fsm = self.books[sym].fsm
        
        # -------------- order filled -----------------
        if of.openclose == "open" and of.status == "success":
            # Is this the order we are waiting for?
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.position_open      = True
                fsm.stop_loss_price    = fsm.pending["stop"]
                fsm.take_profit_price  = fsm.pending["take"]
                fsm.open_volume        = fsm.pending["volume"]
                fsm.enter_market_trade_side = fsm.pending["trend"]
    
                fsm.pending = None      # clear
                old_state   = fsm.state
                fsm.state   = "ENTER_MARKET"
                fsm.log_state_change(old_state, fsm.state, of.orderRef,
                                     "broker filled order")
        # -------------- order rejected ---------------
        elif of.openclose == "open" and of.status != "success":
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.pending = None
                old_state = fsm.state
                fsm.state = "ENTRY_SIGNAL"     # back to previous state
                fsm.log_state_change(old_state, fsm.state, of.orderRef,
                                     "order rejected")
                                     
        # -------------- position closed --------------
        if of.openclose == 'close' and of.status == 'success': # return 'success' if an order is opened/ closed successfully, or when a limit/stop order is filled
            fsm.reset()
    
    
    def on_newsdatafeed(self, nd):
        pass

    def on_weatherdatafeed(self, wd):
        pass
    
    def on_econsdatafeed(self, ed):
        pass
        
    def on_corpAnnouncement(self, ca):
        pass
    
    def on_dailyPLfeed(self, pl):
        pass

    def on_openPositionfeed(self, op, oo, uo):
        pass












#### Version 8 Improvements

1. Instead of computing the ATR over the entire DataFrame history (an O(N) operation), it now works on a small, localized slice of data

In [ ]:
from AlgoAPI import AlgoAPIUtil, AlgoAPI_Backtest
import pandas as pd
import numpy as np
import mplfinance as mpf
from collections import deque
from datetime import datetime, timedelta
    
class BearishFSM: # Finite State Machine

        # ==============================================================
        # ========================  __init__ ===========================
        # ==============================================================
    def __init__(self, df, instrument, evt, account_balance=0, pip_size = 0.0001, pip_value_per_lot = 10, ts = None):
        
        self.df = df
        self.instrument = instrument
        self.evt = evt
        self.account_balance = account_balance
        
        # Enter market flag
        self.position_open = False
        self.pending = None       # ← dict that holds the ticket waiting
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.open_volume = None
        
        self.state = "IDLE"
        self.low_price = None
        
        self.low_price_ts = None
        self.high_price = None
        self.high_price_ts = None

        self.fib_labels = []
        self.fib_dict = {} # not a must to initialize, just for good habit
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.trend_changing_ts = ts # do not reset this, always keep record of 

        # do not reset the folloiwng parameters, test for optimal values
        self.risk_reward_ratio = 2.2
        self.atr_period = 14
        self.atr_multiplier = 7.5 # pip_size = atr_multiplier * atr_value, atr_multiplier = 2.0 means stop loss is 2 ATRs away
        self.risk_percent = 1
        self.dynamic_atr_window = True
        # this is unique in every func call
        self.instrument = instrument
        self.pip_size = pip_size
        self.pip_value_per_lot = pip_value_per_lot
        
        # ==============================================================
        # ========================  calculate_log_state_change =========
        # ==============================================================
        
    def log_state_change(self, old_state, new_state, ts, label=None):
        if 'Datetime' in self.df.columns and ts in self.df.index:
            date_str = str(self.df.loc[ts, 'Datetime'])
        else:
            date_str = str(ts)
        msg = f" State: {old_state} → {new_state}"
        if label:
            msg += f" ({label})"
        self.evt.consoleLog(msg)
        
        # ==============================================================
        # ========================  calculate_lot_size  ================
        # ==============================================================
        
    def calculate_lot_size(self, risk_percent, stop_loss_pips, pip_value_per_lot=10):
        risk_amount = self.account_balance * (risk_percent / 100)
        lot_size = risk_amount / (stop_loss_pips * pip_value_per_lot)
        return round(lot_size, 4)
        
        # ==============================================================
        # ========================  calc_fibonacci_levels  =============
        # ==============================================================
        
    def calc_fibonacci_levels(self, start_ts, end_ts, levels=None, trend="NO",
                         entry_signal=None, engulfing_candle=None,
                         R=None, atr_period=None, atr_multiplier=None,dynamic_atr_window = None):
                             
        if levels is None:
            levels = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]
            
        if dynamic_atr_window is None:
            dynamic_atr_window = self.dynamic_atr_window
            
        if atr_multiplier is None:
            atr_multiplier = self.atr_multiplier
        
        if R is None:
            R = self.risk_reward_ratio
        
        
        df = self.df
        """
        start_idx =  df.index.get_loc(start_ts)
        end_idx = df.index.get_loc(end_ts)
        
        start_ts = df.index[start_idx] # still a timestamp
        end_ts = df.index[end_idx]
        """
        window = df.loc[start_ts : end_ts]
                            
        min_price = window['Low'].min()
        max_price = window['High'].max()
    
        fib_labels = []
        for level in levels:
            if trend == "BE": # bearish, sell
                y = min_price + (max_price - min_price) * level
            else:
                y = max_price - (max_price - min_price) * level
            fib_labels.append((level, y))
        
        if engulfing_candle is None:
            return fib_labels
        
        if atr_period is None:

            if dynamic_atr_window == False:
                # Fixed atr period
                atr_period = self.atr_period
            else: 
                # dynamic atr period
                start_idx = df.index.get_loc(start_ts)
                enter_idx = df.index.get_loc(engulfing_candle)
            
                dynamic_period = enter_idx - start_idx
                atr_period = max(10, min(dynamic_period, 50))
        
        # Normal ATR
        min_start_idx = max(0, df.index.get_loc(start_ts) - 10)
        min_start_ts = df.index[min_start_idx]
        sl = df.loc[min_start_ts : engulfing_candle].copy()
        
        sl['H-L']  = sl['High'] - sl['Low']
        sl['H-PC'] = (sl['High'] - sl['Close'].shift(1)).abs()
        sl['L-PC'] = (sl['Low']  - sl['Close'].shift(1)).abs()
        sl['TR']    = sl[['H-L', 'H-PC', 'L-PC']].max(axis=1)
        
        # ATR: rolling mean of TR
        sl['ATR'] = sl['TR'].rolling(window=atr_period).mean()
         
        # Retrieve ATR value at the engulfing candle index
        atr_value = sl['ATR'].iloc[-1]
        pip_size = atr_multiplier * atr_value
        
        # Retrieve the close price of the engulfing candle
        candle = self.df.loc[engulfing_candle]
        close_price = candle['Close']
        
        # Initialize stop loss and take profit prices
        stop_loss_price = None
        take_profit_price = None
    
        if trend == "BE" :
            stop_loss_price = close_price + pip_size
            take_profit_price = close_price - R * pip_size
        else:  # bullish
            stop_loss_price = close_price - pip_size
            take_profit_price = close_price + R * pip_size
    
        return stop_loss_price, take_profit_price
        
    # ==============================================================
    # ======================== send_entry_order  =============
    # ==============================================================
    
    def send_entry_order(self, ts, trend, entry_price,
                         stop, take, volume):

        # remember what we just asked the broker to do
        self.pending = dict(
            orderRef = ts,         # we use the timestamp as reference
            trend    = trend,      # "BU" / "BE"
            stop     = stop,
            take     = take,
            volume   = volume
        )

        order = AlgoAPIUtil.OrderObject()
        order.instrument   = self.instrument
        order.orderRef     = ts
        order.openclose    = "open"
        order.buysell      = 1 if trend == "BU" else -1
        order.ordertype    = 0          # market
        order.volume       = volume
        order.stopLossLevel= stop
        order.takeProfitLevel = take
        self.evt.sendOrder(order)

        # NOTE: do NOT set state / flags here;
        #       wait for on_orderfeed()
    
    # -------------------------------------------------------------
    # monitor_exit()  – runs while we are in the trade
    # -------------------------------------------------------------
    def monitor_exit(self, ts):
        """
        Called every bar only while self.position_open is True.
        Produces the same log lines you had in your old ENTER_MARKET
        branch.  If you additionally want to *close* the trade,
        send a close-order here.
        """
        if self.take_profit_price is None or self.stop_loss_price is None:
            return      # safety – nothing to do

        high = self.df["High"].loc[ts]
        low  = self.df["Low"].loc[ts]

        if self.enter_market_trade_side == "BU":          # long trade
            if high >= self.take_profit_price:
                self.evt.consoleLog(
                    f"Price touched take_profit at {high:.5f} "
                    f"(take_profit: {self.take_profit_price:.5f}) – "
                    "waiting for broker to close"
                )
            elif low <= self.stop_loss_price:
                self.evt.consoleLog(
                    f"Price touched stop_loss at {low:.5f} "
                    f"(stop_loss: {self.stop_loss_price:.5f}) – "
                    "waiting for broker to close"
                )

        else:                                             # short trade
            if low <= self.take_profit_price:
                self.evt.consoleLog(
                    f"Price touched take_profit at {low:.5f} "
                    f"(take_profit: {self.take_profit_price:.5f}) – "
                    "waiting for broker to close"
                )
            elif high >= self.stop_loss_price:
                self.evt.consoleLog(
                    f"Price touched stop_loss at {high:.5f} "
                    f"(stop_loss: {self.stop_loss_price:.5f}) – "
                    "waiting for broker to close"
                )
                
    # ==============================================================
    # ========================  update()  =============
    # ==============================================================
    
    def update(self, ts: pd.Timestamp):
        
        # 1️⃣ When a position is open we do only exit-logic and leave
        #    the rest of the gigantic state-machine untouched.
        #    (So IDLE / PULLBACK_DETECTED / … cannot fire again.)
        if self.position_open:
            self.monitor_exit(ts)        # <-- will emit console logs
            return                       # <-- early exit
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        prev_ts = df.index[pos - 1] # still a timestamp
        
        if df.index.get_loc(ts) < 3: # Get current timestamp location
            return
        
        #--- 2. prepare frequently-used columns (all remain Series!) ------
        trend   = df["trend"]               # string
        greens  = df["Green"]               # string
        opens   = df["Open"]
        closes  = df["Close"]
        
        lowest_body_price = opens.where(greens == 'G', closes) # if greens == 'G', use values in open, else use values in closes 
        highest_body_price = closes.where(greens == 'G', opens) # the index of this series keeps the same Datetime index as the original df
        
        # selects the row in df where the index = ts
        curr   = df.loc[ts]
        prev   = df.shift(1).loc[ts]       # one-bar shift gives previous values
        
        current_trend = curr['trend']
        prev_trend    = prev['trend']

        if current_trend != prev_trend : 
            self.trend_changing_ts = ts
            if self.state != "ENTER_MARKET":
                self.reset()

        if (current_trend == "BE" and self.enter_market_trade_side == None):
            if self.state == "IDLE":
                if self.check_pullback(greens, opens, closes, ts, trend="BE"): # pullback should eat and cover the previous red candle
                    # logic
                    slices = lowest_body_price.loc[self.trend_changing_ts:ts] # Slicing with .loc[start:end] in pandas includes both endpoints
                    self.low_price = slices.min()
                    self.low_price_ts  = slices.idxmin()   # Return index(Datetime) of first occurrence of minimum over requested axis.
                    # debug
                    old_state = self.state  
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish pullback detected")
                    

            elif self.state == "PULLBACK_DETECTED":
                if lowest_body_price.loc[ts] < self.low_price:
                    # logic
                    slices = highest_body_price.loc[self.low_price_ts:ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    # debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish new low after pullback")

                    
            elif self.state == "BREAK_OF_STRUCTURE":
                # If price breaks the pev high (LH)
                if highest_body_price.loc[ts] > self.high_price:
                    
                    # logic
                    slices = lowest_body_price.loc[self.high_price_ts: ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish break of structure")

                elif self.check_pullback(greens, opens, closes, ts, trend="BE"):
                    
                    if lowest_body_price.shift(2).loc[ts] < self.low_price: #### use 2 timeframe before current ts , can incorporate to 3,4,5 or 6
                    
                        # Logic: Low price for confirming the next Lower high 
                        if pos >= 3:   
                            window = lowest_body_price.iloc[pos-3 : pos+1]
                            self.low_price     = window.min()
                            self.low_price_ts  = window.idxmin()

                        self.fib_labels = self.calc_fibonacci_levels(self.high_price_ts, self.low_price_ts, trend="BE")
                        self.fib_dict = dict(self.fib_labels)
                        
                        # debug
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, ts, "bearish waiting entry")

                    elif highest_body_price.loc[ts] >= self.low_price:
                        
                        # debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bearish back to pullback")

            elif self.state == "WAITING_ENTRY":
                
                fib_618_price = self.fib_dict.get(0.618)
                fib_000_price = self.fib_dict.get(0.000)
                
                # price rebound back to 0.618
                if fib_618_price and df["High"].loc[ts] >= fib_618_price: # reference the "High" price at the exact timestamp
                
                    # debug
                    self.entry_signal_ts = ts
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, ts, "bearish price rebound to 0.618")

                elif fib_618_price and lowest_body_price.loc[ts] <= fib_000_price: # price lower than 0.000
                
                    # Logic: break of structure, set new high point
                    slices = highest_body_price.loc[self.low_price_ts: ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish breaks 0.000 before entry")
            
            elif self.state == "ENTRY_SIGNAL": 
                fib_100_price = self.fib_dict.get(1.000)
                fib_000_price = self.fib_dict.get(0.000)
                

                if highest_body_price.loc[ts] >= fib_100_price: # no matter what, if the candle pass 1.00, we do not trade
                
                    # debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish cancels: price > 1.0")

                # Logic: Bearish Engulfing candle  
                elif (greens.loc[prev_ts] == 'G') and (greens.loc[ts] != 'G') and opens.loc[ts] > closes.loc[prev_ts] and closes.loc[ts] < opens.loc[prev_ts]: # enter market
                    current_slope_30 = df.loc[ts]['slope30']
                    current_slope_120 = df.loc[ts]['slope120']
                    
                    # In bearish trend, a crossover early signal emerges when EMA30 turns upward, while EMA120 is still trending downward or flat.
                    
                    if current_slope_30 > 0 and current_slope_120 < 0: # EMA_30 rising, but EMA_120 decreasing
                        return # no trade, wait for next entry
        
                    # Debug
                    self.engulfing_candle_ts = ts
                    
                    stop, take = self.calc_fibonacci_levels(self.high_price_ts, 
                            self.low_price_ts, trend="BE", entry_signal=self.entry_signal_ts, engulfing_candle=self.engulfing_candle_ts
                            )
                    
                    # Logic to calculate lot_size
                    entry_price = closes.loc[ts]
                    stop_loss_pips = abs(stop - entry_price) / self.pip_size
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips, self.pip_value_per_lot)
                    
                    # Log for debug
                    self.evt.consoleLog(
                        f"DEBUG ticket – side:SELL, instrument:{self.instrument},ab:{self.account_balance}, entry≈{entry_price:.5f} "
                        f"SL={stop:.5f} TP={take:.5f}"
                    )
                    
                    # self.evt.consoleLog(f"Balance: {self.account_balance}, Risk%: {self.risk_percent}, Stop Loss (pips): {stop_loss_pips}, Pip Value: {self.pip_value_per_lot}, Lot Size: {lot_size}")
                    
                    # place selling order
                    self.send_entry_order(ts, trend="BE",
                                entry_price=entry_price,
                                stop=stop, take=take,
                                volume=min(10, lot_size))
                    
                    # Debug
                    old_state = self.state
                    self.state = "PRE_ENTER_MARKET"
                    self.log_state_change(old_state, self.state, ts, "bearish entry: engulfing candle")
                    
                elif lowest_body_price.loc[ts] <= fib_000_price: # If candle breaks 0.00 before there is a Bearish ENGULFING candle
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish breaks 0.000 before engulfing")
            
        # ==============================================================
        # ========================  BULLISH  ============================
        # ==============================================================

        if (current_trend == "BU" and self.enter_market_trade_side == None):
            if self.state == "IDLE":    
                if self.check_pullback(greens, opens, closes, ts, trend="BU"):
                    
                    # Logic
                    slices = highest_body_price[self.trend_changing_ts:ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish pullback detected")

            elif self.state == "PULLBACK_DETECTED":
                if highest_body_price.loc[ts] > self.high_price:
                    
                    # Logic
                    slices = lowest_body_price[self.high_price_ts: ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish new high after pullback")

   
            elif self.state == "BREAK_OF_STRUCTURE":
                # structure broken downward -> restart pull-back search
                if lowest_body_price.loc[ts] < self.low_price:
                    
                    # Logic
                    slices = highest_body_price[self.low_price_ts : ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish break of structure")

                # new 3 red pull-back -> possible Fib leg
                elif self.check_pullback(greens, opens, closes, ts, trend="BU"):
                    
                    if highest_body_price.shift(2).loc[ts] > self.high_price: # fresh HH
                        
                        # Logic
                        if pos >= 3:   
                            window = highest_body_price.iloc[pos-3 : pos+1]
                            self.high_price     = window.max()
                            self.high_price_ts  = window.idxmax()
                        
                        self.fib_labels = self.calc_fibonacci_levels(self.low_price_ts, self.high_price_ts, trend="BU")
                        self.fib_dict = dict(self.fib_labels)
                        
                        # Debug
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, ts, "bullish waiting entry")

                    # price failed to beat prev high: treat as new pull-back
                    elif lowest_body_price.loc[ts] <= self.high_price:
                        
                        # Debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bullish back to pullback")
                        

            elif self.state == "WAITING_ENTRY":
                fib_618_bullish = self.fib_dict.get(0.618)
                fib_000_bullish = self.fib_dict.get(0.000)

                # price retraced to 0.618 – ready for bullish entry
                if fib_618_bullish and df['Low'].loc[ts] <= fib_618_bullish: # count the tip of the candlestick instead of the body
                
                    # Debug
                    self.entry_signal_ts = ts
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, ts, "bullish price retraced to 0.618")

                # price breaks above 0.000 (HH) before touching 0.618
                elif fib_618_bullish and highest_body_price.loc[ts] > fib_000_bullish:
                    
                    # Logic
                    slices = lowest_body_price[self.high_price_ts:ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish breaks above 0.000 before entry")
                
            elif self.state == "ENTRY_SIGNAL":
                fib_100_bullish = self.fib_dict.get(1.000)
                fib_000_bullish = self.fib_dict.get(0.000)

                # cancel if price dumps below 1.000 extension
                if lowest_body_price.loc[ts] <= fib_100_bullish:
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish cancels: price < 1.0")

                # Bullish Engulfing candle
                elif ((greens.loc[prev_ts] != 'G') and (greens.loc[ts] == 'G') and (opens.loc[ts] <= closes.loc[prev_ts]) and (closes.loc[ts] >= opens.loc[prev_ts])):# Enter Market 
                
                    current_slope_30 = df.loc[ts]['slope30']
                    current_slope_120 = df.loc[ts]['slope120']
                    
                    # In bullish trend, a crossover early signal emerges when EMA30 turns downward, while EMA120 is still trending upward or flat.
                    
                    if current_slope_30 < 0 and current_slope_120 > 0: # EMA_30 rising, but EMA_120 decreasing
                        return # no trade, wait for next entry
                    
                    self.engulfing_candle_ts = ts
                    
                    stop, take = self.calc_fibonacci_levels(self.low_price_ts, 
                    self.high_price_ts,trend="BU", entry_signal=self.entry_signal_ts,
                    engulfing_candle=self.engulfing_candle_ts)
                    
                    # calculate volume
                    entry_price = closes.loc[ts]
                    stop_loss_pips = abs(stop - entry_price) / self.pip_size
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips, self.pip_value_per_lot)
                    
                    self.evt.consoleLog(
                        f"DEBUG ticket – side:BUY, instrument: {self.instrument}, ab:{self.account_balance}, entry≈{entry_price:.5f} "
                        f"SL={stop:.5f} TP={take:.5f}"
                    )
                    # place buying order
                    self.send_entry_order(ts, trend="BU",
                              entry_price=entry_price,
                              stop=stop, take=take,
                              volume=min(10, lot_size))
                    # self.evt.consoleLog(f"Balance: {self.account_balance}, Risk%: {self.risk_percent}, Stop Loss (pips): {stop_loss_pips}, Pip Value: {self.pip_value_per_lot}, Lot Size: {lot_size}")
                    
                    # Debug
                    old_state = self.state
                    self.state = "PRE_ENTER_MARKET"
                    self.log_state_change(old_state, self.state, ts, "bullish entry: engulfing candle")
                    
                # invalidated if price breaks above HH first
                elif highest_body_price.loc[ts] >= fib_000_bullish:
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish breaks above HH first")

            elif self.state == "ENTER_MARKET":
                current_high = df['High'].loc[ts]
                current_low = df["Low"].loc[ts]
               
                if current_high >= self.take_profit_price :
                    self.evt.consoleLog(f"Price touched take_profit at {current_high:.5f} (take_profit: {self.take_profit_price:.5f}) – waiting for broker to close")
                elif current_low <= self.stop_loss_price :
                    self.evt.consoleLog(f"Price touched stop_loss at {current_low:.5f} (stop_loss: {self.stop_loss_price:.5f}) – waiting for broker to close")
        
        # ==============================================================
        # ========================  pullback()  ========================
        # ==============================================================     
        
    def check_pullback(self, greens, opens, closes, ts, trend):
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        
        if trend == "BE":
            # Check for 3, 4, 5, or 6 consecutive green candles.
            for window in [3, 4, 5, 6]:

                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                # Boundary check
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                # Check if all are green
                if all(greens.loc[ts_] == 'G' for ts_ in window_ts):
                    # Previous candle must be red
                    if greens.loc[prev_ts] == 'R':
                        # Any green candle closes above the open of previous red candle
                        if any(closes.loc[ts_] >= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False
        
        if trend == "BU":
            for window in [3,4,5,6]:
                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                if all(greens.loc[ts_]!= 'G' for ts_ in window_ts): # all previous not green candles 
                    if greens.loc[prev_ts] == 'G':
                        if any(closes.loc[ts_] <= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False

    def reset(self):
        self.state = "IDLE"
        self.low_price  = self.low_price_ts  =  None
        self.high_price = self.high_price_ts =  None
        self.fib_labels = []
        self.fib_dict = {} 
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.position_open = False
        self.open_volume = None


class InstrumentBar:
    
    """Bar builder + FSM for ONE instrument."""

    ALPHA_30  = 2 / 31
    ALPHA_120 = 2 / 121
    MIN_BARS  = 10
    MAX_BARS  = 225
    
    
    def __init__(self, symbol: str, evt):
        
        self.symbol    = symbol
        self.evt       = evt
        self.last_time = datetime(2010, 1, 1)
        self.fsm       = None
        
        NFLOAT = "float32"
        
        # ------------- 5-min buffer ---------------------------------
        self.raw_5m = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument"])
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "string",
                "Instrument": "object"}))
                
        # ------------- 15-min table ---------------------------------
        self.data = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument", "EMA_30", "EMA_120", "trend","slope30","slope120"])
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "string", "Instrument": "object",
                "EMA_30": NFLOAT, "EMA_120": NFLOAT,
                "trend": "string",
                "slope30": NFLOAT,
                "slope120": NFLOAT
            })
            .set_index("Datetime", drop=False))
        
    def update(self, snap: dict, available_balance: float):
        
        """
        snap = bd[symbol]  – single-instrument snapshot from on_bulkdatafeed
        """
        # -------------------- append 5-min bar ------------------------
        green = ("G" if snap["lastPrice"] > snap["openPrice"]
                 else "R" if snap["lastPrice"] < snap["openPrice"]
                 else "NO")
        
        bar5 = {
            "Datetime": pd.to_datetime(snap["timestamp"]),
            "Open":  snap["openPrice"],
            "High":  snap["highPrice"],
            "Low":   snap["lowPrice"],
            "Close": snap["lastPrice"],
            "Green": green,
            "Instrument": snap["instrument"],
        }
        
        self.raw_5m = pd.concat([self.raw_5m,
                                 pd.DataFrame([bar5])], ignore_index=True)
        
        # ------------------- finished 15-min? -------------------------
        ts = pd.to_datetime(snap["timestamp"])
        if ts < self.last_time + timedelta(minutes=15):
            return
        self.last_time = ts
        
        last3 = self.raw_5m.tail(3)
        if len(last3) < 3:
            return                          # need full 15-minute slice
        self.raw_5m = self.raw_5m.iloc[0:0]  # clear buffer
        
        self._aggregate_15m(last3)
        
        # -------------------- FSM ------------------------------------
        if self.fsm is None:
            self.fsm = BearishFSM(
                self.data.copy(), instrument=self.symbol, evt=self.evt,
                account_balance=np.float32(available_balance),
                pip_size=np.float32(0.0001), pip_value_per_lot=10,
                ts=self.last_time)
        else:
            self.fsm.df = self.data
            self.fsm.account_balance = available_balance
            self.fsm.update(self.last_time)
            
    def _aggregate_15m(self, last3):
        
        n   = len(self.data)
        op  = last3["Open"].iloc[0]
        cl  = last3["Close"].iloc[-1]
        bar = {
            "Datetime": last3["Datetime"].iloc[-1],
            "Open": op,
            "High": last3["High"].max(),
            "Low":  last3["Low"].min(),
            "Close": cl,
            "Green": "G" if cl > op else "R" if cl < op else "NO",
            "Instrument": self.symbol,
        }

        if n == 0:
            ema30 = ema120 = cl
            slope30 = slope120 = 0
        elif n < 30: 
            prev  = self.data.iloc[n - 1]
            ema30  = (1 - self.ALPHA_30)  * prev["EMA_30"]  + self.ALPHA_30  * cl
            ema120 = (1 - self.ALPHA_120) * prev["EMA_120"] + self.ALPHA_120 * cl
            slope30 = slope120 = 0
        else:
            prev  = self.data.iloc[n - 3] ###########################################  optimize the number of bars for calculating the slope: n-bar slope (e.g. 5) bars)
            ema30  = (1 - self.ALPHA_30)  * prev["EMA_30"]  + self.ALPHA_30  * cl
            ema120 = (1 - self.ALPHA_120) * prev["EMA_120"] + self.ALPHA_120 * cl
            slope30 = ema30 - prev["EMA_30"]
            slope120 = ema120 - prev["EMA_120"]
        
        bar["EMA_30"]  = ema30
        bar["EMA_120"] = ema120
        r = ema30 / ema120
        bar["trend"] = "BU" if r > 1.001 else "BE" if r < 0.999 else "NO" ################### change to detect slope
        bar['slope30'] = slope30
        bar['slope120'] = slope120

        self.data = pd.concat([self.data,
                               pd.DataFrame([bar]).set_index("Datetime",
                                                             drop=False)])

        

class AlgoEvent:

    def __init__(self):
        self.evt   = None
        self.books = {}          # symbol → InstrumentBar

    def start(self, mEvt):
        self.evt = AlgoAPI_Backtest.AlgoEvtHandler(self, mEvt)
        self.evt.start()
            
    def on_bulkdatafeed(self, isSync, bd, ab):
        
        balance = ab["availableBalance"]
        
        for sym, snap in bd.items():        # sym: AUDNZD, EURUSD, ... snap: {'instrument': 'AUDNZD', 'timestamp': datetime.datetime(1970, 1, 1, 0, 0), 'askPrice': -1, ...}
            if sym not in self.books:       # first sight → create book
                self.books[sym] = InstrumentBar(sym, self.evt)
            else: 
                self.books[sym].update(snap, balance)

    def on_orderfeed(self, of):
        
        sym = of.instrument
        if sym not in self.books:
            return                        # safety
        
        fsm = self.books[sym].fsm
        
        # -------------- order filled -----------------
        if of.openclose == "open" and of.status == "success":
            # Is this the order we are waiting for?
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.position_open      = True
                fsm.stop_loss_price    = fsm.pending["stop"]
                fsm.take_profit_price  = fsm.pending["take"]
                fsm.open_volume        = fsm.pending["volume"]
                fsm.enter_market_trade_side = fsm.pending["trend"]
    
                fsm.pending = None      # clear
                old_state   = fsm.state
                fsm.state   = "ENTER_MARKET"
                fsm.log_state_change(old_state, fsm.state, of.orderRef,
                                     "broker filled order")
        # -------------- order rejected ---------------
        elif of.openclose == "open" and of.status != "success":
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.pending = None
                old_state = fsm.state
                fsm.state = "ENTRY_SIGNAL"     # back to previous state
                fsm.log_state_change(old_state, fsm.state, of.orderRef,
                                     "order rejected")
                                     
        # -------------- position closed --------------
        if of.openclose == 'close' and of.status == 'success': # return 'success' if an order is opened/ closed successfully, or when a limit/stop order is filled
            fsm.reset()
    
    
    def on_newsdatafeed(self, nd):
        pass

    def on_weatherdatafeed(self, wd):
        pass
    
    def on_econsdatafeed(self, ed):
        pass
        
    def on_corpAnnouncement(self, ca):
        pass
    
    def on_dailyPLfeed(self, pl):
        pass

    def on_openPositionfeed(self, op, oo, uo):
        pass













#### Version 9 Improvements

1. calculates position size using both maximum risk per trade and margin/leverage limits, trading as large as possible while strictly staying within risk and capital constraints.

2. Dynamic and Accurate Pip Value Calculationfor different forex

3. Explore different strategy ( e.g 50-period Simple Moving Average (SMA))

In [ ]:
from AlgoAPI import AlgoAPIUtil, AlgoAPI_Backtest
import pandas as pd
import numpy as np
import mplfinance as mpf
from collections import deque
from datetime import datetime, timedelta
    
class BearishFSM: # Finite State Machine

        # ==============================================================
        # ========================  __init__ ===========================
        # ==============================================================
    def __init__(self, df, instrument, evt, account_balance=0, pip_size = 0.0001, pip_value_per_lot = 10, ts = None):
        
        self.df = df
        self.evt = evt
        self.account_balance = account_balance
        
        # Enter market flag
        self.position_open = False
        self.pending = None       # ← dict that holds the ticket waiting
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.open_volume = None
        
        self.state = "IDLE"
        self.low_price = None
        
        self.low_price_ts = None
        self.high_price = None
        self.high_price_ts = None

        self.fib_labels = []
        self.fib_dict = {} # not a must to initialize, just for good habit
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.trend_changing_ts = ts # do not reset this, always keep record of 
        
##################################################################################################################
####################################### optimize #################################################################
##################################################################################################################

        # do not reset the folloiwng parameters, test for optimal values
        self.risk_reward_ratio = 2.3
        self.atr_period = 14
        self.atr_multiplier = 1.3 # pip_size = atr_multiplier * atr_value, atr_multiplier = 2.0 means stop loss is 2 ATRs away
        self.risk_percent = 0.5
        self.dynamic_atr_window = True
        
##################################################################################################################
##################################################################################################################
##################################################################################################################
        # this is unique in every func call
        self.instrument = instrument
        self.pip_size = pip_size
        self.pip_value_per_lot = pip_value_per_lot
        
        # ==============================================================
        # ========================  calculate_log_state_change =========
        # ==============================================================
        
    def log_state_change(self, old_state, new_state, ts, label=None):
        if 'Datetime' in self.df.columns and ts in self.df.index:
            date_str = str(self.df.loc[ts, 'Datetime'])
        else:
            date_str = str(ts)
        msg = f" State: {old_state} → {new_state}"
        if label:
            msg += f" ({label})"
        # self.evt.consoleLog(msg)
        
        # ==============================================================
        # ========================  calculate_lot_size  ================
        # ==============================================================
        
    def calculate_lot_size(self, risk_percent, stop_loss_pips, pip_value_per_lot=None, max_leverage = 4.9): # base currency vs quote currency
        risk_amount = self.account_balance * (risk_percent / 100)
        lot_risk = risk_amount / (stop_loss_pips * pip_value_per_lot) # 
        
        if self.instrument.endswith('USD'):          # EURUSD, GBPUSD
            price = self.df["Close"].iloc[-1]
            notional_per_lot = 100_000 * price
        else:                                    # USDJPY
            notional_per_lot = 100_000
        
        lot_margin = (self.account_balance * max_leverage) / notional_per_lot
        
        lot_final = min(lot_risk, lot_margin) # all-in 
        
        self.evt.consoleLog(f"balance={self.account_balance}, risk%={risk_percent}, stop_loss_pips={stop_loss_pips}, pip_value_per_lot={pip_value_per_lot}, riskLots={lot_risk:.2f}  marginLots={lot_margin:.2f} final={lot_final:.2f}")
        
        return round(lot_final, 2)
        
        
        # ==============================================================
        # ========================  calc_fibonacci_levels  =============
        # ==============================================================
        
    def calc_fibonacci_levels(self, start_ts, end_ts, levels=None, trend="NO",
                         entry_signal=None, engulfing_candle=None,
                         R=None, atr_period=None, atr_multiplier=None,dynamic_atr_window = None):
                             
        if levels is None:
            levels = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]
            
        if dynamic_atr_window is None:
            dynamic_atr_window = self.dynamic_atr_window
            
        if atr_multiplier is None:
            atr_multiplier = self.atr_multiplier
        
        if R is None:
            R = self.risk_reward_ratio
        
        
        df = self.df
        """
        start_idx =  df.index.get_loc(start_ts)
        end_idx = df.index.get_loc(end_ts)
        
        start_ts = df.index[start_idx] # still a timestamp
        end_ts = df.index[end_idx]
        """
        window = df.loc[start_ts : end_ts]
                            
        min_price = window['Low'].min()
        max_price = window['High'].max()
    
        fib_labels = []
        for level in levels:
            if trend == "BE": # bearish, sell
                y = min_price + (max_price - min_price) * level
            else:
                y = max_price - (max_price - min_price) * level
            fib_labels.append((level, y))
        
        if engulfing_candle is None:
            return fib_labels
        
        if atr_period is None:

            if dynamic_atr_window == False:
                # Fixed atr period
                atr_period = self.atr_period
            else: 
                # dynamic atr period
                start_idx = df.index.get_loc(start_ts)
                enter_idx = df.index.get_loc(engulfing_candle)
            
                dynamic_period = enter_idx - start_idx
                atr_period = max(10, min(dynamic_period, 50))
        
        # Normal ATR
        min_start_idx = max(0, df.index.get_loc(start_ts) - 10)
        min_start_ts = df.index[min_start_idx]
        sl = df.loc[min_start_ts : engulfing_candle].copy()
        
        sl['H-L']  = sl['High'] - sl['Low']
        sl['H-PC'] = (sl['High'] - sl['Close'].shift(1)).abs()
        sl['L-PC'] = (sl['Low']  - sl['Close'].shift(1)).abs()
        sl['TR']    = sl[['H-L', 'H-PC', 'L-PC']].max(axis=1)
        
        # ATR: rolling mean of TR
        sl['ATR'] = sl['TR'].rolling(window=atr_period).mean()
         
        # Retrieve ATR value at the engulfing candle index
        atr_value = sl['ATR'].iloc[-1]
        pip_size = atr_multiplier * atr_value
        
        # Retrieve the close price of the engulfing candle
        candle = self.df.loc[engulfing_candle]
        close_price = candle['Close']
        
        # Initialize stop loss and take profit prices
        stop_loss_price = None
        take_profit_price = None
    
        if trend == "BE" :
            stop_loss_price = close_price + pip_size
            take_profit_price = close_price - R * pip_size
        else:  # bullish
            stop_loss_price = close_price - pip_size
            take_profit_price = close_price + R * pip_size
    
        return stop_loss_price, take_profit_price
        
    # ==============================================================
    # ======================== send_entry_order  =============
    # ==============================================================
    
    def send_entry_order(self, ts, trend, entry_price,
                         stop, take, volume):

        # remember what we just asked the broker to do
        self.pending = dict(
            orderRef = ts,         # we use the timestamp as reference
            trend    = trend,      # "BU" / "BE"
            stop     = stop,
            take     = take,
            volume   = volume
        )

        order = AlgoAPIUtil.OrderObject()
        order.instrument   = self.instrument
        order.orderRef     = ts
        order.openclose    = "open"
        order.buysell      = 1 if trend == "BU" else -1
        order.ordertype    = 0          # market
        order.volume       = volume
        order.stopLossLevel= stop
        order.takeProfitLevel = take
        self.evt.sendOrder(order)

        # NOTE: do NOT set state / flags here;
        #       wait for on_orderfeed()
    
    # -------------------------------------------------------------
    # monitor_exit()  – runs while we are in the trade
    # -------------------------------------------------------------
    def monitor_exit(self, ts):
        """
        Called every bar only while self.position_open is True.
        Produces the same log lines you had in your old ENTER_MARKET
        branch.  If you additionally want to *close* the trade,
        send a close-order here.
        """
        if self.take_profit_price is None or self.stop_loss_price is None:
            return      # safety – nothing to do

        high = self.df["High"].loc[ts]
        low  = self.df["Low"].loc[ts]

        if self.enter_market_trade_side == "BU":          # long trade
            if high >= self.take_profit_price:
                self.evt.consoleLog(
                    f"Price touched take_profit at {high:.5f} "
                    f"(take_profit: {self.take_profit_price:.5f}) – "
                    "waiting for broker to close"
                )
            elif low <= self.stop_loss_price:
                self.evt.consoleLog(
                    f"Price touched stop_loss at {low:.5f} "
                    f"(stop_loss: {self.stop_loss_price:.5f}) – "
                    "waiting for broker to close"
                )

        else:                                             # short trade
            if low <= self.take_profit_price:
                self.evt.consoleLog(
                    f"Price touched take_profit at {low:.5f} "
                    f"(take_profit: {self.take_profit_price:.5f}) – "
                    "waiting for broker to close"
                )
            elif high >= self.stop_loss_price:
                self.evt.consoleLog(
                    f"Price touched stop_loss at {high:.5f} "
                    f"(stop_loss: {self.stop_loss_price:.5f}) – "
                    "waiting for broker to close"
                )
                
    # ==============================================================
    # ========================  update()  =============
    # ==============================================================
    
    def update(self, ts: pd.Timestamp):
        
        # 1️⃣ When a position is open we do only exit-logic and leave
        #    the rest of the gigantic state-machine untouched.
        #    (So IDLE / PULLBACK_DETECTED / … cannot fire again.)
        if self.position_open:
            self.monitor_exit(ts)        # <-- will emit console logs
            return                       # <-- early exit
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        prev_ts = df.index[pos - 1] # still a timestamp
        
        if df.index.get_loc(ts) < 3: # Get current timestamp location
            return
        
        #--- 2. prepare frequently-used columns (all remain Series!) ------
        trend   = df["trend"]               # string
        greens  = df["Green"]               # string
        opens   = df["Open"]
        closes  = df["Close"]
        
        lowest_body_price = opens.where(greens == 'G', closes) # if greens == 'G', use values in open, else use values in closes 
        highest_body_price = closes.where(greens == 'G', opens) # the index of this series keeps the same Datetime index as the original df
        
        # selects the row in df where the index = ts
        curr   = df.loc[ts]
        prev   = df.shift(1).loc[ts]       # one-bar shift gives previous values
        
        current_trend = curr['trend']
        prev_trend    = prev['trend']

        if current_trend != prev_trend : 
            self.trend_changing_ts = ts
            if self.state != "ENTER_MARKET":
                self.reset()

        if (current_trend == "BE" and self.enter_market_trade_side == None):
            if self.state == "IDLE":
                if self.check_pullback(greens, opens, closes, ts, trend="BE"): # pullback should eat and cover the previous red candle
                    # logic
                    slices = lowest_body_price.loc[self.trend_changing_ts:ts] # Slicing with .loc[start:end] in pandas includes both endpoints
                    self.low_price = slices.min()
                    self.low_price_ts  = slices.idxmin()   # Return index(Datetime) of first occurrence of minimum over requested axis.
                    # debug
                    old_state = self.state  
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish pullback detected")
                    

            elif self.state == "PULLBACK_DETECTED":
                if lowest_body_price.loc[ts] < self.low_price:
                    # logic
                    slices = highest_body_price.loc[self.low_price_ts:ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    # debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish new low after pullback")

                    
            elif self.state == "BREAK_OF_STRUCTURE":
                # If price breaks the pev high (LH)
                if highest_body_price.loc[ts] > self.high_price:
                    
                    # logic
                    slices = lowest_body_price.loc[self.high_price_ts: ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish break of structure")

                elif self.check_pullback(greens, opens, closes, ts, trend="BE"):
                    
                    if lowest_body_price.shift(2).loc[ts] < self.low_price: #### use 2 timeframe before current ts , can incorporate to 3,4,5 or 6
                    
                        # Logic: Low price for confirming the next Lower high 
                        if pos >= 3:   
                            window = lowest_body_price.iloc[pos-3 : pos+1]
                            self.low_price     = window.min()
                            self.low_price_ts  = window.idxmin()

                        self.fib_labels = self.calc_fibonacci_levels(self.high_price_ts, self.low_price_ts, trend="BE")
                        self.fib_dict = dict(self.fib_labels)
                        
                        # debug
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, ts, "bearish waiting entry")

                    elif highest_body_price.loc[ts] >= self.low_price:
                        
                        # debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bearish back to pullback")

            elif self.state == "WAITING_ENTRY":
                
                fib_618_price = self.fib_dict.get(0.618)
                fib_000_price = self.fib_dict.get(0.000)
                
                # price rebound back to 0.618
                if fib_618_price and df["High"].loc[ts] >= fib_618_price: # reference the "High" price at the exact timestamp
                
                    # debug
                    self.entry_signal_ts = ts
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, ts, "bearish price rebound to 0.618")

                elif fib_618_price and lowest_body_price.loc[ts] <= fib_000_price: # price lower than 0.000
                
                    # Logic: break of structure, set new high point
                    slices = highest_body_price.loc[self.low_price_ts: ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish breaks 0.000 before entry")
            
            elif self.state == "ENTRY_SIGNAL": 
                fib_100_price = self.fib_dict.get(1.000)
                fib_000_price = self.fib_dict.get(0.000)
                

                if highest_body_price.loc[ts] >= fib_100_price: # no matter what, if the candle pass 1.00, we do not trade
                
                    # debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish cancels: price > 1.0")

                # Logic: Bearish Engulfing candle  
                elif (greens.loc[prev_ts] == 'G') and (greens.loc[ts] != 'G') and opens.loc[ts] > closes.loc[prev_ts] and closes.loc[ts] < opens.loc[prev_ts]: # enter market
                    """
                    current_slope_30 = df.loc[ts]['slope30']
                    current_slope_120 = df.loc[ts]['slope120']
                    
                    # In bearish trend, a crossover early signal emerges when EMA30 turns upward, while EMA120 is still trending downward or flat.
                    
                    if current_slope_30 > 0 and current_slope_120 < 0: # EMA_30 rising, but EMA_120 decreasing
                        return # no trade, wait for next entry
                    """
                    # Debug
                    self.engulfing_candle_ts = ts
                    
                    stop, take = self.calc_fibonacci_levels(self.high_price_ts, 
                            self.low_price_ts, trend="BE", entry_signal=self.entry_signal_ts, engulfing_candle=self.engulfing_candle_ts
                            )
                    
                    # Logic to calculate lot_size
                    entry_price = closes.loc[ts]
                    stop_loss_pips = abs(stop - entry_price) / self.pip_size
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips, self.pip_value_per_lot)
                    
                    # Log for debug
                    self.evt.consoleLog(
                        f"DEBUG ticket – side:SELL, instrument:{self.instrument},ab:{self.account_balance}, entry≈{entry_price:.5f} "
                        f"SL={stop:.5f} TP={take:.5f}"
                    )
                    
                    # self.evt.consoleLog(f"Balance: {self.account_balance}, Risk%: {self.risk_percent}, Stop Loss (pips): {stop_loss_pips}, Pip Value: {self.pip_value_per_lot}, Lot Size: {lot_size}")
                    
                    # place selling order
                    self.send_entry_order(ts, trend="BE",
                                entry_price=entry_price,
                                stop=stop, take=take,
                                volume=lot_size)
                    
                    # Debug
                    old_state = self.state
                    self.state = "PRE_ENTER_MARKET"
                    self.log_state_change(old_state, self.state, ts, "bearish entry: engulfing candle")
                    
                elif lowest_body_price.loc[ts] <= fib_000_price: # If candle breaks 0.00 before there is a Bearish ENGULFING candle
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish breaks 0.000 before engulfing")
            
        # ==============================================================
        # ========================  BULLISH  ============================
        # ==============================================================

        if (current_trend == "BU" and self.enter_market_trade_side == None):
            if self.state == "IDLE":    
                if self.check_pullback(greens, opens, closes, ts, trend="BU"):
                    
                    # Logic
                    slices = highest_body_price[self.trend_changing_ts:ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish pullback detected")

            elif self.state == "PULLBACK_DETECTED":
                if highest_body_price.loc[ts] > self.high_price:
                    
                    # Logic
                    slices = lowest_body_price[self.high_price_ts: ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish new high after pullback")

   
            elif self.state == "BREAK_OF_STRUCTURE":
                # structure broken downward -> restart pull-back search
                if lowest_body_price.loc[ts] < self.low_price:
                    
                    # Logic
                    slices = highest_body_price[self.low_price_ts : ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish break of structure")

                # new 3 red pull-back -> possible Fib leg
                elif self.check_pullback(greens, opens, closes, ts, trend="BU"):
                    
                    if highest_body_price.shift(2).loc[ts] > self.high_price: # fresh HH
                        
                        # Logic
                        if pos >= 3:   
                            window = highest_body_price.iloc[pos-3 : pos+1]
                            self.high_price     = window.max()
                            self.high_price_ts  = window.idxmax()
                        
                        self.fib_labels = self.calc_fibonacci_levels(self.low_price_ts, self.high_price_ts, trend="BU")
                        self.fib_dict = dict(self.fib_labels)
                        
                        # Debug
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, ts, "bullish waiting entry")

                    # price failed to beat prev high: treat as new pull-back
                    elif lowest_body_price.loc[ts] <= self.high_price:
                        
                        # Debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bullish back to pullback")
                        

            elif self.state == "WAITING_ENTRY":
                fib_618_bullish = self.fib_dict.get(0.618)
                fib_000_bullish = self.fib_dict.get(0.000)

                # price retraced to 0.618 – ready for bullish entry
                if fib_618_bullish and df['Low'].loc[ts] <= fib_618_bullish: # count the tip of the candlestick instead of the body
                
                    # Debug
                    self.entry_signal_ts = ts
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, ts, "bullish price retraced to 0.618")

                # price breaks above 0.000 (HH) before touching 0.618
                elif fib_618_bullish and highest_body_price.loc[ts] > fib_000_bullish:
                    
                    # Logic
                    slices = lowest_body_price[self.high_price_ts:ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish breaks above 0.000 before entry")
                
            elif self.state == "ENTRY_SIGNAL":
                fib_100_bullish = self.fib_dict.get(1.000)
                fib_000_bullish = self.fib_dict.get(0.000)

                # cancel if price dumps below 1.000 extension
                if lowest_body_price.loc[ts] <= fib_100_bullish:
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish cancels: price < 1.0")

                # Bullish Engulfing candle
                elif ((greens.loc[prev_ts] != 'G') and (greens.loc[ts] == 'G') and (opens.loc[ts] <= closes.loc[prev_ts]) and (closes.loc[ts] >= opens.loc[prev_ts])):# Enter Market 
                    """
                    current_slope_30 = df.loc[ts]['slope30']
                    current_slope_120 = df.loc[ts]['slope120']
                    
                    # In bullish trend, a crossover early signal emerges when EMA30 turns downward, while EMA120 is still trending upward or flat.
                    
                    if current_slope_30 < 0 and current_slope_120 > 0: # EMA_30 rising, but EMA_120 decreasing
                        return # no trade, wait for next entry
                    """      
                    self.engulfing_candle_ts = ts
                    
                    stop, take = self.calc_fibonacci_levels(self.low_price_ts, 
                    self.high_price_ts,trend="BU", entry_signal=self.entry_signal_ts,
                    engulfing_candle=self.engulfing_candle_ts)
                    
                    # calculate volume
                    entry_price = closes.loc[ts]
                    stop_loss_pips = abs(stop - entry_price) / self.pip_size
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips, self.pip_value_per_lot)
                    
                    self.evt.consoleLog(
                        f"DEBUG ticket – side:BUY, instrument: {self.instrument}, ab:{self.account_balance}, entry≈{entry_price:.5f} "
                        f"SL={stop:.5f} TP={take:.5f}"
                    )
                    # place buying order
                    self.send_entry_order(ts, trend="BU",
                              entry_price=entry_price,
                              stop=stop, take=take,
                              volume=lot_size)
                    # self.evt.consoleLog(f"Balance: {self.account_balance}, Risk%: {self.risk_percent}, Stop Loss (pips): {stop_loss_pips}, Pip Value: {self.pip_value_per_lot}, Lot Size: {lot_size}")
                    
                    # Debug
                    old_state = self.state
                    self.state = "PRE_ENTER_MARKET"
                    self.log_state_change(old_state, self.state, ts, "bullish entry: engulfing candle")
                    
                # invalidated if price breaks above HH first
                elif highest_body_price.loc[ts] >= fib_000_bullish:
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish breaks above HH first")

            elif self.state == "ENTER_MARKET":
                current_high = df['High'].loc[ts]
                current_low = df["Low"].loc[ts]
               
                if current_high >= self.take_profit_price :
                    self.evt.consoleLog(f"Price touched take_profit at {current_high:.5f} (take_profit: {self.take_profit_price:.5f}) – waiting for broker to close")
                elif current_low <= self.stop_loss_price :
                    self.evt.consoleLog(f"Price touched stop_loss at {current_low:.5f} (stop_loss: {self.stop_loss_price:.5f}) – waiting for broker to close")
        
        # ==============================================================
        # ========================  pullback()  ========================
        # ==============================================================     
        
    def check_pullback(self, greens, opens, closes, ts, trend):
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        
        if trend == "BE":
            # Check for 3, 4, 5, or 6 consecutive green candles.
            for window in [3, 4, 5, 6]:

                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                # Boundary check
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                # Check if all are green
                if all(greens.loc[ts_] == 'G' for ts_ in window_ts):
                    # Previous candle must be red
                    if greens.loc[prev_ts] == 'R':
                        # Any green candle closes above the open of previous red candle
                        if any(closes.loc[ts_] >= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False
        
        if trend == "BU":
            for window in [3,4,5,6]:
                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                if all(greens.loc[ts_]!= 'G' for ts_ in window_ts): # all previous not green candles 
                    if greens.loc[prev_ts] == 'G':
                        if any(closes.loc[ts_] <= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False

    def reset(self):
        self.state = "IDLE"
        self.low_price  = self.low_price_ts  =  None
        self.high_price = self.high_price_ts =  None
        self.fib_labels = []
        self.fib_dict = {} 
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.position_open = False
        self.open_volume = None


class InstrumentBar:
    
    """Bar builder + FSM for ONE instrument."""
    
    ALPHA_30  = 2 / 31
    ALPHA_120 = 2 / 121
    MIN_BARS  = 10 
    MAX_BARS  = 700
    
    def __init__(self, symbol: str, evt):
        
        self.symbol    = symbol
        self.evt       = evt
        self.last_time = datetime(2010, 1, 1)
        self.fsm       = None
        
        NFLOAT = "float32"
        
        # ------------- 5-min buffer ---------------------------------
        self.raw_5m = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument"])
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "category",
                "Instrument": "object"}))
                
        # ------------- 15-min table ---------------------------------
        self.data = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument", "EMA_30", "EMA_120", "trend","slope30","slope120", 'SMA_50'])
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "category", "Instrument": "object",
                "EMA_30": NFLOAT, "EMA_120": NFLOAT,
                "trend": "category",
                "slope30": NFLOAT,
                "slope120": NFLOAT,
                'SMA_50': NFLOAT
            })
            .set_index("Datetime", drop=False))
        
    def update(self, snap: dict, available_balance: float):
        
        """
        snap = bd[symbol]  – single-instrument snapshot from on_bulkdatafeed
        """
        # -------------------- append 5-min bar ------------------------
        green = ("G" if snap["lastPrice"] > snap["openPrice"]
                 else "R" if snap["lastPrice"] < snap["openPrice"]
                 else "NO")
        
        bar5 = {
            "Datetime": pd.to_datetime(snap["timestamp"]),
            "Open":  snap["openPrice"],
            "High":  snap["highPrice"],
            "Low":   snap["lowPrice"],
            "Close": snap["lastPrice"],
            "Green": green,
            "Instrument": snap["instrument"],
        }
        
        self.raw_5m = pd.concat([self.raw_5m,
                                 pd.DataFrame([bar5])], ignore_index=True)
        
        
        # ------------------- finished 15-min? -------------------------
        ts = pd.to_datetime(snap["timestamp"])
        if ts < self.last_time + timedelta(minutes=15):
            return
        self.last_time = ts
        
        last3 = self.raw_5m.tail(3)
        if len(last3) < 3:
            return    # need full 15-minute slice
        
        self.raw_5m = self.raw_5m.iloc[0:0]  # clear buffer
        
        self._aggregate_15m(last3)
        
        
        if len(self.data) < self.MIN_BARS:
            return
            
        
        if len(self.data) > self.MAX_BARS:
            self.data = self.data.iloc[-self.MAX_BARS:]
            
        # -------------------- FSM ------------------------------------
        if self.fsm is None:
            if self.symbol == "USDJPY":
                pip_size = 0.01
                price    = self.data["Close"].iloc[-1]
                pip_value_per_lot = 1000 / price        # 1000 JPY ÷ JPYUSD
            else:                                       # EURUSD / GBPUSD
                pip_size = 0.0001
                pip_value_per_lot = 10
                
            self.fsm = BearishFSM(
                self.data, instrument=self.symbol, evt=self.evt,
                account_balance=np.float32(available_balance),
                pip_size=pip_size, pip_value_per_lot=pip_value_per_lot,
                ts=self.last_time)
        else:
            
            self.fsm.df = self.data
            self.fsm.account_balance = available_balance 
            self.fsm.update(self.last_time)
            
    def _aggregate_15m(self, last3):
        
        
        
        n   = len(self.data)
        op  = last3["Open"].iloc[0]
        cl  = last3["Close"].iloc[-1]
        bar = {
            "Datetime": last3["Datetime"].iloc[-1],
            "Open": op,
            "High": last3["High"].max(),
            "Low":  last3["Low"].min(),
            "Close": cl,
            "Green": "G" if cl > op else "R" if cl < op else "NO",
            "Instrument": self.symbol,
        }

        if n == 0:
            ema30 = ema120 = cl
            slope30 = slope120 = 0
        elif n < 30: 
            prev  = self.data.iloc[n - 1]
            ema30  = (1 - self.ALPHA_30)  * prev["EMA_30"]  + self.ALPHA_30  * cl
            ema120 = (1 - self.ALPHA_120) * prev["EMA_120"] + self.ALPHA_120 * cl
            slope30 = slope120 = 0
        else:
            prev  = self.data.iloc[n - 3] ###########################################  optimize the number of bars for calculating the slope: n-bar slope (e.g. 5) bars)
            ema30  = (1 - self.ALPHA_30)  * prev["EMA_30"]  + self.ALPHA_30  * cl
            ema120 = (1 - self.ALPHA_120) * prev["EMA_120"] + self.ALPHA_120 * cl
            slope30 = ema30 - prev["EMA_30"]
            slope120 = ema120 - prev["EMA_120"]
        
        bar["EMA_30"]  = ema30
        bar["EMA_120"] = ema120
        r = ema30 / ema120
        bar["trend"] = "BU" if r > 1.001 else "BE" if r < 0.999 else "NO" ################### change to detect slope
        bar['slope30'] = slope30
        bar['slope120'] = slope120
        

        self.data = pd.concat([self.data, pd.DataFrame([bar]).set_index("Datetime",drop=False)])
        self.data["SMA_50"] = self.data["Close"].rolling(window=50).mean()
        

class AlgoEvent:

    def __init__(self):
        self.evt   = None
        self.books = {}          # symbol → InstrumentBar

    def start(self, mEvt):
        self.evt = AlgoAPI_Backtest.AlgoEvtHandler(self, mEvt)
        self.evt.start()
            
    def on_bulkdatafeed(self, isSync, bd, ab):
        
        balance = ab["availableBalance"]
        
        for sym, snap in bd.items():        # sym: AUDNZD, EURUSD, ... snap: {'instrument': 'AUDNZD', 'timestamp': datetime.datetime(1970, 1, 1, 0, 0), 'askPrice': -1, ...}
            if sym not in self.books:       # first sight → create book
                self.books[sym] = InstrumentBar(sym, self.evt)
            else: 
                self.books[sym].update(snap, balance)

    def on_orderfeed(self, of):
        
        sym = of.instrument
        if sym not in self.books:
            return                        # safety
        
        fsm = self.books[sym].fsm
        
        # -------------- order filled -----------------
        if of.openclose == "open" and of.status == "success":
            # Is this the order we are waiting for?
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.position_open      = True
                fsm.stop_loss_price    = fsm.pending["stop"]
                fsm.take_profit_price  = fsm.pending["take"]
                fsm.open_volume        = fsm.pending["volume"]
                fsm.enter_market_trade_side = fsm.pending["trend"]
    
                fsm.pending = None      # clear
                old_state   = fsm.state
                fsm.state   = "ENTER_MARKET"
                fsm.log_state_change(old_state, fsm.state, of.orderRef,
                                     "broker filled order")
        # -------------- order rejected ---------------
        elif of.openclose == "open" and of.status != "success":
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.pending = None
                old_state = fsm.state
                fsm.state = "ENTRY_SIGNAL"     # back to previous state
                fsm.log_state_change(old_state, fsm.state, of.orderRef,
                                     "order rejected")
                                     
        # -------------- position closed --------------
        if of.openclose == 'close' and of.status == 'success': # return 'success' if an order is opened/ closed successfully, or when a limit/stop order is filled
            fsm.reset()
    
    
    def on_newsdatafeed(self, nd):
        pass

    def on_weatherdatafeed(self, wd):
        pass
    
    def on_econsdatafeed(self, ed):
        pass
        
    def on_corpAnnouncement(self, ca):
        pass
    
    def on_dailyPLfeed(self, pl):
        pass

    def on_openPositionfeed(self, op, oo, uo):
        pass

#### Version 10 Changes

1. Instead of using EMA30 vs EMA120, we try EMA10 vs SMA50 (doesn't improve the performance)

In [ ]:
from AlgoAPI import AlgoAPIUtil, AlgoAPI_Backtest
import pandas as pd
import numpy as np
import mplfinance as mpf
from collections import deque
from datetime import datetime, timedelta
    
class BearishFSM: # Finite State Machine

        # ==============================================================
        # ========================  __init__ ===========================
        # ==============================================================
    def __init__(self, df, instrument, evt, account_balance=0, pip_size = 0.0001, pip_value_per_lot = 10, ts = None):
        
        self.df = df
        self.evt = evt
        self.account_balance = account_balance
        
        # Enter market flag
        self.position_open = False
        self.pending = None       # ← dict that holds the ticket waiting
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.open_volume = None
        
        self.state = "IDLE"
        self.low_price = None
        
        self.low_price_ts = None
        self.high_price = None
        self.high_price_ts = None

        self.fib_labels = []
        self.fib_dict = {} # not a must to initialize, just for good habit
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.trend_changing_ts = ts # do not reset this, always keep record of 
        
##################################################################################################################
####################################### optimize #################################################################
##################################################################################################################

        # do not reset the folloiwng parameters, test for optimal values
        self.risk_reward_ratio = 2.0
        self.atr_period = 14
        self.atr_multiplier = 1.9 # pip_size = atr_multiplier * atr_value, atr_multiplier = 2.0 means stop loss is 2 ATRs away
        self.risk_percent = 0.5
        self.dynamic_atr_window = True
        self.trend_ema10_sma50 = True # If True, the trend will be determined by EMA10 and SMA50, if False, determined by EMA30 and EMA120
        
##################################################################################################################
##################################################################################################################
##################################################################################################################
        # this is unique in every func call
        self.instrument = instrument
        self.pip_size = pip_size
        self.pip_value_per_lot = pip_value_per_lot
        
        # ==============================================================
        # ========================  calculate_log_state_change =========
        # ==============================================================
        
    def log_state_change(self, old_state, new_state, ts, label=None):
        if 'Datetime' in self.df.columns and ts in self.df.index:
            date_str = str(self.df.loc[ts, 'Datetime'])
        else:
            date_str = str(ts)
        msg = f" State: {old_state} → {new_state}"
        if label:
            msg += f" ({label})"
        # self.evt.consoleLog(msg)
        
        # ==============================================================
        # ========================  calculate_lot_size  ================
        # ==============================================================
        
    def calculate_lot_size(self, risk_percent, stop_loss_pips, pip_value_per_lot=None, max_leverage = 4.9): # base currency vs quote currency
        risk_amount = self.account_balance * (risk_percent / 100)
        lot_risk = risk_amount / (stop_loss_pips * pip_value_per_lot) # 
        
        if self.instrument.endswith('USD'):          # EURUSD, GBPUSD
            price = self.df["Close"].iloc[-1]
            notional_per_lot = 100_000 * price
        else:                                    # USDJPY
            notional_per_lot = 100_000
        
        lot_margin = (self.account_balance * max_leverage) / notional_per_lot
        
        lot_final = min(lot_risk, lot_margin) # all-in 
        
        self.evt.consoleLog(f"symbol={self.instrument}. balance={self.account_balance}, risk%={risk_percent}, stop_loss_pips={stop_loss_pips}, pip_value_per_lot={pip_value_per_lot}, riskLots={lot_risk:.2f}  marginLots={lot_margin:.2f} final={lot_final:.2f}")
        
        return round(lot_final, 2)
        
        
        # ==============================================================
        # ========================  calc_fibonacci_levels  =============
        # ==============================================================
        
    def calc_fibonacci_levels(self, start_ts, end_ts, levels=None, trend="NO",
                         entry_signal=None, engulfing_candle=None,
                         R=None, atr_period=None, atr_multiplier=None,dynamic_atr_window = None):
                             
        if levels is None:
            levels = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]
            
        if dynamic_atr_window is None:
            dynamic_atr_window = self.dynamic_atr_window
            
        if atr_multiplier is None:
            atr_multiplier = self.atr_multiplier
        
        if R is None:
            R = self.risk_reward_ratio
        
        
        df = self.df
        """
        start_idx =  df.index.get_loc(start_ts)
        end_idx = df.index.get_loc(end_ts)
        
        start_ts = df.index[start_idx] # still a timestamp
        end_ts = df.index[end_idx]
        """
        window = df.loc[start_ts : end_ts]
                            
        min_price = window['Low'].min()
        max_price = window['High'].max()
    
        fib_labels = []
        for level in levels:
            if trend == "BE": # bearish, sell
                y = min_price + (max_price - min_price) * level
            else:
                y = max_price - (max_price - min_price) * level
            fib_labels.append((level, y))
        
        if engulfing_candle is None:
            return fib_labels
        
        if atr_period is None:

            if dynamic_atr_window == False:
                # Fixed atr period
                atr_period = self.atr_period
            else: 
                # dynamic atr period
                start_idx = df.index.get_loc(start_ts)
                enter_idx = df.index.get_loc(engulfing_candle)
            
                dynamic_period = enter_idx - start_idx
                atr_period = max(10, min(dynamic_period, 50))
        
        # Normal ATR
        min_start_idx = max(0, df.index.get_loc(start_ts) - 10)
        min_start_ts = df.index[min_start_idx]
        sl = df.loc[min_start_ts : engulfing_candle].copy()
        
        sl['H-L']  = sl['High'] - sl['Low']
        sl['H-PC'] = (sl['High'] - sl['Close'].shift(1)).abs()
        sl['L-PC'] = (sl['Low']  - sl['Close'].shift(1)).abs()
        sl['TR']    = sl[['H-L', 'H-PC', 'L-PC']].max(axis=1)
        
        # ATR: rolling mean of TR
        sl['ATR'] = sl['TR'].rolling(window=atr_period).mean()
         
        # Retrieve ATR value at the engulfing candle index
        atr_value = sl['ATR'].iloc[-1]
        pip_size = atr_multiplier * atr_value
        
        # Retrieve the close price of the engulfing candle
        candle = self.df.loc[engulfing_candle]
        close_price = candle['Close']
        
        # Initialize stop loss and take profit prices
        stop_loss_price = None
        take_profit_price = None
    
        if trend == "BE" :
            stop_loss_price = close_price + pip_size
            take_profit_price = close_price - R * pip_size
        else:  # bullish
            stop_loss_price = close_price - pip_size
            take_profit_price = close_price + R * pip_size
    
        return stop_loss_price, take_profit_price
        
    # ==============================================================
    # ======================== send_entry_order  =============
    # ==============================================================
    
    def send_entry_order(self, ts, trend, entry_price,
                         stop, take, volume):

        # remember what we just asked the broker to do
        self.pending = dict(
            orderRef = ts,         # we use the timestamp as reference
            trend    = trend,      # "BU" / "BE"
            stop     = stop,
            take     = take,
            volume   = volume
        )

        order = AlgoAPIUtil.OrderObject()
        order.instrument   = self.instrument
        order.orderRef     = ts
        order.openclose    = "open"
        order.buysell      = 1 if trend == "BU" else -1
        order.ordertype    = 0          # market
        order.volume       = volume
        order.stopLossLevel= stop
        order.takeProfitLevel = take
        self.evt.sendOrder(order)

        # NOTE: do NOT set state / flags here;
        #       wait for on_orderfeed()
    
    # -------------------------------------------------------------
    # monitor_exit()  – runs while we are in the trade
    # -------------------------------------------------------------
    def monitor_exit(self, ts):
        """
        Called every bar only while self.position_open is True.
        Produces the same log lines you had in your old ENTER_MARKET
        branch.  If you additionally want to *close* the trade,
        send a close-order here.
        """
        if self.take_profit_price is None or self.stop_loss_price is None:
            return      # safety – nothing to do

        high = self.df["High"].loc[ts]
        low  = self.df["Low"].loc[ts]

        if self.enter_market_trade_side == "BU":          # long trade
            if high >= self.take_profit_price:
                self.evt.consoleLog(
                    f"Price touched take_profit at {high:.5f} "
                    f"(take_profit: {self.take_profit_price:.5f}) – "
                    "waiting for broker to close"
                )
            elif low <= self.stop_loss_price:
                self.evt.consoleLog(
                    f"Price touched stop_loss at {low:.5f} "
                    f"(stop_loss: {self.stop_loss_price:.5f}) – "
                    "waiting for broker to close"
                )

        else:                                             # short trade
            if low <= self.take_profit_price:
                self.evt.consoleLog(
                    f"Price touched take_profit at {low:.5f} "
                    f"(take_profit: {self.take_profit_price:.5f}) – "
                    "waiting for broker to close"
                )
            elif high >= self.stop_loss_price:
                self.evt.consoleLog(
                    f"Price touched stop_loss at {high:.5f} "
                    f"(stop_loss: {self.stop_loss_price:.5f}) – "
                    "waiting for broker to close"
                )
                
    # ==============================================================
    # ========================  update()  =============
    # ==============================================================
    
    def update(self, ts: pd.Timestamp):
        
        # 1️⃣ When a position is open we do only exit-logic and leave
        #    the rest of the gigantic state-machine untouched.
        #    (So IDLE / PULLBACK_DETECTED / … cannot fire again.)
        if self.position_open:
            self.monitor_exit(ts)        # <-- will emit console logs
            return                       # <-- early exit
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        prev_ts = df.index[pos - 1] # still a timestamp
        
        if df.index.get_loc(ts) < 3: # Get current timestamp location
            return
        
        #--- 2. prepare frequently-used columns (all remain Series!) ------
        trend   = df["trend"]               # string
        trend_SMA = df['trend_SMA']
        greens  = df["Green"]               # string
        opens   = df["Open"]
        closes  = df["Close"]
        
        lowest_body_price = opens.where(greens == 'G', closes) # if greens == 'G', use values in open, else use values in closes 
        highest_body_price = closes.where(greens == 'G', opens) # the index of this series keeps the same Datetime index as the original df
        
        # selects the row in df where the index = ts
        curr   = df.loc[ts]
        prev   = df.shift(1).loc[ts]       # one-bar shift gives previous values
        
        if self.trend_ema10_sma50 == True:  ###########  use SMA50 and EMA10
            current_trend = curr['trend_SMA']
            prev_trend    = prev['trend_SMA']
        else: 
            current_trend = curr['trend'] ############ use EMA120 and EMA30
            prev_trend    = prev['trend']

        if current_trend != prev_trend : 
            self.trend_changing_ts = ts
            if self.state != "ENTER_MARKET":
                self.reset()

        if (current_trend == "BE" and self.enter_market_trade_side == None):
            if self.state == "IDLE":
                if self.check_pullback(greens, opens, closes, ts, trend="BE"): # pullback should eat and cover the previous red candle
                    # logic
                    slices = lowest_body_price.loc[self.trend_changing_ts:ts] # Slicing with .loc[start:end] in pandas includes both endpoints
                    self.low_price = slices.min()
                    self.low_price_ts  = slices.idxmin()   # Return index(Datetime) of first occurrence of minimum over requested axis.
                    # debug
                    old_state = self.state  
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish pullback detected")
                    

            elif self.state == "PULLBACK_DETECTED":
                if lowest_body_price.loc[ts] < self.low_price:
                    # logic
                    slices = highest_body_price.loc[self.low_price_ts:ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    # debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish new low after pullback")

                    
            elif self.state == "BREAK_OF_STRUCTURE":
                # If price breaks the pev high (LH)
                if highest_body_price.loc[ts] > self.high_price:
                    
                    # logic
                    slices = lowest_body_price.loc[self.high_price_ts: ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish break of structure")

                elif self.check_pullback(greens, opens, closes, ts, trend="BE"):
                    
                    if lowest_body_price.shift(2).loc[ts] < self.low_price: #### use 2 timeframe before current ts , can incorporate to 3,4,5 or 6
                    
                        # Logic: Low price for confirming the next Lower high 
                        if pos >= 3:   
                            window = lowest_body_price.iloc[pos-3 : pos+1]
                            self.low_price     = window.min()
                            self.low_price_ts  = window.idxmin()

                        self.fib_labels = self.calc_fibonacci_levels(self.high_price_ts, self.low_price_ts, trend="BE")
                        self.fib_dict = dict(self.fib_labels)
                        
                        # debug
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, ts, "bearish waiting entry")

                    elif highest_body_price.loc[ts] >= self.low_price:
                        
                        # debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bearish back to pullback")

            elif self.state == "WAITING_ENTRY":
                
                fib_618_price = self.fib_dict.get(0.618)
                fib_000_price = self.fib_dict.get(0.000)
                
                # price rebound back to 0.618
                if fib_618_price and df["High"].loc[ts] >= fib_618_price: # reference the "High" price at the exact timestamp
                
                    # debug
                    self.entry_signal_ts = ts
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, ts, "bearish price rebound to 0.618")

                elif fib_618_price and lowest_body_price.loc[ts] <= fib_000_price: # price lower than 0.000
                
                    # Logic: break of structure, set new high point
                    slices = highest_body_price.loc[self.low_price_ts: ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish breaks 0.000 before entry")
            
            elif self.state == "ENTRY_SIGNAL": 
                fib_100_price = self.fib_dict.get(1.000)
                fib_000_price = self.fib_dict.get(0.000)
                

                if highest_body_price.loc[ts] >= fib_100_price: # no matter what, if the candle pass 1.00, we do not trade
                
                    # debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish cancels: price > 1.0")

                # Logic: Bearish Engulfing candle  
                elif (greens.loc[prev_ts] == 'G') and (greens.loc[ts] != 'G') and opens.loc[ts] > closes.loc[prev_ts] and closes.loc[ts] < opens.loc[prev_ts]: # enter market
                    """
                    current_slope_30 = df.loc[ts]['slope30']
                    current_slope_120 = df.loc[ts]['slope120']
                    
                    # In bearish trend, a crossover early signal emerges when EMA30 turns upward, while EMA120 is still trending downward or flat.
                    
                    if current_slope_30 > 0 and current_slope_120 < 0: # EMA_30 rising, but EMA_120 decreasing
                        return # no trade, wait for next entry
                    """
                    # Debug
                    self.engulfing_candle_ts = ts
                    
                    stop, take = self.calc_fibonacci_levels(self.high_price_ts, 
                            self.low_price_ts, trend="BE", entry_signal=self.entry_signal_ts, engulfing_candle=self.engulfing_candle_ts
                            )
                    
                    # Logic to calculate lot_size
                    entry_price = closes.loc[ts]
                    stop_loss_pips = abs(stop - entry_price) / self.pip_size
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips, self.pip_value_per_lot)
                    
                    # Log for debug
                    self.evt.consoleLog(
                        f"DEBUG ticket – side:SELL, instrument:{self.instrument},ab:{self.account_balance}, entry≈{entry_price:.5f} "
                        f"SL={stop:.5f} TP={take:.5f}"
                    )
                    
                    # self.evt.consoleLog(f"Balance: {self.account_balance}, Risk%: {self.risk_percent}, Stop Loss (pips): {stop_loss_pips}, Pip Value: {self.pip_value_per_lot}, Lot Size: {lot_size}")
                    
                    # place selling order
                    self.send_entry_order(ts, trend="BE",
                                entry_price=entry_price,
                                stop=stop, take=take,
                                volume=lot_size)
                    
                    # Debug
                    old_state = self.state
                    self.state = "PRE_ENTER_MARKET"
                    self.log_state_change(old_state, self.state, ts, "bearish entry: engulfing candle")
                    
                elif lowest_body_price.loc[ts] <= fib_000_price: # If candle breaks 0.00 before there is a Bearish ENGULFING candle
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish breaks 0.000 before engulfing")
            
        # ==============================================================
        # ========================  BULLISH  ============================
        # ==============================================================

        if (current_trend == "BU" and self.enter_market_trade_side == None):
            if self.state == "IDLE":    
                if self.check_pullback(greens, opens, closes, ts, trend="BU"):
                    
                    # Logic
                    slices = highest_body_price[self.trend_changing_ts:ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish pullback detected")

            elif self.state == "PULLBACK_DETECTED":
                if highest_body_price.loc[ts] > self.high_price:
                    
                    # Logic
                    slices = lowest_body_price[self.high_price_ts: ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish new high after pullback")

   
            elif self.state == "BREAK_OF_STRUCTURE":
                # structure broken downward -> restart pull-back search
                if lowest_body_price.loc[ts] < self.low_price:
                    
                    # Logic
                    slices = highest_body_price[self.low_price_ts : ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish break of structure")

                # new 3 red pull-back -> possible Fib leg
                elif self.check_pullback(greens, opens, closes, ts, trend="BU"):
                    
                    if highest_body_price.shift(2).loc[ts] > self.high_price: # fresh HH
                        
                        # Logic
                        if pos >= 3:   
                            window = highest_body_price.iloc[pos-3 : pos+1]
                            self.high_price     = window.max()
                            self.high_price_ts  = window.idxmax()
                        
                        self.fib_labels = self.calc_fibonacci_levels(self.low_price_ts, self.high_price_ts, trend="BU")
                        self.fib_dict = dict(self.fib_labels)
                        
                        # Debug
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, ts, "bullish waiting entry")

                    # price failed to beat prev high: treat as new pull-back
                    elif lowest_body_price.loc[ts] <= self.high_price:
                        
                        # Debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bullish back to pullback")
                        

            elif self.state == "WAITING_ENTRY":
                fib_618_bullish = self.fib_dict.get(0.618)
                fib_000_bullish = self.fib_dict.get(0.000)

                # price retraced to 0.618 – ready for bullish entry
                if fib_618_bullish and df['Low'].loc[ts] <= fib_618_bullish: # count the tip of the candlestick instead of the body
                
                    # Debug
                    self.entry_signal_ts = ts
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, ts, "bullish price retraced to 0.618")

                # price breaks above 0.000 (HH) before touching 0.618
                elif fib_618_bullish and highest_body_price.loc[ts] > fib_000_bullish:
                    
                    # Logic
                    slices = lowest_body_price[self.high_price_ts:ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish breaks above 0.000 before entry")
                
            elif self.state == "ENTRY_SIGNAL":
                fib_100_bullish = self.fib_dict.get(1.000)
                fib_000_bullish = self.fib_dict.get(0.000)

                # cancel if price dumps below 1.000 extension
                if lowest_body_price.loc[ts] <= fib_100_bullish:
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish cancels: price < 1.0")

                # Bullish Engulfing candle
                elif ((greens.loc[prev_ts] != 'G') and (greens.loc[ts] == 'G') and (opens.loc[ts] <= closes.loc[prev_ts]) and (closes.loc[ts] >= opens.loc[prev_ts])):# Enter Market 
                    """
                    current_slope_30 = df.loc[ts]['slope30']
                    current_slope_120 = df.loc[ts]['slope120']
                    
                    # In bullish trend, a crossover early signal emerges when EMA30 turns downward, while EMA120 is still trending upward or flat.
                    
                    if current_slope_30 < 0 and current_slope_120 > 0: # EMA_30 rising, but EMA_120 decreasing
                        return # no trade, wait for next entry
                    """      
                    self.engulfing_candle_ts = ts
                    
                    stop, take = self.calc_fibonacci_levels(self.low_price_ts, 
                    self.high_price_ts,trend="BU", entry_signal=self.entry_signal_ts,
                    engulfing_candle=self.engulfing_candle_ts)
                    
                    # calculate volume
                    entry_price = closes.loc[ts]
                    stop_loss_pips = abs(stop - entry_price) / self.pip_size
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips, self.pip_value_per_lot)
                    
                    self.evt.consoleLog(
                        f"DEBUG ticket – side:BUY, instrument: {self.instrument}, ab:{self.account_balance}, entry≈{entry_price:.5f} "
                        f"SL={stop:.5f} TP={take:.5f}"
                    )
                    # place buying order
                    self.send_entry_order(ts, trend="BU",
                              entry_price=entry_price,
                              stop=stop, take=take,
                              volume=lot_size)
                    # self.evt.consoleLog(f"Balance: {self.account_balance}, Risk%: {self.risk_percent}, Stop Loss (pips): {stop_loss_pips}, Pip Value: {self.pip_value_per_lot}, Lot Size: {lot_size}")
                    
                    # Debug
                    old_state = self.state
                    self.state = "PRE_ENTER_MARKET"
                    self.log_state_change(old_state, self.state, ts, "bullish entry: engulfing candle")
                    
                # invalidated if price breaks above HH first
                elif highest_body_price.loc[ts] >= fib_000_bullish:
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish breaks above HH first")

            elif self.state == "ENTER_MARKET":
                current_high = df['High'].loc[ts]
                current_low = df["Low"].loc[ts]
               
                if current_high >= self.take_profit_price :
                    self.evt.consoleLog(f"Price touched take_profit at {current_high:.5f} (take_profit: {self.take_profit_price:.5f}) – waiting for broker to close")
                elif current_low <= self.stop_loss_price :
                    self.evt.consoleLog(f"Price touched stop_loss at {current_low:.5f} (stop_loss: {self.stop_loss_price:.5f}) – waiting for broker to close")
        
        # ==============================================================
        # ========================  pullback()  ========================
        # ==============================================================     
        
    def check_pullback(self, greens, opens, closes, ts, trend):
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        
        if trend == "BE":
            # Check for 3, 4, 5, or 6 consecutive green candles.
            for window in [3, 4, 5, 6]:

                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                # Boundary check
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                # Check if all are green
                if all(greens.loc[ts_] == 'G' for ts_ in window_ts):
                    # Previous candle must be red
                    if greens.loc[prev_ts] == 'R':
                        # Any green candle closes above the open of previous red candle
                        if any(closes.loc[ts_] >= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False
        
        if trend == "BU":
            for window in [3,4,5,6]:
                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                if all(greens.loc[ts_]!= 'G' for ts_ in window_ts): # all previous not green candles 
                    if greens.loc[prev_ts] == 'G':
                        if any(closes.loc[ts_] <= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False

    def reset(self):
        self.state = "IDLE"
        self.low_price  = self.low_price_ts  =  None
        self.high_price = self.high_price_ts =  None
        self.fib_labels = []
        self.fib_dict = {} 
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.position_open = False
        self.open_volume = None


class InstrumentBar:
    
    """Bar builder + FSM for ONE instrument."""
    
    ALPHA_30  = 2 / 31
    ALPHA_120 = 2 / 121
    ALPHA_10 = 2 / 11
    MIN_BARS  = 10 
    MAX_BARS  = 700
    
    def __init__(self, symbol: str, evt):
        
        self.symbol    = symbol
        self.evt       = evt
        self.last_time = datetime(2010, 1, 1)
        self.fsm       = None
        
        NFLOAT = "float32"
        
        # ------------- 5-min buffer ---------------------------------
        self.raw_5m = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument"])
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "category",
                "Instrument": "object"}))
                
        # ------------- 15-min table ---------------------------------
        self.data = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument", "EMA_30", "EMA_120", "trend","slope30","slope120", 'SMA_50','trend_SMA',"EMA_10"])
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "category", "Instrument": "object",
                "EMA_30": NFLOAT, "EMA_120": NFLOAT,
                "trend": "category",
                "slope30": NFLOAT,
                "slope120": NFLOAT,
                'SMA_50': NFLOAT,
                "trend_SMA": "category",
                "EMA_10": NFLOAT
            })
            .set_index("Datetime", drop=False))
        
    def update(self, snap: dict, available_balance: float):
        
        """
        snap = bd[symbol]  – single-instrument snapshot from on_bulkdatafeed
        """
        # -------------------- append 5-min bar ------------------------
        green = ("G" if snap["lastPrice"] > snap["openPrice"]
                 else "R" if snap["lastPrice"] < snap["openPrice"]
                 else "NO")
        
        bar5 = {
            "Datetime": pd.to_datetime(snap["timestamp"]),
            "Open":  snap["openPrice"],
            "High":  snap["highPrice"],
            "Low":   snap["lowPrice"],
            "Close": snap["lastPrice"],
            "Green": green,
            "Instrument": snap["instrument"],
        }
        
        self.raw_5m = pd.concat([self.raw_5m,
                                 pd.DataFrame([bar5])], ignore_index=True)
        
        
        # ------------------- finished 15-min? -------------------------
        ts = pd.to_datetime(snap["timestamp"])
        if ts < self.last_time + timedelta(minutes=15):
            return
        self.last_time = ts
        
        last3 = self.raw_5m.tail(3)
        if len(last3) < 3:
            return    # need full 15-minute slice
        
        self.raw_5m = self.raw_5m.iloc[0:0]  # clear buffer
        
        self._aggregate_15m(last3)
        
        
        if len(self.data) < self.MIN_BARS:
            return
            
        
        if len(self.data) > self.MAX_BARS:
            self.data = self.data.iloc[-self.MAX_BARS:]
            
        # -------------------- FSM ------------------------------------
        if self.fsm is None:
            if self.symbol == "USDJPY":
                pip_size = 0.01
                price    = self.data["Close"].iloc[-1]
                pip_value_per_lot = 1000 / price        # 1000 JPY ÷ JPYUSD
            else:                                       # EURUSD / GBPUSD
                pip_size = 0.0001
                pip_value_per_lot = 10
                
            self.fsm = BearishFSM(
                self.data, instrument=self.symbol, evt=self.evt,
                account_balance=np.float32(available_balance),
                pip_size=pip_size, pip_value_per_lot=pip_value_per_lot,
                ts=self.last_time)
        else:
            
            self.fsm.df = self.data
            self.fsm.account_balance = available_balance 
            self.fsm.update(self.last_time)
            
    def _aggregate_15m(self, last3):
        
        
        
        n   = len(self.data)
        op  = last3["Open"].iloc[0]
        cl  = last3["Close"].iloc[-1]
        bar = {
            "Datetime": last3["Datetime"].iloc[-1],
            "Open": op,
            "High": last3["High"].max(),
            "Low":  last3["Low"].min(),
            "Close": cl,
            "Green": "G" if cl > op else "R" if cl < op else "NO",
            "Instrument": self.symbol,
        }

        if n == 0:
            ema30 = ema120 = ema10 = cl
            slope30 = slope120 = 0
        elif n < 30: 
            prev  = self.data.iloc[n - 1]
            ema10  = (1 - self.ALPHA_10)  * prev["EMA_10"]  + self.ALPHA_10  * cl
            ema30  = (1 - self.ALPHA_30)  * prev["EMA_30"]  + self.ALPHA_30  * cl
            ema120 = (1 - self.ALPHA_120) * prev["EMA_120"] + self.ALPHA_120 * cl
            slope30 = slope120 = 0
        else:
            prev  = self.data.iloc[n - 3] ###########################################  optimize the number of bars for calculating the slope: n-bar slope (e.g. 5) bars)
            ema10  = (1 - self.ALPHA_10)  * prev["EMA_10"]  + self.ALPHA_10  * cl
            ema30  = (1 - self.ALPHA_30)  * prev["EMA_30"]  + self.ALPHA_30  * cl
            ema120 = (1 - self.ALPHA_120) * prev["EMA_120"] + self.ALPHA_120 * cl
            slope30 = ema30 - prev["EMA_30"]
            slope120 = ema120 - prev["EMA_120"]
        
        bar["EMA_30"]  = ema30
        bar["EMA_120"] = ema120
        r = ema30 / ema120
        bar["trend"] = "BU" if r > 1.001 else "BE" if r < 0.999 else "NO" ################### change to detect slope
        bar['slope30'] = slope30
        bar['slope120'] = slope120
        bar["EMA_10"]  = ema10
        
        # ---------- SMA 50 ---------------------------------------------
        if n >= 49:                                    # we already have 49 closes;
            # add the new one (cl)
            sma50 = (self.data["Close"].iloc[-49:].sum() + cl) / 50
        else:
            current_mean = (self.data["Close"].sum() + cl) / (n + 1)
            weight = (n + 1) / 50            # grows from 0.02 … 0.98
            sma50 = weight * current_mean + (1 - weight) * cl
    
        bar["SMA_50"] = sma50
        
        # ---------- trend_SMA (EMA30 vs SMA50) --------------------------
        r2 = ema10 / sma50
        bar["trend_SMA"] = ("BU" if r2 > 1.001 else
                            "BE" if r2 < 0.999 else
                            "NO")
        

        self.data = pd.concat([self.data, pd.DataFrame([bar]).set_index("Datetime",drop=False)])
        


        

class AlgoEvent:

    def __init__(self):
        self.evt   = None
        self.books = {}          # symbol → InstrumentBar

    def start(self, mEvt):
        self.evt = AlgoAPI_Backtest.AlgoEvtHandler(self, mEvt)
        self.evt.start()
            
    def on_bulkdatafeed(self, isSync, bd, ab):
        
        balance = ab["availableBalance"]
        
        for sym, snap in bd.items():        # sym: AUDNZD, EURUSD, ... snap: {'instrument': 'AUDNZD', 'timestamp': datetime.datetime(1970, 1, 1, 0, 0), 'askPrice': -1, ...}
            if sym not in self.books:       # first sight → create book
                self.books[sym] = InstrumentBar(sym, self.evt)
            else: 
                self.books[sym].update(snap, balance)

    def on_orderfeed(self, of):
        
        sym = of.instrument
        if sym not in self.books:
            return                        # safety
        
        fsm = self.books[sym].fsm
        
        # -------------- order filled -----------------
        if of.openclose == "open" and of.status == "success":
            # Is this the order we are waiting for?
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.position_open      = True
                fsm.stop_loss_price    = fsm.pending["stop"]
                fsm.take_profit_price  = fsm.pending["take"]
                fsm.open_volume        = fsm.pending["volume"]
                fsm.enter_market_trade_side = fsm.pending["trend"]
    
                fsm.pending = None      # clear
                old_state   = fsm.state
                fsm.state   = "ENTER_MARKET"
                fsm.log_state_change(old_state, fsm.state, of.orderRef,
                                     "broker filled order")
        # -------------- order rejected ---------------
        elif of.openclose == "open" and of.status != "success":
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.pending = None
                old_state = fsm.state
                fsm.state = "ENTRY_SIGNAL"     # back to previous state
                fsm.log_state_change(old_state, fsm.state, of.orderRef,
                                     "order rejected")
                                     
        # -------------- position closed --------------
        if of.openclose == 'close' and of.status == 'success': # return 'success' if an order is opened/ closed successfully, or when a limit/stop order is filled
            fsm.reset()
    
    
    def on_newsdatafeed(self, nd):
        pass

    def on_weatherdatafeed(self, wd):
        pass
    
    def on_econsdatafeed(self, ed):
        pass
        
    def on_corpAnnouncement(self, ca):
        pass
    
    def on_dailyPLfeed(self, pl):
        pass

    def on_openPositionfeed(self, op, oo, uo):
        pass

#### Version 11 Changes

1. Return to using EMA30 vs EMA120

2. Thanks for the insight from this book -- Quantitative Trading: How to Build Your Own Algorithmic Trading Business By ERNEST P. CHAN, I tried to use less constraint in the algorithm.
    - Before: Only "strong" engulfing allowed
    - Now: "Weak" engulfing is allowed to enter trade

    Result: We pick up more trades, but win-rate does not increase.

In [ ]:
from AlgoAPI import AlgoAPIUtil, AlgoAPI_Backtest
import pandas as pd
import numpy as np
import mplfinance as mpf
from collections import deque
from datetime import datetime, timedelta
    
class BearishFSM: # Finite State Machine

        # ==============================================================
        # ========================  __init__ ===========================
        # ==============================================================
    def __init__(self, df, instrument, evt, account_balance=0, pip_size = 0.0001, fx = 1, fx_base = 1, ts = None):
        
        self.df = df
        self.evt = evt
        self.account_balance = account_balance
        
        # Enter market flag
        self.position_open = False
        self.pending = None       # ← dict that holds the ticket waiting
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.open_volume = None
        
        self.state = "IDLE"
        self.low_price = None
        
        self.low_price_ts = None
        self.high_price = None
        self.high_price_ts = None

        self.fib_labels = []
        self.fib_dict = {} # not a must to initialize, just for good habit
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.trend_changing_ts = ts # do not reset this, always keep record of 
        
##################################################################################################################
####################################### optimize #################################################################
##################################################################################################################

        # do not reset the folloiwng parameters, test for optimal values
        self.risk_reward_ratio = 1.8
        self.atr_period = 14
        self.atr_multiplier = 2 # pip_size = atr_multiplier * atr_value, atr_multiplier = 2.0 means stop loss is 2 ATRs away
        self.risk_percent = 0.5
        self.dynamic_atr_window = False
        # self.trend_ema10_sma50 = False # If True, the trend will be determined by EMA10 and SMA50, if False, determined by EMA30 and EMA120
        
        #### reduce the number of conditions:
        self.weak_engulfing_candle = True
##################################################################################################################
##################################################################################################################
##################################################################################################################
        # this is unique in every func call
        self.instrument = instrument
        self.pip_size = pip_size
        self.fx_usd_quote_currency = fx
        self.fx_base_usd_currency = fx_base
        # ==============================================================
        # ========================  calculate_log_state_change =========
        # ==============================================================
        
    def log_state_change(self, old_state, new_state, ts, label=None):
        if 'Datetime' in self.df.columns and ts in self.df.index:
            date_str = str(self.df.loc[ts, 'Datetime'])
        else:
            date_str = str(ts)
        msg = f" State: {old_state} → {new_state}"
        if label:
            msg += f" ({label})"
        # self.evt.consoleLog(msg)
        
        # ==============================================================
        # ========================  calculate_lot_size  ================
        # ==============================================================
        
    def calculate_lot_size(self, risk_percent, stop_loss_pips, max_leverage = 4.5): # base currency vs quote currency
    
        # 1. Calculate amount to risk
        risk_amount = self.account_balance * (risk_percent / 100)
        
        # 2. calculate pip value per lot (in quote currency)
        pip_value_per_lot_quote = self.pip_size * 100000 # If we are trading AUDJPY, the value will be 0.01 * 100000 = 1000 JPY
        
        # 3. calculate total risk per lot ( in quote currency)
        total_risk_per_lot_quote = stop_loss_pips * pip_value_per_lot_quote
        
        # 4. calculate total risk per lot ( in account currency, i.e. USD)
            # we need real time USD/quote_currency price, in this case, USDJPY
        total_risk_per_lot_USD = total_risk_per_lot_quote / self.fx_usd_quote_currency # if quote currency = USD, this will be 1
        
        # 5. calculate lot size
        lot_size = risk_amount / total_risk_per_lot_USD

        # calculate maximum affordable lot size after using margin (e.g. we are trading AUDUSD)
            # Notional Value per lot (in Base Currency) = 100000 AUD
            
        # Notation Value per lot (in account currency) = 100000 x AUDUSD price
        notation_per_lot_USD = 100000 * self.fx_base_usd_currency
            
        lot_margin = (self.account_balance * max_leverage) / notation_per_lot_USD
        
        lot_final = min(lot_size, lot_margin) # all-in 
        
        self.evt.consoleLog(
            f"{self.instrument} bal={self.account_balance:.2f} risk%={risk_percent} sl={stop_loss_pips} "
            f"riskAmt={risk_amount:.2f} pipVal/lot={pip_value_per_lot_quote:.5f} risk/lot(USD)={total_risk_per_lot_USD}"
            f"riskLot={lot_size:.2f} margLot={lot_margin:.2f} finalLot={lot_final:.2f}"
        )
        
        return round(lot_final, 2)
        
        
        # ==============================================================
        # ========================  calc_fibonacci_levels  =============
        # ==============================================================
        
    def calc_fibonacci_levels(self, start_ts, end_ts, levels=None, trend="NO",
                         entry_signal=None, engulfing_candle=None,
                         R=None, atr_period=None, atr_multiplier=None,dynamic_atr_window = None):
                             
        if levels is None:
            levels = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]
            
        if dynamic_atr_window is None:
            dynamic_atr_window = self.dynamic_atr_window
            
        if atr_multiplier is None:
            atr_multiplier = self.atr_multiplier
        
        if R is None:
            R = self.risk_reward_ratio
        
        
        df = self.df
        """
        start_idx =  df.index.get_loc(start_ts)
        end_idx = df.index.get_loc(end_ts)
        
        start_ts = df.index[start_idx] # still a timestamp
        end_ts = df.index[end_idx]
        """
        window = df.loc[start_ts : end_ts]
                            
        min_price = window['Low'].min()
        max_price = window['High'].max()
    
        fib_labels = []
        for level in levels:
            if trend == "BE": # bearish, sell
                y = min_price + (max_price - min_price) * level
            else:
                y = max_price - (max_price - min_price) * level
            fib_labels.append((level, y))
        
        if engulfing_candle is None:
            return fib_labels
        
        if atr_period is None:

            if dynamic_atr_window == False:
                # Fixed atr period
                atr_period = self.atr_period
            else: 
                # dynamic atr period
                start_idx = df.index.get_loc(start_ts)
                enter_idx = df.index.get_loc(engulfing_candle)
            
                dynamic_period = enter_idx - start_idx
                atr_period = max(10, min(dynamic_period, 50))
        
        # Normal ATR
        min_start_idx = max(0, df.index.get_loc(start_ts) - 10)
        min_start_ts = df.index[min_start_idx]
        sl = df.loc[min_start_ts : engulfing_candle].copy()
        
        sl['H-L']  = sl['High'] - sl['Low']
        sl['H-PC'] = (sl['High'] - sl['Close'].shift(1)).abs()
        sl['L-PC'] = (sl['Low']  - sl['Close'].shift(1)).abs()
        sl['TR']    = sl[['H-L', 'H-PC', 'L-PC']].max(axis=1)
        
        # ATR: rolling mean of TR
        sl['ATR'] = sl['TR'].rolling(window=atr_period).mean()
         
        # Retrieve ATR value at the engulfing candle index
        atr_value = sl['ATR'].iloc[-1]
        pip_size = atr_multiplier * atr_value
        
        # Retrieve the close price of the engulfing candle
        candle = self.df.loc[engulfing_candle]
        close_price = candle['Close']
        
        # Initialize stop loss and take profit prices
        stop_loss_price = None
        take_profit_price = None
    
        if trend == "BE" :
            stop_loss_price = close_price + pip_size
            take_profit_price = close_price - R * pip_size
        else:  # bullish
            stop_loss_price = close_price - pip_size
            take_profit_price = close_price + R * pip_size
    
        return stop_loss_price, take_profit_price
        
    # ==============================================================
    # ======================== send_entry_order  =============
    # ==============================================================
    
    def send_entry_order(self, ts, trend, entry_price,
                         stop, take, volume):

        # remember what we just asked the broker to do
        self.pending = dict(
            orderRef = ts,         # we use the timestamp as reference
            trend    = trend,      # "BU" / "BE"
            stop     = stop,
            take     = take,
            volume   = volume
        )

        order = AlgoAPIUtil.OrderObject()
        order.instrument   = self.instrument
        order.orderRef     = ts
        order.openclose    = "open"
        order.buysell      = 1 if trend == "BU" else -1
        order.ordertype    = 0          # market
        order.volume       = volume
        order.stopLossLevel= stop
        order.takeProfitLevel = take
        self.evt.sendOrder(order)

        # NOTE: do NOT set state / flags here;
        #       wait for on_orderfeed()
    
    # -------------------------------------------------------------
    # monitor_exit()  – runs while we are in the trade
    # -------------------------------------------------------------
    def monitor_exit(self, ts):
        """
        Called every bar only while self.position_open is True.
        Produces the same log lines you had in your old ENTER_MARKET
        branch.  If you additionally want to *close* the trade,
        send a close-order here.
        """
        if self.take_profit_price is None or self.stop_loss_price is None:
            return      # safety – nothing to do

        high = self.df["High"].loc[ts]
        low  = self.df["Low"].loc[ts]

        if self.enter_market_trade_side == "BU":          # long trade
            if high >= self.take_profit_price:
                self.evt.consoleLog(
                    f"{self.instrument} Price touched take_profit at {high:.5f} "
                    f"(take_profit: {self.take_profit_price:.5f}) – "
                    "waiting for broker to close"
                )
            elif low <= self.stop_loss_price:
                self.evt.consoleLog(
                    f"{self.instrument} Price touched stop_loss at {low:.5f} "
                    f"(stop_loss: {self.stop_loss_price:.5f}) – "
                    "waiting for broker to close"
                )

        else:                                             # short trade
            if low <= self.take_profit_price:
                self.evt.consoleLog(
                    f"{self.instrument} Price touched take_profit at {low:.5f} "
                    f"(take_profit: {self.take_profit_price:.5f}) – "
                    "waiting for broker to close"
                )
            elif high >= self.stop_loss_price:
                self.evt.consoleLog(
                    f"{self.instrument} Price touched stop_loss at {high:.5f} "
                    f"(stop_loss: {self.stop_loss_price:.5f}) – "
                    "waiting for broker to close"
                )
                
    # ==============================================================
    # ========================  update()  =============
    # ==============================================================
    
    def update(self, ts: pd.Timestamp):
        
        # 1️⃣ When a position is open we do only exit-logic and leave
        #    the rest of the gigantic state-machine untouched.
        #    (So IDLE / PULLBACK_DETECTED / … cannot fire again.)
        if self.position_open:
            self.monitor_exit(ts)        # <-- will emit console logs
            return                       # <-- early exit
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        prev_ts = df.index[pos - 1] # still a timestamp
        
        if df.index.get_loc(ts) < 3: # Get current timestamp location
            return
        
        #--- 2. prepare frequently-used columns (all remain Series!) ------
        trend   = df["trend"]               # string
        # trend_SMA = df['trend_SMA']
        greens  = df["Green"]               # string
        opens   = df["Open"]
        closes  = df["Close"]
        
        lowest_body_price = opens.where(greens == 'G', closes) # if greens == 'G', use values in open, else use values in closes 
        highest_body_price = closes.where(greens == 'G', opens) # the index of this series keeps the same Datetime index as the original df
        
        # selects the row in df where the index = ts
        curr   = df.loc[ts]
        prev   = df.shift(1).loc[ts]       # one-bar shift gives previous values
        """
        if self.trend_ema10_sma50 == True:  ###########  use SMA50 and EMA10
            current_trend = curr['trend_SMA']
            prev_trend    = prev['trend_SMA']
        else: 
        """
        current_trend = curr['trend'] ############ use EMA120 and EMA30
        prev_trend    = prev['trend']

        if current_trend != prev_trend : 
            self.trend_changing_ts = ts
            if self.state != "ENTER_MARKET":
                self.reset()

        if (current_trend == "BE" and self.enter_market_trade_side == None):
            if self.state == "IDLE":
                if self.check_pullback(greens, opens, closes, ts, trend="BE"): # pullback should eat and cover the previous red candle
                    # logic
                    slices = lowest_body_price.loc[self.trend_changing_ts:ts] # Slicing with .loc[start:end] in pandas includes both endpoints
                    self.low_price = slices.min()
                    self.low_price_ts  = slices.idxmin()   # Return index(Datetime) of first occurrence of minimum over requested axis.
                    # debug
                    old_state = self.state  
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish pullback detected")
                    

            elif self.state == "PULLBACK_DETECTED":
                if lowest_body_price.loc[ts] < self.low_price:
                    # logic
                    slices = highest_body_price.loc[self.low_price_ts:ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    # debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish new low after pullback")

                    
            elif self.state == "BREAK_OF_STRUCTURE":
                # If price breaks the pev high (LH)
                if highest_body_price.loc[ts] > self.high_price:
                    
                    # logic
                    slices = lowest_body_price.loc[self.high_price_ts: ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish break of structure")

                elif self.check_pullback(greens, opens, closes, ts, trend="BE"):
                    
                    if lowest_body_price.shift(2).loc[ts] < self.low_price: #### use 2 timeframe before current ts , can incorporate to 3,4,5 or 6
                    
                        # Logic: Low price for confirming the next Lower high 
                        if pos >= 3:   
                            window = lowest_body_price.iloc[pos-3 : pos+1]
                            self.low_price     = window.min()
                            self.low_price_ts  = window.idxmin()

                        self.fib_labels = self.calc_fibonacci_levels(self.high_price_ts, self.low_price_ts, trend="BE")
                        self.fib_dict = dict(self.fib_labels)
                        
                        # debug
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, ts, "bearish waiting entry")

                    elif highest_body_price.loc[ts] >= self.low_price:
                        
                        # debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bearish back to pullback")

            elif self.state == "WAITING_ENTRY":
                
                fib_618_price = self.fib_dict.get(0.618)
                fib_000_price = self.fib_dict.get(0.000)
                
                # price rebound back to 0.618
                if fib_618_price and df["High"].loc[ts] >= fib_618_price: # reference the "High" price at the exact timestamp
                
                    # debug
                    self.entry_signal_ts = ts
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, ts, "bearish price rebound to 0.618")

                elif fib_618_price and lowest_body_price.loc[ts] <= fib_000_price: # price lower than 0.000
                
                    # Logic: break of structure, set new high point
                    slices = highest_body_price.loc[self.low_price_ts: ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish breaks 0.000 before entry")
            
            elif self.state == "ENTRY_SIGNAL": 
                fib_100_price = self.fib_dict.get(1.000)
                fib_000_price = self.fib_dict.get(0.000)
                

                if highest_body_price.loc[ts] >= fib_100_price: # no matter what, if the candle pass 1.00, we do not trade
                
                    # debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish cancels: price > 1.0")

                # Logic: Bearish Engulfing candle  
                elif (greens.loc[prev_ts] == 'G') and (greens.loc[ts] != 'G'):
                    should_enter = False

                    if self.weak_engulfing_candle:
                        # Condition for weak engulfing
                        if closes.loc[ts] < opens.loc[prev_ts]:
                            should_enter = True
                    else:
                        # Condition for strong engulfing
                        if opens.loc[ts] > closes.loc[prev_ts] and closes.loc[ts] < opens.loc[prev_ts]:
                            should_enter = True
                
                    if should_enter:
                        

                        # Debug
                        self.engulfing_candle_ts = ts
                        
                        stop, take = self.calc_fibonacci_levels(self.high_price_ts, 
                                self.low_price_ts, trend="BE", entry_signal=self.entry_signal_ts, engulfing_candle=self.engulfing_candle_ts
                                )
                        
                        # Logic to calculate lot_size
                        entry_price = closes.loc[ts]
                        stop_loss_pips = abs(stop - entry_price) / self.pip_size
                        lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips)
                        
                        # Log for debug
                        self.evt.consoleLog(
                            f"DEBUG ticket – side:SELL, instrument:{self.instrument},ab:{self.account_balance}, entry≈{entry_price:.5f} "
                            f"SL={stop:.5f} TP={take:.5f}"
                        )
                    
                        
                        # place selling order
                        self.send_entry_order(ts, trend="BE",
                                    entry_price=entry_price,
                                    stop=stop, take=take,
                                    volume=lot_size)
                        
                        # Debug
                        old_state = self.state
                        self.state = "PRE_ENTER_MARKET"
                        self.log_state_change(old_state, self.state, ts, "bearish entry: engulfing candle")
                    
                elif lowest_body_price.loc[ts] <= fib_000_price: # If candle breaks 0.00 before there is a Bearish ENGULFING candle
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish breaks 0.000 before engulfing")
            
        # ==============================================================
        # ========================  BULLISH  ============================
        # ==============================================================

        if (current_trend == "BU" and self.enter_market_trade_side == None):
            if self.state == "IDLE":    
                if self.check_pullback(greens, opens, closes, ts, trend="BU"):
                    
                    # Logic
                    slices = highest_body_price[self.trend_changing_ts:ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish pullback detected")

            elif self.state == "PULLBACK_DETECTED":
                if highest_body_price.loc[ts] > self.high_price:
                    
                    # Logic
                    slices = lowest_body_price[self.high_price_ts: ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish new high after pullback")

   
            elif self.state == "BREAK_OF_STRUCTURE":
                # structure broken downward -> restart pull-back search
                if lowest_body_price.loc[ts] < self.low_price:
                    
                    # Logic
                    slices = highest_body_price[self.low_price_ts : ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish break of structure")

                # new 3 red pull-back -> possible Fib leg
                elif self.check_pullback(greens, opens, closes, ts, trend="BU"):
                    
                    if highest_body_price.shift(2).loc[ts] > self.high_price: # fresh HH
                        
                        # Logic
                        if pos >= 3:   
                            window = highest_body_price.iloc[pos-3 : pos+1]
                            self.high_price     = window.max()
                            self.high_price_ts  = window.idxmax()
                        
                        self.fib_labels = self.calc_fibonacci_levels(self.low_price_ts, self.high_price_ts, trend="BU")
                        self.fib_dict = dict(self.fib_labels)
                        
                        # Debug
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, ts, "bullish waiting entry")

                    # price failed to beat prev high: treat as new pull-back
                    elif lowest_body_price.loc[ts] <= self.high_price:
                        
                        # Debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bullish back to pullback")
                        

            elif self.state == "WAITING_ENTRY":
                fib_618_bullish = self.fib_dict.get(0.618)
                fib_000_bullish = self.fib_dict.get(0.000)

                # price retraced to 0.618 – ready for bullish entry
                if fib_618_bullish and df['Low'].loc[ts] <= fib_618_bullish: # count the tip of the candlestick instead of the body
                
                    # Debug
                    self.entry_signal_ts = ts
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, ts, "bullish price retraced to 0.618")

                # price breaks above 0.000 (HH) before touching 0.618
                elif fib_618_bullish and highest_body_price.loc[ts] > fib_000_bullish:
                    
                    # Logic
                    slices = lowest_body_price[self.high_price_ts:ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish breaks above 0.000 before entry")
                
            elif self.state == "ENTRY_SIGNAL":
                fib_100_bullish = self.fib_dict.get(1.000)
                fib_000_bullish = self.fib_dict.get(0.000)

                # cancel if price dumps below 1.000 extension
                if lowest_body_price.loc[ts] <= fib_100_bullish:
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish cancels: price < 1.0")

                # Bullish Engulfing candle
                elif (greens.loc[prev_ts] != 'G') and (greens.loc[ts] == 'G'):
                    should_enter = False

                    if self.weak_engulfing_candle:
                        # Condition for weak engulfing
                        if (closes.loc[ts] >= opens.loc[prev_ts]):
                            should_enter = True
                    else:
                        # Condition for strong engulfing
                        if (opens.loc[ts] <= closes.loc[prev_ts]) and (closes.loc[ts] >= opens.loc[prev_ts]) :# Enter Market 
                            should_enter = True
                
                    if should_enter:
                    
                        self.engulfing_candle_ts = ts
                        
                        stop, take = self.calc_fibonacci_levels(self.low_price_ts, 
                        self.high_price_ts,trend="BU", entry_signal=self.entry_signal_ts,
                        engulfing_candle=self.engulfing_candle_ts)
                        
                        # calculate volume
                        entry_price = closes.loc[ts]
                        stop_loss_pips = abs(stop - entry_price) / self.pip_size
                        lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips)
                        
                        self.evt.consoleLog(
                            f"DEBUG ticket – side:BUY, instrument: {self.instrument}, ab:{self.account_balance}, entry≈{entry_price:.5f} "
                            f"SL={stop:.5f} TP={take:.5f}"
                        )
                        # place buying order
                        self.send_entry_order(ts, trend="BU",
                                  entry_price=entry_price,
                                  stop=stop, take=take,
                                  volume=lot_size)
                        
                        # Debug
                        old_state = self.state
                        self.state = "PRE_ENTER_MARKET"
                        self.log_state_change(old_state, self.state, ts, "bullish entry: engulfing candle")
                    
                # invalidated if price breaks above HH first
                elif highest_body_price.loc[ts] >= fib_000_bullish:
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish breaks above HH first")

        
        # ==============================================================
        # ========================  pullback()  ========================
        # ==============================================================     
        
    def check_pullback(self, greens, opens, closes, ts, trend):
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        
        if trend == "BE":
            # Check for 3, 4, 5, or 6 consecutive green candles.
            for window in [3, 4, 5, 6]:

                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                # Boundary check
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                # Check if all are green
                if all(greens.loc[ts_] == 'G' for ts_ in window_ts):
                    # Previous candle must be red
                    if greens.loc[prev_ts] == 'R':
                        # Any green candle closes above the open of previous red candle
                        if any(closes.loc[ts_] >= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False
        
        if trend == "BU":
            for window in [3,4,5,6]:
                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                if all(greens.loc[ts_]!= 'G' for ts_ in window_ts): # all previous not green candles 
                    if greens.loc[prev_ts] == 'G':
                        if any(closes.loc[ts_] <= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False

    def reset(self):
        self.state = "IDLE"
        self.low_price  = self.low_price_ts  =  None
        self.high_price = self.high_price_ts =  None
        self.fib_labels = []
        self.fib_dict = {} 
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.position_open = False
        self.open_volume = None


class InstrumentBar:
    
    """Bar builder + FSM for ONE instrument."""
    
    ALPHA_30  = 2 / 31
    ALPHA_120 = 2 / 121
    # ALPHA_10 = 2 / 11
    MIN_BARS  = 120 
    MAX_BARS  = 700
    
    def __init__(self, symbol: str, evt):
        
        self.symbol    = symbol
        self.evt       = evt
        self.last_time = datetime(2010, 1, 1)
        self.fsm       = None
        
        NFLOAT = "float32"
        
        # ------------- 5-min buffer ---------------------------------
        self.raw_5m = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument"])
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "category",
                "Instrument": "object"}))
                
        # ------------- 15-min table ---------------------------------
        self.data = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument", "EMA_30", "EMA_120", "trend",]) # 'SMA_50','trend_SMA',"EMA_10"
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "category", "Instrument": "object",
                "EMA_30": NFLOAT, "EMA_120": NFLOAT,
                "trend": "category",
                # 'SMA_50': NFLOAT,
                # "trend_SMA": "category",
                # "EMA_10": NFLOAT
            })
            .set_index("Datetime", drop=False))
        
    def update(self, snap: dict, available_balance: float):
        
        """
        snap = bd[symbol]  – single-instrument snapshot from on_bulkdatafeed
        """
        # -------------------- append 5-min bar ------------------------
        green = ("G" if snap["lastPrice"] > snap["openPrice"]
                 else "R" if snap["lastPrice"] < snap["openPrice"]
                 else "NO")
        
        bar5 = {
            "Datetime": pd.to_datetime(snap["timestamp"]),
            "Open":  snap["openPrice"],
            "High":  snap["highPrice"],
            "Low":   snap["lowPrice"],
            "Close": snap["lastPrice"],
            "Green": green,
            "Instrument": snap["instrument"],
        }
        
        self.raw_5m = pd.concat([self.raw_5m,
                                 pd.DataFrame([bar5])], ignore_index=True)
                                 
        # avoid  Warning: data month not exists for ... fx error led to array memory explode
        if len(self.raw_5m) > 10:
            self.raw_5m = self.raw_5m.iloc[0:0]
        
        
        # ------------------- finished 15-min? -------------------------
        ts = pd.to_datetime(snap["timestamp"])
        if ts < self.last_time + timedelta(minutes=15):
            return
        self.last_time = ts
        
        last3 = self.raw_5m.tail(3)
        if len(last3) < 3:
            return    # need full 15-minute slice
        
        self.raw_5m = self.raw_5m.iloc[0:0]  # clear buffer
        
        self._aggregate_15m(last3)
        
        
        if len(self.data) < self.MIN_BARS:
            return
            
        
        if len(self.data) > self.MAX_BARS:
            self.data = self.data.iloc[-self.MAX_BARS:]
        
        # -------------------- get latest exchange rate  -----------------
        base_currency = self.symbol[:3] # self.fx_base_usd_currency
        
        
        if base_currency != "USD":
            
            fx_base = self.evt.getExchangeRate(base_currency, "USD")
            # self.evt.consoleLog(f'{base_currency}USD, {fx_base}')
        
        else:
            fx_base = 1
        
        quote_currency = self.symbol[3:]    
        if quote_currency != "USD" : # base currency is not USD
            
            fx = self.evt.getExchangeRate("USD", quote_currency)
            # print out result to console log
            # self.evt.consoleLog(f'USD{quote_currency}, {fx}')
        else: 
            fx = 1
        
            
        # -------------------- FSM ------------------------------------
        if self.fsm is None:
            if "JPY" in self.symbol :
                pip_size = 0.01
            else:                                     
                pip_size = 0.0001
                
            self.fsm = BearishFSM(
                self.data, instrument=self.symbol, evt=self.evt,
                account_balance=np.float32(available_balance),
                pip_size=pip_size,fx=fx,fx_base = fx_base, ts=self.last_time)
        else:
            self.fx_usd_quote_currency = fx
            self.fx_base_usd_currency = fx_base
            self.fsm.df = self.data
            self.fsm.account_balance = available_balance 
            self.fsm.update(self.last_time)
            
    def _aggregate_15m(self, last3):
        
        
        
        n   = len(self.data)
        op  = last3["Open"].iloc[0]
        cl  = last3["Close"].iloc[-1]
        bar = {
            "Datetime": last3["Datetime"].iloc[-1],
            "Open": op,
            "High": last3["High"].max(),
            "Low":  last3["Low"].min(),
            "Close": cl,
            "Green": "G" if cl > op else "R" if cl < op else "NO",
            "Instrument": self.symbol,
        }

        if n == 0:
            ema30 = ema120 = cl #  ema10 =

        elif n < 30: 
            prev  = self.data.iloc[n - 1]
            # ema10  = (1 - self.ALPHA_10)  * prev["EMA_10"]  + self.ALPHA_10  * cl
            ema30  = (1 - self.ALPHA_30)  * prev["EMA_30"]  + self.ALPHA_30  * cl
            ema120 = (1 - self.ALPHA_120) * prev["EMA_120"] + self.ALPHA_120 * cl

        else:
            prev  = self.data.iloc[n - 1] ###########################################  optimize the number of bars for calculating the slope: n-bar slope (e.g. 5) bars)
            # ema10  = (1 - self.ALPHA_10)  * prev["EMA_10"]  + self.ALPHA_10  * cl
            ema30  = (1 - self.ALPHA_30)  * prev["EMA_30"]  + self.ALPHA_30  * cl
            ema120 = (1 - self.ALPHA_120) * prev["EMA_120"] + self.ALPHA_120 * cl

        
        bar["EMA_30"]  = ema30
        bar["EMA_120"] = ema120
        r = ema30 / ema120
        bar["trend"] = "BU" if r > 1.001 else "BE" if r < 0.999 else "NO" ################### change to detect slope

        
        """
        bar["EMA_10"]  = ema10
        # ---------- SMA 50 ---------------------------------------------
        if n >= 49:                                    # we already have 49 closes;
            sma50 = (self.data["Close"].iloc[-49:].sum() + cl) / 50
        else:
            current_mean = (self.data["Close"].sum() + cl) / (n + 1)
            weight = (n + 1) / 50            # grows from 0.02 … 0.98
            sma50 = weight * current_mean + (1 - weight) * cl
    
        bar["SMA_50"] = sma50
        
        # ---------- trend_SMA (EMA30 vs SMA50) --------------------------
        r2 = ema10 / sma50
        bar["trend_SMA"] = ("BU" if r2 > 1.001 else
                            "BE" if r2 < 0.999 else
                            "NO")
        
        """
        self.data = pd.concat([self.data, pd.DataFrame([bar]).set_index("Datetime",drop=False)])
        


        

class AlgoEvent:

    def __init__(self):
        self.evt   = None
        self.books = {}          # symbol → InstrumentBar

    def start(self, mEvt):
        self.evt = AlgoAPI_Backtest.AlgoEvtHandler(self, mEvt)
        self.evt.start()
            
    def on_bulkdatafeed(self, isSync, bd, ab):
        
        balance = ab["availableBalance"]
        
        for sym, snap in bd.items():        # sym: AUDNZD, EURUSD, ... snap: {'instrument': 'AUDNZD', 'timestamp': datetime.datetime(1970, 1, 1, 0, 0), 'askPrice': -1, ...}
            if sym not in self.books:       # first sight → create book
                self.books[sym] = InstrumentBar(sym, self.evt)
            else: 
                self.books[sym].update(snap, balance)

    def on_orderfeed(self, of):
        
        sym = of.instrument
        if sym not in self.books:
            return                        # safety
        
        fsm = self.books[sym].fsm
        
        # -------------- order filled -----------------
        if of.openclose == "open" and of.status == "success":
            # Is this the order we are waiting for?
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.position_open      = True
                fsm.stop_loss_price    = fsm.pending["stop"]
                fsm.take_profit_price  = fsm.pending["take"]
                fsm.open_volume        = fsm.pending["volume"]
                fsm.enter_market_trade_side = fsm.pending["trend"]
    
                fsm.pending = None      # clear
                old_state   = fsm.state
                fsm.state   = "ENTER_MARKET"
                fsm.log_state_change(old_state, fsm.state, of.orderRef,
                                     "broker filled order")
        # -------------- order rejected ---------------
        elif of.openclose == "open" and of.status != "success":
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.pending = None
                old_state = fsm.state
                fsm.state = "ENTRY_SIGNAL"     # back to previous state
                fsm.log_state_change(old_state, fsm.state, of.orderRef,
                                     "order rejected")
                                     
        # -------------- position closed --------------
        if of.openclose == 'close' and of.status == 'success': # return 'success' if an order is opened/ closed successfully, or when a limit/stop order is filled
            fsm.reset()
    
    
    def on_newsdatafeed(self, nd):
        pass

    def on_weatherdatafeed(self, wd):
        pass
    
    def on_econsdatafeed(self, ed):
        pass
        
    def on_corpAnnouncement(self, ca):
        pass
    
    def on_dailyPLfeed(self, pl):
        pass

    def on_openPositionfeed(self, op, oo, uo):
        pass





#### Version 12: Changes

1. Following up on the idea of the book, we enter market when price pullback to 
    - 0.618, 
    - 0.786,
    - 0.71,
    skipping the engulfing candle part;
    And we fine-tune different parameters, including 
    - atr_multiplier,
    - risk_reward_ratio,
    - atr_period

    Result: Among this simplified version, price pullback to 0.71 with atr_multiplier = 2, risk_reward_ratio = 2, and atr_period = 14 performs the best ( However, large profits/losses may occur because we're placing less constraints in our entry points. )
    


In [ ]:
from AlgoAPI import AlgoAPIUtil, AlgoAPI_Backtest
import pandas as pd
import numpy as np
import mplfinance as mpf
from collections import deque
from datetime import datetime, timedelta
    
class BearishFSM: # Finite State Machine

        # ==============================================================
        # ========================  __init__ ===========================
        # ==============================================================
    def __init__(self, df, instrument, evt, account_balance=0, pip_size = 0.0001, fx = 1, fx_base = 1, ts = None):
        
        self.df = df
        self.evt = evt
        self.account_balance = account_balance
        
        # Enter market flag
        self.position_open = False
        self.pending = None       # ← dict that holds the ticket waiting
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.open_volume = None
        
        self.state = "IDLE"
        self.low_price = None
        
        self.low_price_ts = None
        self.high_price = None
        self.high_price_ts = None

        self.fib_labels = []
        self.fib_dict = {} # not a must to initialize, just for good habit
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.trend_changing_ts = ts # do not reset this, always keep record of 
        
##################################################################################################################
####################################### optimize #################################################################
##################################################################################################################

        # do not reset the folloiwng parameters, test for optimal values
        self.risk_reward_ratio = 1.8
        self.atr_period = 14
        self.atr_multiplier = 2 # pip_size = atr_multiplier * atr_value, atr_multiplier = 2.0 means stop loss is 2 ATRs away
        self.risk_percent = 0.5
        self.dynamic_atr_window = False
        # self.trend_ema10_sma50 = False # If True, the trend will be determined by EMA10 and SMA50, if False, determined by EMA30 and EMA120
        
        #### reduce the number of conditions:
        self.weak_engulfing_candle = True
##################################################################################################################
##################################################################################################################
##################################################################################################################
        # this is unique in every func call
        self.instrument = instrument
        self.pip_size = pip_size
        self.fx_usd_quote_currency = fx
        self.fx_base_usd_currency = fx_base
        # ==============================================================
        # ========================  calculate_log_state_change =========
        # ==============================================================
        
    def log_state_change(self, old_state, new_state, ts, label=None):
        if 'Datetime' in self.df.columns and ts in self.df.index:
            date_str = str(self.df.loc[ts, 'Datetime'])
        else:
            date_str = str(ts)
        msg = f" State: {old_state} → {new_state}"
        if label:
            msg += f" ({label})"
        # self.evt.consoleLog(msg)
        
        # ==============================================================
        # ========================  calculate_lot_size  ================
        # ==============================================================
        
    def calculate_lot_size(self, risk_percent, stop_loss_pips, max_leverage = 4.5): # base currency vs quote currency
    
        # 1. Calculate amount to risk
        risk_amount = self.account_balance * (risk_percent / 100)
        
        # 2. calculate pip value per lot (in quote currency)
        pip_value_per_lot_quote = self.pip_size * 100000 # If we are trading AUDJPY, the value will be 0.01 * 100000 = 1000 JPY
        
        # 3. calculate total risk per lot ( in quote currency)
        total_risk_per_lot_quote = stop_loss_pips * pip_value_per_lot_quote
        
        # 4. calculate total risk per lot ( in account currency, i.e. USD)
            # we need real time USD/quote_currency price, in this case, USDJPY
        total_risk_per_lot_USD = total_risk_per_lot_quote / self.fx_usd_quote_currency # if quote currency = USD, this will be 1
        
        # 5. calculate lot size
        lot_size = risk_amount / total_risk_per_lot_USD

        # calculate maximum affordable lot size after using margin (e.g. we are trading AUDUSD)
            # Notional Value per lot (in Base Currency) = 100000 AUD
            
        # Notation Value per lot (in account currency) = 100000 x AUDUSD price
        notation_per_lot_USD = 100000 * self.fx_base_usd_currency
            
        lot_margin = (self.account_balance * max_leverage) / notation_per_lot_USD
        
        lot_final = min(lot_size, lot_margin) # all-in 
        
        self.evt.consoleLog(
            f"{self.instrument} bal={self.account_balance:.2f} risk%={risk_percent} sl={stop_loss_pips} "
            f"riskAmt={risk_amount:.2f} pipVal/lot={pip_value_per_lot_quote:.5f} risk/lot(USD)={total_risk_per_lot_USD}"
            f"riskLot={lot_size:.2f} margLot={lot_margin:.2f} finalLot={lot_final:.2f}"
        )
        
        return round(lot_final, 2)
        
        
        # ==============================================================
        # ========================  calc_fibonacci_levels  =============
        # ==============================================================
        
    def calc_fibonacci_levels(self, start_ts, end_ts, levels=None, trend="NO",
                         entry_signal=None, engulfing_candle=None,
                         R=None, atr_period=None, atr_multiplier=None,dynamic_atr_window = None):
                             
        if levels is None:
            levels = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]
            
        if dynamic_atr_window is None:
            dynamic_atr_window = self.dynamic_atr_window
            
        if atr_multiplier is None:
            atr_multiplier = self.atr_multiplier
        
        if R is None:
            R = self.risk_reward_ratio
        
        
        df = self.df
        """
        start_idx =  df.index.get_loc(start_ts)
        end_idx = df.index.get_loc(end_ts)
        
        start_ts = df.index[start_idx] # still a timestamp
        end_ts = df.index[end_idx]
        """
        window = df.loc[start_ts : end_ts]
                            
        min_price = window['Low'].min()
        max_price = window['High'].max()
    
        fib_labels = []
        for level in levels:
            if trend == "BE": # bearish, sell
                y = min_price + (max_price - min_price) * level
            else:
                y = max_price - (max_price - min_price) * level
            fib_labels.append((level, y))
        
        if engulfing_candle is None:
            return fib_labels
        
        if atr_period is None:

            if dynamic_atr_window == False:
                # Fixed atr period
                atr_period = self.atr_period
            else: 
                # dynamic atr period
                start_idx = df.index.get_loc(start_ts)
                enter_idx = df.index.get_loc(engulfing_candle)
            
                dynamic_period = enter_idx - start_idx
                atr_period = max(10, min(dynamic_period, 50))
        
        # Normal ATR
        min_start_idx = max(0, df.index.get_loc(start_ts) - 10)
        min_start_ts = df.index[min_start_idx]
        sl = df.loc[min_start_ts : engulfing_candle].copy()
        
        sl['H-L']  = sl['High'] - sl['Low']
        sl['H-PC'] = (sl['High'] - sl['Close'].shift(1)).abs()
        sl['L-PC'] = (sl['Low']  - sl['Close'].shift(1)).abs()
        sl['TR']    = sl[['H-L', 'H-PC', 'L-PC']].max(axis=1)
        
        # ATR: rolling mean of TR
        sl['ATR'] = sl['TR'].rolling(window=atr_period).mean()
         
        # Retrieve ATR value at the engulfing candle index
        atr_value = sl['ATR'].iloc[-1]
        pip_size = atr_multiplier * atr_value
        
        # Retrieve the close price of the engulfing candle
        candle = self.df.loc[engulfing_candle]
        close_price = candle['Close']
        
        # Initialize stop loss and take profit prices
        stop_loss_price = None
        take_profit_price = None
    
        if trend == "BE" :
            stop_loss_price = close_price + pip_size
            take_profit_price = close_price - R * pip_size
        else:  # bullish
            stop_loss_price = close_price - pip_size
            take_profit_price = close_price + R * pip_size
    
        return stop_loss_price, take_profit_price
        
    # ==============================================================
    # ======================== send_entry_order  =============
    # ==============================================================
    
    def send_entry_order(self, ts, trend, entry_price,
                         stop, take, volume):

        # remember what we just asked the broker to do
        self.pending = dict(
            orderRef = ts,         # we use the timestamp as reference
            trend    = trend,      # "BU" / "BE"
            stop     = stop,
            take     = take,
            volume   = volume
        )

        order = AlgoAPIUtil.OrderObject()
        order.instrument   = self.instrument
        order.orderRef     = ts
        order.openclose    = "open"
        order.buysell      = 1 if trend == "BU" else -1
        order.ordertype    = 0          # market
        order.volume       = volume
        order.stopLossLevel= stop
        order.takeProfitLevel = take
        self.evt.sendOrder(order)

        # NOTE: do NOT set state / flags here;
        #       wait for on_orderfeed()
    
    # -------------------------------------------------------------
    # monitor_exit()  – runs while we are in the trade
    # -------------------------------------------------------------
    def monitor_exit(self, ts):
        """
        Called every bar only while self.position_open is True.
        Produces the same log lines you had in your old ENTER_MARKET
        branch.  If you additionally want to *close* the trade,
        send a close-order here.
        """
        if self.take_profit_price is None or self.stop_loss_price is None:
            return      # safety – nothing to do

        high = self.df["High"].loc[ts]
        low  = self.df["Low"].loc[ts]

        if self.enter_market_trade_side == "BU":          # long trade
            if high >= self.take_profit_price:
                self.evt.consoleLog(
                    f"{self.instrument} Price touched take_profit at {high:.5f} "
                    f"(take_profit: {self.take_profit_price:.5f}) – "
                    "waiting for broker to close"
                )
            elif low <= self.stop_loss_price:
                self.evt.consoleLog(
                    f"{self.instrument} Price touched stop_loss at {low:.5f} "
                    f"(stop_loss: {self.stop_loss_price:.5f}) – "
                    "waiting for broker to close"
                )

        else:                                             # short trade
            if low <= self.take_profit_price:
                self.evt.consoleLog(
                    f"{self.instrument} Price touched take_profit at {low:.5f} "
                    f"(take_profit: {self.take_profit_price:.5f}) – "
                    "waiting for broker to close"
                )
            elif high >= self.stop_loss_price:
                self.evt.consoleLog(
                    f"{self.instrument} Price touched stop_loss at {high:.5f} "
                    f"(stop_loss: {self.stop_loss_price:.5f}) – "
                    "waiting for broker to close"
                )
                
    # ==============================================================
    # ========================  update()  =============
    # ==============================================================
    
    def update(self, ts: pd.Timestamp):
        
        # 1️⃣ When a position is open we do only exit-logic and leave
        #    the rest of the gigantic state-machine untouched.
        #    (So IDLE / PULLBACK_DETECTED / … cannot fire again.)
        if self.position_open:
            self.monitor_exit(ts)        # <-- will emit console logs
            return                       # <-- early exit
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        prev_ts = df.index[pos - 1] # still a timestamp
        
        if df.index.get_loc(ts) < 3: # Get current timestamp location
            return
        
        #--- 2. prepare frequently-used columns (all remain Series!) ------
        trend   = df["trend"]               # string
        # trend_SMA = df['trend_SMA']
        greens  = df["Green"]               # string
        opens   = df["Open"]
        closes  = df["Close"]
        
        lowest_body_price = opens.where(greens == 'G', closes) # if greens == 'G', use values in open, else use values in closes 
        highest_body_price = closes.where(greens == 'G', opens) # the index of this series keeps the same Datetime index as the original df
        
        # selects the row in df where the index = ts
        curr   = df.loc[ts]
        prev   = df.shift(1).loc[ts]       # one-bar shift gives previous values
        """
        if self.trend_ema10_sma50 == True:  ###########  use SMA50 and EMA10
            current_trend = curr['trend_SMA']
            prev_trend    = prev['trend_SMA']
        else: 
        """
        current_trend = curr['trend'] ############ use EMA120 and EMA30
        prev_trend    = prev['trend']

        if current_trend != prev_trend : 
            self.trend_changing_ts = ts
            if self.state != "ENTER_MARKET":
                self.reset()

        if (current_trend == "BE" and self.enter_market_trade_side == None):
            if self.state == "IDLE":
                if self.check_pullback(greens, opens, closes, ts, trend="BE"): # pullback should eat and cover the previous red candle
                    # logic
                    slices = lowest_body_price.loc[self.trend_changing_ts:ts] # Slicing with .loc[start:end] in pandas includes both endpoints
                    self.low_price = slices.min()
                    self.low_price_ts  = slices.idxmin()   # Return index(Datetime) of first occurrence of minimum over requested axis.
                    # debug
                    old_state = self.state  
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish pullback detected")
                    

            elif self.state == "PULLBACK_DETECTED":
                if lowest_body_price.loc[ts] < self.low_price:
                    # logic
                    slices = highest_body_price.loc[self.low_price_ts:ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    # debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish new low after pullback")

                    
            elif self.state == "BREAK_OF_STRUCTURE":
                # If price breaks the pev high (LH)
                if highest_body_price.loc[ts] > self.high_price:
                    
                    # logic
                    slices = lowest_body_price.loc[self.high_price_ts: ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish break of structure")

                elif self.check_pullback(greens, opens, closes, ts, trend="BE"):
                    
                    if lowest_body_price.shift(2).loc[ts] < self.low_price: #### use 2 timeframe before current ts , can incorporate to 3,4,5 or 6
                    
                        # Logic: Low price for confirming the next Lower high 
                        if pos >= 3:   
                            window = lowest_body_price.iloc[pos-3 : pos+1]
                            self.low_price     = window.min()
                            self.low_price_ts  = window.idxmin()

                        self.fib_labels = self.calc_fibonacci_levels(self.high_price_ts, self.low_price_ts, trend="BE")
                        self.fib_dict = dict(self.fib_labels)
                        
                        # debug
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, ts, "bearish waiting entry")

                    elif highest_body_price.loc[ts] >= self.low_price:
                        
                        # debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bearish back to pullback")

            elif self.state == "WAITING_ENTRY":
                
                fib_618_price = self.fib_dict.get(0.618)
                fib_000_price = self.fib_dict.get(0.000)
                
                # price rebound back to 0.618
                if fib_618_price and df["High"].loc[ts] >= fib_618_price: # reference the "High" price at the exact timestamp
                
                    # debug
                    self.entry_signal_ts = ts
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, ts, "bearish price rebound to 0.618")

                elif fib_618_price and lowest_body_price.loc[ts] <= fib_000_price: # price lower than 0.000
                
                    # Logic: break of structure, set new high point
                    slices = highest_body_price.loc[self.low_price_ts: ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bearish breaks 0.000 before entry")
            
            elif self.state == "ENTRY_SIGNAL": 
                fib_100_price = self.fib_dict.get(1.000)
                fib_000_price = self.fib_dict.get(0.000)
                

                if highest_body_price.loc[ts] >= fib_100_price: # no matter what, if the candle pass 1.00, we do not trade
                
                    # debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bearish cancels: price > 1.0")

                # Logic: Bearish Engulfing candle  
                else: 
                        

                    # Debug
                    self.engulfing_candle_ts = ts
                    
                    stop, take = self.calc_fibonacci_levels(self.high_price_ts, 
                            self.low_price_ts, trend="BE", entry_signal=self.entry_signal_ts, engulfing_candle=self.engulfing_candle_ts
                            )
                    
                    # Logic to calculate lot_size
                    entry_price = closes.loc[ts]
                    stop_loss_pips = abs(stop - entry_price) / self.pip_size
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips)
                    
                    # Log for debug
                    self.evt.consoleLog(
                        f"DEBUG ticket – side:SELL, instrument:{self.instrument},ab:{self.account_balance}, entry≈{entry_price:.5f} "
                        f"SL={stop:.5f} TP={take:.5f}"
                    )
                
                    
                    # place selling order
                    self.send_entry_order(ts, trend="BE",
                                entry_price=entry_price,
                                stop=stop, take=take,
                                volume=lot_size)
                    
                    # Debug
                    old_state = self.state
                    self.state = "PRE_ENTER_MARKET"
                    self.log_state_change(old_state, self.state, ts, "bearish entry: engulfing candle")
                    

            
        # ==============================================================
        # ========================  BULLISH  ============================
        # ==============================================================

        if (current_trend == "BU" and self.enter_market_trade_side == None):
            if self.state == "IDLE":    
                if self.check_pullback(greens, opens, closes, ts, trend="BU"):
                    
                    # Logic
                    slices = highest_body_price[self.trend_changing_ts:ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish pullback detected")

            elif self.state == "PULLBACK_DETECTED":
                if highest_body_price.loc[ts] > self.high_price:
                    
                    # Logic
                    slices = lowest_body_price[self.high_price_ts: ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish new high after pullback")

   
            elif self.state == "BREAK_OF_STRUCTURE":
                # structure broken downward -> restart pull-back search
                if lowest_body_price.loc[ts] < self.low_price:
                    
                    # Logic
                    slices = highest_body_price[self.low_price_ts : ts]
                    self.high_price = slices.max()
                    self.high_price_ts = slices.idxmax()
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish break of structure")

                # new 3 red pull-back -> possible Fib leg
                elif self.check_pullback(greens, opens, closes, ts, trend="BU"):
                    
                    if highest_body_price.shift(2).loc[ts] > self.high_price: # fresh HH
                        
                        # Logic
                        if pos >= 3:   
                            window = highest_body_price.iloc[pos-3 : pos+1]
                            self.high_price     = window.max()
                            self.high_price_ts  = window.idxmax()
                        
                        self.fib_labels = self.calc_fibonacci_levels(self.low_price_ts, self.high_price_ts, trend="BU")
                        self.fib_dict = dict(self.fib_labels)
                        
                        # Debug
                        old_state = self.state
                        self.state = "WAITING_ENTRY"
                        self.log_state_change(old_state, self.state, ts, "bullish waiting entry")

                    # price failed to beat prev high: treat as new pull-back
                    elif lowest_body_price.loc[ts] <= self.high_price:
                        
                        # Debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bullish back to pullback")
                        

            elif self.state == "WAITING_ENTRY":
                fib_618_bullish = self.fib_dict.get(0.618)
                fib_000_bullish = self.fib_dict.get(0.000)

                # price retraced to 0.618 – ready for bullish entry
                if fib_618_bullish and df['Low'].loc[ts] <= fib_618_bullish: # count the tip of the candlestick instead of the body
                
                    # Debug
                    self.entry_signal_ts = ts
                    old_state = self.state
                    self.state = "ENTRY_SIGNAL"
                    self.log_state_change(old_state, self.state, ts, "bullish price retraced to 0.618")

                # price breaks above 0.000 (HH) before touching 0.618
                elif fib_618_bullish and highest_body_price.loc[ts] > fib_000_bullish:
                    
                    # Logic
                    slices = lowest_body_price[self.high_price_ts:ts]
                    self.low_price = slices.min()
                    self.low_price_ts = slices.idxmin()
                    
                    # Debug
                    old_state = self.state
                    self.state = "BREAK_OF_STRUCTURE"
                    self.log_state_change(old_state, self.state, ts, "bullish breaks above 0.000 before entry")
                
            elif self.state == "ENTRY_SIGNAL":
                fib_100_bullish = self.fib_dict.get(1.000)
                fib_000_bullish = self.fib_dict.get(0.000)

                # cancel if price dumps below 1.000 extension
                if lowest_body_price.loc[ts] <= fib_100_bullish:
                    
                    # Debug
                    old_state = self.state
                    self.state = "PULLBACK_DETECTED"
                    self.log_state_change(old_state, self.state, ts, "bullish cancels: price < 1.0")

                # Bullish Engulfing candle
                else: 
                    
                    self.engulfing_candle_ts = ts
                    
                    stop, take = self.calc_fibonacci_levels(self.low_price_ts, 
                    self.high_price_ts,trend="BU", entry_signal=self.entry_signal_ts,
                    engulfing_candle=self.engulfing_candle_ts)
                    
                    # calculate volume
                    entry_price = closes.loc[ts]
                    stop_loss_pips = abs(stop - entry_price) / self.pip_size
                    lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips)
                    
                    self.evt.consoleLog(
                        f"DEBUG ticket – side:BUY, instrument: {self.instrument}, ab:{self.account_balance}, entry≈{entry_price:.5f} "
                        f"SL={stop:.5f} TP={take:.5f}"
                    )
                    # place buying order
                    self.send_entry_order(ts, trend="BU",
                              entry_price=entry_price,
                              stop=stop, take=take,
                              volume=lot_size)
                    
                    # Debug
                    old_state = self.state
                    self.state = "PRE_ENTER_MARKET"
                    self.log_state_change(old_state, self.state, ts, "bullish entry: engulfing candle")
                    


        
        # ==============================================================
        # ========================  pullback()  ========================
        # ==============================================================     
        
    def check_pullback(self, greens, opens, closes, ts, trend):
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        
        if trend == "BE":
            # Check for 3, 4, 5, or 6 consecutive green candles.
            for window in [3, 4, 5, 6]:

                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                # Boundary check
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                # Check if all are green
                if all(greens.loc[ts_] == 'G' for ts_ in window_ts):
                    # Previous candle must be red
                    if greens.loc[prev_ts] == 'R':
                        # Any green candle closes above the open of previous red candle
                        if any(closes.loc[ts_] >= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False
        
        if trend == "BU":
            for window in [3,4,5,6]:
                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                if all(greens.loc[ts_]!= 'G' for ts_ in window_ts): # all previous not green candles 
                    if greens.loc[prev_ts] == 'G':
                        if any(closes.loc[ts_] <= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False

    def reset(self):
        self.state = "IDLE"
        self.low_price  = self.low_price_ts  =  None
        self.high_price = self.high_price_ts =  None
        self.fib_labels = []
        self.fib_dict = {} 
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.position_open = False
        self.open_volume = None


class InstrumentBar:
    
    """Bar builder + FSM for ONE instrument."""
    
    ALPHA_30  = 2 / 31
    ALPHA_120 = 2 / 121
    # ALPHA_10 = 2 / 11
    MIN_BARS  = 120 
    MAX_BARS  = 700
    
    def __init__(self, symbol: str, evt):
        
        self.symbol    = symbol
        self.evt       = evt
        self.last_time = datetime(2010, 1, 1)
        self.fsm       = None
        
        NFLOAT = "float32"
        
        # ------------- 5-min buffer ---------------------------------
        self.raw_5m = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument"])
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "category",
                "Instrument": "object"}))
                
        # ------------- 15-min table ---------------------------------
        self.data = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument", "EMA_30", "EMA_120", "trend",]) # 'SMA_50','trend_SMA',"EMA_10"
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "category", "Instrument": "object",
                "EMA_30": NFLOAT, "EMA_120": NFLOAT,
                "trend": "category",
                # 'SMA_50': NFLOAT,
                # "trend_SMA": "category",
                # "EMA_10": NFLOAT
            })
            .set_index("Datetime", drop=False))
        
    def update(self, snap: dict, available_balance: float):
        
        """
        snap = bd[symbol]  – single-instrument snapshot from on_bulkdatafeed
        """
        # -------------------- append 5-min bar ------------------------
        green = ("G" if snap["lastPrice"] > snap["openPrice"]
                 else "R" if snap["lastPrice"] < snap["openPrice"]
                 else "NO")
        
        bar5 = {
            "Datetime": pd.to_datetime(snap["timestamp"]),
            "Open":  snap["openPrice"],
            "High":  snap["highPrice"],
            "Low":   snap["lowPrice"],
            "Close": snap["lastPrice"],
            "Green": green,
            "Instrument": snap["instrument"],
        }
        
        self.raw_5m = pd.concat([self.raw_5m,
                                 pd.DataFrame([bar5])], ignore_index=True)
                                 
        # avoid  Warning: data month not exists for ... fx error led to array memory explode
        if len(self.raw_5m) > 10:
            self.raw_5m = self.raw_5m.iloc[0:0]
        
        
        # ------------------- finished 15-min? -------------------------
        ts = pd.to_datetime(snap["timestamp"])
        if ts < self.last_time + timedelta(minutes=15):
            return
        self.last_time = ts
        
        last3 = self.raw_5m.tail(3)
        if len(last3) < 3:
            return    # need full 15-minute slice
        
        self.raw_5m = self.raw_5m.iloc[0:0]  # clear buffer
        
        self._aggregate_15m(last3)
        
        
        if len(self.data) < self.MIN_BARS:
            return
            
        
        if len(self.data) > self.MAX_BARS:
            self.data = self.data.iloc[-self.MAX_BARS:]
        
        # -------------------- get latest exchange rate  -----------------
        base_currency = self.symbol[:3] # self.fx_base_usd_currency
        
        
        if base_currency != "USD":
            
            fx_base = self.evt.getExchangeRate(base_currency, "USD")
            # self.evt.consoleLog(f'{base_currency}USD, {fx_base}')
        
        else:
            fx_base = 1
        
        quote_currency = self.symbol[3:]    
        if quote_currency != "USD" : # base currency is not USD
            
            fx = self.evt.getExchangeRate("USD", quote_currency)
            # print out result to console log
            # self.evt.consoleLog(f'USD{quote_currency}, {fx}')
        else: 
            fx = 1
        
            
        # -------------------- FSM ------------------------------------
        if self.fsm is None:
            if "JPY" in self.symbol :
                pip_size = 0.01
            else:                                     
                pip_size = 0.0001
                
            self.fsm = BearishFSM(
                self.data, instrument=self.symbol, evt=self.evt,
                account_balance=np.float32(available_balance),
                pip_size=pip_size,fx=fx,fx_base = fx_base, ts=self.last_time)
        else:
            self.fx_usd_quote_currency = fx
            self.fx_base_usd_currency = fx_base
            self.fsm.df = self.data
            self.fsm.account_balance = available_balance 
            self.fsm.update(self.last_time)
            
    def _aggregate_15m(self, last3):
        
        
        
        n   = len(self.data)
        op  = last3["Open"].iloc[0]
        cl  = last3["Close"].iloc[-1]
        bar = {
            "Datetime": last3["Datetime"].iloc[-1],
            "Open": op,
            "High": last3["High"].max(),
            "Low":  last3["Low"].min(),
            "Close": cl,
            "Green": "G" if cl > op else "R" if cl < op else "NO",
            "Instrument": self.symbol,
        }

        if n == 0:
            ema30 = ema120 = cl #  ema10 =

        elif n < 30: 
            prev  = self.data.iloc[n - 1]
            # ema10  = (1 - self.ALPHA_10)  * prev["EMA_10"]  + self.ALPHA_10  * cl
            ema30  = (1 - self.ALPHA_30)  * prev["EMA_30"]  + self.ALPHA_30  * cl
            ema120 = (1 - self.ALPHA_120) * prev["EMA_120"] + self.ALPHA_120 * cl

        else:
            prev  = self.data.iloc[n - 1] ###########################################  optimize the number of bars for calculating the slope: n-bar slope (e.g. 5) bars)
            # ema10  = (1 - self.ALPHA_10)  * prev["EMA_10"]  + self.ALPHA_10  * cl
            ema30  = (1 - self.ALPHA_30)  * prev["EMA_30"]  + self.ALPHA_30  * cl
            ema120 = (1 - self.ALPHA_120) * prev["EMA_120"] + self.ALPHA_120 * cl

        
        bar["EMA_30"]  = ema30
        bar["EMA_120"] = ema120
        r = ema30 / ema120
        bar["trend"] = "BU" if r > 1.001 else "BE" if r < 0.999 else "NO" ################### change to detect slope

        
        """
        bar["EMA_10"]  = ema10
        # ---------- SMA 50 ---------------------------------------------
        if n >= 49:                                    # we already have 49 closes;
            sma50 = (self.data["Close"].iloc[-49:].sum() + cl) / 50
        else:
            current_mean = (self.data["Close"].sum() + cl) / (n + 1)
            weight = (n + 1) / 50            # grows from 0.02 … 0.98
            sma50 = weight * current_mean + (1 - weight) * cl
    
        bar["SMA_50"] = sma50
        
        # ---------- trend_SMA (EMA30 vs SMA50) --------------------------
        r2 = ema10 / sma50
        bar["trend_SMA"] = ("BU" if r2 > 1.001 else
                            "BE" if r2 < 0.999 else
                            "NO")
        
        """
        self.data = pd.concat([self.data, pd.DataFrame([bar]).set_index("Datetime",drop=False)])
        


        

class AlgoEvent:

    def __init__(self):
        self.evt   = None
        self.books = {}          # symbol → InstrumentBar

    def start(self, mEvt):
        self.evt = AlgoAPI_Backtest.AlgoEvtHandler(self, mEvt)
        self.evt.start()
            
    def on_bulkdatafeed(self, isSync, bd, ab):
        
        balance = ab["availableBalance"]
        
        for sym, snap in bd.items():        # sym: AUDNZD, EURUSD, ... snap: {'instrument': 'AUDNZD', 'timestamp': datetime.datetime(1970, 1, 1, 0, 0), 'askPrice': -1, ...}
            if sym not in self.books:       # first sight → create book
                self.books[sym] = InstrumentBar(sym, self.evt)
            else: 
                self.books[sym].update(snap, balance)

    def on_orderfeed(self, of):
        
        sym = of.instrument
        if sym not in self.books:
            return                        # safety
        
        fsm = self.books[sym].fsm
        
        # -------------- order filled -----------------
        if of.openclose == "open" and of.status == "success":
            # Is this the order we are waiting for?
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.position_open      = True
                fsm.stop_loss_price    = fsm.pending["stop"]
                fsm.take_profit_price  = fsm.pending["take"]
                fsm.open_volume        = fsm.pending["volume"]
                fsm.enter_market_trade_side = fsm.pending["trend"]
    
                fsm.pending = None      # clear
                old_state   = fsm.state
                fsm.state   = "ENTER_MARKET"
                fsm.log_state_change(old_state, fsm.state, of.orderRef,
                                     "broker filled order")
        # -------------- order rejected ---------------
        elif of.openclose == "open" and of.status != "success":
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.pending = None
                old_state = fsm.state
                fsm.state = "ENTRY_SIGNAL"     # back to previous state
                fsm.log_state_change(old_state, fsm.state, of.orderRef,
                                     "order rejected")
                                     
        # -------------- position closed --------------
        if of.openclose == 'close' and of.status == 'success': # return 'success' if an order is opened/ closed successfully, or when a limit/stop order is filled
            fsm.reset()
    
    
    def on_newsdatafeed(self, nd):
        pass

    def on_weatherdatafeed(self, wd):
        pass
    
    def on_econsdatafeed(self, ed):
        pass
        
    def on_corpAnnouncement(self, ca):
        pass
    
    def on_dailyPLfeed(self, pl):
        pass

    def on_openPositionfeed(self, op, oo, uo):
        pass





## Incorporating Statistical Analysis

We spend too much time in refining and backtesting different technical analysis, however, while financial data can exhibit periodic or cyclical patterns, it is also heavily influenced by randomness and external shocks. Traditional technical analysis methods often struggle to capture these complexities, especially in the presence of noise and non-linear relationships.

So, our plan is to use statistical analysis and machine learning techniques, i.e.

- use z-index to filter out trades with low probability of winning, base on historical data
- Evaluate model performance using rigorous out-of-sample testing

#### Version 13 changes

1. Each trade entry (long or short) now requires the Z-score to meet a threshold, configurable per instrument. If the Z-score is not met at the entry signal, no trade is placed and the strategy returns to a pullback-waiting state.

2. The system supports multiple forex pairs, each with individual settings for long/short permissions and Z-score thresholds. Trading directions can be enabled or disabled per symbol.

3. All trades are logged with entry/exit details, Z-score, and stop-loss pip size, to facilitate further statistical analysis

4. Trades are only executed if the calculated atr stop-loss falls within a defined pip range (8–30 pips), preventing unrealistically small/large pip size

In [ ]:
from AlgoAPI import AlgoAPIUtil, AlgoAPI_Backtest
import pandas as pd
import numpy as np
import mplfinance as mpf
from datetime import datetime, timedelta
    
class BearishFSM: # Finite State Machine

        # ==============================================================
        # ========================  __init__ ===========================
        # ==============================================================
    def __init__(self, df, instrument, evt, account_balance=0, pip_size = 0.0001, fx = 1, fx_base = 1, ts = None, allow_long = False,long_z_index = None, allow_short = False, short_z_index = None):
        
        self.df = df
        self.evt = evt
        self.account_balance = account_balance
        
        # Enter market flag
        self.position_open = False
        self.pending = None       # ← dict that holds the ticket waiting
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.open_volume = None
        
        self.state = "IDLE"
        self.low_price = None
        
        self.low_price_ts = None
        self.high_price = None
        self.high_price_ts = None

        self.fib_labels = []
        self.fib_dict = {} # not a must to initialize, just for good habit
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.trend_changing_ts = ts # do not reset this, always keep record of 
        
##################################################################################################################
####################################### optimize #################################################################
##################################################################################################################

        # do not reset the folloiwng parameters, test for optimal values
        self.risk_reward_ratio = 1.8
        self.atr_period = 14
        self.atr_multiplier = 2.25 # pip_size = atr_multiplier * atr_value, atr_multiplier = 2.0 means stop loss is 2 ATRs away
        self.risk_percent = 0.5
        self.dynamic_atr_window = False
        # self.trend_ema10_sma50 = False # If True, the trend will be determined by EMA10 and SMA50, if False, determined by EMA30 and EMA120
        
##################################################################################################################
##################################################################################################################
##################################################################################################################
        # this is unique in every func call
        self.instrument = instrument
        self.pip_size = pip_size
        self.fx_usd_quote_currency = fx
        self.fx_base_usd_currency = fx_base
        
        self.allow_long = allow_long
        self.long_z_index = long_z_index
        self.allow_short = allow_short
        self.short_z_index = short_z_index

    # ==============================================================
    # ========================  current_zscore             =========
    # ==============================================================
    def current_zscore(self, ts):
        if "Zscore" not in self.df.columns:
            return np.nan
        return float(self.df["Zscore"].loc[ts])
    # ==============================================================
    # ========================  calculate_log_state_change =========
    # ============================================================== 
    def log_state_change(self, old_state, new_state, ts, label=None):
        if 'Datetime' in self.df.columns and ts in self.df.index:
            date_str = str(self.df.loc[ts, 'Datetime'])
        else:
            date_str = str(ts)
        msg = f" State: {old_state} → {new_state}"
        if label:
            msg += f" ({label})"
        # self.evt.consoleLog(msg)
        
        # ==============================================================
        # ========================  calculate_lot_size  ================
        # ==============================================================
        
    def calculate_lot_size(self, risk_percent, stop_loss_pips, max_leverage = 4.5): # base currency vs quote currency
    
        # 1. Calculate amount to risk
        risk_amount = self.account_balance * (risk_percent / 100)
        
        # 2. calculate pip value per lot (in quote currency)
        pip_value_per_lot_quote = self.pip_size * 100000 # If we are trading AUDJPY, the value will be 0.01 * 100000 = 1000 JPY
        
        # 3. calculate total risk per lot ( in quote currency)
        total_risk_per_lot_quote = stop_loss_pips * pip_value_per_lot_quote
        
        # 4. calculate total risk per lot ( in account currency, i.e. USD)
            # we need real time USD/quote_currency price, in this case, USDJPY
        self.evt.consoleLog(self.fx_usd_quote_currency)
        total_risk_per_lot_USD = total_risk_per_lot_quote / self.fx_usd_quote_currency # if quote currency = USD, this will be 1
        
        # 5. calculate lot size
        lot_size = risk_amount / total_risk_per_lot_USD

        # calculate maximum affordable lot size after using margin (e.g. we are trading AUDUSD)
            # Notional Value per lot (in Base Currency) = 100000 AUD
            
        # Notation Value per lot (in account currency) = 100000 x AUDUSD price
        notation_per_lot_USD = 100000 * self.fx_base_usd_currency
            
        lot_margin = (self.account_balance * max_leverage) / notation_per_lot_USD
        
        lot_final = min(lot_size, lot_margin) # all-in 
        
        self.evt.consoleLog(
            f"{self.instrument} bal={self.account_balance:.2f} risk%={risk_percent} sl={stop_loss_pips} "
            f"riskAmt={risk_amount:.2f} pipVal/lot={pip_value_per_lot_quote:.5f} risk/lot(USD)={total_risk_per_lot_USD}"
            f"riskLot={lot_size:.2f} margLot={lot_margin:.2f} finalLot={lot_final:.2f}"
        )
        
        return round(lot_final, 2)
        
        
        # ==============================================================
        # ========================  calc_fibonacci_levels  =============
        # ==============================================================
        
    def calc_fibonacci_levels(self, start_ts, end_ts, levels=None, trend="NO",
                         entry_signal=None, engulfing_candle=None,
                         R=None, atr_period=None, atr_multiplier=None,dynamic_atr_window = None):
                             
        if levels is None:
            levels = [0, 0.236, 0.382, 0.5, 0.618, 0.710, 0.786, 1]
            
        if dynamic_atr_window is None:
            dynamic_atr_window = self.dynamic_atr_window
            
        if atr_multiplier is None:
            atr_multiplier = self.atr_multiplier
        
        if R is None:
            R = self.risk_reward_ratio
        
        
        df = self.df

        window = df.loc[start_ts : end_ts]
                            
        min_price = window['Low'].min()
        max_price = window['High'].max()
    
        fib_labels = []
        for level in levels:
            if trend == "BE": # bearish, sell
                y = min_price + (max_price - min_price) * level
            else:
                y = max_price - (max_price - min_price) * level
            fib_labels.append((level, y))
        
        if engulfing_candle is None:
            return fib_labels
        
        if atr_period is None:

            if dynamic_atr_window == False:
                # Fixed atr period
                atr_period = self.atr_period
            else: 
                # dynamic atr period
                start_idx = df.index.get_loc(start_ts)
                enter_idx = df.index.get_loc(engulfing_candle)
            
                dynamic_period = enter_idx - start_idx
                atr_period = max(10, min(dynamic_period, 50))
        
        # Normal ATR
        min_start_idx = max(0, df.index.get_loc(start_ts) - 10)
        min_start_ts = df.index[min_start_idx]
        sl = df.loc[min_start_ts : engulfing_candle].copy()
        
        sl['H-L']  = sl['High'] - sl['Low']
        sl['H-PC'] = (sl['High'] - sl['Close'].shift(1)).abs()
        sl['L-PC'] = (sl['Low']  - sl['Close'].shift(1)).abs()
        sl['TR']    = sl[['H-L', 'H-PC', 'L-PC']].max(axis=1)
        
        # ATR: rolling mean of TR
        sl['ATR'] = sl['TR'].rolling(window=atr_period).mean()
         
        # Retrieve ATR value at the engulfing candle index
        atr_value = sl['ATR'].iloc[-1]
        pip_size = atr_multiplier * atr_value
        
        # Retrieve the close price of the engulfing candle
        candle = self.df.loc[engulfing_candle]
        close_price = candle['Close']
        
        # Initialize stop loss and take profit prices
        stop_loss_price = None
        take_profit_price = None
    
        if trend == "BE" :
            stop_loss_price = close_price + pip_size
            take_profit_price = close_price - R * pip_size
        else:  # bullish
            stop_loss_price = close_price - pip_size
            take_profit_price = close_price + R * pip_size
    
        return stop_loss_price, take_profit_price
        
    # ==============================================================
    # ======================== send_entry_order  =============
    # ==============================================================
    
    def send_entry_order(self, ts, trend, entry_price,
                         stop, take, volume, stoploss_pip_size):

        # remember what we just asked the broker to do
        self.pending = dict(
            orderRef = ts,         # we use the timestamp as reference
            trend    = trend,      # "BU" / "BE"
            stop     = stop,
            take     = take,
            volume   = volume,
            stoploss_pip = stoploss_pip_size
        )

        order = AlgoAPIUtil.OrderObject()
        order.instrument   = self.instrument
        order.orderRef     = ts
        order.openclose    = "open"
        order.buysell      = 1 if trend == "BU" else -1
        order.ordertype    = 0          # market
        order.volume       = volume
        order.stopLossLevel= stop
        order.takeProfitLevel = take
        self.evt.sendOrder(order)

        # NOTE: do NOT set state / flags here;
        #       wait for on_orderfeed()
    
    # ==============================================================
    # ========================  update()  =============
    # ==============================================================
    
    def update(self, ts: pd.Timestamp):
        
        # 1️⃣ When a position is open we do only exit-logic and leave
        #    the rest of the gigantic state-machine untouched.
        #    (So IDLE / PULLBACK_DETECTED / … cannot fire again.)
        if self.position_open:
            return                       # <-- early exit
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        
        if df.index.get_loc(ts) < 3: # Get current timestamp location
            return
        
        #--- 2. prepare frequently-used columns (all remain Series!) ------
        greens  = df["Green"]               # string
        opens   = df["Open"]
        closes  = df["Close"]
        
        lowest_body_price = opens.where(greens == 'G', closes) # if greens == 'G', use values in open, else use values in closes 
        highest_body_price = closes.where(greens == 'G', opens) # the index of this series keeps the same Datetime index as the original df
        
        # selects the row in df where the index = ts
        curr   = df.loc[ts]
        prev   = df.shift(1).loc[ts]       # one-bar shift gives previous values

        current_trend = curr['trend'] ############ use EMA120 and EMA30
        prev_trend    = prev['trend']

        if current_trend != prev_trend : 
            self.trend_changing_ts = ts
            if self.state != "ENTER_MARKET":
                self.reset()
                
        # ==============================================================
        # ========================  BEARISH  ============================
        # ==============================================================
        if self.allow_short == True: 
            if (current_trend == "BE" and self.enter_market_trade_side == None):
                if self.state == "IDLE":
                    if self.check_pullback(greens, opens, closes, ts, trend="BE"): # pullback should eat and cover the previous red candle
                        # logic
                        slices = lowest_body_price.loc[self.trend_changing_ts:ts] # Slicing with .loc[start:end] in pandas includes both endpoints
                        self.low_price = slices.min()
                        self.low_price_ts  = slices.idxmin()   # Return index(Datetime) of first occurrence of minimum over requested axis.
                        # debug
                        old_state = self.state  
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bearish pullback detected")
                        return
                        
    
                elif self.state == "PULLBACK_DETECTED":
                    if lowest_body_price.loc[ts] < self.low_price:
                        # logic
                        slices = highest_body_price.loc[self.low_price_ts:ts]
                        self.high_price = slices.max()
                        self.high_price_ts = slices.idxmax()
                        # debug
                        old_state = self.state
                        self.state = "BREAK_OF_STRUCTURE"
                        self.log_state_change(old_state, self.state, ts, "bearish new low after pullback")
                        return
    
                        
                elif self.state == "BREAK_OF_STRUCTURE":
                    # If price breaks the pev high (LH)
                    if highest_body_price.loc[ts] > self.high_price:
                        
                        # logic
                        slices = lowest_body_price.loc[self.high_price_ts: ts]
                        self.low_price = slices.min()
                        self.low_price_ts = slices.idxmin()
                        
                        # debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bearish break of structure")
                        return
    
                    elif self.check_pullback(greens, opens, closes, ts, trend="BE"):
                        
                        if lowest_body_price.shift(2).loc[ts] < self.low_price: #### use 2 timeframe before current ts , can incorporate to 3,4,5 or 6
                        
                            # Logic: Low price for confirming the next Lower high 
                            if pos >= 3:   
                                window = lowest_body_price.iloc[pos-3 : pos+1]
                                self.low_price     = window.min()
                                self.low_price_ts  = window.idxmin()
    
                            self.fib_labels = self.calc_fibonacci_levels(self.high_price_ts, self.low_price_ts, trend="BE")
                            self.fib_dict = dict(self.fib_labels)
                            
                            # debug
                            old_state = self.state
                            self.state = "WAITING_ENTRY"
                            self.log_state_change(old_state, self.state, ts, "bearish waiting entry")
                            return
    
                        elif highest_body_price.loc[ts] >= self.low_price:
                            
                            # debug
                            old_state = self.state
                            self.state = "PULLBACK_DETECTED"
                            self.log_state_change(old_state, self.state, ts, "bearish back to pullback")
                            return
    
                elif self.state == "WAITING_ENTRY":
                    
                    fib_710_price = self.fib_dict.get(0.710)
                    fib_000_price = self.fib_dict.get(0.000)
                    
                    
                    # price rebound back to 0.710 &
                    if fib_710_price and df["High"].loc[ts] >= fib_710_price: # reference the "High" price at the exact timestamp
                    
                        # debug
                        self.entry_signal_ts = ts
                        old_state = self.state
                        self.state = "ENTRY_SIGNAL"
                        self.log_state_change(old_state, self.state, ts, "bearish price rebound to 0.710")
                        return
    
                    elif fib_710_price and lowest_body_price.loc[ts] <= fib_000_price: # price lower than 0.000
                    
                        # Logic: break of structure, set new high point
                        slices = highest_body_price.loc[self.low_price_ts: ts]
                        self.high_price = slices.max()
                        self.high_price_ts = slices.idxmax()
                        
                        # debug
                        old_state = self.state
                        self.state = "BREAK_OF_STRUCTURE"
                        self.log_state_change(old_state, self.state, ts, "bearish breaks 0.000 before entry")
                        return
                
                elif self.state == "ENTRY_SIGNAL": 
                    fib_100_price = self.fib_dict.get(1.000)
                    z = df["Zscore"].loc[ts]
                    
                    if highest_body_price.loc[ts] >= fib_100_price: # no matter what, if the candle pass 1.00, we do not trade
                    
                        # debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bearish cancels: price > 1.0")
                        return
    
                    # Logic: Bearish Engulfing candle  
                    elif z <= self.short_z_index: 
                            
                        # Debug
                        self.engulfing_candle_ts = ts
                        
                        stop, take = self.calc_fibonacci_levels(self.high_price_ts, 
                                self.low_price_ts, trend="BE", entry_signal=self.entry_signal_ts, engulfing_candle=self.engulfing_candle_ts
                                )
                        
                        # Logic to calculate lot_size
                        entry_price = closes.loc[ts]
                        stop_loss_pips = abs(stop - entry_price) / self.pip_size

                        if stop_loss_pips >= 30 or stop_loss_pips <= 8: 
                            # don't trade
                            old_state = self.state
                            self.state = "PULLBACK_DETECTED"
                            self.log_state_change(old_state, self.state, ts, "stop_loss_pips > 30, returning to pullback state")
                            return


                        lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips)
                        """
                        # Log for debug
                        self.evt.consoleLog(
                            f"DEBUG ticket – side:SELL, instrument:{self.instrument},ab:{self.account_balance}, entry≈{entry_price:.5f} "
                            f"SL={stop:.5f} TP={take:.5f}"
                        )
                    
                        """
                        # place selling order
                        self.send_entry_order(ts, trend="BE",
                                    entry_price=entry_price,
                                    stop=stop, take=take,
                                    volume=lot_size, stoploss_pip_size = stop_loss_pips)
                        
                        # Debug
                        old_state = self.state
                        self.state = "PRE_ENTER_MARKET"
                        self.log_state_change(old_state, self.state, ts, "bearish entry: engulfing candle")
                        return
                        
                    else: 
                        # don't trade
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "Z-score not met, returning to pullback state")
                        return
                    
        # ==============================================================
        # ========================  BULLISH  ============================
        # ==============================================================
        
        if self.allow_long == True:
            if (current_trend == "BU" and self.enter_market_trade_side == None):
                if self.state == "IDLE":    
                    if self.check_pullback(greens, opens, closes, ts, trend="BU"):
                        
                        # Logic
                        slices = highest_body_price[self.trend_changing_ts:ts]
                        self.high_price = slices.max()
                        self.high_price_ts = slices.idxmax()
                        
                        # Debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bullish pullback detected")
                        return
    
                elif self.state == "PULLBACK_DETECTED":
                    if highest_body_price.loc[ts] > self.high_price:
                        
                        # Logic
                        slices = lowest_body_price[self.high_price_ts: ts]
                        self.low_price = slices.min()
                        self.low_price_ts = slices.idxmin()
                        
                        # Debug
                        old_state = self.state
                        self.state = "BREAK_OF_STRUCTURE"
                        self.log_state_change(old_state, self.state, ts, "bullish new high after pullback")
                        return
    
       
                elif self.state == "BREAK_OF_STRUCTURE":
                    # structure broken downward -> restart pull-back search
                    if lowest_body_price.loc[ts] < self.low_price:
                        
                        # Logic
                        slices = highest_body_price[self.low_price_ts : ts]
                        self.high_price = slices.max()
                        self.high_price_ts = slices.idxmax()
                        
                        # Debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bullish break of structure")
                        return
    
                    # new 3 red pull-back -> possible Fib leg
                    elif self.check_pullback(greens, opens, closes, ts, trend="BU"):
                        
                        if highest_body_price.shift(2).loc[ts] > self.high_price: # fresh HH
                            
                            # Logic
                            if pos >= 3:   
                                window = highest_body_price.iloc[pos-3 : pos+1]
                                self.high_price     = window.max()
                                self.high_price_ts  = window.idxmax()
                            
                            self.fib_labels = self.calc_fibonacci_levels(self.low_price_ts, self.high_price_ts, trend="BU")
                            self.fib_dict = dict(self.fib_labels)
                            
                            # Debug
                            old_state = self.state
                            self.state = "WAITING_ENTRY"
                            self.log_state_change(old_state, self.state, ts, "bullish waiting entry")
                            return
                        
                    
                        # price failed to beat prev high: treat as new pull-back
                        elif lowest_body_price.loc[ts] <= self.high_price:
                            
                            # Debug
                            old_state = self.state
                            self.state = "PULLBACK_DETECTED"
                            self.log_state_change(old_state, self.state, ts, "bullish back to pullback")
                            return
    
                elif self.state == "WAITING_ENTRY":
                    fib_710_bullish = self.fib_dict.get(0.710)
                    fib_000_bullish = self.fib_dict.get(0.000)
    
                    # price retraced to 0.710 – ready for bullish entry
                    if fib_710_bullish and df['Low'].loc[ts] <= fib_710_bullish: # count the tip of the candlestick instead of the body
                    
                        # Debug
                        self.entry_signal_ts = ts
                        old_state = self.state
                        self.state = "ENTRY_SIGNAL"
                        self.log_state_change(old_state, self.state, ts, "bullish price retraced to 0.710")
                        return
    
                    # price breaks above 0.000 (HH) before touching 0.710
                    elif fib_710_bullish and highest_body_price.loc[ts] > fib_000_bullish:
                        
                        # Logic
                        slices = lowest_body_price[self.high_price_ts:ts]
                        self.low_price = slices.min()
                        self.low_price_ts = slices.idxmin()
                        
                        # Debug
                        old_state = self.state
                        self.state = "BREAK_OF_STRUCTURE"
                        self.log_state_change(old_state, self.state, ts, "bullish breaks above 0.000 before entry")
                        return
                    
                elif self.state == "ENTRY_SIGNAL":
                    fib_100_bullish = self.fib_dict.get(1.000)
                    fib_000_bullish = self.fib_dict.get(0.000)
                    z = df['Zscore'].loc[ts]
                    # cancel if price dumps below 1.000 extension
                    if lowest_body_price.loc[ts] <= fib_100_bullish:
                        
                        # Debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bullish cancels: price < 1.0")
                        return
    
                    # Bullish Engulfing candle
                    elif z <= self.long_z_index: 
                        
                        self.engulfing_candle_ts = ts
                        
                        stop, take = self.calc_fibonacci_levels(self.low_price_ts, 
                        self.high_price_ts,trend="BU", entry_signal=self.entry_signal_ts,
                        engulfing_candle=self.engulfing_candle_ts)
                        
                        # calculate volume
                        entry_price = closes.loc[ts]
                        stop_loss_pips = abs(stop - entry_price) / self.pip_size

                        if stop_loss_pips >= 30 or stop_loss_pips <= 8: 
                            # don't trade
                            old_state = self.state
                            self.state = "PULLBACK_DETECTED"
                            self.log_state_change(old_state, self.state, ts, "stop_loss_pips > 30, returning to pullback state")
                            return  

                        lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips)
                        """
                        self.evt.consoleLog(
                            f"DEBUG ticket – side:BUY, instrument: {self.instrument}, ab:{self.account_balance}, entry≈{entry_price:.5f} "
                            f"SL={stop:.5f} TP={take:.5f}"
                        )
                        """
                        # place buying order
                        self.send_entry_order(ts, trend="BU",
                                  entry_price=entry_price,
                                  stop=stop, take=take,
                                  volume=lot_size, stoploss_pip_size = stop_loss_pips)
                        
                        # Debug
                        old_state = self.state
                        self.state = "PRE_ENTER_MARKET"
                        self.log_state_change(old_state, self.state, ts, "bullish entry: engulfing candle")
                        
                    else:
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "Z-score not met, returning to pullback state")
                        return
                        


        
        # ==============================================================
        # ========================  pullback()  ========================
        # ==============================================================     
        
    def check_pullback(self, greens, opens, closes, ts, trend):
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        
        if trend == "BE":
            # Check for 3, 4, 5, or 6 consecutive green candles.
            for window in [3, 4, 5, 6]:

                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                # Boundary check
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                # Check if all are green
                if all(greens.loc[ts_] == 'G' for ts_ in window_ts):
                    # Previous candle must be red
                    if greens.loc[prev_ts] == 'R':
                        # Any green candle closes above the open of previous red candle
                        if any(closes.loc[ts_] >= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False
        
        if trend == "BU":
            for window in [3,4,5,6]:
                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                if all(greens.loc[ts_]!= 'G' for ts_ in window_ts): # all previous not green candles 
                    if greens.loc[prev_ts] == 'G':
                        if any(closes.loc[ts_] <= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False

    def reset(self):
        self.state = "IDLE"
        self.low_price  = self.low_price_ts  =  None
        self.high_price = self.high_price_ts =  None
        self.fib_labels = []
        self.fib_dict = {} 
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.position_open = False
        self.open_volume = None


class InstrumentBar:
    
    """Bar builder + FSM for ONE instrument."""
    
    ALPHA_30  = 2 / 31
    ALPHA_120 = 2 / 121
    MIN_BARS  = 200 
    MAX_BARS  = 700
    
    def __init__(self, symbol: str, evt):
        
        self.symbol    = symbol
        self.evt       = evt
        self.last_time = datetime(2010, 1, 1)
        self.fsm       = None
        
        # placeholders; will be updated externally
        self.allow_long = False
        self.long_z_index = None
        self.allow_short = False
        self.short_z_index = None
        
        NFLOAT = "float32"
        
        # ------------- 5-min buffer ---------------------------------
        self.raw_5m = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument"])
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "category",
                "Instrument": "object"}))
                
        # ------------- 15-min table ---------------------------------
        self.data = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument", "EMA_30", "EMA_120", "trend","Zscore"]) 
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "category", "Instrument": "object",
                "EMA_30": NFLOAT, "EMA_120": NFLOAT,
                "trend": "category",
                "Zscore": NFLOAT
            })
            .set_index("Datetime", drop=False))
        
    def update(self, snap: dict, available_balance: float):
        
        """
        snap = bd[symbol]  – single-instrument snapshot from on_bulkdatafeed
        """
        # -------------------- append 5-min bar ------------------------
        green = ("G" if snap["lastPrice"] > snap["openPrice"]
                 else "R" if snap["lastPrice"] < snap["openPrice"]
                 else "NO")
        
        bar5 = {
            "Datetime": pd.to_datetime(snap["timestamp"]),
            "Open":  snap["openPrice"],
            "High":  snap["highPrice"],
            "Low":   snap["lowPrice"],
            "Close": snap["lastPrice"],
            "Green": green,
            "Instrument": snap["instrument"],
        }
        
        self.raw_5m = pd.concat([self.raw_5m,
                                 pd.DataFrame([bar5])], ignore_index=True)
                                 
        # avoid  Warning: data month not exists for ... fx error led to array memory explode
        if len(self.raw_5m) > 10:
            self.raw_5m = self.raw_5m.iloc[0:0]
        
        
        # ------------------- finished 15-min? -------------------------
        ts = pd.to_datetime(snap["timestamp"])
        if ts < self.last_time + timedelta(minutes=15):
            return
        self.last_time = ts
        
        last3 = self.raw_5m.tail(3)
        if len(last3) < 3:
            return    # need full 15-minute slice
        
        self.raw_5m = self.raw_5m.iloc[0:0]  # clear buffer
        
        self._aggregate_15m(last3)
        
        
        if len(self.data) < self.MIN_BARS:
            return
            
        
        if len(self.data) > self.MAX_BARS:
            self.data = self.data.iloc[-self.MAX_BARS:]
        
        # -------------------- get latest exchange rate  -----------------
        base_currency = self.symbol[:3] # self.fx_base_usd_currency
        
        
        if base_currency != "USD":
            
            fx_base = self.evt.getExchangeRate(base_currency, "USD")

        
        else:
            fx_base = 1
        
        quote_currency = self.symbol[3:]    
        if quote_currency != "USD" : # base currency is not USD
            
            fx = self.evt.getExchangeRate("USD", quote_currency)

        else: 
            fx = 1
        
            
        # -------------------- FSM ------------------------------------
        if self.fsm is None:
            if "JPY" in self.symbol :
                pip_size = 0.01
            else:                                     
                pip_size = 0.0001
                
            self.fsm = BearishFSM(
                self.data, 
                instrument=self.symbol, 
                evt=self.evt,
                account_balance=np.float32(available_balance),
                pip_size=pip_size,
                fx=fx,
                fx_base = fx_base, 
                ts=self.last_time, 
                allow_long = self.allow_long, 
                long_z_index = self.long_z_index, 
                allow_short = self.allow_short, 
                short_z_index = self.short_z_index)
        else:
            self.fx_usd_quote_currency = fx
            self.fx_base_usd_currency = fx_base
            self.fsm.df = self.data
            self.fsm.account_balance = available_balance 
            self.fsm.update(self.last_time)
            
    def _aggregate_15m(self, last3):
        
        
        
        n   = len(self.data)
        op  = last3["Open"].iloc[0]
        cl  = last3["Close"].iloc[-1]
        bar = {
            "Datetime": last3["Datetime"].iloc[-1],
            "Open": op,
            "High": last3["High"].max(),
            "Low":  last3["Low"].min(),
            "Close": cl,
            "Green": "G" if cl > op else "R" if cl < op else "NO",
            "Instrument": self.symbol,
        }

        if n == 0:
            ema30 = ema120 = cl #  ema10 =

        elif n < 30: 
            prev  = self.data.iloc[n - 1]
            ema30  = (1 - self.ALPHA_30)  * prev["EMA_30"]  + self.ALPHA_30  * cl
            ema120 = (1 - self.ALPHA_120) * prev["EMA_120"] + self.ALPHA_120 * cl

        else:
            prev  = self.data.iloc[n - 1] 
            ema30  = (1 - self.ALPHA_30)  * prev["EMA_30"]  + self.ALPHA_30  * cl
            ema120 = (1 - self.ALPHA_120) * prev["EMA_120"] + self.ALPHA_120 * cl

        
        bar["EMA_30"]  = ema30
        bar["EMA_120"] = ema120
        r = ema30 / ema120
        bar["trend"] = "BU" if r > 1.001 else "BE" if r < 0.999 else "NO" 

        # ---------- rolling Z-score (30-bar look-back) ----------------------
        window = 200                        # feel free to optimise
        if n >= window-1:                  # we have enough history
            mean = self.data["Close"].iloc[-(window-1):].mean()
            std  = self.data["Close"].iloc[-(window-1):].std(ddof=0)
            z    = (cl - mean) / std if std > 0 else 0
        else:
            z = 0                          # not enough data yet

        bar["Zscore"] = np.float32(z)

        self.data = pd.concat([self.data, pd.DataFrame([bar]).set_index("Datetime",drop=False)])

class AlgoEvent:

    def __init__(self):
        self.evt   = None
        self.books = {}          # symbol → InstrumentBar
        self.trades = []
        # Initialize instrument configuration:
        # symbol → dict with keys: allow_long, long_z_index, allow_short, short_z_index
        self.instrument_cfg = {
            'USDJPY': {
                "allow_long": True,
                "long_z_index": 1.05, # verified
                "allow_short": False,
                "short_z_index": None
            }, 
           'USDCAD': {
                "allow_long": True,
                "long_z_index": 1.18, # 2020-2023 1.29 if 2024 use 1.18
                "allow_short": False,
                "short_z_index": None
            },
            'CADJPY': {
                "allow_long": True,
                "long_z_index": 0.80, # un-verified, 
                "allow_short": False,
                "short_z_index": None
            },
            'GBPUSD': {
                "allow_long": True,
                "long_z_index": 1.38, # 2020-2023 1.69, if 2024 use 1.38
                "allow_short": False,
                "short_z_index": None
            },
            'AUDCAD': {
                "allow_long": True,
                "long_z_index": 1.01, # un-verified, if 2024 use 1.38
                "allow_short": False,
                "short_z_index": None
            },
            
            'EURCHF': {
                "allow_long": True,
                "long_z_index": 1.18, # un-verified
                "allow_short": True,
                "short_z_index": -0.68 # un-verified
            },
            'EURCAD': {
                "allow_long": False,
                "long_z_index": None,
                "allow_short": True,
                "short_z_index": -1.42 # un-verified, to small amount
            },
            
        }
            
    def start(self, mEvt):
        self.evt = AlgoAPI_Backtest.AlgoEvtHandler(self, mEvt)
        self.evt.start()
        self.analyse_zscores()
        self.analyse_stoploss_impact()
        
    def analyse_stoploss_impact(self):
        df = pd.DataFrame(self.trades).dropna(subset=["exit_ts", "pnl", "stoploss_pip", "symbol"])
        if df.empty:
            self.evt.consoleLog("No trades to analyze (missing stoploss_pip).")
            return
        
         # 🔍 Analyze tight stop-loss trades (< 8 pips)
        small_sl = df[df["stoploss_pip"] < 8]
        if not small_sl.empty:
            winners_small_sl = small_sl[small_sl["pnl"] > 0]
            win_rate_small = len(winners_small_sl) / len(small_sl)
    
            self.evt.consoleLog(f"\n⚠️ Trades with SL < 8 pips: {len(small_sl)}")
            self.evt.consoleLog(f"Winning trades    : {len(winners_small_sl)}")
            self.evt.consoleLog(f"Win rate          : {win_rate_small:.2%}")
            self.evt.consoleLog(f"Example SLs       : {small_sl['stoploss_pip'].head(5).tolist()}")
                
        sl_thresholds = [10, 15, 20, 25, 30]
    
        self.evt.consoleLog("\n====== 📉 Stop-Loss Impact Analysis by Symbol ======")
    
        for symbol in df["symbol"].unique():
            df_sym = df[df["symbol"] == symbol]
            self.evt.consoleLog(f"\n=== Symbol: {symbol} ===")
            
            for threshold in sl_thresholds:
                df_filtered = df_sym[df_sym["stoploss_pip"] >= threshold]
                if df_filtered.empty:
                    continue
    
                winners = df_filtered[df_filtered["pnl"] > 0]
                win_rate = len(winners) / len(df_filtered)
                avg_pnl = df_filtered["pnl"].mean()
                self.evt.consoleLog(f"SL ≥ {threshold} pips → Trades: {len(df_filtered)}, Win Rate: {win_rate:.2%}")
                self.evt.consoleLog(f"Avg PnL: {avg_pnl:.2f}")

        
    def analyse_zscores(self):

        df = pd.DataFrame(self.trades).dropna(subset=["exit_ts", "pnl", "entry_z"])
        
        if df.empty:
            self.evt.consoleLog("No trades to analyze.")
            return
        
        def opt_threshold(z_w, z_l, num=50):
            all_z = np.concatenate([z_w, z_l])
            thresholds = np.linspace(all_z.min(), all_z.max(), num)
            best = thresholds[0]
            best_j = -np.inf
            for t in thresholds:
                tpr = np.mean(z_w <= t)
                fpr = np.mean(z_l <= t)
                j = tpr - fpr  # equivalent to Youden's J :contentReference[oaicite:4]{index=4}
                if j > best_j:
                    best_j, best = j, t
                    best_tpr,best_fpr = tpr, fpr
            return best, best_j, best_tpr,best_fpr
    
        
        
         # Loop through each instrument (symbol)
        for symbol in df["symbol"].unique():
            df_symbol = df[df["symbol"] == symbol]
            self.evt.consoleLog(f"\n==================== {symbol} ====================")
    
            for side_name, side_sign in [("LONG", 1), ("SHORT", -1)]:
                df_side = df_symbol[df_symbol["side_sign"] == side_sign]
                if df_side.empty:
                    self.evt.consoleLog(f"\n⚠️  No {side_name} trades recorded for {symbol}.\n")
                    continue
    
                winners = df_side[df_side["pnl"] > 0]
                losers  = df_side[df_side["pnl"] <= 0]
    
                if winners.empty:
                    self.evt.consoleLog(f"\n📊 {side_name} TRADES (Symbol: {symbol})\nNo winning trades.\n")
                    continue
    
                z_winners = winners["entry_z"]
                z_losers = losers["entry_z"]
                
                # Compute optimal threshold
                best_z, best_j, tpr, fpr = opt_threshold(z_winners, z_losers)
                # Calculate true/false positive rates & trade count

                n_trades_pct = np.mean(df_side["entry_z"] <= best_z)

    
                self.evt.consoleLog(f"\n📊 {side_name} TRADES (Symbol: {symbol})")
                self.evt.consoleLog(f"Winning Z-score range : {z_winners.min():.2f} → {z_winners.max():.2f}")
                self.evt.consoleLog(f"Mean Z of winners     : {z_winners.mean():.2f}")
                self.evt.consoleLog(f"Median Z of winners   : {z_winners.median():.2f}")
                self.evt.consoleLog(f"StdDev Z of winners   : {z_winners.std():.2f}")
                self.evt.consoleLog(f"IQR (25%-75%)         : {np.percentile(z_winners, 25):.2f} – {np.percentile(z_winners, 75):.2f}")
    
                self.evt.consoleLog(f"Mean Z of losers      : {z_losers.mean():.2f}")
                self.evt.consoleLog(f"Median Z of losers   : {z_losers.median():.2f}")
                self.evt.consoleLog(f"StdDev Z of winners   : {z_losers.std():.2f}")
                self.evt.consoleLog(f"IQR (25%-75%)         : {np.percentile(z_losers, 25):.2f} – {np.percentile(z_losers, 75):.2f}")
                
                self.evt.consoleLog(f"Count winners/losers  : {len(z_winners)} / {len(z_losers)}")
                self.evt.consoleLog(f"✅ Optimal Z thresh : {best_z:.2f}   (Youden J = {best_j:.2f})")
                self.evt.consoleLog(f"📈 TPR (winners hit) : {tpr:.2f}, FPR (losers hit) : {fpr:.2f}")
                self.evt.consoleLog(f"📊 % of trades triggered: {n_trades_pct}\n")

    def on_bulkdatafeed(self, isSync, bd, ab):
        
        balance = ab["availableBalance"]
        
        for sym, snap in bd.items():        # sym: AUDNZD, EURUSD, ... snap: {'instrument': 'AUDNZD', 'timestamp': datetime.datetime(1970, 1, 1, 0, 0), 'askPrice': -1, ...}
            if sym not in self.books:       # first sight → create book

                book = InstrumentBar(sym, self.evt)

                # Apply matching config if exists
                cfg = self.instrument_cfg.get(sym)
                if cfg:
                    book.allow_long = cfg["allow_long"]
                    book.long_z_index = cfg["long_z_index"]
                    book.allow_short = cfg["allow_short"]
                    book.short_z_index = cfg["short_z_index"]
                
                self.books[sym] = book
            else: 
                self.books[sym].update(snap, balance)

    def on_orderfeed(self, of):
        
        sym = of.instrument
        if sym not in self.books:
            return                        # safety
        
        fsm = self.books[sym].fsm
        
        # -------------- order filled -----------------
        if of.openclose == "open" and of.status == "success":
            # Is this the order we are waiting for?
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.position_open      = True
                fsm.stop_loss_price    = fsm.pending["stop"]
                fsm.take_profit_price  = fsm.pending["take"]
                fsm.open_volume        = fsm.pending["volume"]
                fsm.enter_market_trade_side = fsm.pending["trend"]
                stoploss_pip_size = fsm.pending["stoploss_pip"]
    
                fsm.pending = None      # clear
                old_state   = fsm.state
                fsm.state   = "ENTER_MARKET"
                fsm.log_state_change(old_state, fsm.state, of.orderRef,
                                     "broker filled order")
            # --------- store entry info ---------------------------------
            self.trades.append(dict(
                orderRef     = of.orderRef,
                symbol       = of.instrument,
                side_sign    = 1 if of.buysell == 1 else -1,   # +1 = long, −1 = short
                volume       = of.fill_volume,
                entry_price  = of.fill_price,
                entry_ts     = of.insertTime,
                entry_z      = fsm.current_zscore(of.orderRef),  # helper shown earlier
                stoploss_pip = stoploss_pip_size,
                exit_ts      = None,
                pnl          = None
            ))
        # -------------- order rejected ---------------
        elif of.openclose == "open" and of.status != "success":
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.pending = None
                old_state = fsm.state
                fsm.state = "ENTRY_SIGNAL"     # back to previous state
                fsm.log_state_change(old_state, fsm.state, of.orderRef,
                                     "order rejected")
                                     
        # -------------- position closed --------------
        if of.openclose == 'close' and of.status == 'success': # return 'success' if an order is opened/ closed successfully, or when a limit/stop order is filled
            
            for t in reversed(self.trades):
                if t["symbol"] == of.instrument and t["exit_ts"] is None:
                    t["exit_ts"]    = of.insertTime
                    t["exit_price"] = of.fill_price

                    # signed P/L per unit (positive = winner)
                    #   BUY  →  (exit - entry) * +1
                    #   SELL →  (exit - entry) * -1
                    t["pnl"] = (of.fill_price - t["entry_price"]) * t["side_sign"] \
                            * t["volume"]                         # in “price × volume” units
                    break
            fsm.reset()
            # --------- store entry info ---------------------------------
    
    
    def on_newsdatafeed(self, nd):
        pass

    def on_weatherdatafeed(self, wd):
        pass
    
    def on_econsdatafeed(self, ed):
        pass
        
    def on_corpAnnouncement(self, ca):
        pass
    
    def on_dailyPLfeed(self, pl):
        pass

    def on_openPositionfeed(self, op, oo, uo):
        pass










#### Version 14 changes

1. Now, both a short-term (Zscore, e.g. 30 bars) and a longer-term (Zscore_7, e.g. 120 bars) Z-score are calculated and stored for each forex, providing a stricter statistical filter for entries (previously, only one Z-score threshold was used).


In [ ]:
from AlgoAPI import AlgoAPIUtil, AlgoAPI_Backtest
import pandas as pd
import numpy as np
import mplfinance as mpf
from datetime import datetime, timedelta
    
class BearishFSM: # Finite State Machine

        # ==============================================================
        # ========================  __init__ ===========================
        # ==============================================================
    def __init__(self, df, instrument, evt, account_balance=0, pip_size = 0.0001, fx = 1, fx_base = 1, ts = None, allow_long = False, long_z_index_short = None, long_z_index_long = None, allow_short = False, short_z_index_short = None, short_z_index_long = None):
        
        self.df = df
        self.evt = evt
        self.account_balance = account_balance
        
        # Enter market flag
        self.position_open = False
        self.pending = None       # ← dict that holds the ticket waiting
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.open_volume = None
        
        self.state = "IDLE"
        self.low_price = None
        
        self.low_price_ts = None
        self.high_price = None
        self.high_price_ts = None

        self.fib_labels = []
        self.fib_dict = {} # not a must to initialize, just for good habit
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.trend_changing_ts = ts # do not reset this, always keep record of 
        
##################################################################################################################
####################################### optimize #################################################################
##################################################################################################################

        # do not reset the folloiwng parameters, test for optimal values
        self.risk_reward_ratio = 2
        self.atr_period = 14
        self.atr_multiplier = 2.5 # pip_size = atr_multiplier * atr_value, atr_multiplier = 2.0 means stop loss is 2 ATRs away
        self.risk_percent = 1
        self.dynamic_atr_window = False
        # self.trend_ema10_sma50 = False # If True, the trend will be determined by EMA10 and SMA50, if False, determined by EMA30 and EMA120
        
##################################################################################################################
##################################################################################################################
##################################################################################################################
        # this is unique in every func call
        self.instrument = instrument
        self.pip_size = pip_size
        self.fx_usd_quote_currency = fx
        self.fx_base_usd_currency = fx_base
        
        self.allow_long = allow_long
        self.long_z_index_short = long_z_index_short
        self.long_z_index_long = long_z_index_long
        self.allow_short = allow_short
        self.short_z_index_short = short_z_index_short
        self.short_z_index_long = short_z_index_long

    # ==============================================================
    # ========================  current_zscore             =========
    # ==============================================================
    def current_zscore(self, ts):
        if "Zscore" not in self.df.columns:
            return np.nan
        return float(self.df["Zscore"].loc[ts])
    def current_zscore_7_days(self, ts):
        if "Zscore_7" not in self.df.columns:
            return np.nan
        return float(self.df["Zscore_7"].loc[ts])
    # ==============================================================
    # ========================  calculate_log_state_change =========
    # ============================================================== 
    def log_state_change(self, old_state, new_state, ts, label=None):
        msg = f"{self.instrument} State: {old_state} → {new_state}"
        if label:
            msg += f" ({label})"
        # self.evt.consoleLog(msg)
        
        # ==============================================================
        # ========================  calculate_lot_size  ================
        # ==============================================================
        
    def calculate_lot_size(self, risk_percent, stop_loss_pips, max_leverage = 4.5): # base currency vs quote currency
    
        # 1. Calculate amount to risk
        risk_amount = self.account_balance * (risk_percent / 100)
        
        # 2. calculate pip value per lot (in quote currency)
        pip_value_per_lot_quote = self.pip_size * 100000 # If we are trading AUDJPY, the value will be 0.01 * 100000 = 1000 JPY
        
        # 3. calculate total risk per lot ( in quote currency)
        total_risk_per_lot_quote = stop_loss_pips * pip_value_per_lot_quote
        
        # 4. calculate total risk per lot ( in account currency, i.e. USD)
            # we need real time USD/quote_currency price, in this case, USDJPY
        self.evt.consoleLog(self.fx_usd_quote_currency)
        total_risk_per_lot_USD = total_risk_per_lot_quote / self.fx_usd_quote_currency # if quote currency = USD, this will be 1
        
        # 5. calculate lot size
        lot_size = risk_amount / total_risk_per_lot_USD

        # calculate maximum affordable lot size after using margin (e.g. we are trading AUDUSD)
            # Notional Value per lot (in Base Currency) = 100000 AUD
            
        # Notation Value per lot (in account currency) = 100000 x AUDUSD price
        notation_per_lot_USD = 100000 * self.fx_base_usd_currency
            
        lot_margin = (self.account_balance * max_leverage) / notation_per_lot_USD
        
        lot_final = min(lot_size, lot_margin) # all-in 
        
        self.evt.consoleLog(
            f"{self.instrument} bal={self.account_balance:.2f} risk%={risk_percent} sl={stop_loss_pips} "
            f"riskAmt={risk_amount:.2f} pipVal/lot={pip_value_per_lot_quote:.5f} risk/lot(USD)={total_risk_per_lot_USD}"
            f"riskLot={lot_size:.2f} margLot={lot_margin:.2f} finalLot={lot_final:.2f}"
        )
        
        return round(lot_final, 2)
        
        
        # ==============================================================
        # ========================  calc_fibonacci_levels  =============
        # ==============================================================
        
    def calc_fibonacci_levels(self, start_ts, end_ts, levels=None, trend="NO",
                         entry_signal=None, engulfing_candle=None,
                         R=None, atr_period=None, atr_multiplier=None,dynamic_atr_window = None):
                             
        if levels is None:
            levels = [0, 0.236, 0.382, 0.5, 0.618, 0.710, 0.786, 1]
            
        if dynamic_atr_window is None:
            dynamic_atr_window = self.dynamic_atr_window
            
        if atr_multiplier is None:
            atr_multiplier = self.atr_multiplier
        
        if R is None:
            R = self.risk_reward_ratio
        
        
        df = self.df

        window = df.loc[start_ts : end_ts]
                            
        min_price = window['Low'].min()
        max_price = window['High'].max()
    
        fib_labels = []
        for level in levels:
            if trend == "BE": # bearish, sell
                y = min_price + (max_price - min_price) * level
            else:
                y = max_price - (max_price - min_price) * level
            fib_labels.append((level, y))
        
        if engulfing_candle is None:
            return fib_labels
        
        if atr_period is None:

            if dynamic_atr_window == False:
                # Fixed atr period
                atr_period = self.atr_period
            else: 
                # dynamic atr period
                start_idx = df.index.get_loc(start_ts)
                enter_idx = df.index.get_loc(engulfing_candle)
            
                dynamic_period = enter_idx - start_idx
                atr_period = max(10, min(dynamic_period, 50))
        
        # Normal ATR
        min_start_idx = max(0, df.index.get_loc(start_ts) - 10)
        min_start_ts = df.index[min_start_idx]
        sl = df.loc[min_start_ts : engulfing_candle].copy()
        
        sl['H-L']  = sl['High'] - sl['Low']
        sl['H-PC'] = (sl['High'] - sl['Close'].shift(1)).abs()
        sl['L-PC'] = (sl['Low']  - sl['Close'].shift(1)).abs()
        sl['TR']    = sl[['H-L', 'H-PC', 'L-PC']].max(axis=1)
        
        # ATR: rolling mean of TR
        sl['ATR'] = sl['TR'].rolling(window=atr_period).mean()
         
        # Retrieve ATR value at the engulfing candle index
        atr_value = sl['ATR'].iloc[-1]
        pip_size = atr_multiplier * atr_value
        
        # Retrieve the close price of the engulfing candle
        candle = self.df.loc[engulfing_candle]
        close_price = candle['Close']
        
        # Initialize stop loss and take profit prices
        stop_loss_price = None
        take_profit_price = None
    
        if trend == "BE" :
            stop_loss_price = close_price + pip_size
            take_profit_price = close_price - R * pip_size
        else:  # bullish
            stop_loss_price = close_price - pip_size
            take_profit_price = close_price + R * pip_size
    
        return stop_loss_price, take_profit_price
        
    # ==============================================================
    # ======================== send_entry_order  =============
    # ==============================================================
    
    def send_entry_order(self, ts, trend, entry_price,
                         stop, take, volume, stoploss_pip_size):

        # remember what we just asked the broker to do
        self.pending = dict(
            orderRef = ts,         # we use the timestamp as reference
            trend    = trend,      # "BU" / "BE"
            stop     = stop,
            take     = take,
            volume   = volume,
            stoploss_pip = stoploss_pip_size
        )

        order = AlgoAPIUtil.OrderObject()
        order.instrument   = self.instrument
        order.orderRef     = ts
        order.openclose    = "open"
        order.buysell      = 1 if trend == "BU" else -1
        order.ordertype    = 0          # market
        order.volume       = volume
        order.stopLossLevel= stop
        order.takeProfitLevel = take
        self.evt.sendOrder(order)

        # NOTE: do NOT set state / flags here;
        #       wait for on_orderfeed()
    
    # ==============================================================
    # ========================  update()  =============
    # ==============================================================
    
    def update(self, ts: pd.Timestamp):
        
        # 1️⃣ When a position is open we do only exit-logic and leave
        #    the rest of the gigantic state-machine untouched.
        #    (So IDLE / PULLBACK_DETECTED / … cannot fire again.)
        if self.position_open:
            return                       # <-- early exit
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        
        if df.index.get_loc(ts) < 3: # Get current timestamp location
            return
        
        #--- 2. prepare frequently-used columns (all remain Series!) ------
        greens  = df["Green"]               # string
        opens   = df["Open"]
        closes  = df["Close"]
        
        lowest_body_price = opens.where(greens == 'G', closes) # if greens == 'G', use values in open, else use values in closes 
        highest_body_price = closes.where(greens == 'G', opens) # the index of this series keeps the same Datetime index as the original df
        
        # selects the row in df where the index = ts
        curr   = df.loc[ts]
        prev   = df.shift(1).loc[ts]       # one-bar shift gives previous values

        current_trend = curr['trend'] ############ use EMA120 and EMA30
        prev_trend    = prev['trend']

        if current_trend != prev_trend : 
            self.trend_changing_ts = ts
            if self.state != "ENTER_MARKET":
                self.reset()
                
        # ==============================================================
        # ========================  BEARISH  ============================
        # ==============================================================
        if self.allow_short == True: 
            if (current_trend == "BE" and self.enter_market_trade_side == None):
                if self.state == "IDLE":
                    if self.check_pullback(greens, opens, closes, ts, trend="BE"): # pullback should eat and cover the previous red candle
                        # logic
                        slices = lowest_body_price.loc[self.trend_changing_ts:ts] # Slicing with .loc[start:end] in pandas includes both endpoints
                        self.low_price = slices.min()
                        self.low_price_ts  = slices.idxmin()   # Return index(Datetime) of first occurrence of minimum over requested axis.
                        # debug
                        old_state = self.state  
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bearish pullback detected")
                        
    
                elif self.state == "PULLBACK_DETECTED":
                    if lowest_body_price.loc[ts] < self.low_price:
                        # logic
                        slices = highest_body_price.loc[self.low_price_ts:ts]
                        self.high_price = slices.max()
                        self.high_price_ts = slices.idxmax()
                        # debug
                        old_state = self.state
                        self.state = "BREAK_OF_STRUCTURE"
                        self.log_state_change(old_state, self.state, ts, "bearish new low after pullback")
    
                        
                elif self.state == "BREAK_OF_STRUCTURE":
                    # If price breaks the pev high (LH)
                    if highest_body_price.loc[ts] > self.high_price:
                        
                        # logic
                        slices = lowest_body_price.loc[self.high_price_ts: ts]
                        self.low_price = slices.min()
                        self.low_price_ts = slices.idxmin()
                        
                        # debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bearish break of structure")
    
                    elif self.check_pullback(greens, opens, closes, ts, trend="BE"):
                        
                        if lowest_body_price.shift(2).loc[ts] < self.low_price: #### use 2 timeframe before current ts , can incorporate to 3,4,5 or 6
                        
                            # Logic: Low price for confirming the next Lower high 
                            if pos >= 3:   
                                window = lowest_body_price.iloc[pos-3 : pos+1]
                                self.low_price     = window.min()
                                self.low_price_ts  = window.idxmin()
    
                            self.fib_labels = self.calc_fibonacci_levels(self.high_price_ts, self.low_price_ts, trend="BE")
                            self.fib_dict = dict(self.fib_labels)
                            
                            # debug
                            old_state = self.state
                            self.state = "WAITING_ENTRY"
                            self.log_state_change(old_state, self.state, ts, "bearish waiting entry")
    
                        elif highest_body_price.loc[ts] >= self.low_price:
                            
                            # debug
                            old_state = self.state
                            self.state = "PULLBACK_DETECTED"
                            self.log_state_change(old_state, self.state, ts, "bearish back to pullback")
    
                elif self.state == "WAITING_ENTRY":
                    
                    fib_710_price = self.fib_dict.get(0.710)
                    fib_000_price = self.fib_dict.get(0.000)
                    
                    
                    # price rebound back to 0.710 &
                    if fib_710_price and df["High"].loc[ts] >= fib_710_price: # reference the "High" price at the exact timestamp
                    
                        # debug
                        self.entry_signal_ts = ts
                        old_state = self.state
                        self.state = "ENTRY_SIGNAL"
                        self.log_state_change(old_state, self.state, ts, "bearish price rebound to 0.710")
    
                    elif fib_710_price and lowest_body_price.loc[ts] <= fib_000_price: # price lower than 0.000
                    
                        # Logic: break of structure, set new high point
                        slices = highest_body_price.loc[self.low_price_ts: ts]
                        self.high_price = slices.max()
                        self.high_price_ts = slices.idxmax()
                        
                        # debug
                        old_state = self.state
                        self.state = "BREAK_OF_STRUCTURE"
                        self.log_state_change(old_state, self.state, ts, "bearish breaks 0.000 before entry")
                
                elif self.state == "ENTRY_SIGNAL": 
                    fib_100_price = self.fib_dict.get(1.000)
                    z_short = df['Zscore'].loc[ts]
                    z_long = df['Zscore_7'].loc[ts]
                    
                    if highest_body_price.loc[ts] >= fib_100_price: # no matter what, if the candle pass 1.00, we do not trade
                    
                        # debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bearish cancels: price > 1.0")
    
                    # Logic: Bearish Engulfing candle  
                    elif z_short <= self.short_z_index_short and z_long <= self.short_z_index_long :
                            
                        # Debug
                        self.engulfing_candle_ts = ts
                        
                        stop, take = self.calc_fibonacci_levels(self.high_price_ts, 
                                self.low_price_ts, trend="BE", entry_signal=self.entry_signal_ts, engulfing_candle=self.engulfing_candle_ts
                                )
                        
                        # Logic to calculate lot_size
                        entry_price = closes.loc[ts]
                        stop_loss_pips = abs(stop - entry_price) / self.pip_size
                        lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips)
                        
                        if self.instrument == "EURCHF" and stop_loss_pips <= 8: 
                            lot_size = 0.01
                        
                            
                        """
                        if stop_loss_pips >= 30 : 
                            lot_size = 0.01
                            
                        # Log for debug
                        self.evt.consoleLog(
                            f"DEBUG ticket – side:SELL, instrument:{self.instrument},ab:{self.account_balance}, entry≈{entry_price:.5f} "
                            f"SL={stop:.5f} TP={take:.5f}"
                        )
                    
                        """
                        # place selling order
                        self.send_entry_order(ts, trend="BE",
                                    entry_price=entry_price,
                                    stop=stop, take=take,
                                    volume=lot_size, stoploss_pip_size = stop_loss_pips)
                        
                        # Debug
                        old_state = self.state
                        self.state = "PRE_ENTER_MARKET"
                        self.log_state_change(old_state, self.state, ts, "bearish entry: engulfing candle")
                    else:
                        self.state = "PULLBACK_DETECTED"
                        return
                    
        # ==============================================================
        # ========================  BULLISH  ============================
        # ==============================================================
        
        if self.allow_long == True:
            if (current_trend == "BU" and self.enter_market_trade_side == None):
                if self.state == "IDLE":    
                    if self.check_pullback(greens, opens, closes, ts, trend="BU"):
                        
                        # Logic
                        slices = highest_body_price[self.trend_changing_ts:ts]
                        self.high_price = slices.max()
                        self.high_price_ts = slices.idxmax()
                        
                        # Debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bullish pullback detected")
    
                elif self.state == "PULLBACK_DETECTED":
                    if highest_body_price.loc[ts] > self.high_price:
                        
                        # Logic
                        slices = lowest_body_price[self.high_price_ts: ts]
                        self.low_price = slices.min()
                        self.low_price_ts = slices.idxmin()
                        
                        # Debug
                        old_state = self.state
                        self.state = "BREAK_OF_STRUCTURE"
                        self.log_state_change(old_state, self.state, ts, "bullish new high after pullback")
    
       
                elif self.state == "BREAK_OF_STRUCTURE":
                    # structure broken downward -> restart pull-back search
                    if lowest_body_price.loc[ts] < self.low_price:
                        
                        # Logic
                        slices = highest_body_price[self.low_price_ts : ts]
                        self.high_price = slices.max()
                        self.high_price_ts = slices.idxmax()
                        
                        # Debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bullish break of structure")
    
                    # new 3 red pull-back -> possible Fib leg
                    elif self.check_pullback(greens, opens, closes, ts, trend="BU"):
                        
                        if highest_body_price.shift(2).loc[ts] > self.high_price: # fresh HH
                            
                            # Logic
                            if pos >= 3:   
                                window = highest_body_price.iloc[pos-3 : pos+1]
                                self.high_price     = window.max()
                                self.high_price_ts  = window.idxmax()
                            
                            self.fib_labels = self.calc_fibonacci_levels(self.low_price_ts, self.high_price_ts, trend="BU")
                            self.fib_dict = dict(self.fib_labels)
                            
                            # Debug
                            old_state = self.state
                            self.state = "WAITING_ENTRY"
                            self.log_state_change(old_state, self.state, ts, "bullish waiting entry")
    
                        # price failed to beat prev high: treat as new pull-back
                        elif lowest_body_price.loc[ts] <= self.high_price:
                            
                            # Debug
                            old_state = self.state
                            self.state = "PULLBACK_DETECTED"
                            self.log_state_change(old_state, self.state, ts, "bullish back to pullback")
                            
    
                elif self.state == "WAITING_ENTRY":
                    fib_710_bullish = self.fib_dict.get(0.710)
                    fib_000_bullish = self.fib_dict.get(0.000)
    
                    # price retraced to 0.710 – ready for bullish entry
                    if fib_710_bullish and df['Low'].loc[ts] <= fib_710_bullish: # count the tip of the candlestick instead of the body
                    
                        # Debug
                        self.entry_signal_ts = ts
                        old_state = self.state
                        self.state = "ENTRY_SIGNAL"
                        self.log_state_change(old_state, self.state, ts, "bullish price retraced to 0.710")
    
                    # price breaks above 0.000 (HH) before touching 0.710
                    elif fib_710_bullish and highest_body_price.loc[ts] > fib_000_bullish:
                        
                        # Logic
                        slices = lowest_body_price[self.high_price_ts:ts]
                        self.low_price = slices.min()
                        self.low_price_ts = slices.idxmin()
                        
                        # Debug
                        old_state = self.state
                        self.state = "BREAK_OF_STRUCTURE"
                        self.log_state_change(old_state, self.state, ts, "bullish breaks above 0.000 before entry")
                    
                elif self.state == "ENTRY_SIGNAL":
                    fib_100_bullish = self.fib_dict.get(1.000)
                    fib_000_bullish = self.fib_dict.get(0.000)
                    z_short = df['Zscore'].loc[ts]
                    z_long = df['Zscore_7'].loc[ts]
                    # cancel if price dumps below 1.000 extension
                    if lowest_body_price.loc[ts] <= fib_100_bullish:
                        
                        # Debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bullish cancels: price < 1.0")
    
                    # Bullish Engulfing candle
                    elif z_short <= self.long_z_index_short and z_long <= self.long_z_index_long: # self.long_z_index: 
                        
                        self.engulfing_candle_ts = ts
                        
                        stop, take = self.calc_fibonacci_levels(self.low_price_ts, 
                        self.high_price_ts,trend="BU", entry_signal=self.entry_signal_ts,
                        engulfing_candle=self.engulfing_candle_ts)
                        
                        # calculate volume
                        entry_price = closes.loc[ts]
                        stop_loss_pips = abs(stop - entry_price) / self.pip_size
                        lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips)
                        


                        """
                        if stop_loss_pips >= 30 : 
                            lot_size = 0.01
                            
                        self.evt.consoleLog(
                            f"DEBUG ticket – side:BUY, instrument: {self.instrument}, ab:{self.account_balance}, entry≈{entry_price:.5f} "
                            f"SL={stop:.5f} TP={take:.5f}"
                        )
                        """
                        # place buying order
                        self.send_entry_order(ts, trend="BU",
                                  entry_price=entry_price,
                                  stop=stop, take=take,
                                  volume=lot_size, stoploss_pip_size = stop_loss_pips)
                        
                        # Debug
                        old_state = self.state
                        self.state = "PRE_ENTER_MARKET"
                        self.log_state_change(old_state, self.state, ts, "bullish entry: engulfing candle")
                    
                    else:
                        self.state = "PULLBACK_DETECTED"
                        return
                        


        
        # ==============================================================
        # ========================  pullback()  ========================
        # ==============================================================     
        
    def check_pullback(self, greens, opens, closes, ts, trend):
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        
        if trend == "BE":
            # Check for 3, 4, 5, or 6 consecutive green candles.
            for window in [3, 4, 5, 6]:

                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                # Boundary check
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                # Check if all are green
                if all(greens.loc[ts_] == 'G' for ts_ in window_ts):
                    # Previous candle must be red
                    if greens.loc[prev_ts] == 'R':
                        # Any green candle closes above the open of previous red candle
                        if any(closes.loc[ts_] >= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False
        
        if trend == "BU":
            for window in [3,4,5,6]:
                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                if all(greens.loc[ts_]!= 'G' for ts_ in window_ts): # all previous not green candles 
                    if greens.loc[prev_ts] == 'G':
                        if any(closes.loc[ts_] <= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False

    def reset(self):
        self.state = "IDLE"
        self.low_price  = self.low_price_ts  =  None
        self.high_price = self.high_price_ts =  None
        self.fib_labels = []
        self.fib_dict = {} 
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.position_open = False
        self.open_volume = None


class InstrumentBar:
    
    """Bar builder + FSM for ONE instrument."""
    
    ALPHA_30  = 2 / 31
    ALPHA_120 = 2 / 121
    MIN_BARS  = 200 
    MAX_BARS  = 700
    
    def __init__(self, symbol: str, evt):
        
        self.symbol    = symbol
        self.evt       = evt
        self.last_time = datetime(2010, 1, 1)
        self.fsm       = None
        
        # placeholders; will be updated externally
        self.allow_long = False
        self.long_z_index_short = None
        self.long_z_index_long = None
        self.allow_short = False
        self.short_z_index_short = None
        self.short_z_index_long = None
        
        NFLOAT = "float32"
        
        # ------------- 5-min buffer ---------------------------------
        self.raw_5m = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument"])
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "category",
                "Instrument": "object"}))
                
        # ------------- 15-min table ---------------------------------
        self.data = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument", "EMA_30", "EMA_120", "trend","Zscore", "Zscore_7"]) 
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "category", "Instrument": "object",
                "EMA_30": NFLOAT, "EMA_120": NFLOAT,
                "trend": "category",
                "Zscore": NFLOAT,
                "Zscore_7": NFLOAT,
            })
            .set_index("Datetime", drop=False))
        
    def update(self, snap: dict, available_balance: float):
        
        """
        snap = bd[symbol]  – single-instrument snapshot from on_bulkdatafeed
        """
        # -------------------- append 5-min bar ------------------------
        green = ("G" if snap["lastPrice"] > snap["openPrice"]
                 else "R" if snap["lastPrice"] < snap["openPrice"]
                 else "NO")
        
        bar5 = {
            "Datetime": pd.to_datetime(snap["timestamp"]),
            "Open":  snap["openPrice"],
            "High":  snap["highPrice"],
            "Low":   snap["lowPrice"],
            "Close": snap["lastPrice"],
            "Green": green,
            "Instrument": snap["instrument"],
        }
        
        self.raw_5m = pd.concat([self.raw_5m,
                                 pd.DataFrame([bar5])], ignore_index=True)
                                 
        # avoid  Warning: data month not exists for ... fx error led to array memory explode
        if len(self.raw_5m) > 10:
            self.raw_5m = self.raw_5m.iloc[0:0]
        
        
        # ------------------- finished 15-min? -------------------------
        ts = pd.to_datetime(snap["timestamp"])
        if ts < self.last_time + timedelta(minutes=15):
            return
        self.last_time = ts
        
        last3 = self.raw_5m.tail(3)
        if len(last3) < 3:
            return    # need full 15-minute slice
        
        self.raw_5m = self.raw_5m.iloc[0:0]  # clear buffer
        
        self._aggregate_15m(last3)
        
        
        if len(self.data) < self.MIN_BARS:
            return
            
        
        if len(self.data) > self.MAX_BARS:
            self.data = self.data.iloc[-self.MAX_BARS:]
        
        # -------------------- get latest exchange rate  -----------------
        base_currency = self.symbol[:3] # self.fx_base_usd_currency
        
        
        if base_currency != "USD":
            
            fx_base = self.evt.getExchangeRate(base_currency, "USD")

        
        else:
            fx_base = 1
        
        quote_currency = self.symbol[3:]    
        if quote_currency != "USD" : # base currency is not USD
            
            fx = self.evt.getExchangeRate("USD", quote_currency)

        else: 
            fx = 1
        
            
        # -------------------- FSM ------------------------------------
        if self.fsm is None:
            if "JPY" in self.symbol :
                pip_size = 0.01
            else:                                     
                pip_size = 0.0001
                
            self.fsm = BearishFSM(
                self.data, 
                instrument=self.symbol, 
                evt=self.evt,
                account_balance=np.float32(available_balance),
                pip_size=pip_size,
                fx=fx,
                fx_base = fx_base, 
                ts=self.last_time, 
                allow_long = self.allow_long, 
                long_z_index_short = self.long_z_index_short, 
                long_z_index_long = self.long_z_index_long, 
                allow_short = self.allow_short, 
                short_z_index_short = self.short_z_index_short,
                short_z_index_long = self.short_z_index_long)
        else:
            self.fx_usd_quote_currency = fx
            self.fx_base_usd_currency = fx_base
            self.fsm.df = self.data
            self.fsm.account_balance = available_balance 
            # self.evt.consoleLog(self.fsm.instrument , available_balance)
            self.fsm.update(self.last_time)
            
    def _aggregate_15m(self, last3):
        
        
        
        n   = len(self.data)
        op  = last3["Open"].iloc[0]
        cl  = last3["Close"].iloc[-1]
        bar = {
            "Datetime": last3["Datetime"].iloc[-1],
            "Open": op,
            "High": last3["High"].max(),
            "Low":  last3["Low"].min(),
            "Close": cl,
            "Green": "G" if cl > op else "R" if cl < op else "NO",
            "Instrument": self.symbol,
        }

        if n == 0:
            ema30 = ema120 = cl #  ema10 =

        elif n < 30: 
            prev  = self.data.iloc[n - 1]
            ema30  = (1 - self.ALPHA_30)  * prev["EMA_30"]  + self.ALPHA_30  * cl
            ema120 = (1 - self.ALPHA_120) * prev["EMA_120"] + self.ALPHA_120 * cl

        else:
            prev  = self.data.iloc[n - 1] 
            ema30  = (1 - self.ALPHA_30)  * prev["EMA_30"]  + self.ALPHA_30  * cl
            ema120 = (1 - self.ALPHA_120) * prev["EMA_120"] + self.ALPHA_120 * cl

        
        bar["EMA_30"]  = ema30
        bar["EMA_120"] = ema120
        r = ema30 / ema120
        bar["trend"] = "BU" if r > 1.001 else "BE" if r < 0.999 else "NO" 

        # ---------- rolling Z-score (30-bar look-back) ----------------------
        window = 30 # 6 hours # 144                       # 2 days
        if n >= window-1:                  # we have enough history
            mean = self.data["Close"].iloc[-(window-1):].mean()
            std  = self.data["Close"].iloc[-(window-1):].std(ddof=0)
            z    = (cl - mean) / std if std > 0 else 0
        else:
            z = 0                          # not enough data yet
            
        # ---------- rolling Z-score (30-bar look-back) ----------------------
        window = 120 # 3 days                      # 7 days
        if n >= window-1:                  # we have enough history
            mean = self.data["Close"].iloc[-(window-1):].mean()
            std  = self.data["Close"].iloc[-(window-1):].std(ddof=0)
            z_7    = (cl - mean) / std if std > 0 else 0
        else:
            z_7 = 0
        bar["Zscore"] = np.float32(z)
        bar["Zscore_7"] = np.float32(z_7)

        self.data = pd.concat([self.data, pd.DataFrame([bar]).set_index("Datetime",drop=False)])

class AlgoEvent:

    def __init__(self):
        self.evt   = None
        self.books = {}          # symbol → InstrumentBar
        self.trades = []
        # Initialize instrument configuration:
        # symbol → dict with keys: allow_long, long_z_index, allow_short, short_z_index
        self.instrument_cfg = {
            'USDJPY': {          # 2024 earn 23k
                "allow_long": True,
                "long_z_index_short": -1.70, # 1.79 to overfit 21-23 data
                "long_z_index_long": 1.18,
                "allow_short": False,
                "short_z_index_short": None,
                "short_z_index_long": None,
            },
            'GBPJPY': {
                "allow_long": False,
                "long_z_index_short": None, # 1.79 to overfit 21-23 data
                "long_z_index_long": None,
                "allow_short": True,
                "short_z_index_short": 2.17,
                "short_z_index_long": -0.42, 
            },
            'GBPAUD': {            # wait until test, but 2024 didnt lose money
                "allow_long": True,
                "long_z_index_short": 100,
                "long_z_index_long": 0.61,
                "allow_short": False,
                "short_z_index_short": None,
                "short_z_index_long": None,
            },
            'EURCAD': {            #  2024 earn 30k
                "allow_long": False,
                "long_z_index_short": None,
                "long_z_index_long": None,
                "allow_short": True,
                "short_z_index_short": 1.42,
                "short_z_index_long": -0.64,
            },
            'EURCHF': {  # 21-23 earn 18k ,24 lose 2k, need to use z-index trained from 21-24
                "allow_long": False,
                "long_z_index_short": None,
                "long_z_index_long": None,
                "allow_short": True,
                "short_z_index_short": 1.68,
                "short_z_index_long": -1.27,
                # pip less than 8 no trade
            },
            'GBPUSD': {   # 2024 + 17k
                "allow_long": True,
                "long_z_index_short": -0.28, # un-verified, 
                "long_z_index_long": 1.18,
                "allow_short": False,
                "short_z_index_short": None,
                "short_z_index_long": None,
            },
 
        }
            
    def start(self, mEvt):
        self.evt = AlgoAPI_Backtest.AlgoEvtHandler(self, mEvt)
        self.evt.start()
        
        self.analyse_dual_zscores()
        self.analyse_stoploss_impact()
        
    def analyse_stoploss_impact(self):
        df = pd.DataFrame(self.trades).dropna(subset=["exit_ts", "pnl", "stoploss_pip", "symbol"])
        if df.empty:
            self.evt.consoleLog("No trades to analyze (missing stoploss_pip).")
            return
        
         # 🔍 Analyze tight stop-loss trades (< 8 pips)
        small_sl = df[df["stoploss_pip"] < 8]
        if not small_sl.empty:
            winners_small_sl = small_sl[small_sl["pnl"] > 0]
            win_rate_small = len(winners_small_sl) / len(small_sl)
    
            self.evt.consoleLog(f"\n⚠️ Trades with SL < 8 pips: {len(small_sl)}")
            self.evt.consoleLog(f"Winning trades    : {len(winners_small_sl)}")
            self.evt.consoleLog(f"Win rate          : {win_rate_small:.2%}")
            self.evt.consoleLog(f"Example SLs       : {small_sl['stoploss_pip'].head(5).tolist()}")
                
        sl_thresholds = [20, 25, 30, 35, 40, 45]
    
        self.evt.consoleLog("\n====== 📉 Stop-Loss Impact Analysis by Symbol ======")
    
        for symbol in df["symbol"].unique():
            df_sym = df[df["symbol"] == symbol]
            self.evt.consoleLog(f"\n=== Symbol: {symbol} ===")
            
            for threshold in sl_thresholds:
                df_filtered = df_sym[df_sym["stoploss_pip"] >= threshold]
                if df_filtered.empty:
                    continue
    
                winners = df_filtered[df_filtered["pnl"] > 0]
                win_rate = len(winners) / len(df_filtered)
                avg_pnl = df_filtered["pnl"].mean()
                self.evt.consoleLog(f"SL ≥ {threshold} pips → Trades: {len(df_filtered)}, Win Rate: {win_rate:.2%}")
                self.evt.consoleLog(f"Avg PnL: {avg_pnl:.2f}")

        
    def analyse_dual_zscores(self):
        df = pd.DataFrame(self.trades).dropna(subset=["exit_ts", "pnl", "entry_z", "entry_z_7", "symbol"])
        if df.empty:
            self.evt.consoleLog("No trades with dual Z-scores to analyze.")
            return
    
        def opt_threshold(z_w, z_l, num=50):
            all_z = np.concatenate([z_w, z_l])
            thresholds = np.linspace(all_z.min(), all_z.max(), num)
            best_j, best_t = -np.inf, thresholds[0]
            for t in thresholds:
                tpr = np.mean(z_w <= t)
                fpr = np.mean(z_l <= t)
                j = tpr - fpr
                if j > best_j:
                    best_j, best_t, best_tpr, best_fpr = j, t, tpr, fpr
            return best_t, best_j, best_tpr, best_fpr
    
        for sym in sorted(df["symbol"].unique()):
            self.evt.consoleLog(f"\n==================== {sym} ====================")
            df_sym = df[df["symbol"] == sym]
    
            for side_name, side_sign in [("LONG", 1), ("SHORT", -1)]:
                self.evt.consoleLog(f"\n📊 {side_name} TRADES (Symbol: {sym}):")
                df_side = df_sym[df_sym["side_sign"] == side_sign]
                if df_side.empty:
                    self.evt.consoleLog(f"⚠️ No {side_name} trades recorded.\n")
                    continue
    
                winners = df_side[df_side["pnl"] > 0]
                losers  = df_side[df_side["pnl"] <= 0]
                if winners.empty:
                    self.evt.consoleLog("No winning trades.\n")
                    continue
    
                for z_field, label in [("entry_z", "Short-term Z"), ("entry_z_7", "Long-term Z")]:
                    zw = winners[z_field]
                    zl = losers[z_field]
    
                    self.evt.consoleLog(f"\n--- {label} analysis ---")
                    self.evt.consoleLog(f"Winning {label} range: {zw.min():.2f} → {zw.max():.2f}")
                    self.evt.consoleLog(f"Mean/Median/Std (winners): {zw.mean():.2f}, {zw.median():.2f}, {zw.std():.2f}")
                    q1, q3 = np.percentile(zw, [25, 75])
                    self.evt.consoleLog(f"IQR (winners): {q1:.2f} – {q3:.2f}")
    
                    self.evt.consoleLog(f"Mean/Median/Std (losers): {zl.mean():.2f}, {zl.median():.2f}, {zl.std():.2f}")
                    q1l, q3l = np.percentile(zl, [25, 75])
                    self.evt.consoleLog(f"IQR (losers): {q1l:.2f} – {q3l:.2f}")
    
                    best_t, best_j, tpr, fpr = opt_threshold(zw.values, zl.values)
                    pct = np.mean(df_side[z_field] <= best_t)
                    self.evt.consoleLog(f"Count (w/l): {len(zw)} / {len(zl)}")
                    self.evt.consoleLog(f"✅ Optimal {label} threshold: {best_t:.2f} (J = {best_j:.2f})")
                    self.evt.consoleLog(f"📈 TPR = {tpr:.2%}, FPR = {fpr:.2%}, capture = {pct:.2%}")
    
    def on_marketdatafeed(self, md, ab):
        balance = ab["availableBalance"]
        
        sym = md.instrument
        # self.evt.consoleLog(md.instrument)
        if sym not in self.books: 
            book = InstrumentBar(sym, self.evt)
            
            # get pre-defined z-index and short/long
            cfg = self.instrument_cfg.get(sym)
            if cfg:
                book.allow_long = cfg["allow_long"]
                book.long_z_index_short = cfg["long_z_index_short"]
                book.long_z_index_long = cfg["long_z_index_long"]
                book.allow_short = cfg["allow_short"]
                book.short_z_index_short = cfg["short_z_index_short"]
                book.short_z_index_long = cfg["short_z_index_long"]
            self.books[sym] = book
        else:
            snap = {}
            snap['instrument'] = md.instrument
            snap['timestamp']  = md.timestamp
            snap['lastPrice']  = md.lastPrice
            snap['openPrice']  = md.openPrice
            snap['highPrice']  = md.highPrice
            snap['lowPrice']  = md.lowPrice
            # self.evt.consoleLog(snap)
            self.books[sym].update(snap, balance)
            
            
    """   
    def on_bulkdatafeed(self, isSync, bd, ab):
        
        balance = ab["availableBalance"]
        for sym, snap in bd.items():        # sym: AUDNZD, EURUSD, ... snap: {'instrument': 'AUDNZD', 'timestamp': datetime.datetime(1970, 1, 1, 0, 0), 'askPrice': -1, ...}
            if sym not in self.books:       # first sight → create book

                book = InstrumentBar(sym, self.evt)

                # Apply matching config if exists
                cfg = self.instrument_cfg.get(sym)
                if cfg:
                    book.allow_long = cfg["allow_long"]
                    book.long_z_index_short = cfg["long_z_index_short"]
                    book.long_z_index_long = cfg["long_z_index_long"]
                    book.allow_short = cfg["allow_short"]
                    book.short_z_index_short = cfg["short_z_index_short"]
                    book.short_z_index_long = cfg["short_z_index_long"]
                self.books[sym] = book
            else: 
                self.books[sym].update(snap, balance)
    """
    def on_orderfeed(self, of):
        
        sym = of.instrument
        if sym not in self.books:
            return                        # safety
        
        fsm = self.books[sym].fsm
        
        # -------------- order filled -----------------
        if of.openclose == "open" and of.status == "success":
            # Is this the order we are waiting for?
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.position_open      = True
                fsm.stop_loss_price    = fsm.pending["stop"]
                fsm.take_profit_price  = fsm.pending["take"]
                fsm.open_volume        = fsm.pending["volume"]
                fsm.enter_market_trade_side = fsm.pending["trend"]
                stoploss_pip_size = fsm.pending["stoploss_pip"]
    
                fsm.pending = None      # clear
                old_state   = fsm.state
                fsm.state   = "ENTER_MARKET"
                fsm.log_state_change(old_state, fsm.state, of.orderRef,
                                     "broker filled order")
            # --------- store entry info ---------------------------------
            self.trades.append(dict(
                orderRef     = of.orderRef,
                symbol       = of.instrument,
                side_sign    = 1 if of.buysell == 1 else -1,   # +1 = long, −1 = short
                volume       = of.fill_volume,
                entry_price  = of.fill_price,
                entry_ts     = of.insertTime,
                entry_z      = fsm.current_zscore(of.orderRef),  # helper shown earlier
                entry_z_7    = fsm.current_zscore_7_days(of.orderRef),
                stoploss_pip = stoploss_pip_size,
                exit_ts      = None,
                pnl          = None
            ))
        # -------------- order rejected ---------------
        elif of.openclose == "open" and of.status != "success":
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.pending = None
                old_state = fsm.state
                fsm.state = "ENTRY_SIGNAL"     # back to previous state
                fsm.log_state_change(old_state, fsm.state, of.orderRef,
                                     "order rejected")
                                     
        # -------------- position closed --------------
        if of.openclose == 'close' and of.status == 'success': # return 'success' if an order is opened/ closed successfully, or when a limit/stop order is filled
            
            for t in reversed(self.trades):
                if t["symbol"] == of.instrument and t["exit_ts"] is None:
                    t["exit_ts"]    = of.insertTime
                    t["exit_price"] = of.fill_price

                    # signed P/L per unit (positive = winner)
                    #   BUY  →  (exit - entry) * +1
                    #   SELL →  (exit - entry) * -1
                    t["pnl"] = (of.fill_price - t["entry_price"]) * t["side_sign"] \
                            * t["volume"]                         # in “price × volume” units
                    break
            fsm.reset()
            # --------- store entry info ---------------------------------
    
    
    def on_newsdatafeed(self, nd):
        pass

    def on_weatherdatafeed(self, wd):
        pass
    
    def on_econsdatafeed(self, ed):
        pass
        
    def on_corpAnnouncement(self, ca):
        pass
    
    def on_dailyPLfeed(self, pl):
        pass

    def on_openPositionfeed(self, op, oo, uo):
        pass












## Final version of forex algo-trading

In [ ]:
from AlgoAPI import AlgoAPIUtil, AlgoAPI_Livetest, AlgoAPI_Backtest
import pandas as pd
import numpy as np
import mplfinance as mpf
from datetime import datetime, timedelta
    
class BearishFSM: # Finite State Machine

        # ==============================================================
        # ========================  __init__ ===========================
        # ==============================================================
    def __init__(self, df, instrument, evt, account_balance=0, pip_size = 0.0001, fx = 1, fx_base = 1, ts = None, allow_long = False, long_z_index_short = None, long_z_index_long = None, allow_short = False, short_z_index_short = None, short_z_index_long = None):
        
        self.df = df
        self.evt = evt
        self.account_balance = account_balance
        
        # Enter market flag
        self.position_open = False
        self.pending = None       # ← dict that holds the ticket waiting
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.open_volume = None
        
        self.state = "IDLE"
        self.low_price = None
        
        self.low_price_ts = None
        self.high_price = None
        self.high_price_ts = None

        self.fib_labels = []
        self.fib_dict = {} # not a must to initialize, just for good habit
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.trend_changing_ts = ts # do not reset this, always keep record of 
        
##################################################################################################################
####################################### optimize #################################################################
##################################################################################################################

        # do not reset the folloiwng parameters, test for optimal values
        self.risk_reward_ratio = 2
        self.atr_period = 14
        self.atr_multiplier = 2.5 # pip_size = atr_multiplier * atr_value, atr_multiplier = 2.0 means stop loss is 2 ATRs away
        self.risk_percent = 3
        self.dynamic_atr_window = False
        # self.trend_ema10_sma50 = False # If True, the trend will be determined by EMA10 and SMA50, if False, determined by EMA30 and EMA120
        
##################################################################################################################
##################################################################################################################
##################################################################################################################
        # this is unique in every func call
        self.instrument = instrument
        self.pip_size = pip_size
        self.fx_usd_quote_currency = fx
        self.fx_base_usd_currency = fx_base
        
        self.allow_long = allow_long
        self.long_z_index_short = long_z_index_short
        self.long_z_index_long = long_z_index_long
        self.allow_short = allow_short
        self.short_z_index_short = short_z_index_short
        self.short_z_index_long = short_z_index_long

    # ==============================================================
    # ========================  current_zscore             =========
    # ==============================================================
    def current_zscore(self, ts):
        if "Zscore" not in self.df.columns:
            return np.nan
        return float(self.df["Zscore"].loc[ts])
    def current_zscore_7_days(self, ts):
        if "Zscore_7" not in self.df.columns:
            return np.nan
        return float(self.df["Zscore_7"].loc[ts])
    # ==============================================================
    # ========================  calculate_log_state_change =========
    # ============================================================== 
    def log_state_change(self, old_state, new_state, ts, label=None):
        msg = f"{self.instrument} State: {old_state} → {new_state}"
        if label:
            msg += f" ({label})"
        self.evt.consoleLog(msg)
        
        # ==============================================================
        # ========================  calculate_lot_size  ================
        # ==============================================================
        
    def calculate_lot_size(self, risk_percent, stop_loss_pips, max_leverage = 4.5): # base currency vs quote currency
    
        # 1. Calculate amount to risk
        risk_amount = self.account_balance * (risk_percent / 100)
        
        # 2. calculate pip value per lot (in quote currency)
        pip_value_per_lot_quote = self.pip_size * 100000 # If we are trading AUDJPY, the value will be 0.01 * 100000 = 1000 JPY
        
        # 3. calculate total risk per lot ( in quote currency)
        total_risk_per_lot_quote = stop_loss_pips * pip_value_per_lot_quote
        
        # 4. calculate total risk per lot ( in account currency, i.e. USD)
            # we need real time USD/quote_currency price, in this case, USDJPY
        self.evt.consoleLog(self.fx_usd_quote_currency)
        total_risk_per_lot_USD = total_risk_per_lot_quote / self.fx_usd_quote_currency # if quote currency = USD, this will be 1
        
        # 5. calculate lot size
        lot_size = risk_amount / total_risk_per_lot_USD

        # calculate maximum affordable lot size after using margin (e.g. we are trading AUDUSD)
            # Notional Value per lot (in Base Currency) = 100000 AUD
            
        # Notation Value per lot (in account currency) = 100000 x AUDUSD price
        notation_per_lot_USD = 100000 * self.fx_base_usd_currency
            
        lot_margin = (self.account_balance * max_leverage) / notation_per_lot_USD
        
        lot_final = min(lot_size, lot_margin) # all-in 
        
        self.evt.consoleLog(
            f"{self.instrument} bal={self.account_balance:.2f} risk%={risk_percent} sl={stop_loss_pips} "
            f"riskAmt={risk_amount:.2f} pipVal/lot={pip_value_per_lot_quote:.5f} risk/lot(USD)={total_risk_per_lot_USD}"
            f"riskLot={lot_size:.2f} margLot={lot_margin:.2f} finalLot={lot_final:.2f}"
        )
        
        return round(lot_final, 2)
        
        
        # ==============================================================
        # ========================  calc_fibonacci_levels  =============
        # ==============================================================
        
    def calc_fibonacci_levels(self, start_ts, end_ts, levels=None, trend="NO",
                         entry_signal=None, engulfing_candle=None,
                         R=None, atr_period=None, atr_multiplier=None,dynamic_atr_window = None):
                             
        if levels is None:
            levels = [0, 0.236, 0.382, 0.5, 0.618, 0.710, 0.786, 1]
            
        if dynamic_atr_window is None:
            dynamic_atr_window = self.dynamic_atr_window
            
        if atr_multiplier is None:
            atr_multiplier = self.atr_multiplier
        
        if R is None:
            R = self.risk_reward_ratio
        
        
        df = self.df

        window = df.loc[start_ts : end_ts]
                            
        min_price = window['Low'].min()
        max_price = window['High'].max()
    
        fib_labels = []
        for level in levels:
            if trend == "BE": # bearish, sell
                y = min_price + (max_price - min_price) * level
            else:
                y = max_price - (max_price - min_price) * level
            fib_labels.append((level, y))
        
        if engulfing_candle is None:
            return fib_labels
        
        if atr_period is None:

            if dynamic_atr_window == False:
                # Fixed atr period
                atr_period = self.atr_period
            else: 
                # dynamic atr period
                start_idx = df.index.get_loc(start_ts)
                enter_idx = df.index.get_loc(engulfing_candle)
            
                dynamic_period = enter_idx - start_idx
                atr_period = max(10, min(dynamic_period, 50))
        
        # Normal ATR
        min_start_idx = max(0, df.index.get_loc(start_ts) - 10)
        min_start_ts = df.index[min_start_idx]
        sl = df.loc[min_start_ts : engulfing_candle].copy()
        
        sl['H-L']  = sl['High'] - sl['Low']
        sl['H-PC'] = (sl['High'] - sl['Close'].shift(1)).abs()
        sl['L-PC'] = (sl['Low']  - sl['Close'].shift(1)).abs()
        sl['TR']    = sl[['H-L', 'H-PC', 'L-PC']].max(axis=1)
        
        # ATR: rolling mean of TR
        sl['ATR'] = sl['TR'].rolling(window=atr_period).mean()
         
        # Retrieve ATR value at the engulfing candle index
        atr_value = sl['ATR'].iloc[-1]
        pip_size = atr_multiplier * atr_value
        
        # Retrieve the close price of the engulfing candle
        candle = self.df.loc[engulfing_candle]
        close_price = candle['Close']
        
        # Initialize stop loss and take profit prices
        stop_loss_price = None
        take_profit_price = None
    
        if trend == "BE" :
            stop_loss_price = close_price + pip_size
            take_profit_price = close_price - R * pip_size
        else:  # bullish
            stop_loss_price = close_price - pip_size
            take_profit_price = close_price + R * pip_size
    
        return stop_loss_price, take_profit_price
        
    # ==============================================================
    # ======================== send_entry_order  =============
    # ==============================================================
    
    def send_entry_order(self, ts, trend, entry_price,
                         stop, take, volume, stoploss_pip_size):

        # remember what we just asked the broker to do
        self.pending = dict(
            orderRef = ts,         # we use the timestamp as reference
            trend    = trend,      # "BU" / "BE"
            stop     = stop,
            take     = take,
            volume   = volume,
            stoploss_pip = stoploss_pip_size
        )

        order = AlgoAPIUtil.OrderObject()
        order.instrument   = self.instrument
        order.orderRef     = ts
        order.openclose    = "open"
        order.buysell      = 1 if trend == "BU" else -1
        order.ordertype    = 0          # market
        order.volume       = volume
        order.stopLossLevel= stop
        order.takeProfitLevel = take
        order.holdtime = 604800  #max holding time is 1 week
        self.evt.sendOrder(order)

        # NOTE: do NOT set state / flags here;
        #       wait for on_orderfeed()
    
    # ==============================================================
    # ========================  update()  =============
    # ==============================================================
    
    def update(self, ts: pd.Timestamp):
        
        # 1️⃣ When a position is open we do only exit-logic and leave
        #    the rest of the gigantic state-machine untouched.
        #    (So IDLE / PULLBACK_DETECTED / … cannot fire again.)
        if self.position_open:
            return                       # <-- early exit
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        
        if df.index.get_loc(ts) < 3: # Get current timestamp location
            return
        
        #--- 2. prepare frequently-used columns (all remain Series!) ------
        greens  = df["Green"]               # string
        opens   = df["Open"]
        closes  = df["Close"]
        
        lowest_body_price = opens.where(greens == 'G', closes) # if greens == 'G', use values in open, else use values in closes 
        highest_body_price = closes.where(greens == 'G', opens) # the index of this series keeps the same Datetime index as the original df
        
        # selects the row in df where the index = ts
        curr   = df.loc[ts]
        prev   = df.shift(1).loc[ts]       # one-bar shift gives previous values

        current_trend = curr['trend'] ############ use EMA120 and EMA30
        prev_trend    = prev['trend']

        if current_trend != prev_trend : 
            self.trend_changing_ts = ts
            if self.state != "ENTER_MARKET":
                self.reset()
                
        # ==============================================================
        # ========================  BEARISH  ============================
        # ==============================================================
        if self.allow_short == True: 
            if (current_trend == "BE" and self.enter_market_trade_side == None):
                if self.state == "IDLE":
                    if self.check_pullback(greens, opens, closes, ts, trend="BE"): # pullback should eat and cover the previous red candle
                        # logic
                        slices = lowest_body_price.loc[self.trend_changing_ts:ts] # Slicing with .loc[start:end] in pandas includes both endpoints
                        self.low_price = slices.min()
                        self.low_price_ts  = slices.idxmin()   # Return index(Datetime) of first occurrence of minimum over requested axis.
                        # debug
                        old_state = self.state  
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bearish pullback detected")
                        
    
                elif self.state == "PULLBACK_DETECTED":
                    if lowest_body_price.loc[ts] < self.low_price:
                        # logic
                        slices = highest_body_price.loc[self.low_price_ts:ts]
                        self.high_price = slices.max()
                        self.high_price_ts = slices.idxmax()
                        # debug
                        old_state = self.state
                        self.state = "BREAK_OF_STRUCTURE"
                        self.log_state_change(old_state, self.state, ts, "bearish new low after pullback")
    
                        
                elif self.state == "BREAK_OF_STRUCTURE":
                    # If price breaks the pev high (LH)
                    if highest_body_price.loc[ts] > self.high_price:
                        
                        # logic
                        slices = lowest_body_price.loc[self.high_price_ts: ts]
                        self.low_price = slices.min()
                        self.low_price_ts = slices.idxmin()
                        
                        # debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bearish break of structure")
    
                    elif self.check_pullback(greens, opens, closes, ts, trend="BE"):
                        
                        if lowest_body_price.shift(2).loc[ts] < self.low_price: #### use 2 timeframe before current ts , can incorporate to 3,4,5 or 6
                        
                            # Logic: Low price for confirming the next Lower high 
                            if pos >= 3:   
                                window = lowest_body_price.iloc[pos-3 : pos+1]
                                self.low_price     = window.min()
                                self.low_price_ts  = window.idxmin()
    
                            self.fib_labels = self.calc_fibonacci_levels(self.high_price_ts, self.low_price_ts, trend="BE")
                            self.fib_dict = dict(self.fib_labels)
                            
                            # debug
                            old_state = self.state
                            self.state = "WAITING_ENTRY"
                            self.log_state_change(old_state, self.state, ts, "bearish waiting entry")
    
                        elif highest_body_price.loc[ts] >= self.low_price:
                            
                            # debug
                            old_state = self.state
                            self.state = "PULLBACK_DETECTED"
                            self.log_state_change(old_state, self.state, ts, "bearish back to pullback")
    
                elif self.state == "WAITING_ENTRY":
                    
                    fib_710_price = self.fib_dict.get(0.710)
                    fib_000_price = self.fib_dict.get(0.000)
                    
                    
                    # price rebound back to 0.710 &
                    if fib_710_price and df["High"].loc[ts] >= fib_710_price: # reference the "High" price at the exact timestamp
                    
                        # debug
                        self.entry_signal_ts = ts
                        old_state = self.state
                        self.state = "ENTRY_SIGNAL"
                        self.log_state_change(old_state, self.state, ts, "bearish price rebound to 0.710")
    
                    elif fib_710_price and lowest_body_price.loc[ts] <= fib_000_price: # price lower than 0.000
                    
                        # Logic: break of structure, set new high point
                        slices = highest_body_price.loc[self.low_price_ts: ts]
                        self.high_price = slices.max()
                        self.high_price_ts = slices.idxmax()
                        
                        # debug
                        old_state = self.state
                        self.state = "BREAK_OF_STRUCTURE"
                        self.log_state_change(old_state, self.state, ts, "bearish breaks 0.000 before entry")
                
                elif self.state == "ENTRY_SIGNAL": 
                    fib_100_price = self.fib_dict.get(1.000)
                    z_short = df['Zscore'].loc[ts]
                    z_long = df['Zscore_7'].loc[ts]
                    
                    if highest_body_price.loc[ts] >= fib_100_price: # no matter what, if the candle pass 1.00, we do not trade
                    
                        # debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bearish cancels: price > 1.0")
    
                    # Logic: Bearish Engulfing candle  
                    elif z_short <= self.short_z_index_short and z_long <= self.short_z_index_long :
                            
                        # Debug
                        self.engulfing_candle_ts = ts
                        
                        stop, take = self.calc_fibonacci_levels(self.high_price_ts, 
                                self.low_price_ts, trend="BE", entry_signal=self.entry_signal_ts, engulfing_candle=self.engulfing_candle_ts
                                )
                        
                        # Logic to calculate lot_size
                        entry_price = closes.loc[ts]
                        stop_loss_pips = abs(stop - entry_price) / self.pip_size
                        lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips)
                        
                        if self.instrument == "EURCHF" and stop_loss_pips <= 8: 
                            lot_size = 0.01
                        
                            
                        """
                        if stop_loss_pips >= 30 : 
                            lot_size = 0.01
                            
                        # Log for debug
                        self.evt.consoleLog(
                            f"DEBUG ticket – side:SELL, instrument:{self.instrument},ab:{self.account_balance}, entry≈{entry_price:.5f} "
                            f"SL={stop:.5f} TP={take:.5f}"
                        )
                    
                        """
                        # place selling order
                        self.send_entry_order(ts, trend="BE",
                                    entry_price=entry_price,
                                    stop=stop, take=take,
                                    volume=lot_size, stoploss_pip_size = stop_loss_pips)
                        
                        # Debug
                        old_state = self.state
                        self.state = "PRE_ENTER_MARKET"
                        self.log_state_change(old_state, self.state, ts, "bearish entry: engulfing candle")
                    else:
                        self.state = "PULLBACK_DETECTED"
                        return
                    
        # ==============================================================
        # ========================  BULLISH  ============================
        # ==============================================================
        
        if self.allow_long == True:
            if (current_trend == "BU" and self.enter_market_trade_side == None):
                if self.state == "IDLE":    
                    if self.check_pullback(greens, opens, closes, ts, trend="BU"):
                        
                        # Logic
                        slices = highest_body_price[self.trend_changing_ts:ts]
                        self.high_price = slices.max()
                        self.high_price_ts = slices.idxmax()
                        
                        # Debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bullish pullback detected")
    
                elif self.state == "PULLBACK_DETECTED":
                    if highest_body_price.loc[ts] > self.high_price:
                        
                        # Logic
                        slices = lowest_body_price[self.high_price_ts: ts]
                        self.low_price = slices.min()
                        self.low_price_ts = slices.idxmin()
                        
                        # Debug
                        old_state = self.state
                        self.state = "BREAK_OF_STRUCTURE"
                        self.log_state_change(old_state, self.state, ts, "bullish new high after pullback")
    
       
                elif self.state == "BREAK_OF_STRUCTURE":
                    # structure broken downward -> restart pull-back search
                    if lowest_body_price.loc[ts] < self.low_price:
                        
                        # Logic
                        slices = highest_body_price[self.low_price_ts : ts]
                        self.high_price = slices.max()
                        self.high_price_ts = slices.idxmax()
                        
                        # Debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bullish break of structure")
    
                    # new 3 red pull-back -> possible Fib leg
                    elif self.check_pullback(greens, opens, closes, ts, trend="BU"):
                        
                        if highest_body_price.shift(2).loc[ts] > self.high_price: # fresh HH
                            
                            # Logic
                            if pos >= 3:   
                                window = highest_body_price.iloc[pos-3 : pos+1]
                                self.high_price     = window.max()
                                self.high_price_ts  = window.idxmax()
                            
                            self.fib_labels = self.calc_fibonacci_levels(self.low_price_ts, self.high_price_ts, trend="BU")
                            self.fib_dict = dict(self.fib_labels)
                            
                            # Debug
                            old_state = self.state
                            self.state = "WAITING_ENTRY"
                            self.log_state_change(old_state, self.state, ts, "bullish waiting entry")
    
                        # price failed to beat prev high: treat as new pull-back
                        elif lowest_body_price.loc[ts] <= self.high_price:
                            
                            # Debug
                            old_state = self.state
                            self.state = "PULLBACK_DETECTED"
                            self.log_state_change(old_state, self.state, ts, "bullish back to pullback")
                            
    
                elif self.state == "WAITING_ENTRY":
                    fib_710_bullish = self.fib_dict.get(0.710)
                    fib_000_bullish = self.fib_dict.get(0.000)
    
                    # price retraced to 0.710 – ready for bullish entry
                    if fib_710_bullish and df['Low'].loc[ts] <= fib_710_bullish: # count the tip of the candlestick instead of the body
                    
                        # Debug
                        self.entry_signal_ts = ts
                        old_state = self.state
                        self.state = "ENTRY_SIGNAL"
                        self.log_state_change(old_state, self.state, ts, "bullish price retraced to 0.710")
    
                    # price breaks above 0.000 (HH) before touching 0.710
                    elif fib_710_bullish and highest_body_price.loc[ts] > fib_000_bullish:
                        
                        # Logic
                        slices = lowest_body_price[self.high_price_ts:ts]
                        self.low_price = slices.min()
                        self.low_price_ts = slices.idxmin()
                        
                        # Debug
                        old_state = self.state
                        self.state = "BREAK_OF_STRUCTURE"
                        self.log_state_change(old_state, self.state, ts, "bullish breaks above 0.000 before entry")
                    
                elif self.state == "ENTRY_SIGNAL":
                    fib_100_bullish = self.fib_dict.get(1.000)
                    fib_000_bullish = self.fib_dict.get(0.000)
                    z_short = df['Zscore'].loc[ts]
                    z_long = df['Zscore_7'].loc[ts]
                    # cancel if price dumps below 1.000 extension
                    if lowest_body_price.loc[ts] <= fib_100_bullish:
                        
                        # Debug
                        old_state = self.state
                        self.state = "PULLBACK_DETECTED"
                        self.log_state_change(old_state, self.state, ts, "bullish cancels: price < 1.0")
    
                    # Bullish Engulfing candle
                    elif z_short <= self.long_z_index_short and z_long <= self.long_z_index_long: # self.long_z_index: 
                        
                        self.engulfing_candle_ts = ts
                        
                        stop, take = self.calc_fibonacci_levels(self.low_price_ts, 
                        self.high_price_ts,trend="BU", entry_signal=self.entry_signal_ts,
                        engulfing_candle=self.engulfing_candle_ts)
                        
                        # calculate volume
                        entry_price = closes.loc[ts]
                        stop_loss_pips = abs(stop - entry_price) / self.pip_size
                        lot_size = self.calculate_lot_size(self.risk_percent,stop_loss_pips)
                        


                        """
                        if stop_loss_pips >= 30 : 
                            lot_size = 0.01
                            
                        self.evt.consoleLog(
                            f"DEBUG ticket – side:BUY, instrument: {self.instrument}, ab:{self.account_balance}, entry≈{entry_price:.5f} "
                            f"SL={stop:.5f} TP={take:.5f}"
                        )
                        """
                        # place buying order
                        self.send_entry_order(ts, trend="BU",
                                  entry_price=entry_price,
                                  stop=stop, take=take,
                                  volume=lot_size, stoploss_pip_size = stop_loss_pips)
                        
                        # Debug
                        old_state = self.state
                        self.state = "PRE_ENTER_MARKET"
                        self.log_state_change(old_state, self.state, ts, "bullish entry: engulfing candle")
                    
                    else:
                        self.state = "PULLBACK_DETECTED"
                        return
                        


        
        # ==============================================================
        # ========================  pullback()  ========================
        # ==============================================================     
        
    def check_pullback(self, greens, opens, closes, ts, trend):
        
        df = self.df
        pos = df.index.get_loc(ts) # get current timestamp integer position
        
        if trend == "BE":
            # Check for 3, 4, 5, or 6 consecutive green candles.
            for window in [3, 4, 5, 6]:

                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                # Boundary check
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                # Check if all are green
                if all(greens.loc[ts_] == 'G' for ts_ in window_ts):
                    # Previous candle must be red
                    if greens.loc[prev_ts] == 'R':
                        # Any green candle closes above the open of previous red candle
                        if any(closes.loc[ts_] >= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False
        
        if trend == "BU":
            for window in [3,4,5,6]:
                start_idx = pos - window + 1
                prev_idx = start_idx - 1
                
                if prev_idx < 0:
                    continue
                
                start_ts = df.index[start_idx]
                prev_ts = df.index[prev_idx]
                
                window_ts = df.index[start_idx : pos + 1] # output an array of timestamp
                
                if all(greens.loc[ts_]!= 'G' for ts_ in window_ts): # all previous not green candles 
                    if greens.loc[prev_ts] == 'G':
                        if any(closes.loc[ts_] <= opens.loc[prev_ts] for ts_ in window_ts):
                            return True
            return False

    def reset(self):
        self.state = "IDLE"
        self.low_price  = self.low_price_ts  =  None
        self.high_price = self.high_price_ts =  None
        self.fib_labels = []
        self.fib_dict = {} 
        self.entry_signal_ts = None
        self.engulfing_candle_ts = None
        self.stop_loss_price = None
        self.take_profit_price = None
        self.enter_market_trade_side = None
        self.position_open = False
        self.open_volume = None


class InstrumentBar:
    
    """Bar builder + FSM for ONE instrument."""
    
    ALPHA_30  = 2 / 31
    ALPHA_120 = 2 / 121
    MIN_BARS  = 200 
    MAX_BARS  = 700
    
    def __init__(self, symbol: str, evt):
        
        self.symbol    = symbol
        self.evt       = evt
        self.last_time = datetime(2010, 1, 1)
        self.fsm       = None
        
        # placeholders; will be updated externally
        self.allow_long = False
        self.long_z_index_short = None
        self.long_z_index_long = None
        self.allow_short = False
        self.short_z_index_short = None
        self.short_z_index_long = None
        
        NFLOAT = "float32"
        
        # ------------- 5-min buffer ---------------------------------
        self.raw_5m = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument"])
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "category",
                "Instrument": "object"}))
                
        # ------------- 15-min table ---------------------------------
        self.data = (pd.DataFrame(columns=[
                "Datetime", "Open", "High", "Low", "Close",
                "Green", "Instrument", "EMA_30", "EMA_120", "trend","Zscore", "Zscore_7"]) 
            .astype({
                "Datetime": "datetime64[ns]",
                "Open":  NFLOAT,
                "High":  NFLOAT,
                "Low":   NFLOAT,
                "Close": NFLOAT,
                "Green": "category", "Instrument": "object",
                "EMA_30": NFLOAT, "EMA_120": NFLOAT,
                "trend": "category",
                "Zscore": NFLOAT,
                "Zscore_7": NFLOAT,
            })
            .set_index("Datetime", drop=False))
        
    def update(self, snap: dict, available_balance: float):
        
        """
        snap = bd[symbol]  – single-instrument snapshot from on_bulkdatafeed
        """
        # -------------------- append 5-min bar ------------------------
        green = ("G" if snap["lastPrice"] > snap["openPrice"]
                 else "R" if snap["lastPrice"] < snap["openPrice"]
                 else "NO")
        
        bar5 = {
            "Datetime": pd.to_datetime(snap["timestamp"]),
            "Open":  snap["openPrice"],
            "High":  snap["highPrice"],
            "Low":   snap["lowPrice"],
            "Close": snap["lastPrice"],
            "Green": green,
            "Instrument": snap["instrument"],
        }
        
        self.raw_5m = pd.concat([self.raw_5m,
                                 pd.DataFrame([bar5])], ignore_index=True)
                                 
        # avoid  Warning: data month not exists for ... fx error led to array memory explode
        if len(self.raw_5m) > 10:
            self.raw_5m = self.raw_5m.iloc[0:0]
        
        
        # ------------------- finished 15-min? -------------------------
        ts = pd.to_datetime(snap["timestamp"])
        if ts < self.last_time + timedelta(minutes=15):
            return
        self.last_time = ts
        
        last3 = self.raw_5m.tail(3)
        if len(last3) < 3:
            return    # need full 15-minute slice
        
        self.raw_5m = self.raw_5m.iloc[0:0]  # clear buffer
        
        self._aggregate_15m(last3)
        
        
        if len(self.data) < self.MIN_BARS:
            return
            
        
        if len(self.data) > self.MAX_BARS:
            self.data = self.data.iloc[-self.MAX_BARS:]
        
        # -------------------- get latest exchange rate  -----------------
        base_currency = self.symbol[:3] # self.fx_base_usd_currency
        
        
        if base_currency != "USD":
            
            fx_base = self.evt.getExchangeRate(base_currency, "USD")

        
        else:
            fx_base = 1
        
        quote_currency = self.symbol[3:]    
        if quote_currency != "USD" : # base currency is not USD
            
            fx = self.evt.getExchangeRate("USD", quote_currency)

        else: 
            fx = 1
        
            
        # -------------------- FSM ------------------------------------
        if self.fsm is None:
            if "JPY" in self.symbol :
                pip_size = 0.01
            else:                                     
                pip_size = 0.0001
                
            self.fsm = BearishFSM(
                self.data, 
                instrument=self.symbol, 
                evt=self.evt,
                account_balance=np.float32(available_balance),
                pip_size=pip_size,
                fx=fx,
                fx_base = fx_base, 
                ts=self.last_time, 
                allow_long = self.allow_long, 
                long_z_index_short = self.long_z_index_short, 
                long_z_index_long = self.long_z_index_long, 
                allow_short = self.allow_short, 
                short_z_index_short = self.short_z_index_short,
                short_z_index_long = self.short_z_index_long)
        else:
            self.fx_usd_quote_currency = fx
            self.fx_base_usd_currency = fx_base
            self.fsm.df = self.data
            self.fsm.account_balance = available_balance 
            # self.evt.consoleLog(self.fsm.instrument , available_balance)
            self.fsm.update(self.last_time)
            
    def _aggregate_15m(self, last3):
        
        
        
        n   = len(self.data)
        op  = last3["Open"].iloc[0]
        cl  = last3["Close"].iloc[-1]
        bar = {
            "Datetime": last3["Datetime"].iloc[-1],
            "Open": op,
            "High": last3["High"].max(),
            "Low":  last3["Low"].min(),
            "Close": cl,
            "Green": "G" if cl > op else "R" if cl < op else "NO",
            "Instrument": self.symbol,
        }

        if n == 0:
            ema30 = ema120 = cl #  ema10 =

        elif n < 30: 
            prev  = self.data.iloc[n - 1]
            ema30  = (1 - self.ALPHA_30)  * prev["EMA_30"]  + self.ALPHA_30  * cl
            ema120 = (1 - self.ALPHA_120) * prev["EMA_120"] + self.ALPHA_120 * cl

        else:
            prev  = self.data.iloc[n - 1] 
            ema30  = (1 - self.ALPHA_30)  * prev["EMA_30"]  + self.ALPHA_30  * cl
            ema120 = (1 - self.ALPHA_120) * prev["EMA_120"] + self.ALPHA_120 * cl

        
        bar["EMA_30"]  = ema30
        bar["EMA_120"] = ema120
        r = ema30 / ema120
        bar["trend"] = "BU" if r > 1.001 else "BE" if r < 0.999 else "NO" 

        # ---------- rolling Z-score (30-bar look-back) ----------------------
        window = 30 # 6 hours # 144                       # 2 days
        if n >= window-1:                  # we have enough history
            mean = self.data["Close"].iloc[-(window-1):].mean()
            std  = self.data["Close"].iloc[-(window-1):].std(ddof=0)
            z    = (cl - mean) / std if std > 0 else 0
        else:
            z = 0                          # not enough data yet
            
        # ---------- rolling Z-score (30-bar look-back) ----------------------
        window = 120 # 3 days                      # 7 days
        if n >= window-1:                  # we have enough history
            mean = self.data["Close"].iloc[-(window-1):].mean()
            std  = self.data["Close"].iloc[-(window-1):].std(ddof=0)
            z_7    = (cl - mean) / std if std > 0 else 0
        else:
            z_7 = 0
        bar["Zscore"] = np.float32(z)
        bar["Zscore_7"] = np.float32(z_7)

        self.data = pd.concat([self.data, pd.DataFrame([bar]).set_index("Datetime",drop=False)])

class AlgoEvent:

    def __init__(self):
        self.evt   = None
        self.books = {}          # symbol → InstrumentBar
        self.trades = []
        # Initialize instrument configuration:
        # symbol → dict with keys: allow_long, long_z_index, allow_short, short_z_index
        self.instrument_cfg = {
            'USDJPY': {          # 2024 earn 23k
                "allow_long": True,
                "long_z_index_short": -1.70, # 1.79 to overfit 21-23 data
                "long_z_index_long": 1.18,
                "allow_short": False,
                "short_z_index_short": None,
                "short_z_index_long": None,
            },
            'GBPJPY': {
                "allow_long": False,
                "long_z_index_short": None, # 1.79 to overfit 21-23 data
                "long_z_index_long": None,
                "allow_short": True,
                "short_z_index_short": 2.17,
                "short_z_index_long": -0.42, 
            },
            'GBPAUD': {            # wait until test, but 2024 didnt lose money
                "allow_long": True,
                "long_z_index_short": 100,
                "long_z_index_long": 0.61,
                "allow_short": False,
                "short_z_index_short": None,
                "short_z_index_long": None,
            },
            'EURCAD': {            #  2024 earn 30k
                "allow_long": False,
                "long_z_index_short": None,
                "long_z_index_long": None,
                "allow_short": True,
                "short_z_index_short": 1.42,
                "short_z_index_long": -0.64,
            },
            'EURCHF': {  # 21-23 earn 18k ,24 lose 2k, need to use z-index trained from 21-24
                "allow_long": False,
                "long_z_index_short": None,
                "long_z_index_long": None,
                "allow_short": True,
                "short_z_index_short": 1.68,
                "short_z_index_long": -1.27,
                # pip less than 8 no trade
            },
            'GBPUSD': {   # 2024 + 17k
                "allow_long": True,
                "long_z_index_short": -0.28, # un-verified, 
                "long_z_index_long": 1.18,
                "allow_short": False,
                "short_z_index_short": None,
                "short_z_index_long": None,
            },
 
        }
            
    def start(self, mEvt):
        self.evt = AlgoAPI_Backtest.AlgoEvtHandler(self, mEvt)
        self.evt.start()
        
        self.analyse_dual_zscores()
        self.analyse_stoploss_impact()
        
    def analyse_stoploss_impact(self):
        df = pd.DataFrame(self.trades).dropna(subset=["exit_ts", "pnl", "stoploss_pip", "symbol"])
        if df.empty:
            self.evt.consoleLog("No trades to analyze (missing stoploss_pip).")
            return
        
         # 🔍 Analyze tight stop-loss trades (< 8 pips)
        small_sl = df[df["stoploss_pip"] < 8]
        if not small_sl.empty:
            winners_small_sl = small_sl[small_sl["pnl"] > 0]
            win_rate_small = len(winners_small_sl) / len(small_sl)
    
            self.evt.consoleLog(f"\n⚠️ Trades with SL < 8 pips: {len(small_sl)}")
            self.evt.consoleLog(f"Winning trades    : {len(winners_small_sl)}")
            self.evt.consoleLog(f"Win rate          : {win_rate_small:.2%}")
            self.evt.consoleLog(f"Example SLs       : {small_sl['stoploss_pip'].head(5).tolist()}")
                
        sl_thresholds = [20, 25, 30, 35, 40, 45]
    
        self.evt.consoleLog("\n====== 📉 Stop-Loss Impact Analysis by Symbol ======")
    
        for symbol in df["symbol"].unique():
            df_sym = df[df["symbol"] == symbol]
            self.evt.consoleLog(f"\n=== Symbol: {symbol} ===")
            
            for threshold in sl_thresholds:
                df_filtered = df_sym[df_sym["stoploss_pip"] >= threshold]
                if df_filtered.empty:
                    continue
    
                winners = df_filtered[df_filtered["pnl"] > 0]
                win_rate = len(winners) / len(df_filtered)
                avg_pnl = df_filtered["pnl"].mean()
                self.evt.consoleLog(f"SL ≥ {threshold} pips → Trades: {len(df_filtered)}, Win Rate: {win_rate:.2%}")
                self.evt.consoleLog(f"Avg PnL: {avg_pnl:.2f}")

        
    def analyse_dual_zscores(self):
        df = pd.DataFrame(self.trades).dropna(subset=["exit_ts", "pnl", "entry_z", "entry_z_7", "symbol"])
        if df.empty:
            self.evt.consoleLog("No trades with dual Z-scores to analyze.")
            return
    
        def opt_threshold(z_w, z_l, num=50):
            all_z = np.concatenate([z_w, z_l])
            thresholds = np.linspace(all_z.min(), all_z.max(), num)
            best_j, best_t = -np.inf, thresholds[0]
            for t in thresholds:
                tpr = np.mean(z_w <= t)
                fpr = np.mean(z_l <= t)
                j = tpr - fpr
                if j > best_j:
                    best_j, best_t, best_tpr, best_fpr = j, t, tpr, fpr
            return best_t, best_j, best_tpr, best_fpr
    
        for sym in sorted(df["symbol"].unique()):
            self.evt.consoleLog(f"\n==================== {sym} ====================")
            df_sym = df[df["symbol"] == sym]
    
            for side_name, side_sign in [("LONG", 1), ("SHORT", -1)]:
                self.evt.consoleLog(f"\n📊 {side_name} TRADES (Symbol: {sym}):")
                df_side = df_sym[df_sym["side_sign"] == side_sign]
                if df_side.empty:
                    self.evt.consoleLog(f"⚠️ No {side_name} trades recorded.\n")
                    continue
    
                winners = df_side[df_side["pnl"] > 0]
                losers  = df_side[df_side["pnl"] <= 0]
                if winners.empty:
                    self.evt.consoleLog("No winning trades.\n")
                    continue
    
                for z_field, label in [("entry_z", "Short-term Z"), ("entry_z_7", "Long-term Z")]:
                    zw = winners[z_field]
                    zl = losers[z_field]
    
                    self.evt.consoleLog(f"\n--- {label} analysis ---")
                    self.evt.consoleLog(f"Winning {label} range: {zw.min():.2f} → {zw.max():.2f}")
                    self.evt.consoleLog(f"Mean/Median/Std (winners): {zw.mean():.2f}, {zw.median():.2f}, {zw.std():.2f}")
                    q1, q3 = np.percentile(zw, [25, 75])
                    self.evt.consoleLog(f"IQR (winners): {q1:.2f} – {q3:.2f}")
    
                    self.evt.consoleLog(f"Mean/Median/Std (losers): {zl.mean():.2f}, {zl.median():.2f}, {zl.std():.2f}")
                    q1l, q3l = np.percentile(zl, [25, 75])
                    self.evt.consoleLog(f"IQR (losers): {q1l:.2f} – {q3l:.2f}")
    
                    best_t, best_j, tpr, fpr = opt_threshold(zw.values, zl.values)
                    pct = np.mean(df_side[z_field] <= best_t)
                    self.evt.consoleLog(f"Count (w/l): {len(zw)} / {len(zl)}")
                    self.evt.consoleLog(f"✅ Optimal {label} threshold: {best_t:.2f} (J = {best_j:.2f})")
                    self.evt.consoleLog(f"📈 TPR = {tpr:.2%}, FPR = {fpr:.2%}, capture = {pct:.2%}")
    
    def on_marketdatafeed(self, md, ab):
        balance = ab["availableBalance"]
        
        sym = md.instrument
        if sym not in self.books: 
            book = InstrumentBar(sym, self.evt)
            
            # get pre-defined z-index and short/long
            cfg = self.instrument_cfg.get(sym)
            if cfg:
                book.allow_long = cfg["allow_long"]
                book.long_z_index_short = cfg["long_z_index_short"]
                book.long_z_index_long = cfg["long_z_index_long"]
                book.allow_short = cfg["allow_short"]
                book.short_z_index_short = cfg["short_z_index_short"]
                book.short_z_index_long = cfg["short_z_index_long"]
            self.books[sym] = book
        else:
            snap = {}
            snap['instrument'] = md.instrument
            snap['timestamp']  = md.timestamp
            snap['lastPrice']  = md.lastPrice
            snap['openPrice']  = md.openPrice
            snap['highPrice']  = md.highPrice
            snap['lowPrice']  = md.lowPrice
            self.books[sym].update(snap, balance)
            
            
    """   
    def on_bulkdatafeed(self, isSync, bd, ab):
        
        balance = ab["availableBalance"]
        for sym, snap in bd.items():        # sym: AUDNZD, EURUSD, ... snap: {'instrument': 'AUDNZD', 'timestamp': datetime.datetime(1970, 1, 1, 0, 0), 'askPrice': -1, ...}
            if sym not in self.books:       # first sight → create book

                book = InstrumentBar(sym, self.evt)

                # Apply matching config if exists
                cfg = self.instrument_cfg.get(sym)
                if cfg:
                    book.allow_long = cfg["allow_long"]
                    book.long_z_index_short = cfg["long_z_index_short"]
                    book.long_z_index_long = cfg["long_z_index_long"]
                    book.allow_short = cfg["allow_short"]
                    book.short_z_index_short = cfg["short_z_index_short"]
                    book.short_z_index_long = cfg["short_z_index_long"]
                self.books[sym] = book
            else: 
                self.books[sym].update(snap, balance)
    """
    def on_orderfeed(self, of):
        
        sym = of.instrument
        if sym not in self.books:
            return                        # safety
        
        fsm = self.books[sym].fsm
        
        # -------------- order filled -----------------
        if of.openclose == "open" and of.status == "success":
            # Is this the order we are waiting for?
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.position_open      = True
                fsm.stop_loss_price    = fsm.pending["stop"]
                fsm.take_profit_price  = fsm.pending["take"]
                fsm.open_volume        = fsm.pending["volume"]
                fsm.enter_market_trade_side = fsm.pending["trend"]
                stoploss_pip_size = fsm.pending["stoploss_pip"]
    
                fsm.pending = None      # clear
                old_state   = fsm.state
                fsm.state   = "ENTER_MARKET"
                fsm.log_state_change(old_state, fsm.state, of.orderRef,
                                     "broker filled order")
            # --------- store entry info ---------------------------------
            self.trades.append(dict(
                orderRef     = of.orderRef,
                symbol       = of.instrument,
                side_sign    = 1 if of.buysell == 1 else -1,   # +1 = long, −1 = short
                volume       = of.fill_volume,
                entry_price  = of.fill_price,
                entry_ts     = of.insertTime,
                entry_z      = fsm.current_zscore(of.orderRef),  # helper shown earlier
                entry_z_7    = fsm.current_zscore_7_days(of.orderRef),
                stoploss_pip = stoploss_pip_size,
                exit_ts      = None,
                pnl          = None
            ))
        # -------------- order rejected ---------------
        elif of.openclose == "open" and of.status != "success":
            if fsm.pending and of.orderRef == fsm.pending["orderRef"]:
                fsm.pending = None
                old_state = fsm.state
                fsm.state = "ENTRY_SIGNAL"     # back to previous state
                fsm.log_state_change(old_state, fsm.state, of.orderRef,
                                     "order rejected")
                                     
        # -------------- position closed --------------
        if of.openclose == 'close' and of.status == 'success': # return 'success' if an order is opened/ closed successfully, or when a limit/stop order is filled
            
            for t in reversed(self.trades):
                if t["symbol"] == of.instrument and t["exit_ts"] is None:
                    t["exit_ts"]    = of.insertTime
                    t["exit_price"] = of.fill_price

                    # signed P/L per unit (positive = winner)
                    #   BUY  →  (exit - entry) * +1
                    #   SELL →  (exit - entry) * -1
                    t["pnl"] = (of.fill_price - t["entry_price"]) * t["side_sign"] \
                            * t["volume"]                         # in “price × volume” units
                    break
            fsm.reset()
            # --------- store entry info ---------------------------------
    
    
    def on_newsdatafeed(self, nd):
        pass

    def on_weatherdatafeed(self, wd):
        pass
    
    def on_econsdatafeed(self, ed):
        pass
        
    def on_corpAnnouncement(self, ca):
        pass
    
    def on_dailyPLfeed(self, pl):
        pass

    def on_openPositionfeed(self, op, oo, uo):
        pass















